# Creating Node Definitions 

This notebook will examine the creation of definitions for standard or custom libraries.

Aspects covered include:
1. Creating "robust" definitions from nodegraph implementations.
2. Creating efficient definitions from source code implementations.
3. "Publishing" definitions to libraries, including the core MaterialX libraries.

## Core Library Setup

In [18]:
# MaterialX and MaterialX utilities
import MaterialX as mx
import mtlxutils.mxfile as mxf
from mtlxutils.mxnodegraph import MtlxNodeGraph as mxg
# For markdown display
from IPython.display import display_markdown

# Version check
from mtlxutils.mxbase import *
haveVersion1387 = haveVersion(1, 38, 7) 
if not haveVersion1387:
    print("** Warning: Recommended minimum version is 1.38.7 for tutorials. Have version: ", mx.__version__)

stdlib = mx.createDocument()
searchPath = mx.getDefaultDataSearchPath()
libraryFolders = mx.getDefaultDataLibraryFolders()
try:
    libFiles = mx.loadLibraries(libraryFolders, searchPath, stdlib)
    print('Loaded %s standard library definitions' % len(stdlib.getNodeDefs()))
except mx.Exception as err:
    print('Failed to load standard library definitions: "', err, '"')

doc = mx.createDocument()
doc.importLibrary(stdlib)

# Write predicate
def skipLibraryElement(elem):
    return not elem.hasSourceUri()

Loaded 721 standard library definitions


## 1. Creating Definitions from Compound Graphs

Creating definitions from compound node graphs is the easiest way to create new definitions without worrying about shading language implementations if the language is supported. 

The basic logic for publishing from a nodegraph entails:
* Copying a compound graph
* Turning the copy into a functional graph
* Creating a 
* definition based on the input and output interfaces on the functional graph
* Adding in additional meta-data tagging as outlined in the definitions learning section:
  * Node group
  * Version
  * Namespace
  * UI properties
* Creating a reference between the definition and the functional graph
 

### Helpers

There is currently a helper interface on <a href="https://materialx.org/docs/api/class_document.html" target="_blank">Document</a> called `addNodeDefFromGraph()` that encapsulates the required logic for the most part. It does not:
* Handle creating of definitions which inherit from each other, nor 
* Update versioning on existing definitions with different versions.

The 1.38.7 version of the utility has some issues with it which is being looked at. These issues can be addressed by patching up the results.

The following is a simple utility wrapper sets up the creation parameters, calls `adNodeDefFromGraph()` and returns the definition (`nodedef`) and the functional graph created. 

In [19]:
def createDefinitionAndFunctionalGraph(nodeGraph, cparam):
    '''
    Example of creating a definition. This has a fixed version, nodegroup and graph names. 
    '''
    definition = doc.addNodeDefFromGraph(nodeGraph, cparam['nodedefName'],
                            cparam['category'], cparam['version'], cparam['defaultversion'], 
                            cparam['nodegroup'], cparam['nodegraphName'])
    funcgraph = doc.getNodeGraph(cparam['nodegraphName'])

    return definition, funcgraph

The `getNodeGroups()` utility will scan for existing node group names. Using existing group names allows for 
* Any associated semantic meanings to be discoverable by code generation. For example `texture2d` and `pbr` have semantic meanings. 
* Naming consistency and avoidance of excessive partitioning into too many disparate groups.
Integrations may which to run this type of logic to examine for existing nodegroups. 

In [20]:
def getNodeGroups(library):
    '''
    Find all the existing node group names on definitions 
    '''
    nodeGroups = set()
    #mx::StringVec nodeGroups;
    for nd in library.getNodeDefs():
        group = nd.getNodeGroup()
        if group:
            nodeGroups.add(group)

    return nodeGroups

The `findCompoundGraphs()` utility scans and returns a list of compound `nodegraphs` in a document.
A list of library file names is passed in as we wish to avoid any compound graphs that exist in a definition
library.

In [21]:
def findCompoundGraphs(doc, libFiles):
    '''
    Search for compound graphs in a document. Skips any graphs found in 
    library files (passed in a a list of source URIs)
    '''
    compoundGraphs = []

    nodeGraphs = doc.getNodeGraphs()
    for nodeGraph in nodeGraphs:
        # Skip any nodegraph which is from a library
        if nodeGraph.getSourceUri() in libFiles:
            continue

        # Skip functional graphs
        if nodeGraph.getNodeDef():
            continue

        compoundGraphs.append(nodeGraph)
    return compoundGraphs

### Example
As an example we will load in an example compound graph and use it to create a definition. The Python utility <a href="createdefinition.py" target="_blank">(`createdefinition`)</a> encapsulates this logic and provides a command line interface for various options.

A compound nodegraph has no definition association. This can be tested by calling `getNodeDef()` on the nodegraph.
If a `nodedef` is returned then it is instead a functional graph.

Once created, we will want to export the definitions and functional graphs into either a new document or an existing definition library document. To do so the contents need to be copied into the desired document.

The utility function `addDefinitionToDocument()` will copy a definition, functional graph pair either 1 or 2 seprate documents. As there is no "copy" function from one document to another, an empty definition and graph needs to be created first and then the contents copied over using the `copyContentFrom()` interface respectively.

As the association between the functional graph and definition resides on nodegraph, this will be copied over
when `copyContentFrom()` is called. If the association is stored on an `implementation` instead then that must
also be copied over.

In [22]:
def addDefinitionToDocument(definition, funcgraph, defDoc, graphDoc=None, defComment='', graphComment=''):
    '''
    Copy a definition and functional node graph to a new document.
    
    '''
    if definition and funcgraph:

        # Add definition comment
        if defComment:
            comment = defDoc.addChildOfCategory('comment')
            comment.setDocString(defComment)                 

        # Create a new definition, and copy the content over. Make sure
        # to use the existing name and category.
        newDef = defDoc.addNodeDef(definition.getName(), '', definition.getCategory())
        newDef.copyContentFrom(definition)        

        if not graphDoc:
            graphDoc = defDoc

        # Add graph comment
        if graphComment:
            comment = graphDoc.addChildOfCategory('comment')
            comment.setDocString(graphComment)                 

        # Create a new graph and copy the contents over. This will result in a functional graph.
        # Use the definiton document if no graph document specified
        newGraph = graphDoc.addNodeGraph(funcgraph.getName())
        newGraph.copyContentFrom(funcgraph)

Some additional utilities are proved to write the contents of the new definition document for display.

In [23]:
def writeDocToString(doc):
    writeOptions = mx.XmlWriteOptions()
    writeOptions.writeXIncludeEnable = False
    writeOptions.elementPredicate = mxf.MtlxFile.skipLibraryElement

    documentContents = mx.writeToXmlString(doc, writeOptions)
    return documentContents

def writeDocToMarkdown(documentContents):
    display_markdown('```xml\n' + documentContents + '\n```\n', raw=True)

def writeToMarkdown(val):
    display_markdown(val, raw=True)    

The main logic loads in an example file, and creates a new document for each definition/functional graph pair.

For the 1.38.7 version API, the following creation parameters are set:

* An arbitrary `category` name is chosen -- which is the compound node graph name. In general this should be set by the user.
* The "standard" prefixes of `ND_` and `NG_` are used as node definition and node graph name prefixes.
* A version is always added. For initial definitions the version number of `1.0` and the flag is set to indicate that this is the default version.
* A node group is always added. This setting is difficult to infer based on just the compound node graph, so will probably require
user input.

In [24]:

doc, libFiles = mxf.MtlxFile.createWorkingDocument()
mx.readFromXmlFile(doc, mx.FilePath('./data/test_procedural.mtlx'))

availableGroupNames = getNodeGroups(doc)
print('> Available node group names', ', '.join(availableGroupNames))

compoundGraphs = findCompoundGraphs(doc, libFiles)
for nodeGraph in compoundGraphs: 
    cparam = {}
    cparam['nodedefName'] = 'ND_' + nodeGraph.getName()
    cparam['category'] = nodeGraph.getName()
    cparam['version'] = '1.0'
    cparam['defaultversion'] = True
    cparam['nodegroup'] = 'texture2d'
    cparam['nodegraphName'] = 'NG_' + nodeGraph.getName()    
    definition, funcgraph = createDefinitionAndFunctionalGraph(nodeGraph, cparam)

    defDoc = mx.createDocument()
    addDefinitionToDocument(definition, funcgraph, defDoc)
    if defDoc:
        documentContents = writeDocToString(defDoc)
        writeDocToMarkdown(documentContents)
    break

> Available node group names translation, light, global, compositing, convolution2d, shader, organization, procedural2d, texture3d, geometric, procedural, channel, pbr, conditional, math, adjustment, procedural3d, colortransform, material, texture2d, application


```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <nodedef name="ND_myProcedural" node="myProcedural" nodegroup="texture2d" version="1.0" isdefaultversion="true">
    <output name="output_color3" type="color3" />
  </nodedef>
  <nodegraph name="NG_myProcedural" nodedef="ND_myProcedural">
    <ramplr name="ramplr_color3" type="color3" xpos="7.246377" ypos="0.594828">
      <input name="valuer" type="color3" value="0.088713, 0.647922, 0.168014" />
      <input name="texcoord" type="vector2" nodename="modulo_vector2" />
    </ramplr>
    <output name="output_color3" type="color3" xpos="17.391304" ypos="0.000000" nodename="ifgreater_color3" />
    <ifgreater name="ifgreater_color3" type="color3" xpos="12.318841" ypos="-0.508621">
      <input name="in1" type="color3" nodename="ramplr_color3" />
      <input name="in2" type="color3" nodename="noise2d_color3" />
      <input name="value1" type="float" nodename="separate2_vector2" />
      <input name="value2" type="float" interfacename="threshold" />
    </ifgreater>
    <texcoord name="texcoord_vector2" type="vector2" xpos="-7.963768" ypos="-0.482759" />
    <separate2 name="separate2_vector2" type="multioutput" xpos="7.246377" ypos="-2.362069">
      <input name="in" type="vector2" nodename="modulo_vector2" />
    </separate2>
    <modulo name="modulo_vector2" type="vector2" xpos="2.173913" ypos="-0.172414">
      <input name="in1" type="vector2" nodename="multiply_vector2" />
      <input name="in2" type="vector2" interfacename="modulo" />
    </modulo>
    <input name="modulo" type="vector2" value="2.1, 1.1" xpos="-2.884058" ypos="0.724138" />
    <noise2d name="noise2d_color3" type="color3" xpos="7.246377" ypos="2.206897">
      <input name="texcoord" type="vector2" nodename="modulo_vector2" />
      <input name="amplitude" type="vector3" interfacename="noiseAmplitude" />
    </noise2d>
    <multiply name="multiply_vector2" type="vector2" xpos="-2.898551" ypos="-0.862069">
      <input name="in1" type="vector2" nodename="texcoord_vector2" />
      <input name="in2" type="vector2" interfacename="repeat" />
    </multiply>
    <input name="threshold" type="float" value="1.2" xpos="7.202899" ypos="-0.646552" />
    <input name="repeat" type="vector2" value="13.2, 9.9" xpos="-7.963768" ypos="0.534483" />
    <input name="noiseAmplitude" type="vector3" value="1.3, 1.7, 4.8" xpos="4.326087" ypos="3.060345" />
  </nodegraph>
</materialx>

```


### 1.38.7 Definition Patching

For 1.38.7 we need to patch the result. This includes 
* Setting the documentation string and adding any namespace.
* Moving the interface inputs from the graph to the definition. ( Note that this is checked during Document `validate()` and will produce warnings if the functional graph has any inputs ). 
* Removing any undesired attributes on nodes and ports.
Both the definition and functional graph need to be patched.

The utility `patchDefinition` encapsulates this logic.

In [25]:
def patchDefinition(definition, funcgraph, documentation, namespace):
    if documentation:
        definition.setDocString(documentation)
    if namespace:
        definition.setNamespace(namespace)
        funcgraph.setNamespace(namespace)

    if not funcgraph:
        return

    for graphChild in funcgraph.getChildren():
        graphChild.removeAttribute('xpos')
        graphChild.removeAttribute('ypos')

    filterAttributes = { 'nodegraph', 'nodename', 'channels', 'interfacename', 'xpos', 'ypos' }

    # Transfer input interface from the graph to the nodedef
    for input in funcgraph.getInputs():
        nodeDefInput = definition.addInput(input.getName(), input.getType())
        if nodeDefInput:
            nodeDefInput.copyContentFrom(input)
            for filterAttribute in filterAttributes:
                nodeDefInput.removeAttribute(filterAttribute);
            nodeDefInput.setSourceUri('')
            input.setInterfaceName(nodeDefInput.getName())

    # Remove interface from the nodegraph
    for input in funcgraph.getInputs():
        funcgraph.removeInput(input.getName())

    # Copy the output interface from the graph to the nodedef
    for output in funcgraph.getOutputs():
        nodeDefOutput = definition.getOutput(output.getName())
        if nodeDefOutput:
            definition.removeOutput(output.getName())
        definition.addOutput(output.getName(), output.getType())
        if nodeDefOutput:
            nodeDefOutput.copyContentFrom(output)
            for filterAttribute in filterAttributes:
                nodeDefOutput.removeAttribute(filterAttribute)
            nodeDefOutput.setSourceUri('')    


Running with the patch results in the new corrected definition:

In [26]:
# Run definition creation again with patching logic
doc, libFiles = mxf.MtlxFile.createWorkingDocument()
mx.readFromXmlFile(doc, mx.FilePath('./data/test_procedural.mtlx'))

compoundGraphs = findCompoundGraphs(doc, libFiles)
for nodeGraph in compoundGraphs: 
    cparam = {}
    cparam['nodedefName'] = 'ND_' + nodeGraph.getName()
    cparam['category'] = nodeGraph.getName()
    cparam['version'] = '1.0'
    cparam['defaultversion'] = True
    cparam['nodegroup'] = 'texture2d'
    cparam['nodegraphName'] = 'NG_' + nodeGraph.getName()      
    definition, funcgraph = createDefinitionAndFunctionalGraph(nodeGraph, cparam)

    # Add documentation and namespace as well as patch up definition and functional graph
    documentation = 'Documentation for new definition: ' + nodeGraph.getName()
    namespace = 'mynamespace'
    patchDefinition(definition, funcgraph, documentation, namespace)

    defDoc = mx.createDocument()
    graphDoc = None
    defComment = ' Definition: nodeGraph.getName() '
    graphComment = ' Functional graph for definition: nodeGraph.getName() '
    
    addDefinitionToDocument(definition, funcgraph, defDoc, graphDoc, defComment, graphComment)
    if defDoc:
        documentContents = writeDocToString(defDoc)
        writeDocToMarkdown(documentContents)

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Definition: nodeGraph.getName() -->
  <nodedef name="ND_myProcedural" node="myProcedural" nodegroup="texture2d" version="1.0" isdefaultversion="true" doc="Documentation for new definition: myProcedural" namespace="mynamespace">
    <input name="modulo" type="vector2" value="2.1, 1.1" />
    <input name="threshold" type="float" value="1.2" />
    <input name="repeat" type="vector2" value="13.2, 9.9" />
    <input name="noiseAmplitude" type="vector3" value="1.3, 1.7, 4.8" />
    <output name="output_color3" type="color3" />
  </nodedef>
  <!-- Functional graph for definition: nodeGraph.getName() -->
  <nodegraph name="NG_myProcedural" nodedef="ND_myProcedural" namespace="mynamespace">
    <ramplr name="ramplr_color3" type="color3">
      <input name="valuer" type="color3" value="0.088713, 0.647922, 0.168014" />
      <input name="texcoord" type="vector2" nodename="modulo_vector2" />
    </ramplr>
    <output name="output_color3" type="color3" nodename="ifgreater_color3" />
    <ifgreater name="ifgreater_color3" type="color3">
      <input name="in1" type="color3" nodename="ramplr_color3" />
      <input name="in2" type="color3" nodename="noise2d_color3" />
      <input name="value1" type="float" nodename="separate2_vector2" />
      <input name="value2" type="float" interfacename="threshold" />
    </ifgreater>
    <texcoord name="texcoord_vector2" type="vector2" />
    <separate2 name="separate2_vector2" type="multioutput">
      <input name="in" type="vector2" nodename="modulo_vector2" />
    </separate2>
    <modulo name="modulo_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="multiply_vector2" />
      <input name="in2" type="vector2" interfacename="modulo" />
    </modulo>
    <noise2d name="noise2d_color3" type="color3">
      <input name="texcoord" type="vector2" nodename="modulo_vector2" />
      <input name="amplitude" type="vector3" interfacename="noiseAmplitude" />
    </noise2d>
    <multiply name="multiply_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="texcoord_vector2" />
      <input name="in2" type="vector2" interfacename="repeat" />
    </multiply>
  </nodegraph>
</materialx>

```


## 2. Creating Definitions from Source Code

When creating a custom node in MaterialX there are basically three things that needs to be created:

* A `<nodedef>` element specifying the signature of the node.
* An `<implementation>` element that tells the code generator where it can find the source code for the node, for a particular target/language. You need one such element for each target you want to support. For the standard library: GLSL (and Vulkan, ESSL variants), OSL, MDL, and MSL should be supported. 
* The source code for the node.




### Creating the Interface

There are no specific tools to directly creation `nodedef` interfaces. We will use a simple example which just adds 2 colors together:
```xml
  <!-- Definition of a simple node <myad>, adding two colors. -->
  <nodedef name="ND_myadd_color3" node="myadd">
    <input name="in1" type="color3" value="1.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 1.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  ```
Note that it is a good practice to have a default routing from the input to the output if a node instance is disabled (is a pass-through).
This can be done by setting the `mx.Output.DEFAULT_INPUT_ATTRIBUTE` attribute on an output. Note that it is only valid to set this on
an output in a definition.  In this case the default value for the output is the input "in1".


In [27]:
def addNodeDefinition(doc, category, type):
    '''
    Add a node definition which uses the standard naming convention.
    No definition is created if a node of the same name already exists in the document 
    '''
    prefix = 'ND_'
    nodedefName = prefix + category + '_' + type 

    existingDef = doc.getChild(nodedefName)
    if existingDef:
        return None

    newDef = doc.addChildOfCategory('nodedef', nodedefName)
    newDef.setNodeString(category)
    return newDef

doc, libFiles = mxf.MtlxFile.createWorkingDocument()
category = 'myadd'
output_type = 'color3'
comment = doc.addChildOfCategory('comment')
comment.setDocString(' Definition of a simple node <myadd>, adding two colors. ')
newDef = addNodeDefinition(doc, category, output_type)

# Example way to add in inputs and outputs
inputs = [ ["in1", "color3", "1.0, 0.0, 0.0"], ["in2", "color3", "0.0, 0.0, 0.0"] ]
outputs = [ ["out", "color3", "in1"] ]
for input in inputs:
    newInput = newDef.addInput(input[0], input[1])
    newInput.setValueString(input[2])
for output in outputs:
    newOutput = newDef.addOutput(output[0], output[1])
    if output[2]:
        newOutput.setAttribute(mx.Output.DEFAULT_INPUT_ATTRIBUTE, output[2])

documentContents = writeDocToString(doc)
writeDocToMarkdown(documentContents)

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Definition of a simple node <myadd>, adding two colors. -->
  <nodedef name="ND_myadd_color3" node="myadd">
    <input name="in1" type="color3" value="1.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
</materialx>

```



As an alternative, the interface could be created as a compound `nodegraph` first using existing graph editing tools and then create a definition
based on it. For example this graph was created interactively in the MaterialX Graph Editor:
```xml
<nodegraph name="myadd">
  <input name="in1" type="color3" value="1, 0, 0" xpos="11.021739" ypos="-3.568965" />
  <input name="in2" type="color3" value="0, 0, 0" xpos="11.115942" ypos="-2.051724" />
  <output name="out" type="color3" xpos="13.456522" ypos="-3.284483" />
</nodegraph>
```
A utility called `copyValueElements()` is used to copy inputs and outputs over.
> Note that `copyValueElements()` replaces the attributes on the node so these need to be cached and restored.

In [28]:
def createDefinitionFromGraphReference(doc, category, output_type, refNodeGraph):
    newDef = addNodeDefinition(doc, category, output_type)

    # Copy the children over from the nodegraph
    newDefAttrs = newDef.getAttributeNames()
    newDefAttrValues = {}
    for newDefAttr in newDefAttrs:
        attr = newDef.getAttribute(newDefAttr)
        newDefAttrValues[newDefAttr] = attr
    newDef.copyContentFrom(refNodeGraph)
    for newDefAttr in newDefAttrs:
        newDef.setAttribute(newDefAttr, newDefAttrValues[newDefAttr])

    # Filter out undesired attributes including connections and ui position
    filterAttributes = { 'nodegraph', 'nodename', 'channels', 'interfacename', 'xpos', 'ypos' }
    for child in newDef.getChildren():
        for filterAttribute in filterAttributes:
            child.removeAttribute(filterAttribute)

    return newDef


As in the previous example the default input value is set manually. It cannot be set compound nodegraph before hand as this is considered to be invalid.

In [29]:

# Read in reference nodegraph
refdoc, reflibFiles = mxf.MtlxFile.createWorkingDocument()
mx.readFromXmlFile(refdoc, mx.FilePath('./data/myadd_compound_graph.mtlx'))

category = 'myadd'
output_type = 'color3'
refNodeGraph = refdoc.getNodeGraph('myadd')

sourceCodeDoc = None
inline_definition = None
if refNodeGraph:
    # Create a new empty definition
    sourceCodeDoc, libFiles = mxf.MtlxFile.createWorkingDocument()
    comment = sourceCodeDoc.addChildOfCategory('comment')
    comment.setDocString(' Definition of a simple node <myadd>, adding two colors. ')
    inline_definition = createDefinitionFromGraphReference(sourceCodeDoc, category, output_type, refNodeGraph)
    for child in inline_definition.getOutputs():
        child.setAttribute(mx.Output.DEFAULT_INPUT_ATTRIBUTE, 'in1')

if sourceCodeDoc:
    docString = writeDocToString(sourceCodeDoc)
    writeDocToMarkdown(docString)

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Definition of a simple node <myadd>, adding two colors. -->
  <nodedef name="ND_myadd_color3" node="myadd">
    <input name="in1" type="color3" value="1, 0, 0" />
    <input name="in2" type="color3" value="0, 0, 0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
</materialx>

```


### Creating Implementations

To support the standard shading languages and implementation will be made for each target.
At this point a choice needs to be made on whether the code can be inlined or not. If it can then the `implementation` element
will hold the code in it's `sourcecode` attribute. If not then additional source code files need to be created and a `file` and `function`
attribute need to be added. 

#### Inlined Source Code

For this implementation example we will first show inlined code which uses tokens to represent arguments. Tokens are use '{{' and '}}' delimiters.

In this example the code becomes:
```
 '{{in1}} + {{in2}}'
 ```
 where `in1` corresponds to the nodedef input `in1` and `in2` to the nodedef input `in2`

In [30]:
def getTargetDefs(doc):
    targets = []
    for element in doc.getChildren():
        if element.getCategory() == 'targetdef':
            if element.getName() != 'essl':
                targets.append(element.getName())
    return targets

def createImplementations(doc, nodedef, targets):
    '''
    Create implementation elements for a set of targets based on a given definition (nodedef).
    All implementation names are of the form:
    
        IM_<category>_<output type>_<target>
    
    '''
    prefix = 'IM_'
    category = nodedef.getNodeString() 
    type = nodedef.getType()
    implName = prefix + category + '_' + type 
    impls = []
    for target in targets:
        #newline = doc.addChildOfCategory('newline')
        comment = doc.addChildOfCategory('comment')
        comment.setDocString(' Implementation of <%s> for target %s and type %s ' % (category, target, type))
        impl = doc.addImplementation(implName + '_' + target)
        impl.setNodeDef(newDef)
        impl.setTarget(target)
        impls.append(impl)

    return impls


def setImplementationSourceCode_v1(doc, nodedef, targets, sourceCode):
    category = nodedef.getNodeString() 
    type = nodedef.getType()
    implName = 'IM_' + category + '_' + type 
    for target, code in zip(targets, sourceCode):
        impl = doc.getImplementation(implName + '_' + target)
        if impl:
            impl.setAttribute('sourcecode', code)

# Create the implementations for all targets based on the nodedef
inlined_doc = mx.createDocument()
inlined_doc.copyContentFrom(sourceCodeDoc)
targets = getTargetDefs(inlined_doc)
inlined_impls = createImplementations(inlined_doc, newDef, targets)

# Set the source code for all targets based on the nodedef.
# In this case all inline implementations are identical.
sourceCode = [ '{{in1}} + {{in2}}' ]
sourceCode = sourceCode * len(targets)
setImplementationSourceCode_v1(inlined_doc, newDef, targets, sourceCode)

docString = mxf.MtlxFile.writeDocumentToString(inlined_doc, mxf.MtlxFile.skipLibraryElement)
writeDocToMarkdown(docString)

mxf.MtlxFile.writeDocumentToFile(inlined_doc, './data/myadd_definition.mtlx', mxf.MtlxFile.skipLibraryElement)

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Definition of a simple node <myadd>, adding two colors. -->
  <nodedef name="ND_myadd_color3" node="myadd">
    <input name="in1" type="color3" value="1, 0, 0" />
    <input name="in2" type="color3" value="0, 0, 0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <!-- Implementation of <myadd> for target genglsl and type color3 -->
  <implementation name="IM_myadd_color3_genglsl" nodedef="ND_myadd_color3" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <!-- Implementation of <myadd> for target genmdl and type color3 -->
  <implementation name="IM_myadd_color3_genmdl" nodedef="ND_myadd_color3" target="genmdl" sourcecode="{{in1}} + {{in2}}" />
  <!-- Implementation of <myadd> for target genmsl and type color3 -->
  <implementation name="IM_myadd_color3_genmsl" nodedef="ND_myadd_color3" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <!-- Implementation of <myadd> for target genosl and type color3 -->
  <implementation name="IM_myadd_color3_genosl" nodedef="ND_myadd_color3" target="genosl" sourcecode="{{in1}} + {{in2}}" />
</materialx>

```


#### Source Code Files For Implementations

If the code cannot be inlined then a new function name is required, with the general guideline to prefix the function name with the string `mx_` followed by catagory and type. For consistency the file names for source code will use the same convention.

Thus for this example:
* `mx_myadd_color3` is the function name and
* `mx_myadd_color3.<shader language extension>` is used for the shader name, where `<shader language extension>` is the native shading language suffix name (e.g. `osl` for the OSL shading language or `msl` for Metal) 

The utility function `setImplementationSourceCode_v1` is extended to differentiate between inline and file source code and called `setImplementationSourceCode` 

In [31]:
def setImplementationSourceCode(doc, nodedef, targets, sourceCode, inlined):
    type = nodedef.getType()
    category = nodedef.getNodeString()
    implName = 'IM_' + category + '_' + type 

    if inlined:
        for target, code in zip(targets, sourceCode):
            impl = doc.getImplementation(implName + '_' + target)
            if impl:
                    impl.setAttribute('sourcecode', code)
    else:
        functionName = 'mx_' + category + '_' + type
        for target, code in zip(targets, sourceCode):
            impl = doc.getImplementation(implName + '_' + target)
            if impl:
                fileName = functionName
                impl.setFunction(functionName)
                fileExtension = target.removeprefix('gen')
                fileName = functionName + '.' + fileExtension
                impl.setFile(fileName)

                # Note: A possible option to add here would be to create the actual source files.

filesource_doc = mx.createDocument()
filesource_doc.copyContentFrom(doc)

targets = getTargetDefs(filesource_doc)
createImplementations(filesource_doc, newDef, targets)

sourceCode = [ 'placeholder text']
sourceCode = sourceCode * len(targets)
setImplementationSourceCode(filesource_doc, newDef, targets, sourceCode, False)

docString = mxf.MtlxFile.writeDocumentToString(filesource_doc, mxf.MtlxFile.skipLibraryElement)
writeDocToMarkdown(docString)

mxf.MtlxFile.writeDocumentToFile(filesource_doc, './data/myadd_definition_file.mtlx', mxf.MtlxFile.skipLibraryElement)

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Definition of a simple node <myadd>, adding two colors. -->
  <nodedef name="ND_myadd_color3" node="myadd">
    <input name="in1" type="color3" value="1.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <!-- Implementation of <myadd> for target genglsl and type color3 -->
  <implementation name="IM_myadd_color3_genglsl" nodedef="ND_myadd_color3" target="genglsl" function="mx_myadd_color3" file="mx_myadd_color3.glsl" />
  <!-- Implementation of <myadd> for target genmdl and type color3 -->
  <implementation name="IM_myadd_color3_genmdl" nodedef="ND_myadd_color3" target="genmdl" function="mx_myadd_color3" file="mx_myadd_color3.mdl" />
  <!-- Implementation of <myadd> for target genmsl and type color3 -->
  <implementation name="IM_myadd_color3_genmsl" nodedef="ND_myadd_color3" target="genmsl" function="mx_myadd_color3" file="mx_myadd_color3.msl" />
  <!-- Implementation of <myadd> for target genosl and type color3 -->
  <implementation name="IM_myadd_color3_genosl" nodedef="ND_myadd_color3" target="genosl" function="mx_myadd_color3" file="mx_myadd_color3.osl" />
</materialx>

```


## 4. Adding Definitions To A "Library"

When a definition is refined to the point where it can be made available generally, there are a few choices to how they are organized and where they will reside. For instance a new definition could either be part of a custom library or potentially contributed back the the MaterialX standard libraries.

### "Standard" Library Organization

Below shows a layout for how the standard libraries are organized on the left.

<img src="../documents/images/definition_library_org.jpg" width="80%">

For example if we consider the groupin on the left to be `stdlib`, then is composed:
* A separate file `stdlib_defs.mtlx` containing all definitions 
* A separate file `stdlib_ng.mtlx` containing all Functional node graph implementations.
* A separate file containing all per-target source code implementation reference. Files are of in per-target sub-folders and of the form: `<target>/stdlib_<target>_impl.mtlx` files.  For example `genglsl/stdlib_genglsl_impl.mtlx` is the implementation file for GLSL (genglsl target)).

This structure is repeated for the pbr library `pbrlib`.

Higher level functional nodegraph implementation only libraries such as `bxdf` are built on top of `pbrilb` and `stdlib`. 

### Custom Library Organization
In the diagram we show a custom library (on the right) which reflects the "standard" libraries. 

There are however many choices as shown on the far right.
For instance functional nodegraphs could be kept separate from other graphs, and/or they could be kept within the file as definition or as two separate files. The same holds true implementations and implementation files. For instance the implementation, functional nodegraph and definition could all
reside in the same file as a self-contained grouping.

### Semantic Groupings
Different attributes could be used for organization such as `category`, `node group`, `namespace`and `version`.


### Library Identification and Dependencies
Note that there is no formal concept of a library and thus no concept of library or definition depeencies. 
For example `stdlib` is just a folder name where definitions reside. The definitions themselves have no reference to a given library identifier.
If a definition from the `bxdf` library is created without loading in  `stdlib` and / or `pbrlib` then this dependency may only 
be detected at graph evaluation time (e.g. for code generation)

Also as noted, include dependencies are not recommended to be specified explicitly as they are file references. There is no
concept of library identifier dependence.



### Adding A Custom Definition to the "Standard" library

A desirable feature is to be able to add definitions into the MaterialX standard libraries.
As definitions have no delineation within a file (beyond an XML comment string), an initial option is to just 
append the definitions, implementations, and functional graphs into the appropriate files.

At time of writing, utilities to aid in this process are under discussion / design currently, with a possible recommended
workflow forth-coming. Note that version `1.38.8` is the minimum version to be able to preserve comments and
newlines properly on XML load and save.

#### Example 1: Definition with Functional Graph Implementation 
In this example, we will take the nodegraph from the Marble example and produce separate documents using the
`addDefinitionToDocument()`.


In [32]:
doc, libFiles = mxf.MtlxFile.createWorkingDocument()
mx.readFromXmlFile(doc, 'data/standard_surface_marble_solid.mtlx')

nodeGraph = doc.getNodeGraph('NG_marble1')

category = 'mymarble'

cparam = {}
cparam['nodedefName'] = 'ND_' + category
cparam['category'] = category
cparam['version'] = '1.0'
cparam['defaultversion'] = True
cparam['nodegroup'] = 'texture2d'
cparam['nodegraphName'] = 'NG_' + category      
definition, funcgraph = createDefinitionAndFunctionalGraph(nodeGraph, cparam)

# Add documentation and namespace as well as patch up definition and functional graph
documentation = ' Custom marble texture defintion: ' + category
namespace = ''
patchDefinition(definition, funcgraph, documentation, namespace)

defDoc = mx.createDocument()
graphDoc = mx.createDocument()
addDefinitionToDocument(definition, funcgraph, defDoc, graphDoc, ' Custom marble definition: : mymarble ', ' Functional graph implementation of custom marble: mymarble ')

writeToMarkdown('### Definition document')
documentContents = writeDocToString(defDoc)
writeDocToMarkdown(documentContents)

writeToMarkdown('### Functional Graph document')
documentContents = writeDocToString(graphDoc)
writeDocToMarkdown(documentContents)



### Definition document

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Custom marble definition: : mymarble -->
  <nodedef name="ND_mymarble" node="mymarble" nodegroup="texture2d" version="1.0" isdefaultversion="true" doc=" Custom marble texture defintion: mymarble">
    <input name="base_color_1" type="color3" value="0.8, 0.8, 0.8" uiname="Color 1" uifolder="Marble Color" />
    <input name="base_color_2" type="color3" value="0.1, 0.1, 0.3" uiname="Color 2" uifolder="Marble Color" />
    <input name="noise_scale_1" type="float" value="6.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 1" uifolder="Marble Noise" />
    <input name="noise_scale_2" type="float" value="4.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 2" uifolder="Marble Noise" />
    <input name="noise_power" type="float" value="3.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Power" uifolder="Marble Noise" />
    <input name="noise_octaves" type="integer" value="3" uisoftmin="1" uisoftmax="8" uiname="Octaves" uifolder="Marble Noise" />
    <output name="out" type="color3" />
  </nodedef>
</materialx>

```


### Functional Graph document

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <!-- Functional graph implementation of custom marble: mymarble -->
  <nodegraph name="NG_mymarble" nodedef="ND_mymarble">
    <position name="obj_pos" type="vector3" />
    <dotproduct name="add_xyz" type="float">
      <input name="in1" type="vector3" nodename="obj_pos" />
      <input name="in2" type="vector3" value="1, 1, 1" />
    </dotproduct>
    <multiply name="scale_xyz" type="float">
      <input name="in1" type="float" nodename="add_xyz" />
      <input name="in2" type="float" interfacename="noise_scale_1" />
    </multiply>
    <multiply name="scale_pos" type="vector3">
      <input name="in1" type="vector3" nodename="obj_pos" />
      <input name="in2" type="float" interfacename="noise_scale_2" />
    </multiply>
    <fractal3d name="noise" type="float">
      <input name="octaves" type="integer" interfacename="noise_octaves" />
      <input name="position" type="vector3" nodename="scale_pos" />
    </fractal3d>
    <multiply name="scale_noise" type="float">
      <input name="in1" type="float" nodename="noise" />
      <input name="in2" type="float" value="3.0" />
    </multiply>
    <add name="sum" type="float">
      <input name="in1" type="float" nodename="scale_xyz" />
      <input name="in2" type="float" nodename="scale_noise" />
    </add>
    <sin name="sin" type="float">
      <input name="in" type="float" nodename="sum" />
    </sin>
    <multiply name="scale" type="float">
      <input name="in1" type="float" nodename="sin" />
      <input name="in2" type="float" value="0.5" />
    </multiply>
    <add name="bias" type="float">
      <input name="in1" type="float" nodename="scale" />
      <input name="in2" type="float" value="0.5" />
    </add>
    <power name="power" type="float">
      <input name="in1" type="float" nodename="bias" />
      <input name="in2" type="float" interfacename="noise_power" />
    </power>
    <mix name="color_mix" type="color3">
      <input name="bg" type="color3" interfacename="base_color_1" />
      <input name="fg" type="color3" interfacename="base_color_2" />
      <input name="mix" type="float" nodename="power" />
    </mix>
    <output name="out" type="color3" nodename="color_mix" />
  </nodegraph>
</materialx>

```


 If the `stdlib` files are used instead then the definition and graph will be appended to
existing files.

In [33]:
def getStandardLibraryFilePaths(library, targets=[]):
    '''
    Get file paths based on a "standard" library configuration
    '''    
    DEFS_POSTFIX = '_defs'
    GRAPH_POSTFIX = '_ng'
    MTLX_EXTENSION = 'mtlx'
    IMPL_POSTFIX = '_impl'

    rootFilePath = mx.FilePath(library)

    defFilePath = mx.FilePath(library + DEFS_POSTFIX)
    defFilePath.addExtension(MTLX_EXTENSION)
    defFilePath = rootFilePath / defFilePath

    graphFilePath = mx.FilePath(library + GRAPH_POSTFIX)
    graphFilePath.addExtension(MTLX_EXTENSION)
    graphFilePath = rootFilePath / graphFilePath

    implFilePaths = []
    for target in targets:
        targetRoot = mx.FilePath(target)
        targetPath = mx.FilePath(library + '_' + target + IMPL_POSTFIX)
        targetPath.addExtension(MTLX_EXTENSION)
        targetPath = rootFilePath / targetRoot / targetPath
        implFilePaths.append(targetPath)

    return defFilePath, graphFilePath, implFilePaths


targets = getTargetDefs(doc)
libraryName = 'stdlib'
defFilePath, graphFilePath, implFilePaths = getStandardLibraryFilePaths(libraryName, targets)
writeToMarkdown('### File Paths for Library: %s ' % libraryName)
writeToMarkdown('* Definition File: %s' % defFilePath.asString())
writeToMarkdown('* Functional Graph File: %s' % graphFilePath.asString())
for implPath, target in zip(implFilePaths, targets):    
    writeToMarkdown('* Target( %s ) implementation file: %s' % (target, implPath.asString()))

### File Paths for Library: stdlib 

* Definition File: stdlib\stdlib_defs.mtlx

* Functional Graph File: stdlib\stdlib_ng.mtlx

* Target( genglsl ) implementation file: stdlib\genglsl\stdlib_genglsl_impl.mtlx

* Target( genmdl ) implementation file: stdlib\genmdl\stdlib_genmdl_impl.mtlx

* Target( genmsl ) implementation file: stdlib\genmsl\stdlib_genmsl_impl.mtlx

* Target( genosl ) implementation file: stdlib\genosl\stdlib_genosl_impl.mtlx

With these file names available we can load in these documents, append to them and write them back out.
Note that we turn on preservation of both comments and newlines so as to not lose any of the original formatting.

In [34]:
targets = getTargetDefs(doc)
libraryName = 'stdlib'
defFilePath, graphFilePath, implFilePaths = getStandardLibraryFilePaths(libraryName, targets)

defaultLibFolder = mx.getDefaultDataLibraryFolders()
defaultSearchPath = mx.getDefaultDataSearchPath()

# Read in files relative to default library search path
defDoc = mx.createDocument()
defFilePath = mx.FilePath(defaultLibFolder[0]) / defFilePath
mx.readFromXmlFile(defDoc, defFilePath, defaultSearchPath)

graphDoc = mx.createDocument()
graphFilePath = mx.FilePath(defaultLibFolder[0]) / graphFilePath
mx.readFromXmlFile(graphDoc, graphFilePath, defaultSearchPath)

# Append the definitions and functional graph to each document
addDefinitionToDocument(definition, funcgraph, defDoc, graphDoc, ' Custom marble definition: : mymarble ', ' Functional graph implementation of custom marble: mymarble ')

# Examine the document
writeOptions = mx.XmlWriteOptions()
writeOptions.writeXIncludeEnable = False
writeOptions.elementPredicate = mxf.MtlxFile.skipLibraryElement

documentContents = mx.writeToXmlString(defDoc, writeOptions)
text = '<details><summary>Standard Libray Definitions with New Definiont</summary>\n\n' + '```xml\n' + documentContents + '```\n' + '</details>\n' 
display_markdown(text , raw=True)  

documentContents = mx.writeToXmlString(graphDoc, writeOptions)
text = '<details><summary>Standard Libray Graphs with New Graph</summary>\n\n' + '```xml\n' + documentContents + '```\n' + '</details>\n' 
display_markdown(text , raw=True)  

# Confirm and clean-up
findGraph = graphDoc.getNodeGraph(funcgraph.getName())
if findGraph:
    print('Functional graph added to: %s' % graphFilePath.asString())
    graphDoc.removeChild(funcgraph.getName())
    findGraph = graphDoc.getNodeGraph(funcgraph.getName())

findDef = defDoc.getNodeDef(definition.getName())
if findDef:
    print('Definition added to: %s' % defFilePath.asString())
    defDoc.removeChild(definition.getName())
    findDef = defDoc.getNodeGraph(definition.getName())


<details><summary>Standard Libray Definitions with New Definiont</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <typedef name="boolean" />
  <typedef name="integer" />
  <typedef name="float" />
  <typedef name="color3" semantic="color" />
  <typedef name="color4" semantic="color" />
  <typedef name="vector2" />
  <typedef name="vector3" />
  <typedef name="vector4" />
  <typedef name="matrix33" />
  <typedef name="matrix44" />
  <typedef name="string" />
  <typedef name="filename" />
  <typedef name="geomname" />
  <typedef name="surfaceshader" semantic="shader" context="surface" />
  <typedef name="displacementshader" semantic="shader" context="displacement" />
  <typedef name="volumeshader" semantic="shader" context="volume" />
  <typedef name="lightshader" semantic="shader" context="light" />
  <typedef name="material" semantic="material" />
  <typedef name="none" />
  <typedef name="integerarray" />
  <typedef name="floatarray" />
  <typedef name="color3array" semantic="color" />
  <typedef name="color4array" semantic="color" />
  <typedef name="vector2array" />
  <typedef name="vector3array" />
  <typedef name="vector4array" />
  <typedef name="stringarray" />
  <typedef name="geomnamearray" />
  <unittypedef name="distance" />
  <unitdef name="UD_stdlib_distance" unittype="distance">
    <unit name="nanometer" scale="0.000000001" />
    <unit name="micron" scale="0.000001" />
    <unit name="millimeter" scale="0.001" />
    <unit name="centimeter" scale="0.01" />
    <unit name="inch" scale="0.0254" />
    <unit name="foot" scale="0.3048" />
    <unit name="yard" scale="0.9144" />
    <unit name="meter" scale="1.0" />
    <unit name="kilometer" scale="1000.0" />
    <unit name="mile" scale="1609.344" />
  </unitdef>
  <unittypedef name="angle" />
  <unitdef name="UD_stdlib_angle" unittype="angle">
    <unit name="degree" scale="1.0" />
    <unit name="radian" scale="57.295779513" />
  </unitdef>
  <geompropdef name="Pobject" type="vector3" geomprop="position" space="object" />
  <geompropdef name="Nobject" type="vector3" geomprop="normal" space="object" />
  <geompropdef name="Tobject" type="vector3" geomprop="tangent" space="object" index="0" />
  <geompropdef name="Bobject" type="vector3" geomprop="bitangent" space="object" index="0" />
  <geompropdef name="Pworld" type="vector3" geomprop="position" space="world" />
  <geompropdef name="Nworld" type="vector3" geomprop="normal" space="world" />
  <geompropdef name="Tworld" type="vector3" geomprop="tangent" space="world" index="0" />
  <geompropdef name="Bworld" type="vector3" geomprop="bitangent" space="world" index="0" />
  <geompropdef name="UV0" type="vector2" geomprop="texcoord" index="0" />
  <nodedef name="ND_surfacematerial" node="surfacematerial" nodegroup="material">
    <input name="surfaceshader" type="surfaceshader" value="" />
    <input name="displacementshader" type="displacementshader" value="" />
    <output name="out" type="material" />
  </nodedef>
  <nodedef name="ND_volumematerial" node="volumematerial" nodegroup="material">
    <input name="volumeshader" type="volumeshader" value="" />
    <output name="out" type="material" />
  </nodedef>
  <nodedef name="ND_surface_unlit" node="surface_unlit" nodegroup="shader" doc="Construct a surface shader from emission and transmission values.">
    <input name="emission" type="float" value="1.0" doc="Surface emission amount." />
    <input name="emission_color" type="color3" value="1,1,1" doc="Surface emission color." />
    <input name="transmission" type="float" value="0.0" doc="Surface transmission amount." />
    <input name="transmission_color" type="color3" value="1,1,1" doc="Surface transmission color." />
    <input name="opacity" type="float" value="1.0" doc="Surface cutout opacity." />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_image_float" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="float" value="0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_image_color3" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_color4" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_vector2" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_vector3" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_vector4" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_float" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_color3" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_color4" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_vector2" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_vector3" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_vector4" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_float" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_color3" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_color4" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_vector2" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_vector3" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_vector4" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_float" node="constant" nodegroup="procedural">
    <input name="value" type="float" value="0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_constant_color3" node="constant" nodegroup="procedural">
    <input name="value" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_color4" node="constant" nodegroup="procedural">
    <input name="value" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_vector2" node="constant" nodegroup="procedural">
    <input name="value" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_vector3" node="constant" nodegroup="procedural">
    <input name="value" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_vector4" node="constant" nodegroup="procedural">
    <input name="value" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_boolean" node="constant" nodegroup="procedural">
    <input name="value" type="boolean" value="false" />
    <output name="out" type="boolean" default="false" />
  </nodedef>
  <nodedef name="ND_constant_integer" node="constant" nodegroup="procedural">
    <input name="value" type="integer" value="0" />
    <output name="out" type="integer" default="0" />
  </nodedef>
  <nodedef name="ND_constant_matrix33" node="constant" nodegroup="procedural">
    <input name="value" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" default="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
  </nodedef>
  <nodedef name="ND_constant_matrix44" node="constant" nodegroup="procedural">
    <input name="value" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" default="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
  </nodedef>
  <nodedef name="ND_constant_string" node="constant" nodegroup="procedural">
    <input name="value" type="string" value="" uniform="true" />
    <output name="out" type="string" default="" />
  </nodedef>
  <nodedef name="ND_constant_filename" node="constant" nodegroup="procedural">
    <input name="value" type="filename" value="" uniform="true" />
    <output name="out" type="filename" default="" />
  </nodedef>
  <nodedef name="ND_ramplr_float" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="float" value="0.0" />
    <input name="valuer" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_color3" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuer" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_color4" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuer" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_vector2" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="vector2" value="0.0, 0.0" />
    <input name="valuer" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_vector3" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuer" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_vector4" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuer" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_float" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="float" value="0.0" />
    <input name="valueb" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_color3" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valueb" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_color4" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valueb" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_vector2" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="vector2" value="0.0, 0.0" />
    <input name="valueb" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_vector3" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valueb" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_vector4" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valueb" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_float" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="float" value="0.0" />
    <input name="valuetr" type="float" value="0.0" />
    <input name="valuebl" type="float" value="0.0" />
    <input name="valuebr" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_color3" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuetr" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuebl" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuebr" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_color4" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuetr" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebl" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebr" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_vector2" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="vector2" value="0.0, 0.0" />
    <input name="valuetr" type="vector2" value="0.0, 0.0" />
    <input name="valuebl" type="vector2" value="0.0, 0.0" />
    <input name="valuebr" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_vector3" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuetr" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuebl" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuebr" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_vector4" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuetr" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebl" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebr" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_float" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="float" value="0.0" uiname="Left" />
    <input name="valuer" type="float" value="0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_color3" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="color3" value="0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="color3" value="0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_color4" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_vector2" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="vector2" value="0.0, 0.0" uiname="Left" />
    <input name="valuer" type="vector2" value="0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_vector3" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="vector3" value="0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="vector3" value="0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_vector4" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_float" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="float" value="0.0" uiname="Top" />
    <input name="valueb" type="float" value="0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_splittb_color3" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="color3" value="0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="color3" value="0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_color4" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_vector2" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="vector2" value="0.0, 0.0" uiname="Top" />
    <input name="valueb" type="vector2" value="0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_vector3" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="vector3" value="0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="vector3" value="0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_vector4" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_float" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color3" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color4" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector2" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector2" value="1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector3" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector4" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color3FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color4FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector2FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector3FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector4FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_float" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color3" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color4" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector2" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector2" value="1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector3" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector4" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color3FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color4FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector2FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector3FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector4FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_float" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color3" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color4" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector2" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector2" value="1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector3" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector4" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color3FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color4FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector2FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector3FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector4FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_cellnoise2d_float" node="cellnoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_cellnoise3d_float" node="cellnoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise2d_float" node="worleynoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise2d_vector2" node="worleynoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise2d_vector3" node="worleynoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise3d_float" node="worleynoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise3d_vector2" node="worleynoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise3d_vector3" node="worleynoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_unifiednoise2d_float" node="unifiednoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" uifolder="Common" defaultgeomprop="UV0" doc="The input 2d space. Default is the first texture coordinates." />
    <input name="freq" type="vector2" uiname="Frequency" uifolder="Common" value="1, 1" doc="Adjusts the noise frequency, with higher values producing smaller noise shapes. Default is (1,1)." />
    <input name="offset" type="vector2" uiname="Offset" uifolder="Common" value="0, 0" doc="Shift the noise in 2d space. Default is (0,0)." />
    <input name="jitter" type="float" uiname="Jitter" uifolder="Common" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjust uniformity of Worley noise; for other noise types jitters the results." />
    <input name="outmin" type="float" uiname="Output Min" uifolder="Post Process" value="0" doc="The lowest values fit to the noise. Default is 0.0." />
    <input name="outmax" type="float" uiname="Output Max" uifolder="Post Process" value="1" doc="The highest values fit to the noise. Default is 1.0." />
    <input name="clampoutput" type="boolean" uiname="Clamp Output" uifolder="Post Process" value="true" doc="Clamp the output to the min and max output values." />
    <input name="octaves" type="integer" uiname="Octaves" uifolder="Fractal" value="3" doc="The number of octaves of Fractal noise to be generated. Default is 3." />
    <input name="lacunarity" type="float" uiname="Lacunarity" uifolder="Fractal" value="2" doc="The exponential scale between successive octaves of Fractal noise. Default is 2.0." />
    <input name="diminish" type="float" uiname="Diminish" uifolder="Fractal" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="The rate at which noise amplitude is diminished for each octave of Fractal noise. Default is 0.5." />
    <input name="type" type="integer" uiname="Noise Type" uifolder="Common" uisoftmin="0" uisoftmax="3" value="0" enum="Perlin,Cell,Worley,Fractal" enumvalues="0,1,2,3" doc="Menu to select the type of noise: Perlin, Cell, Worley, or Fractal. Default is Perlin." />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_unifiednoise3d_float" node="unifiednoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" uifolder="Common" defaultgeomprop="Pobject" doc="The input 3d space. Default is position in object-space." />
    <input name="freq" type="vector3" uiname="Frequency" uifolder="Common" value="1, 1, 1" doc="Adjusts the noise frequency, with higher values producing smaller noise shapes. Default is (1,1,1)." />
    <input name="offset" type="vector3" uiname="Offset" uifolder="Common" value="0, 0, 0" doc="Shift the noise in 3d space. Default is (0,0,0)." />
    <input name="jitter" type="float" uiname="Jitter" uifolder="Common" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjust uniformity of Worley noise; for other noise types jitters the results." />
    <input name="outmin" type="float" uiname="Output Min" uifolder="Post Process" value="0" doc="The lowest values fit to the noise. Default is 0.0." />
    <input name="outmax" type="float" uiname="Output Max" uifolder="Post Process" value="1" doc="The highest values fit to the noise. Default is 1.0." />
    <input name="clampoutput" type="boolean" uiname="Clamp Output" uifolder="Post Process" value="true" doc="Clamp the output to the min and max output values." />
    <input name="octaves" type="integer" uiname="Octaves" uifolder="Fractal" value="3" doc="The number of octaves of Fractal noise to be generated. Default is 3." />
    <input name="lacunarity" type="float" uiname="Lacunarity" uifolder="Fractal" value="2" doc="The exponential scale between successive octaves of Fractal noise. Default is 2.0." />
    <input name="diminish" type="float" uiname="Diminish" uifolder="Fractal" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="The rate at which noise amplitude is diminished for each octave of Fractal noise. Default is 0.5." />
    <input name="type" type="integer" uiname="Noise Type" uifolder="Common" uisoftmin="0" uisoftmax="3" value="0" enum="Perlin,Cell,Worley,Fractal" enumvalues="0,1,2,3" doc="Menu to select the type of noise: Perlin, Cell, Worley, or Fractal. Default is Perlin." />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_checkerboard_color3" node="checkerboard" nodegroup="procedural2d">
    <input name="color1" type="color3" uiname="Color 1" value="1.0, 1.0, 1.0" doc="The first color used in the checkerboard pattern." />
    <input name="color2" type="color3" uiname="Color 2" value="0.0, 0.0, 0.0" doc="The second color used in the checkerboard pattern." />
    <input name="freq" type="vector2" uiname="Frequency" value="8, 8" doc="The frequency of checkers, with higher values producing smaller squares. Default is (8, 8)." />
    <input name="offset" type="vector2" uiname="Offset" value="0, 0" doc="Shift the pattern in 2d space. Default is (0, 0)." />
    <input name="texcoord" type="vector2" uiname="Texture Coordinates" defaultgeomprop="UV0" doc="The input 2d space. Default is the first texture coordinates." />
    <output name="out" type="color3" />
  </nodedef>
  <nodedef name="ND_position_vector3" node="position" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_normal_vector3" node="normal" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <output name="out" type="vector3" default="0.0, 1.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tangent_vector3" node="tangent" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector3" default="1.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_bitangent_vector3" node="bitangent" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 1.0" />
  </nodedef>
  <nodedef name="ND_texcoord_vector2" node="texcoord" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_texcoord_vector3" node="texcoord" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geomcolor_float" node="geomcolor" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_geomcolor_color3" node="geomcolor" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geomcolor_color4" node="geomcolor" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_integer" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="integer" value="0" />
    <output name="out" type="integer" default="0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_boolean" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="boolean" value="false" />
    <output name="out" type="boolean" default="false" />
  </nodedef>
  <nodedef name="ND_geompropvalue_string" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="string" value="" uniform="true" />
    <output name="out" type="string" default="" />
  </nodedef>
  <nodedef name="ND_geompropvalue_float" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="float" value="0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_color3" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_color4" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_vector2" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_vector3" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_vector4" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_bump_vector3" node="bump" nodegroup="geometric">
    <input name="height" type="float" uiname="Height" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Amount to offset the surface normal." />
    <input name="scale" type="float" uiname="Scale" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Scalar to adjust the height amount." />
    <input name="normal" type="vector3" uiname="Normal" defaultgeomprop="Nworld" doc="Surface normal; defaults to the current world-space normal." />
    <input name="tangent" type="vector3" uiname="Tangent" defaultgeomprop="Tworld" doc="Surface tangent vector, defaults to the current world-space tangent vector." />
    <output name="out" type="vector3" doc="Offset surface normal; connect this to a shader's 'normal' input." />
  </nodedef>
  <nodedef name="ND_ambientocclusion_float" node="ambientocclusion" nodegroup="global">
    <input name="coneangle" type="float" value="90.0" unittype="angle" unit="degree" />
    <input name="maxdistance" type="float" value="1e38" />
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_frame_float" node="frame" nodegroup="application">
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_time_float" node="time" nodegroup="application">
    <input name="fps" type="float" value="24.0" />
    <output name="out" type="float" default="0.041666667" />
  </nodedef>
  <nodedef name="ND_add_float" node="add" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color3" node="add" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color4" node="add" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector2" node="add" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector3" node="add" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector4" node="add" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix33" node="add" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix44" node="add" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color3FA" node="add" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color4FA" node="add" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector2FA" node="add" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector3FA" node="add" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector4FA" node="add" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix33FA" node="add" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix44FA" node="add" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_float" node="subtract" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color3" node="subtract" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color4" node="subtract" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector2" node="subtract" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector3" node="subtract" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector4" node="subtract" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix33" node="subtract" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix44" node="subtract" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color3FA" node="subtract" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color4FA" node="subtract" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector2FA" node="subtract" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector3FA" node="subtract" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector4FA" node="subtract" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix33FA" node="subtract" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix44FA" node="subtract" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_float" node="multiply" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color3" node="multiply" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color4" node="multiply" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector2" node="multiply" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector3" node="multiply" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector4" node="multiply" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_matrix33" node="multiply" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_matrix44" node="multiply" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color3FA" node="multiply" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color4FA" node="multiply" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector2FA" node="multiply" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector3FA" node="multiply" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector4FA" node="multiply" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_float" node="divide" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color3" node="divide" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color4" node="divide" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector2" node="divide" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector3" node="divide" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector4" node="divide" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_matrix33" node="divide" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_matrix44" node="divide" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color3FA" node="divide" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color4FA" node="divide" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector2FA" node="divide" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector3FA" node="divide" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector4FA" node="divide" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_float" node="modulo" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color3" node="modulo" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color4" node="modulo" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector2" node="modulo" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector3" node="modulo" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector4" node="modulo" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color3FA" node="modulo" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color4FA" node="modulo" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector2FA" node="modulo" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector3FA" node="modulo" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector4FA" node="modulo" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_invert_float" node="invert" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color3" node="invert" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color4" node="invert" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector2" node="invert" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector3" node="invert" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector4" node="invert" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color3FA" node="invert" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color4FA" node="invert" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector2FA" node="invert" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector3FA" node="invert" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector4FA" node="invert" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_float" node="absval" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_color3" node="absval" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_color4" node="absval" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_vector2" node="absval" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_vector3" node="absval" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_vector4" node="absval" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_float" node="floor" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_color3" node="floor" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_color4" node="floor" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_vector2" node="floor" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_vector3" node="floor" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_vector4" node="floor" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_integer" node="floor" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="integer" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_float" node="ceil" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_color3" node="ceil" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_color4" node="ceil" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_vector2" node="ceil" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_vector3" node="ceil" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_vector4" node="ceil" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_integer" node="ceil" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="integer" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_power_float" node="power" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color3" node="power" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color4" node="power" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector2" node="power" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector3" node="power" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector4" node="power" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color3FA" node="power" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color4FA" node="power" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector2FA" node="power" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector3FA" node="power" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector4FA" node="power" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_float" node="sin" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_float" node="cos" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_float" node="tan" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_float" node="asin" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_float" node="acos" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_float" node="atan2" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_vector2" node="sin" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_vector2" node="cos" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_vector2" node="tan" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_vector2" node="asin" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_vector2" node="acos" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_vector2" node="atan2" nodegroup="math">
    <input name="in1" type="vector2" value="1.0, 1.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_vector3" node="sin" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_vector3" node="cos" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_vector3" node="tan" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_vector3" node="asin" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_vector3" node="acos" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_vector3" node="atan2" nodegroup="math">
    <input name="in1" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_vector4" node="sin" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_vector4" node="cos" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_vector4" node="tan" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_vector4" node="asin" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_vector4" node="acos" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_vector4" node="atan2" nodegroup="math">
    <input name="in1" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sqrt_float" node="sqrt" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_float" node="ln" nodegroup="math">
    <input name="in" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_float" node="exp" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sqrt_vector2" node="sqrt" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_vector2" node="ln" nodegroup="math">
    <input name="in" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_vector2" node="exp" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sqrt_vector3" node="sqrt" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_vector3" node="ln" nodegroup="math">
    <input name="in" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_vector3" node="exp" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sqrt_vector4" node="sqrt" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_vector4" node="ln" nodegroup="math">
    <input name="in" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_vector4" node="exp" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_float" node="sign" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_color3" node="sign" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_color4" node="sign" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_vector2" node="sign" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_vector3" node="sign" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_vector4" node="sign" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_float" node="clamp" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color3" node="clamp" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="color3" value="0.0, 0.0, 0.0" />
    <input name="high" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color4" node="clamp" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector2" node="clamp" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="vector2" value="0.0, 0.0" />
    <input name="high" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector3" node="clamp" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="high" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector4" node="clamp" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color3FA" node="clamp" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color4FA" node="clamp" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector2FA" node="clamp" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector3FA" node="clamp" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector4FA" node="clamp" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_min_float" node="min" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color3" node="min" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color4" node="min" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector2" node="min" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector3" node="min" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector4" node="min" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color3FA" node="min" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color4FA" node="min" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector2FA" node="min" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector3FA" node="min" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector4FA" node="min" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_float" node="max" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color3" node="max" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color4" node="max" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector2" node="max" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector3" node="max" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector4" node="max" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color3FA" node="max" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color4FA" node="max" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector2FA" node="max" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector3FA" node="max" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector4FA" node="max" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_normalize_vector2" node="normalize" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_normalize_vector3" node="normalize" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_normalize_vector4" node="normalize" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_magnitude_vector2" node="magnitude" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_magnitude_vector3" node="magnitude" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_magnitude_vector4" node="magnitude" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_distance_vector2" node="distance" nodegroup="math">
    <input name="in1" type="vector2" uiname="in1" value="0.0, 0.0" />
    <input name="in2" type="vector2" uiname="in2" value="0.0, 0.0" />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_distance_vector3" node="distance" nodegroup="math">
    <input name="in1" type="vector3" uiname="in1" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" uiname="in2" value="0.0, 0.0, 0.0" />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_distance_vector4" node="distance" nodegroup="math">
    <input name="in1" type="vector4" uiname="in1" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" uiname="in2" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_dotproduct_vector2" node="dotproduct" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_dotproduct_vector3" node="dotproduct" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_dotproduct_vector4" node="dotproduct" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_crossproduct_vector3" node="crossproduct" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_transformpoint_vector3" node="transformpoint" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="fromspace" type="string" value="" uniform="true" />
    <input name="tospace" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformvector_vector3" node="transformvector" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="fromspace" type="string" value="" uniform="true" />
    <input name="tospace" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformnormal_vector3" node="transformnormal" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 1.0" />
    <input name="fromspace" type="string" value="" uniform="true" />
    <input name="tospace" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector2M3" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="mat" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector3" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mat" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector3M4" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mat" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector4" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mat" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_normalmap" node="normalmap" nodegroup="math">
    <input name="in" type="vector3" value="0.5, 0.5, 1.0" />
    <input name="space" type="string" value="tangent" enum="tangent, object" uniform="true" />
    <input name="scale" type="float" value="1.0" />
    <input name="normal" type="vector3" defaultgeomprop="Nworld" />
    <input name="tangent" type="vector3" defaultgeomprop="Tworld" />
    <output name="out" type="vector3" defaultinput="normal" />
  </nodedef>
  <nodedef name="ND_transpose_matrix33" node="transpose" nodegroup="math">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transpose_matrix44" node="transpose" nodegroup="math">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_determinant_matrix33" node="determinant" nodegroup="math">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_determinant_matrix44" node="determinant" nodegroup="math">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_invertmatrix_matrix33" node="invertmatrix" nodegroup="math">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invertmatrix_matrix44" node="invertmatrix" nodegroup="math">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rotate2d_vector2" node="rotate2d" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="float" value="0.0" unittype="angle" unit="degree" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rotate3d_vector3" node="rotate3d" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="0.0" unittype="angle" unit="degree" />
    <input name="axis" type="vector3" value="0.0, 1.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_place2d_vector2" node="place2d" nodegroup="math">
    <input name="texcoord" type="vector2" value="0.0, 0.0" />
    <input name="pivot" type="vector2" value="0.0,0.0" />
    <input name="scale" type="vector2" value="1.0,1.0" />
    <input name="rotate" type="float" value="0.0" unittype="angle" unit="degree" />
    <input name="offset" type="vector2" value="0.0,0.0" />
    <input name="operationorder" type="integer" value="0" enum="SRT, TRS" enumvalues="0, 1" />
    <output name="out" type="vector2" defaultinput="texcoord" />
  </nodedef>
  <nodedef name="ND_arrayappend_integer_integerarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="integer" value="0" />
    <input name="in2" type="integer" value="0" />
    <output name="out" type="integerarray" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_integerarray_integerarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="integerarray" value="" />
    <input name="in2" type="integer" value="0" />
    <output name="out" type="integerarray" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_float_floatarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="floatarray" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_floatarray_floatarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="floatarray" value="" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="floatarray" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_color3_color3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_color3array_color3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color3array" value="" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_color4_color4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_color4array_color4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color4array" value="" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector2_vector2array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector2array_vector2array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector2array" value="" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector3_vector3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector3array_vector3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector3array" value="" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector4_vector4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector4array_vector4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector4array" value="" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_string_stringarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="string" value="" />
    <input name="in2" type="string" value="" />
    <output name="out" type="stringarray" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_stringarray_stringarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="stringarray" value="" />
    <input name="in2" type="string" value="" />
    <output name="out" type="stringarray" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_remap_float" node="remap" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color3" node="remap" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="color3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color4" node="remap" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector2" node="remap" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="vector2" value="0.0, 0.0" />
    <input name="inhigh" type="vector2" value="1.0, 1.0" />
    <input name="outlow" type="vector2" value="0.0, 0.0" />
    <input name="outhigh" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector3" node="remap" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector4" node="remap" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color3FA" node="remap" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color4FA" node="remap" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector2FA" node="remap" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector3FA" node="remap" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector4FA" node="remap" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_float" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color3" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="color3" value="0.0, 0.0, 0.0" />
    <input name="high" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color4" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector2" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="vector2" value="0.0, 0.0" />
    <input name="high" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector3" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="high" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector4" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color3FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color4FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector2FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector3FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector4FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_float" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_color3" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_color4" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_vector2" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_vector3" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_vector4" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_luminance_color3" node="luminance" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_luminance_color4" node="luminance" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rgbtohsv_color3" node="rgbtohsv" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rgbtohsv_color4" node="rgbtohsv" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvtorgb_color3" node="hsvtorgb" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvtorgb_color4" node="hsvtorgb" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_float" node="contrast" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color3" node="contrast" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="color3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="color3" value="0.5, 0.5, 0.5" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color4" node="contrast" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="color4" value="0.5, 0.5, 0.5, 0.5" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector2" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="vector2" value="1.0, 1.0" />
    <input name="pivot" type="vector2" value="0.5, 0.5" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector3" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="vector3" value="0.5, 0.5, 0.5" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector4" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="vector4" value="0.5, 0.5, 0.5, 0.5" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color3FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color4FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector2FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector3FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector4FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_float" node="range" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color3" node="range" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="color3" value="1.0, 1.0, 1.0" />
    <input name="gamma" type="color3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="color3" value="1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color4" node="range" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="gamma" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector2" node="range" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="vector2" value="0.0, 0.0" />
    <input name="inhigh" type="vector2" value="1.0, 1.0" />
    <input name="gamma" type="vector2" value="1.0, 1.0" />
    <input name="outlow" type="vector2" value="0.0, 0.0" />
    <input name="outhigh" type="vector2" value="1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector3" node="range" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="gamma" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector4" node="range" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="gamma" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color3FA" node="range" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color4FA" node="range" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector2FA" node="range" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector3FA" node="range" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector4FA" node="range" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvadjust_color3" node="hsvadjust" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="0.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvadjust_color4" node="hsvadjust" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="0.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_saturate_color3" node="saturate" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_saturate_color4" node="saturate" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_colorcorrect_color3" node="colorcorrect" nodegroup="adjustment">
    <input name="in" type="color3" uiname="Input Color" value="1, 1, 1" doc="The input color to be adjusted." />
    <input name="hue" type="float" uiname="Hue" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Rotates the color hue, with values wrapping at 0-1 boundaries." />
    <input name="saturation" type="float" uiname="Saturation" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjusts the input color saturation level." />
    <input name="gamma" type="float" uiname="Gamma" uisoftmin="0.0" uisoftmax="3.0" value="1" doc="Applies a gamma correction to the color." />
    <input name="lift" type="float" uiname="Lift" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Raise the dark color values, leaving the white values unchanged." />
    <input name="gain" type="float" uiname="Gain" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Multiplier increases lighter color values, leaving black values unchanged." />
    <input name="contrast" type="float" uiname="Contrast" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Linearly increase or decrease the color contrast." />
    <input name="contrastpivot" type="float" uiname="Contrast Pivot" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="Pivot value around which contrast applies. This value will not change as contrast is adjusted." />
    <input name="exposure" type="float" uiname="Exposure" uisoftmin="-1.0" uisoftmax="1.0" value="0" doc="Multplier which increases or decreases color brightness by 2^value." />
    <output name="out" type="color3" />
  </nodedef>
  <nodedef name="ND_colorcorrect_color4" node="colorcorrect" nodegroup="adjustment">
    <input name="in" type="color4" uiname="Input Color" value="1, 1, 1, 0" doc="The input color to be adjusted." />
    <input name="hue" type="float" uiname="Hue" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Rotates the color hue, with values wrapping at 0-1 boundaries." />
    <input name="saturation" type="float" uiname="Saturation" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjusts the input color saturation level." />
    <input name="gamma" type="float" uiname="Gamma" uisoftmin="0.0" uisoftmax="3.0" value="1" doc="Applies a gamma correction to the color." />
    <input name="lift" type="float" uiname="Lift" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Raise the dark color values, leaving the white values unchanged." />
    <input name="gain" type="float" uiname="Gain" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Multiplier increases lighter color values, leaving black values unchanged." />
    <input name="contrast" type="float" uiname="Contrast" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Linearly increase or decrease the color contrast." />
    <input name="contrastpivot" type="float" uiname="Contrast Pivot" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="Pivot value around which contrast applies. This value will not change as contrast is adjusted." />
    <input name="exposure" type="float" uiname="Exposure" uisoftmin="-1.0" uisoftmax="1.0" value="0" doc="Multplier which increases or decreases color brightness by 2^value." />
    <output name="out" type="color4" />
  </nodedef>
  <nodedef name="ND_premult_color4" node="premult" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_unpremult_color4" node="unpremult" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_plus_float" node="plus" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_plus_color3" node="plus" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_plus_color4" node="plus" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_minus_float" node="minus" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_minus_color3" node="minus" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_minus_color4" node="minus" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_difference_float" node="difference" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_difference_color3" node="difference" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_difference_color4" node="difference" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_burn_float" node="burn" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_burn_color3" node="burn" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_burn_color4" node="burn" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_dodge_float" node="dodge" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_dodge_color3" node="dodge" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_dodge_color4" node="dodge" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_screen_float" node="screen" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_screen_color3" node="screen" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_screen_color4" node="screen" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_overlay_float" node="overlay" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_overlay_color3" node="overlay" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_overlay_color4" node="overlay" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_disjointover_color4" node="disjointover" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_in_color4" node="in" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mask_color4" node="mask" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_matte_color4" node="matte" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_out_color4" node="out" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_over_color4" node="over" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_inside_float" node="inside" nodegroup="compositing">
    <input name="in" type="float" value="0.0" />
    <input name="mask" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_inside_color3" node="inside" nodegroup="compositing">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_inside_color4" node="inside" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_outside_float" node="outside" nodegroup="compositing">
    <input name="in" type="float" value="0.0" />
    <input name="mask" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_outside_color3" node="outside" nodegroup="compositing">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_outside_color4" node="outside" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_mix_float" node="mix" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color3" node="mix" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color3_color3" node="mix" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="color3" value="0.0, 0.0, 0.0" uisoftmin="0,0,0" uisoftmax="1,1,1" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color4" node="mix" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color4_color4" node="mix" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="color4" value="0.0, 0.0, 0.0, 0.0" uisoftmin="0,0,0,0" uisoftmax="1,1,1,1" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector2" node="mix" nodegroup="compositing">
    <input name="fg" type="vector2" value="0.0, 0.0" />
    <input name="bg" type="vector2" value="0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="vector2" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector2_vector2" node="mix" nodegroup="compositing">
    <input name="fg" type="vector2" value="0.0, 0.0" />
    <input name="bg" type="vector2" value="0.0, 0.0" />
    <input name="mix" type="vector2" value="0.0, 0.0" uisoftmin="0,0" uisoftmax="1,1" />
    <output name="out" type="vector2" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector3" node="mix" nodegroup="compositing">
    <input name="fg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="vector3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector3_vector3" node="mix" nodegroup="compositing">
    <input name="fg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="vector3" value="0.0, 0.0, 0.0" uisoftmin="0,0,0" uisoftmax="1,1,1" />
    <output name="out" type="vector3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector4" node="mix" nodegroup="compositing">
    <input name="fg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="vector4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector4_vector4" node="mix" nodegroup="compositing">
    <input name="fg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="vector4" value="0.0, 0.0, 0.0, 0.0" uisoftmin="0,0,0,0" uisoftmax="1,1,1,1" />
    <output name="out" type="vector4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_surfaceshader" node="mix" nodegroup="compositing">
    <input name="fg" type="surfaceshader" value="" />
    <input name="bg" type="surfaceshader" value="" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="surfaceshader" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_displacementshader" node="mix" nodegroup="compositing">
    <input name="fg" type="displacementshader" value="" />
    <input name="bg" type="displacementshader" value="" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="displacementshader" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_volumeshader" node="mix" nodegroup="compositing">
    <input name="fg" type="volumeshader" value="" />
    <input name="bg" type="volumeshader" value="" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="volumeshader" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_ifgreater_float" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color3" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color4" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector2" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector3" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector4" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_floatI" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color3I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color4I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector2I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector3I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector4I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_float" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color3" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color4" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector2" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector3" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector4" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_floatI" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color3I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color4I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector2I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector3I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector4I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_float" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color3" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color4" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector2" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector3" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector4" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_floatI" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color3I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color4I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector2I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector3I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector4I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_floatB" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color3B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color4B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector2B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector3B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector4B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_float" node="switch" nodegroup="conditional">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <input name="in5" type="float" value="0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color3" node="switch" nodegroup="conditional">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="color3" value="0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color4" node="switch" nodegroup="conditional">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector2" node="switch" nodegroup="conditional">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <input name="in3" type="vector2" value="0.0, 0.0" />
    <input name="in4" type="vector2" value="0.0, 0.0" />
    <input name="in5" type="vector2" value="0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector3" node="switch" nodegroup="conditional">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector4" node="switch" nodegroup="conditional">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_floatI" node="switch" nodegroup="conditional">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <input name="in5" type="float" value="0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color3I" node="switch" nodegroup="conditional">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="color3" value="0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color4I" node="switch" nodegroup="conditional">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector2I" node="switch" nodegroup="conditional">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <input name="in3" type="vector2" value="0.0, 0.0" />
    <input name="in4" type="vector2" value="0.0, 0.0" />
    <input name="in5" type="vector2" value="0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector3I" node="switch" nodegroup="conditional">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector4I" node="switch" nodegroup="conditional">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_convert_float_color3" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_color4" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_vector2" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_vector3" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_vector4" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector2_vector3" node="convert" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector3_color3" node="convert" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector3_vector2" node="convert" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector3_vector4" node="convert" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector4_color4" node="convert" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector4_vector3" node="convert" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color3_vector3" node="convert" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color4_vector4" node="convert" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color3_color4" node="convert" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color4_color3" node="convert" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_boolean_float" node="convert" nodegroup="channel">
    <input name="in" type="boolean" value="false" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_convert_integer_float" node="convert" nodegroup="channel">
    <input name="in" type="integer" value="0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_convert_color3_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert color3 to shader">
    <input name="in" type="color3" value="0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_color4_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert color4  to shader">
    <input name="in" type="color4" value="0, 0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_float_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert float to shader">
    <input name="in" type="float" value="0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_vector2_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert vector2 to shader">
    <input name="in" type="vector2" value="0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_vector3_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert vector2 to shader">
    <input name="in" type="vector3" value="0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_vector4_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert vector4 to shader">
    <input name="in" type="vector4" value="0, 0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_integer_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert integer to shader">
    <input name="in" type="integer" value="0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_boolean_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert boolean to shader">
    <input name="in" type="boolean" value="false" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_swizzle_float_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_float" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="r" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rr" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_float" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="r" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rr" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_float" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="x" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_float" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="x" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_float" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="x" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_combine2_vector2" node="combine2" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine2_color4CF" node="combine2" nodegroup="channel">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine2_vector4VF" node="combine2" nodegroup="channel">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine2_vector4VV" node="combine2" nodegroup="channel">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine3_color3" node="combine3" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine3_vector3" node="combine3" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine4_color4" node="combine4" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine4_vector4" node="combine4" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_extract_color3" node="extract" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="2" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_color4" node="extract" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="3" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_vector2" node="extract" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="1" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_vector3" node="extract" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="2" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_vector4" node="extract" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="3" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate2_vector2" node="separate2" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="outx" type="float" default="0.0" />
    <output name="outy" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate3_color3" node="separate3" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="outr" type="float" default="0.0" />
    <output name="outg" type="float" default="0.0" />
    <output name="outb" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate3_vector3" node="separate3" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="outx" type="float" default="0.0" />
    <output name="outy" type="float" default="0.0" />
    <output name="outz" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate4_color4" node="separate4" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="outr" type="float" default="0.0" />
    <output name="outg" type="float" default="0.0" />
    <output name="outb" type="float" default="0.0" />
    <output name="outa" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate4_vector4" node="separate4" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="outx" type="float" default="0.0" />
    <output name="outy" type="float" default="0.0" />
    <output name="outz" type="float" default="0.0" />
    <output name="outw" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_blur_float" node="blur" nodegroup="convolution2d">
    <input name="in" type="float" value="0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_color3" node="blur" nodegroup="convolution2d">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_color4" node="blur" nodegroup="convolution2d">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_vector2" node="blur" nodegroup="convolution2d">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_vector3" node="blur" nodegroup="convolution2d">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_vector4" node="blur" nodegroup="convolution2d">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_heighttonormal_vector3" node="heighttonormal" nodegroup="convolution2d">
    <input name="in" type="float" value="0.0" />
    <input name="scale" type="float" value="1.0" />
    <output name="out" type="vector3" default="0.5, 0.5, 1.0" />
  </nodedef>
  <nodedef name="ND_dot_float" node="dot" nodegroup="organization">
    <input name="in" type="float" value="0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_color3" node="dot" nodegroup="organization">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_color4" node="dot" nodegroup="organization">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_vector2" node="dot" nodegroup="organization">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_vector3" node="dot" nodegroup="organization">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_vector4" node="dot" nodegroup="organization">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_boolean" node="dot" nodegroup="organization">
    <input name="in" type="boolean" value="false" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="boolean" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_integer" node="dot" nodegroup="organization">
    <input name="in" type="integer" value="0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="integer" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_matrix33" node="dot" nodegroup="organization">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="matrix33" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_matrix44" node="dot" nodegroup="organization">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="matrix44" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_string" node="dot" nodegroup="organization">
    <input name="in" type="string" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="string" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_filename" node="dot" nodegroup="organization">
    <input name="in" type="filename" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="filename" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_surfaceshader" node="dot" nodegroup="organization">
    <input name="in" type="surfaceshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="surfaceshader" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_displacementshader" node="dot" nodegroup="organization">
    <input name="in" type="displacementshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="displacementshader" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_volumeshader" node="dot" nodegroup="organization">
    <input name="in" type="volumeshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="volumeshader" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_lightshader" node="dot" nodegroup="organization">
    <input name="in" type="lightshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="lightshader" defaultinput="in" />
  </nodedef>
  <!-- Custom marble definition: : mymarble -->
  <nodedef name="ND_mymarble" node="mymarble" nodegroup="texture2d" version="1.0" isdefaultversion="true" doc=" Custom marble texture defintion: mymarble">
    <input name="base_color_1" type="color3" value="0.8, 0.8, 0.8" uiname="Color 1" uifolder="Marble Color" />
    <input name="base_color_2" type="color3" value="0.1, 0.1, 0.3" uiname="Color 2" uifolder="Marble Color" />
    <input name="noise_scale_1" type="float" value="6.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 1" uifolder="Marble Noise" />
    <input name="noise_scale_2" type="float" value="4.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 2" uifolder="Marble Noise" />
    <input name="noise_power" type="float" value="3.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Power" uifolder="Marble Noise" />
    <input name="noise_octaves" type="integer" value="3" uisoftmin="1" uisoftmax="8" uiname="Octaves" uifolder="Marble Noise" />
    <output name="out" type="color3" />
  </nodedef>
</materialx>
```
</details>


<details><summary>Standard Libray Graphs with New Graph</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <nodegraph name="NG_tiledimage_float" nodedef="ND_tiledimage_float">
    <multiply name="N_mult_float" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="uvtiling" />
    </multiply>
    <subtract name="N_sub_float" type="vector2">
      <input name="in1" type="vector2" nodename="N_mult_float" />
      <input name="in2" type="vector2" interfacename="uvoffset" />
    </subtract>
    <divide name="N_divtilesize_float" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_float" />
      <input name="in2" type="vector2" interfacename="realworldimagesize" />
    </divide>
    <multiply name="N_multtilesize_float" type="vector2">
      <input name="in1" type="vector2" nodename="N_divtilesize_float" />
      <input name="in2" type="vector2" interfacename="realworldtilesize" />
    </multiply>
    <image name="N_img_float" type="float">
      <input name="file" type="filename" interfacename="file" />
      <input name="default" type="float" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_multtilesize_float" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <output name="out" type="float" nodename="N_img_float" />
  </nodegraph>
  <nodegraph name="NG_tiledimage_color3" nodedef="ND_tiledimage_color3">
    <multiply name="N_mult_color3" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="uvtiling" />
    </multiply>
    <subtract name="N_sub_color3" type="vector2">
      <input name="in1" type="vector2" nodename="N_mult_color3" />
      <input name="in2" type="vector2" interfacename="uvoffset" />
    </subtract>
    <divide name="N_divtilesize_color3" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_color3" />
      <input name="in2" type="vector2" interfacename="realworldimagesize" />
    </divide>
    <multiply name="N_multtilesize_color3" type="vector2">
      <input name="in1" type="vector2" nodename="N_divtilesize_color3" />
      <input name="in2" type="vector2" interfacename="realworldtilesize" />
    </multiply>
    <image name="N_img_color3" type="color3">
      <input name="file" type="filename" interfacename="file" />
      <input name="default" type="color3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_multtilesize_color3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <output name="out" type="color3" nodename="N_img_color3" />
  </nodegraph>
  <nodegraph name="NG_tiledimage_color4" nodedef="ND_tiledimage_color4">
    <multiply name="N_mult_color4" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="uvtiling" />
    </multiply>
    <subtract name="N_sub_color4" type="vector2">
      <input name="in1" type="vector2" nodename="N_mult_color4" />
      <input name="in2" type="vector2" interfacename="uvoffset" />
    </subtract>
    <divide name="N_divtilesize_color4" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_color4" />
      <input name="in2" type="vector2" interfacename="realworldimagesize" />
    </divide>
    <multiply name="N_multtilesize_color4" type="vector2">
      <input name="in1" type="vector2" nodename="N_divtilesize_color4" />
      <input name="in2" type="vector2" interfacename="realworldtilesize" />
    </multiply>
    <image name="N_img_color4" type="color4">
      <input name="file" type="filename" interfacename="file" />
      <input name="default" type="color4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_multtilesize_color4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <output name="out" type="color4" nodename="N_img_color4" />
  </nodegraph>
  <nodegraph name="NG_tiledimage_vector2" nodedef="ND_tiledimage_vector2">
    <multiply name="N_mult_vector2" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="uvtiling" />
    </multiply>
    <subtract name="N_sub_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_mult_vector2" />
      <input name="in2" type="vector2" interfacename="uvoffset" />
    </subtract>
    <divide name="N_divtilesize_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_vector2" />
      <input name="in2" type="vector2" interfacename="realworldimagesize" />
    </divide>
    <multiply name="N_multtilesize_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_divtilesize_vector2" />
      <input name="in2" type="vector2" interfacename="realworldtilesize" />
    </multiply>
    <image name="N_img_vector2" type="vector2">
      <input name="file" type="filename" interfacename="file" />
      <input name="default" type="vector2" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_multtilesize_vector2" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <output name="out" type="vector2" nodename="N_img_vector2" />
  </nodegraph>
  <nodegraph name="NG_tiledimage_vector3" nodedef="ND_tiledimage_vector3">
    <multiply name="N_mult_vector3" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="uvtiling" />
    </multiply>
    <subtract name="N_sub_vector3" type="vector2">
      <input name="in1" type="vector2" nodename="N_mult_vector3" />
      <input name="in2" type="vector2" interfacename="uvoffset" />
    </subtract>
    <divide name="N_divtilesize_vector3" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_vector3" />
      <input name="in2" type="vector2" interfacename="realworldimagesize" />
    </divide>
    <multiply name="N_multtilesize_vector3" type="vector2">
      <input name="in1" type="vector2" nodename="N_divtilesize_vector3" />
      <input name="in2" type="vector2" interfacename="realworldtilesize" />
    </multiply>
    <image name="N_img_vector3" type="vector3">
      <input name="file" type="filename" interfacename="file" />
      <input name="default" type="vector3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_multtilesize_vector3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <output name="out" type="vector3" nodename="N_img_vector3" />
  </nodegraph>
  <nodegraph name="NG_tiledimage_vector4" nodedef="ND_tiledimage_vector4">
    <multiply name="N_mult_vector4" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="uvtiling" />
    </multiply>
    <subtract name="N_sub_vector4" type="vector2">
      <input name="in1" type="vector2" nodename="N_mult_vector4" />
      <input name="in2" type="vector2" interfacename="uvoffset" />
    </subtract>
    <divide name="N_divtilesize_vector4" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_vector4" />
      <input name="in2" type="vector2" interfacename="realworldimagesize" />
    </divide>
    <multiply name="N_multtilesize_vector4" type="vector2">
      <input name="in1" type="vector2" nodename="N_divtilesize_vector4" />
      <input name="in2" type="vector2" interfacename="realworldtilesize" />
    </multiply>
    <image name="N_img_vector4" type="vector4">
      <input name="file" type="filename" interfacename="file" />
      <input name="default" type="vector4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_multtilesize_vector4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <output name="out" type="vector4" nodename="N_img_vector4" />
  </nodegraph>
  <nodegraph name="NG_triplanarprojection_float" nodedef="ND_triplanarprojection_float">
    <extract name="N_extX_float" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_extY_float" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="1" />
    </extract>
    <extract name="N_extZ_float" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="2" />
    </extract>
    <combine2 name="N_vecYZ_float" type="vector2">
      <input name="in1" type="float" nodename="N_extY_float" />
      <input name="in2" type="float" nodename="N_extZ_float" />
    </combine2>
    <combine2 name="N_vecXZ_float" type="vector2">
      <input name="in1" type="float" nodename="N_extX_float" />
      <input name="in2" type="float" nodename="N_extZ_float" />
    </combine2>
    <combine2 name="N_vecXY_float" type="vector2">
      <input name="in1" type="float" nodename="N_extX_float" />
      <input name="in2" type="float" nodename="N_extY_float" />
    </combine2>
    <image name="N_imgX_float" type="float">
      <input name="file" type="filename" interfacename="filex" />
      <input name="layer" type="string" interfacename="layerx" />
      <input name="default" type="float" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecYZ_float" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgY_float" type="float">
      <input name="file" type="filename" interfacename="filey" />
      <input name="layer" type="string" interfacename="layery" />
      <input name="default" type="float" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXZ_float" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgZ_float" type="float">
      <input name="file" type="filename" interfacename="filez" />
      <input name="layer" type="string" interfacename="layerz" />
      <input name="default" type="float" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXY_float" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <normalize name="N_norm_float" type="vector3">
      <input name="in" type="vector3" interfacename="normal" />
    </normalize>
    <dotproduct name="N_dotX_float" type="float">
      <input name="in1" type="vector3" nodename="N_norm_float" />
      <input name="in2" type="vector3" value="1.0, 0.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotY_float" type="float">
      <input name="in1" type="vector3" nodename="N_norm_float" />
      <input name="in2" type="vector3" value="0.0, 1.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotZ_float" type="float">
      <input name="in1" type="vector3" nodename="N_norm_float" />
      <input name="in2" type="vector3" value="0.0, 0.0, 1.0" />
    </dotproduct>
    <absval name="N_blendX_float" type="float">
      <input name="in" type="float" nodename="N_dotX_float" />
    </absval>
    <absval name="N_blendY_float" type="float">
      <input name="in" type="float" nodename="N_dotY_float" />
    </absval>
    <absval name="N_blendZ_float" type="float">
      <input name="in" type="float" nodename="N_dotZ_float" />
    </absval>
    <multiply name="N_nX_float" type="float">
      <input name="in1" type="float" nodename="N_imgX_float" />
      <input name="in2" type="float" nodename="N_blendX_float" />
    </multiply>
    <multiply name="N_nY_float" type="float">
      <input name="in1" type="float" nodename="N_imgY_float" />
      <input name="in2" type="float" nodename="N_blendY_float" />
    </multiply>
    <multiply name="N_nZ_float" type="float">
      <input name="in1" type="float" nodename="N_imgZ_float" />
      <input name="in2" type="float" nodename="N_blendZ_float" />
    </multiply>
    <add name="N_add1_float" type="float">
      <input name="in1" type="float" nodename="N_nX_float" />
      <input name="in2" type="float" nodename="N_nY_float" />
    </add>
    <add name="N_add2_float" type="float">
      <input name="in1" type="float" nodename="N_add1_float" />
      <input name="in2" type="float" nodename="N_nZ_float" />
    </add>
    <output name="out" type="float" nodename="N_add2_float" />
  </nodegraph>
  <nodegraph name="NG_triplanarprojection_color3" nodedef="ND_triplanarprojection_color3">
    <extract name="N_extX_color3" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_extY_color3" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="1" />
    </extract>
    <extract name="N_extZ_color3" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="2" />
    </extract>
    <combine2 name="N_vecYZ_color3" type="vector2">
      <input name="in1" type="float" nodename="N_extY_color3" />
      <input name="in2" type="float" nodename="N_extZ_color3" />
    </combine2>
    <combine2 name="N_vecXZ_color3" type="vector2">
      <input name="in1" type="float" nodename="N_extX_color3" />
      <input name="in2" type="float" nodename="N_extZ_color3" />
    </combine2>
    <combine2 name="N_vecXY_color3" type="vector2">
      <input name="in1" type="float" nodename="N_extX_color3" />
      <input name="in2" type="float" nodename="N_extY_color3" />
    </combine2>
    <image name="N_imgX_color3" type="color3">
      <input name="file" type="filename" interfacename="filex" />
      <input name="layer" type="string" interfacename="layerx" />
      <input name="default" type="color3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecYZ_color3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgY_color3" type="color3">
      <input name="file" type="filename" interfacename="filey" />
      <input name="layer" type="string" interfacename="layery" />
      <input name="default" type="color3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXZ_color3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgZ_color3" type="color3">
      <input name="file" type="filename" interfacename="filez" />
      <input name="layer" type="string" interfacename="layerz" />
      <input name="default" type="color3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXY_color3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <normalize name="N_norm_color3" type="vector3">
      <input name="in" type="vector3" interfacename="normal" />
    </normalize>
    <dotproduct name="N_dotX_color3" type="float">
      <input name="in1" type="vector3" nodename="N_norm_color3" />
      <input name="in2" type="vector3" value="1.0, 0.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotY_color3" type="float">
      <input name="in1" type="vector3" nodename="N_norm_color3" />
      <input name="in2" type="vector3" value="0.0, 1.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotZ_color3" type="float">
      <input name="in1" type="vector3" nodename="N_norm_color3" />
      <input name="in2" type="vector3" value="0.0, 0.0, 1.0" />
    </dotproduct>
    <absval name="N_blendX_color3" type="float">
      <input name="in" type="float" nodename="N_dotX_color3" />
    </absval>
    <absval name="N_blendY_color3" type="float">
      <input name="in" type="float" nodename="N_dotY_color3" />
    </absval>
    <absval name="N_blendZ_color3" type="float">
      <input name="in" type="float" nodename="N_dotZ_color3" />
    </absval>
    <multiply name="N_nX_color3" type="color3">
      <input name="in1" type="color3" nodename="N_imgX_color3" />
      <input name="in2" type="float" nodename="N_blendX_color3" />
    </multiply>
    <multiply name="N_nY_color3" type="color3">
      <input name="in1" type="color3" nodename="N_imgY_color3" />
      <input name="in2" type="float" nodename="N_blendY_color3" />
    </multiply>
    <multiply name="N_nZ_color3" type="color3">
      <input name="in1" type="color3" nodename="N_imgZ_color3" />
      <input name="in2" type="float" nodename="N_blendZ_color3" />
    </multiply>
    <add name="N_add1_color3" type="color3">
      <input name="in1" type="color3" nodename="N_nX_color3" />
      <input name="in2" type="color3" nodename="N_nY_color3" />
    </add>
    <add name="N_add2_color3" type="color3">
      <input name="in1" type="color3" nodename="N_add1_color3" />
      <input name="in2" type="color3" nodename="N_nZ_color3" />
    </add>
    <output name="out" type="color3" nodename="N_add2_color3" />
  </nodegraph>
  <nodegraph name="NG_triplanarprojection_color4" nodedef="ND_triplanarprojection_color4">
    <extract name="N_extX_color4" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_extY_color4" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="1" />
    </extract>
    <extract name="N_extZ_color4" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="2" />
    </extract>
    <combine2 name="N_vecYZ_color4" type="vector2">
      <input name="in1" type="float" nodename="N_extY_color4" />
      <input name="in2" type="float" nodename="N_extZ_color4" />
    </combine2>
    <combine2 name="N_vecXZ_color4" type="vector2">
      <input name="in1" type="float" nodename="N_extX_color4" />
      <input name="in2" type="float" nodename="N_extZ_color4" />
    </combine2>
    <combine2 name="N_vecXY_color4" type="vector2">
      <input name="in1" type="float" nodename="N_extX_color4" />
      <input name="in2" type="float" nodename="N_extY_color4" />
    </combine2>
    <image name="N_imgX_color4" type="color4">
      <input name="file" type="filename" interfacename="filex" />
      <input name="layer" type="string" interfacename="layerx" />
      <input name="default" type="color4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecYZ_color4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgY_color4" type="color4">
      <input name="file" type="filename" interfacename="filey" />
      <input name="layer" type="string" interfacename="layery" />
      <input name="default" type="color4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXZ_color4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgZ_color4" type="color4">
      <input name="file" type="filename" interfacename="filez" />
      <input name="layer" type="string" interfacename="layerz" />
      <input name="default" type="color4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXY_color4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <normalize name="N_norm_color4" type="vector3">
      <input name="in" type="vector3" interfacename="normal" />
    </normalize>
    <dotproduct name="N_dotX_color4" type="float">
      <input name="in1" type="vector3" nodename="N_norm_color4" />
      <input name="in2" type="vector3" value="1.0, 0.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotY_color4" type="float">
      <input name="in1" type="vector3" nodename="N_norm_color4" />
      <input name="in2" type="vector3" value="0.0, 1.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotZ_color4" type="float">
      <input name="in1" type="vector3" nodename="N_norm_color4" />
      <input name="in2" type="vector3" value="0.0, 0.0, 1.0" />
    </dotproduct>
    <absval name="N_blendX_color4" type="float">
      <input name="in" type="float" nodename="N_dotX_color4" />
    </absval>
    <absval name="N_blendY_color4" type="float">
      <input name="in" type="float" nodename="N_dotY_color4" />
    </absval>
    <absval name="N_blendZ_color4" type="float">
      <input name="in" type="float" nodename="N_dotZ_color4" />
    </absval>
    <multiply name="N_nX_color4" type="color4">
      <input name="in1" type="color4" nodename="N_imgX_color4" />
      <input name="in2" type="float" nodename="N_blendX_color4" />
    </multiply>
    <multiply name="N_nY_color4" type="color4">
      <input name="in1" type="color4" nodename="N_imgY_color4" />
      <input name="in2" type="float" nodename="N_blendY_color4" />
    </multiply>
    <multiply name="N_nZ_color4" type="color4">
      <input name="in1" type="color4" nodename="N_imgZ_color4" />
      <input name="in2" type="float" nodename="N_blendZ_color4" />
    </multiply>
    <add name="N_add1_color4" type="color4">
      <input name="in1" type="color4" nodename="N_nX_color4" />
      <input name="in2" type="color4" nodename="N_nY_color4" />
    </add>
    <add name="N_add2_color4" type="color4">
      <input name="in1" type="color4" nodename="N_add1_color4" />
      <input name="in2" type="color4" nodename="N_nZ_color4" />
    </add>
    <output name="out" type="color4" nodename="N_add2_color4" />
  </nodegraph>
  <nodegraph name="NG_triplanarprojection_vector2" nodedef="ND_triplanarprojection_vector2">
    <extract name="N_extX_vector2" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_extY_vector2" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="1" />
    </extract>
    <extract name="N_extZ_vector2" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="2" />
    </extract>
    <combine2 name="N_vecYZ_vector2" type="vector2">
      <input name="in1" type="float" nodename="N_extY_vector2" />
      <input name="in2" type="float" nodename="N_extZ_vector2" />
    </combine2>
    <combine2 name="N_vecXZ_vector2" type="vector2">
      <input name="in1" type="float" nodename="N_extX_vector2" />
      <input name="in2" type="float" nodename="N_extZ_vector2" />
    </combine2>
    <combine2 name="N_vecXY_vector2" type="vector2">
      <input name="in1" type="float" nodename="N_extX_vector2" />
      <input name="in2" type="float" nodename="N_extY_vector2" />
    </combine2>
    <image name="N_imgX_vector2" type="vector2">
      <input name="file" type="filename" interfacename="filex" />
      <input name="layer" type="string" interfacename="layerx" />
      <input name="default" type="vector2" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecYZ_vector2" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgY_vector2" type="vector2">
      <input name="file" type="filename" interfacename="filey" />
      <input name="layer" type="string" interfacename="layery" />
      <input name="default" type="vector2" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXZ_vector2" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgZ_vector2" type="vector2">
      <input name="file" type="filename" interfacename="filez" />
      <input name="layer" type="string" interfacename="layerz" />
      <input name="default" type="vector2" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXY_vector2" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <normalize name="N_norm_vector2" type="vector3">
      <input name="in" type="vector3" interfacename="normal" />
    </normalize>
    <dotproduct name="N_dotX_vector2" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector2" />
      <input name="in2" type="vector3" value="1.0, 0.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotY_vector2" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector2" />
      <input name="in2" type="vector3" value="0.0, 1.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotZ_vector2" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector2" />
      <input name="in2" type="vector3" value="0.0, 0.0, 1.0" />
    </dotproduct>
    <absval name="N_blendX_vector2" type="float">
      <input name="in" type="float" nodename="N_dotX_vector2" />
    </absval>
    <absval name="N_blendY_vector2" type="float">
      <input name="in" type="float" nodename="N_dotY_vector2" />
    </absval>
    <absval name="N_blendZ_vector2" type="float">
      <input name="in" type="float" nodename="N_dotZ_vector2" />
    </absval>
    <multiply name="N_nX_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_imgX_vector2" />
      <input name="in2" type="float" nodename="N_blendX_vector2" />
    </multiply>
    <multiply name="N_nY_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_imgY_vector2" />
      <input name="in2" type="float" nodename="N_blendY_vector2" />
    </multiply>
    <multiply name="N_nZ_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_imgZ_vector2" />
      <input name="in2" type="float" nodename="N_blendZ_vector2" />
    </multiply>
    <add name="N_add1_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_nX_vector2" />
      <input name="in2" type="vector2" nodename="N_nY_vector2" />
    </add>
    <add name="N_add2_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_add1_vector2" />
      <input name="in2" type="vector2" nodename="N_nZ_vector2" />
    </add>
    <output name="out" type="vector2" nodename="N_add2_vector2" />
  </nodegraph>
  <nodegraph name="NG_triplanarprojection_vector3" nodedef="ND_triplanarprojection_vector3">
    <extract name="N_extX_vector3" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_extY_vector3" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="1" />
    </extract>
    <extract name="N_extZ_vector3" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="2" />
    </extract>
    <combine2 name="N_vecYZ_vector3" type="vector2">
      <input name="in1" type="float" nodename="N_extY_vector3" />
      <input name="in2" type="float" nodename="N_extZ_vector3" />
    </combine2>
    <combine2 name="N_vecXZ_vector3" type="vector2">
      <input name="in1" type="float" nodename="N_extX_vector3" />
      <input name="in2" type="float" nodename="N_extZ_vector3" />
    </combine2>
    <combine2 name="N_vecXY_vector3" type="vector2">
      <input name="in1" type="float" nodename="N_extX_vector3" />
      <input name="in2" type="float" nodename="N_extY_vector3" />
    </combine2>
    <image name="N_imgX_vector3" type="vector3">
      <input name="file" type="filename" interfacename="filex" />
      <input name="layer" type="string" interfacename="layerx" />
      <input name="default" type="vector3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecYZ_vector3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgY_vector3" type="vector3">
      <input name="file" type="filename" interfacename="filey" />
      <input name="layer" type="string" interfacename="layery" />
      <input name="default" type="vector3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXZ_vector3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgZ_vector3" type="vector3">
      <input name="file" type="filename" interfacename="filez" />
      <input name="layer" type="string" interfacename="layerz" />
      <input name="default" type="vector3" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXY_vector3" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <normalize name="N_norm_vector3" type="vector3">
      <input name="in" type="vector3" interfacename="normal" />
    </normalize>
    <dotproduct name="N_dotX_vector3" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector3" />
      <input name="in2" type="vector3" value="1.0, 0.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotY_vector3" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector3" />
      <input name="in2" type="vector3" value="0.0, 1.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotZ_vector3" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector3" />
      <input name="in2" type="vector3" value="0.0, 0.0, 1.0" />
    </dotproduct>
    <absval name="N_blendX_vector3" type="float">
      <input name="in" type="float" nodename="N_dotX_vector3" />
    </absval>
    <absval name="N_blendY_vector3" type="float">
      <input name="in" type="float" nodename="N_dotY_vector3" />
    </absval>
    <absval name="N_blendZ_vector3" type="float">
      <input name="in" type="float" nodename="N_dotZ_vector3" />
    </absval>
    <multiply name="N_nX_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_imgX_vector3" />
      <input name="in2" type="float" nodename="N_blendX_vector3" />
    </multiply>
    <multiply name="N_nY_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_imgY_vector3" />
      <input name="in2" type="float" nodename="N_blendY_vector3" />
    </multiply>
    <multiply name="N_nZ_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_imgZ_vector3" />
      <input name="in2" type="float" nodename="N_blendZ_vector3" />
    </multiply>
    <add name="N_add1_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_nX_vector3" />
      <input name="in2" type="vector3" nodename="N_nY_vector3" />
    </add>
    <add name="N_add2_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_add1_vector3" />
      <input name="in2" type="vector3" nodename="N_nZ_vector3" />
    </add>
    <output name="out" type="vector3" nodename="N_add2_vector3" />
  </nodegraph>
  <nodegraph name="NG_triplanarprojection_vector4" nodedef="ND_triplanarprojection_vector4">
    <extract name="N_extX_vector4" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_extY_vector4" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="1" />
    </extract>
    <extract name="N_extZ_vector4" type="float">
      <input name="in" type="vector3" interfacename="position" />
      <input name="index" type="integer" value="2" />
    </extract>
    <combine2 name="N_vecYZ_vector4" type="vector2">
      <input name="in1" type="float" nodename="N_extY_vector4" />
      <input name="in2" type="float" nodename="N_extZ_vector4" />
    </combine2>
    <combine2 name="N_vecXZ_vector4" type="vector2">
      <input name="in1" type="float" nodename="N_extX_vector4" />
      <input name="in2" type="float" nodename="N_extZ_vector4" />
    </combine2>
    <combine2 name="N_vecXY_vector4" type="vector2">
      <input name="in1" type="float" nodename="N_extX_vector4" />
      <input name="in2" type="float" nodename="N_extY_vector4" />
    </combine2>
    <image name="N_imgX_vector4" type="vector4">
      <input name="file" type="filename" interfacename="filex" />
      <input name="layer" type="string" interfacename="layerx" />
      <input name="default" type="vector4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecYZ_vector4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgY_vector4" type="vector4">
      <input name="file" type="filename" interfacename="filey" />
      <input name="layer" type="string" interfacename="layery" />
      <input name="default" type="vector4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXZ_vector4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <image name="N_imgZ_vector4" type="vector4">
      <input name="file" type="filename" interfacename="filez" />
      <input name="layer" type="string" interfacename="layerz" />
      <input name="default" type="vector4" interfacename="default" />
      <input name="texcoord" type="vector2" nodename="N_vecXY_vector4" />
      <input name="uaddressmode" type="string" value="periodic" />
      <input name="vaddressmode" type="string" value="periodic" />
      <input name="filtertype" type="string" interfacename="filtertype" />
      <input name="framerange" type="string" interfacename="framerange" />
      <input name="frameoffset" type="integer" interfacename="frameoffset" />
      <input name="frameendaction" type="string" interfacename="frameendaction" />
    </image>
    <normalize name="N_norm_vector4" type="vector3">
      <input name="in" type="vector3" interfacename="normal" />
    </normalize>
    <dotproduct name="N_dotX_vector4" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector4" />
      <input name="in2" type="vector3" value="1.0, 0.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotY_vector4" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector4" />
      <input name="in2" type="vector3" value="0.0, 1.0, 0.0" />
    </dotproduct>
    <dotproduct name="N_dotZ_vector4" type="float">
      <input name="in1" type="vector3" nodename="N_norm_vector4" />
      <input name="in2" type="vector3" value="0.0, 0.0, 1.0" />
    </dotproduct>
    <absval name="N_blendX_vector4" type="float">
      <input name="in" type="float" nodename="N_dotX_vector4" />
    </absval>
    <absval name="N_blendY_vector4" type="float">
      <input name="in" type="float" nodename="N_dotY_vector4" />
    </absval>
    <absval name="N_blendZ_vector4" type="float">
      <input name="in" type="float" nodename="N_dotZ_vector4" />
    </absval>
    <multiply name="N_nX_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_imgX_vector4" />
      <input name="in2" type="float" nodename="N_blendX_vector4" />
    </multiply>
    <multiply name="N_nY_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_imgY_vector4" />
      <input name="in2" type="float" nodename="N_blendY_vector4" />
    </multiply>
    <multiply name="N_nZ_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_imgZ_vector4" />
      <input name="in2" type="float" nodename="N_blendZ_vector4" />
    </multiply>
    <add name="N_add1_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_nX_vector4" />
      <input name="in2" type="vector4" nodename="N_nY_vector4" />
    </add>
    <add name="N_add2_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_add1_vector4" />
      <input name="in2" type="vector4" nodename="N_nZ_vector4" />
    </add>
    <output name="out" type="vector4" nodename="N_add2_vector4" />
  </nodegraph>
  <nodegraph name="NG_ramp4_float" nodedef="ND_ramp4_float">
    <clamp name="N_txclamp_float" type="vector2">
      <input name="in" type="vector2" interfacename="texcoord" />
    </clamp>
    <extract name="N_s_float" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_float" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_t_float" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_float" />
      <input name="index" type="integer" value="1" />
    </extract>
    <mix name="N_mixtop_float" type="float">
      <input name="bg" type="float" interfacename="valuetl" />
      <input name="fg" type="float" interfacename="valuetr" />
      <input name="mix" type="float" nodename="N_s_float" />
    </mix>
    <mix name="N_mixbot_float" type="float">
      <input name="bg" type="float" interfacename="valuebl" />
      <input name="fg" type="float" interfacename="valuebr" />
      <input name="mix" type="float" nodename="N_s_float" />
    </mix>
    <mix name="N_mix_float" type="float">
      <input name="bg" type="float" nodename="N_mixtop_float" />
      <input name="fg" type="float" nodename="N_mixbot_float" />
      <input name="mix" type="float" nodename="N_t_float" />
    </mix>
    <output name="out" type="float" nodename="N_mix_float" />
  </nodegraph>
  <nodegraph name="NG_ramp4_color3" nodedef="ND_ramp4_color3">
    <clamp name="N_txclamp_color3" type="vector2">
      <input name="in" type="vector2" interfacename="texcoord" />
    </clamp>
    <extract name="N_s_color3" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_color3" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_t_color3" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_color3" />
      <input name="index" type="integer" value="1" />
    </extract>
    <mix name="N_mixtop_color3" type="color3">
      <input name="bg" type="color3" interfacename="valuetl" />
      <input name="fg" type="color3" interfacename="valuetr" />
      <input name="mix" type="float" nodename="N_s_color3" />
    </mix>
    <mix name="N_mixbot_color3" type="color3">
      <input name="bg" type="color3" interfacename="valuebl" />
      <input name="fg" type="color3" interfacename="valuebr" />
      <input name="mix" type="float" nodename="N_s_color3" />
    </mix>
    <mix name="N_mix_color3" type="color3">
      <input name="bg" type="color3" nodename="N_mixtop_color3" />
      <input name="fg" type="color3" nodename="N_mixbot_color3" />
      <input name="mix" type="float" nodename="N_t_color3" />
    </mix>
    <output name="out" type="color3" nodename="N_mix_color3" />
  </nodegraph>
  <nodegraph name="NG_ramp4_color4" nodedef="ND_ramp4_color4">
    <clamp name="N_txclamp_color4" type="vector2">
      <input name="in" type="vector2" interfacename="texcoord" />
    </clamp>
    <extract name="N_s_color4" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_color4" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_t_color4" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_color4" />
      <input name="index" type="integer" value="1" />
    </extract>
    <mix name="N_mixtop_color4" type="color4">
      <input name="bg" type="color4" interfacename="valuetl" />
      <input name="fg" type="color4" interfacename="valuetr" />
      <input name="mix" type="float" nodename="N_s_color4" />
    </mix>
    <mix name="N_mixbot_color4" type="color4">
      <input name="bg" type="color4" interfacename="valuebl" />
      <input name="fg" type="color4" interfacename="valuebr" />
      <input name="mix" type="float" nodename="N_s_color4" />
    </mix>
    <mix name="N_mix_color4" type="color4">
      <input name="bg" type="color4" nodename="N_mixtop_color4" />
      <input name="fg" type="color4" nodename="N_mixbot_color4" />
      <input name="mix" type="float" nodename="N_t_color4" />
    </mix>
    <output name="out" type="color4" nodename="N_mix_color4" />
  </nodegraph>
  <nodegraph name="NG_ramp4_vector2" nodedef="ND_ramp4_vector2">
    <clamp name="N_txclamp_vector2" type="vector2">
      <input name="in" type="vector2" interfacename="texcoord" />
    </clamp>
    <extract name="N_s_vector2" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_vector2" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_t_vector2" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_vector2" />
      <input name="index" type="integer" value="1" />
    </extract>
    <mix name="N_mixtop_vector2" type="vector2">
      <input name="bg" type="vector2" interfacename="valuetl" />
      <input name="fg" type="vector2" interfacename="valuetr" />
      <input name="mix" type="float" nodename="N_s_vector2" />
    </mix>
    <mix name="N_mixbot_vector2" type="vector2">
      <input name="bg" type="vector2" interfacename="valuebl" />
      <input name="fg" type="vector2" interfacename="valuebr" />
      <input name="mix" type="float" nodename="N_s_vector2" />
    </mix>
    <mix name="N_mix_vector2" type="vector2">
      <input name="bg" type="vector2" nodename="N_mixtop_vector2" />
      <input name="fg" type="vector2" nodename="N_mixbot_vector2" />
      <input name="mix" type="float" nodename="N_t_vector2" />
    </mix>
    <output name="out" type="vector2" nodename="N_mix_vector2" />
  </nodegraph>
  <nodegraph name="NG_ramp4_vector3" nodedef="ND_ramp4_vector3">
    <clamp name="N_txclamp_vector3" type="vector2">
      <input name="in" type="vector2" interfacename="texcoord" />
    </clamp>
    <extract name="N_s_vector3" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_vector3" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_t_vector3" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_vector3" />
      <input name="index" type="integer" value="1" />
    </extract>
    <mix name="N_mixtop_vector3" type="vector3">
      <input name="bg" type="vector3" interfacename="valuetl" />
      <input name="fg" type="vector3" interfacename="valuetr" />
      <input name="mix" type="float" nodename="N_s_vector3" />
    </mix>
    <mix name="N_mixbot_vector3" type="vector3">
      <input name="bg" type="vector3" interfacename="valuebl" />
      <input name="fg" type="vector3" interfacename="valuebr" />
      <input name="mix" type="float" nodename="N_s_vector3" />
    </mix>
    <mix name="N_mix_vector3" type="vector3">
      <input name="bg" type="vector3" nodename="N_mixtop_vector3" />
      <input name="fg" type="vector3" nodename="N_mixbot_vector3" />
      <input name="mix" type="float" nodename="N_t_vector3" />
    </mix>
    <output name="out" type="vector3" nodename="N_mix_vector3" />
  </nodegraph>
  <nodegraph name="NG_ramp4_vector4" nodedef="ND_ramp4_vector4">
    <clamp name="N_txclamp_vector4" type="vector2">
      <input name="in" type="vector2" interfacename="texcoord" />
    </clamp>
    <extract name="N_s_vector4" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_vector4" />
      <input name="index" type="integer" value="0" />
    </extract>
    <extract name="N_t_vector4" type="float">
      <input name="in" type="vector2" nodename="N_txclamp_vector4" />
      <input name="index" type="integer" value="1" />
    </extract>
    <mix name="N_mixtop_vector4" type="vector4">
      <input name="bg" type="vector4" interfacename="valuetl" />
      <input name="fg" type="vector4" interfacename="valuetr" />
      <input name="mix" type="float" nodename="N_s_vector4" />
    </mix>
    <mix name="N_mixbot_vector4" type="vector4">
      <input name="bg" type="vector4" interfacename="valuebl" />
      <input name="fg" type="vector4" interfacename="valuebr" />
      <input name="mix" type="float" nodename="N_s_vector4" />
    </mix>
    <mix name="N_mix_vector4" type="vector4">
      <input name="bg" type="vector4" nodename="N_mixtop_vector4" />
      <input name="fg" type="vector4" nodename="N_mixbot_vector4" />
      <input name="mix" type="float" nodename="N_t_vector4" />
    </mix>
    <output name="out" type="vector4" nodename="N_mix_vector4" />
  </nodegraph>
  <nodegraph name="NG_unifiednoise2d_float" nodedef="ND_unifiednoise2d_float">
    <range name="N_range" type="float">
      <input name="in" type="float" nodename="N_switch_type" />
      <input name="outlow" type="float" interfacename="outmin" />
      <input name="outhigh" type="float" interfacename="outmax" />
      <input name="doclamp" type="boolean" interfacename="clampoutput" />
      <input name="inlow" type="float" value="0" />
      <input name="inhigh" type="float" value="1" />
    </range>
    <switch name="N_switch_type" type="float">
      <input name="in1" type="float" nodename="N_perlin_noise2d" />
      <input name="in2" type="float" nodename="N_cellnoise2d" />
      <input name="in3" type="float" nodename="N_worleynoise2d" />
      <input name="in4" type="float" nodename="N_fractal3d" />
      <input name="which" type="integer" interfacename="type" />
    </switch>
    <noise2d name="N_perlin_noise2d" type="float">
      <input name="texcoord" type="vector2" nodename="N_apply_cell_jitter" />
      <input name="amplitude" type="float" value="0.5" />
      <input name="pivot" type="float" value="0.5" />
    </noise2d>
    <cellnoise2d name="N_cellnoise2d" type="float">
      <input name="texcoord" type="vector2" nodename="N_apply_cell_jitter" />
    </cellnoise2d>
    <worleynoise2d name="N_worleynoise2d" type="float">
      <input name="texcoord" type="vector2" nodename="N_apply_offset" />
      <input name="jitter" type="float" interfacename="jitter" />
    </worleynoise2d>
    <fractal3d name="N_fractal3d" type="float">
      <input name="octaves" type="integer" interfacename="octaves" />
      <input name="lacunarity" type="float" interfacename="lacunarity" />
      <input name="diminish" type="float" interfacename="diminish" />
      <input name="position" type="vector3" nodename="N_combine_with_jitter" />
      <input name="amplitude" type="float" value="1" />
    </fractal3d>
    <rotate2d name="N_apply_cell_jitter" type="vector2">
      <input name="in" type="vector2" nodename="N_apply_offset" />
      <input name="amount" type="float" nodename="N_cell_jitter_mult" />
    </rotate2d>
    <add name="N_apply_offset" type="vector2">
      <input name="in1" type="vector2" nodename="N_apply_freq" />
      <input name="in2" type="vector2" interfacename="offset" />
    </add>
    <combine3 name="N_combine_with_jitter" type="vector3">
      <input name="in1" type="float" nodename="N_separate" output="outx" />
      <input name="in2" type="float" nodename="N_separate" output="outy" />
      <input name="in3" type="float" nodename="N_cell_jitter_mult" />
    </combine3>
    <multiply name="N_cell_jitter_mult" type="float">
      <input name="in1" type="float" nodename="N_jitter_minus_1" />
      <input name="in2" type="float" value="90000" />
    </multiply>
    <multiply name="N_apply_freq" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="freq" />
    </multiply>
    <separate2 name="N_separate" type="multioutput">
      <input name="in" type="vector2" nodename="N_apply_offset" />
    </separate2>
    <subtract name="N_jitter_minus_1" type="float">
      <input name="in1" type="float" interfacename="jitter" />
      <input name="in2" type="float" value="1" />
    </subtract>
    <output name="out" type="float" nodename="N_range" />
  </nodegraph>
  <nodegraph name="NG_unifiednoise3d_float" nodedef="ND_unifiednoise3d_float">
    <range name="N_range" type="float">
      <input name="in" type="float" nodename="N_switch_type" />
      <input name="outlow" type="float" interfacename="outmin" />
      <input name="outhigh" type="float" interfacename="outmax" />
      <input name="doclamp" type="boolean" interfacename="clampoutput" />
      <input name="inlow" type="float" value="0" />
      <input name="inhigh" type="float" value="1" />
    </range>
    <switch name="N_switch_type" type="float">
      <input name="in1" type="float" nodename="N_perlin_noise3d" />
      <input name="in2" type="float" nodename="N_cellnoise3d" />
      <input name="in3" type="float" nodename="N_worleynoise3d" />
      <input name="in4" type="float" nodename="N_fractal3d" />
      <input name="which" type="integer" interfacename="type" />
    </switch>
    <noise3d name="N_perlin_noise3d" type="float">
      <input name="position" type="vector3" nodename="N_apply_cell_jitter" />
      <input name="amplitude" type="float" value="0.5" />
      <input name="pivot" type="float" value="0.5" />
    </noise3d>
    <cellnoise3d name="N_cellnoise3d" type="float">
      <input name="position" type="vector3" nodename="N_apply_cell_jitter" />
    </cellnoise3d>
    <worleynoise3d name="N_worleynoise3d" type="float">
      <input name="position" type="vector3" nodename="N_apply_offset" />
      <input name="jitter" type="float" interfacename="jitter" />
    </worleynoise3d>
    <fractal3d name="N_fractal3d" type="float">
      <input name="octaves" type="integer" interfacename="octaves" />
      <input name="lacunarity" type="float" interfacename="lacunarity" />
      <input name="diminish" type="float" interfacename="diminish" />
      <input name="position" type="vector3" nodename="N_apply_cell_jitter" />
      <input name="amplitude" type="float" value="1" />
    </fractal3d>
    <rotate3d name="N_apply_cell_jitter" type="vector3">
      <input name="in" type="vector3" nodename="N_apply_offset" />
      <input name="amount" type="float" nodename="N_cell_jitter_mult" />
      <input name="axis" type="vector3" value="0.1, 1, 0" />
    </rotate3d>
    <add name="N_apply_offset" type="vector3">
      <input name="in1" type="vector3" nodename="N_apply_freq" />
      <input name="in2" type="vector3" interfacename="offset" />
    </add>
    <multiply name="N_cell_jitter_mult" type="float">
      <input name="in1" type="float" nodename="N_jitter_minus_one" />
      <input name="in2" type="float" value="90000" />
    </multiply>
    <multiply name="N_apply_freq" type="vector3">
      <input name="in1" type="vector3" interfacename="position" />
      <input name="in2" type="vector3" interfacename="freq" />
    </multiply>
    <subtract name="N_jitter_minus_one" type="float">
      <input name="in1" type="float" interfacename="jitter" />
      <input name="in2" type="float" value="1" />
    </subtract>
    <output name="out" type="float" nodename="N_range" />
  </nodegraph>
  <nodegraph name="NG_checkerboard_color3" nodedef="ND_checkerboard_color3">
    <separate2 name="N_mtlxseparate2" type="multioutput">
      <input name="in" type="vector2" nodename="N_mtlxplace2d1" />
    </separate2>
    <modulo name="N_mtlxmodulo1" type="float">
      <input name="in1" type="float" nodename="N_mtlxseparate2" output="outx" />
      <input name="in2" type="float" value="2" />
    </modulo>
    <multiply name="N_mult" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="freq" />
    </multiply>
    <modulo name="N_mtlxmodulo2" type="float">
      <input name="in1" type="float" nodename="N_mtlxseparate2" output="outy" />
      <input name="in2" type="float" value="2" />
    </modulo>
    <floor name="N_mtlxfloor2" type="float">
      <input name="in" type="float" nodename="N_mtlxmodulo1" />
    </floor>
    <floor name="N_mtlxfloor3" type="float">
      <input name="in" type="float" nodename="N_mtlxmodulo2" />
    </floor>
    <add name="N_mtlxadd2" type="float">
      <input name="in1" type="float" nodename="N_mtlxfloor2" />
      <input name="in2" type="float" nodename="N_mtlxfloor3" />
    </add>
    <ifequal name="N_mtlxifequal2" type="float">
      <input name="value1" type="float" nodename="N_mtlxadd2" />
      <input name="value2" type="float" value="1" />
      <input name="in1" type="float" value="1" />
      <input name="in2" type="float" value="0" />
    </ifequal>
    <place2d name="N_mtlxplace2d1" type="vector2">
      <input name="texcoord" type="vector2" nodename="N_mult" />
      <input name="offset" type="vector2" interfacename="offset" />
    </place2d>
    <mix name="N_mix_color3" type="color3">
      <input name="bg" type="color3" interfacename="color2" />
      <input name="fg" type="color3" interfacename="color1" />
      <input name="mix" type="float" nodename="N_mtlxifequal2" />
    </mix>
    <output name="out" type="color3" nodename="N_mix_color3" />
  </nodegraph>
  <nodegraph name="NG_bump_vector3" nodedef="ND_bump_vector3">
    <heighttonormal name="N_heighttonormal" type="vector3">
      <input name="in" type="float" interfacename="height" />
    </heighttonormal>
    <normalmap name="N_normalmap" type="vector3">
      <input name="in" type="vector3" nodename="N_heighttonormal" />
      <input name="normal" type="vector3" interfacename="normal" />
      <input name="scale" type="float" interfacename="scale" />
      <input name="tangent" type="vector3" interfacename="tangent" />
    </normalmap>
    <output name="out" type="vector3" nodename="N_normalmap" />
  </nodegraph>
  <nodegraph name="NG_place2d_vector2" nodedef="ND_place2d_vector2">
    <subtract name="N_subpivot" type="vector2">
      <input name="in1" type="vector2" interfacename="texcoord" />
      <input name="in2" type="vector2" interfacename="pivot" />
    </subtract>
    <divide name="N_applyscale" type="vector2">
      <input name="in1" type="vector2" nodename="N_subpivot" />
      <input name="in2" type="vector2" interfacename="scale" />
    </divide>
    <rotate2d name="N_applyrot" type="vector2">
      <input name="in" type="vector2" nodename="N_applyscale" />
      <input name="amount" type="float" interfacename="rotate" />
    </rotate2d>
    <subtract name="N_applyoffset" type="vector2">
      <input name="in1" type="vector2" nodename="N_applyrot" />
      <input name="in2" type="vector2" interfacename="offset" />
    </subtract>
    <add name="N_addpivot" type="vector2">
      <input name="in1" type="vector2" nodename="N_applyoffset" />
      <input name="in2" type="vector2" interfacename="pivot" />
    </add>
    <subtract name="N_applyoffset2" type="vector2">
      <input name="in1" type="vector2" nodename="N_subpivot" />
      <input name="in2" type="vector2" interfacename="offset" />
    </subtract>
    <rotate2d name="N_applyrot2" type="vector2">
      <input name="in" type="vector2" nodename="N_applyoffset2" />
      <input name="amount" type="float" interfacename="rotate" />
    </rotate2d>
    <divide name="N_applyscale2" type="vector2">
      <input name="in1" type="vector2" nodename="N_applyrot2" />
      <input name="in2" type="vector2" interfacename="scale" />
    </divide>
    <add name="N_addpivot2" type="vector2">
      <input name="in1" type="vector2" nodename="N_applyscale2" />
      <input name="in2" type="vector2" interfacename="pivot" />
    </add>
    <switch name="N_switch_operationorder" type="vector2">
      <input name="in1" type="vector2" nodename="N_addpivot" />
      <input name="in2" type="vector2" nodename="N_addpivot2" />
      <input name="which" type="integer" interfacename="operationorder" />
    </switch>
    <output name="out" type="vector2" nodename="N_switch_operationorder" />
  </nodegraph>
  <nodegraph name="NG_distance_vector2" nodedef="ND_distance_vector2">
    <subtract name="N_mtlxsubtract" type="vector2">
      <input name="in1" type="vector2" interfacename="in1" />
      <input name="in2" type="vector2" interfacename="in2" />
    </subtract>
    <magnitude name="N_mtlxmagnitude" type="float">
      <input name="in" type="vector2" nodename="N_mtlxsubtract" />
    </magnitude>
    <output name="out" type="float" nodename="N_mtlxmagnitude" />
  </nodegraph>
  <nodegraph name="NG_distance_vector3" nodedef="ND_distance_vector3">
    <subtract name="N_mtlxsubtract" type="vector3">
      <input name="in1" type="vector3" interfacename="in1" />
      <input name="in2" type="vector3" interfacename="in2" />
    </subtract>
    <magnitude name="N_mtlxmagnitude" type="float">
      <input name="in" type="vector3" nodename="N_mtlxsubtract" />
    </magnitude>
    <output name="out" type="float" nodename="N_mtlxmagnitude" />
  </nodegraph>
  <nodegraph name="NG_distance_vector4" nodedef="ND_distance_vector4">
    <subtract name="N_mtlxsubtract" type="vector4">
      <input name="in1" type="vector4" interfacename="in1" />
      <input name="in2" type="vector4" interfacename="in2" />
    </subtract>
    <magnitude name="N_mtlxmagnitude" type="float">
      <input name="in" type="vector4" nodename="N_mtlxsubtract" />
    </magnitude>
    <output name="out" type="float" nodename="N_mtlxmagnitude" />
  </nodegraph>
  <nodegraph name="NG_contrast_float" nodedef="ND_contrast_float">
    <subtract name="N_sub_float" type="float">
      <input name="in1" type="float" interfacename="in" />
      <input name="in2" type="float" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_float" type="float">
      <input name="in1" type="float" nodename="N_sub_float" />
      <input name="in2" type="float" interfacename="amount" />
    </multiply>
    <add name="N_add_float" type="float">
      <input name="in1" type="float" nodename="N_mul_float" />
      <input name="in2" type="float" interfacename="pivot" />
    </add>
    <output name="out" type="float" nodename="N_add_float" />
  </nodegraph>
  <nodegraph name="NG_contrast_color3" nodedef="ND_contrast_color3">
    <subtract name="N_sub_color3" type="color3">
      <input name="in1" type="color3" interfacename="in" />
      <input name="in2" type="color3" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_color3" type="color3">
      <input name="in1" type="color3" nodename="N_sub_color3" />
      <input name="in2" type="color3" interfacename="amount" />
    </multiply>
    <add name="N_add_color3" type="color3">
      <input name="in1" type="color3" nodename="N_mul_color3" />
      <input name="in2" type="color3" interfacename="pivot" />
    </add>
    <output name="out" type="color3" nodename="N_add_color3" />
  </nodegraph>
  <nodegraph name="NG_contrast_color4" nodedef="ND_contrast_color4">
    <subtract name="N_sub_color4" type="color4">
      <input name="in1" type="color4" interfacename="in" />
      <input name="in2" type="color4" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_color4" type="color4">
      <input name="in1" type="color4" nodename="N_sub_color4" />
      <input name="in2" type="color4" interfacename="amount" />
    </multiply>
    <add name="N_add_color4" type="color4">
      <input name="in1" type="color4" nodename="N_mul_color4" />
      <input name="in2" type="color4" interfacename="pivot" />
    </add>
    <output name="out" type="color4" nodename="N_add_color4" />
  </nodegraph>
  <nodegraph name="NG_contrast_vector2" nodedef="ND_contrast_vector2">
    <subtract name="N_sub_vector2" type="vector2">
      <input name="in1" type="vector2" interfacename="in" />
      <input name="in2" type="vector2" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_vector2" />
      <input name="in2" type="vector2" interfacename="amount" />
    </multiply>
    <add name="N_add_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_mul_vector2" />
      <input name="in2" type="vector2" interfacename="pivot" />
    </add>
    <output name="out" type="vector2" nodename="N_add_vector2" />
  </nodegraph>
  <nodegraph name="NG_contrast_vector3" nodedef="ND_contrast_vector3">
    <subtract name="N_sub_vector3" type="vector3">
      <input name="in1" type="vector3" interfacename="in" />
      <input name="in2" type="vector3" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_sub_vector3" />
      <input name="in2" type="vector3" interfacename="amount" />
    </multiply>
    <add name="N_add_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_mul_vector3" />
      <input name="in2" type="vector3" interfacename="pivot" />
    </add>
    <output name="out" type="vector3" nodename="N_add_vector3" />
  </nodegraph>
  <nodegraph name="NG_contrast_vector4" nodedef="ND_contrast_vector4">
    <subtract name="N_sub_vector4" type="vector4">
      <input name="in1" type="vector4" interfacename="in" />
      <input name="in2" type="vector4" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_sub_vector4" />
      <input name="in2" type="vector4" interfacename="amount" />
    </multiply>
    <add name="N_add_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_mul_vector4" />
      <input name="in2" type="vector4" interfacename="pivot" />
    </add>
    <output name="out" type="vector4" nodename="N_add_vector4" />
  </nodegraph>
  <nodegraph name="NG_contrast_color3FA" nodedef="ND_contrast_color3FA">
    <subtract name="N_sub_color3FA" type="color3">
      <input name="in1" type="color3" interfacename="in" />
      <input name="in2" type="float" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_color3FA" type="color3">
      <input name="in1" type="color3" nodename="N_sub_color3FA" />
      <input name="in2" type="float" interfacename="amount" />
    </multiply>
    <add name="N_add_color3FA" type="color3">
      <input name="in1" type="color3" nodename="N_mul_color3FA" />
      <input name="in2" type="float" interfacename="pivot" />
    </add>
    <output name="out" type="color3" nodename="N_add_color3FA" />
  </nodegraph>
  <nodegraph name="NG_contrast_color4FA" nodedef="ND_contrast_color4FA">
    <subtract name="N_sub_color4FA" type="color4">
      <input name="in1" type="color4" interfacename="in" />
      <input name="in2" type="float" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_color4FA" type="color4">
      <input name="in1" type="color4" nodename="N_sub_color4FA" />
      <input name="in2" type="float" interfacename="amount" />
    </multiply>
    <add name="N_add_color4FA" type="color4">
      <input name="in1" type="color4" nodename="N_mul_color4FA" />
      <input name="in2" type="float" interfacename="pivot" />
    </add>
    <output name="out" type="color4" nodename="N_add_color4FA" />
  </nodegraph>
  <nodegraph name="NG_contrast_vector2FA" nodedef="ND_contrast_vector2FA">
    <subtract name="N_sub_vector2FA" type="vector2">
      <input name="in1" type="vector2" interfacename="in" />
      <input name="in2" type="float" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_vector2FA" type="vector2">
      <input name="in1" type="vector2" nodename="N_sub_vector2FA" />
      <input name="in2" type="float" interfacename="amount" />
    </multiply>
    <add name="N_add_vector2FA" type="vector2">
      <input name="in1" type="vector2" nodename="N_mul_vector2FA" />
      <input name="in2" type="float" interfacename="pivot" />
    </add>
    <output name="out" type="vector2" nodename="N_add_vector2FA" />
  </nodegraph>
  <nodegraph name="NG_contrast_vector3FA" nodedef="ND_contrast_vector3FA">
    <subtract name="N_sub_vector3FA" type="vector3">
      <input name="in1" type="vector3" interfacename="in" />
      <input name="in2" type="float" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_vector3FA" type="vector3">
      <input name="in1" type="vector3" nodename="N_sub_vector3FA" />
      <input name="in2" type="float" interfacename="amount" />
    </multiply>
    <add name="N_add_vector3FA" type="vector3">
      <input name="in1" type="vector3" nodename="N_mul_vector3FA" />
      <input name="in2" type="float" interfacename="pivot" />
    </add>
    <output name="out" type="vector3" nodename="N_add_vector3FA" />
  </nodegraph>
  <nodegraph name="NG_contrast_vector4FA" nodedef="ND_contrast_vector4FA">
    <subtract name="N_sub_vector4FA" type="vector4">
      <input name="in1" type="vector4" interfacename="in" />
      <input name="in2" type="float" interfacename="pivot" />
    </subtract>
    <multiply name="N_mul_vector4FA" type="vector4">
      <input name="in1" type="vector4" nodename="N_sub_vector4FA" />
      <input name="in2" type="float" interfacename="amount" />
    </multiply>
    <add name="N_add_vector4FA" type="vector4">
      <input name="in1" type="vector4" nodename="N_mul_vector4FA" />
      <input name="in2" type="float" interfacename="pivot" />
    </add>
    <output name="out" type="vector4" nodename="N_add_vector4FA" />
  </nodegraph>
  <nodegraph name="NG_range_float" nodedef="ND_range_float">
    <remap name="N_remap1_float" type="float">
      <input name="in" type="float" interfacename="in" />
      <input name="inlow" type="float" interfacename="inlow" />
      <input name="inhigh" type="float" interfacename="inhigh" />
      <input name="outlow" type="float" value="0.0" />
      <input name="outhigh" type="float" value="1.0" />
    </remap>
    <divide name="N_recip_float" type="float">
      <input name="in1" type="float" value="1.0" />
      <input name="in2" type="float" interfacename="gamma" />
    </divide>
    <absval name="N_abs_float" type="float">
      <input name="in" type="float" nodename="N_remap1_float" />
    </absval>
    <power name="N_pow_float" type="float">
      <input name="in1" type="float" nodename="N_abs_float" />
      <input name="in2" type="float" nodename="N_recip_float" />
    </power>
    <sign name="N_sign_float" type="float">
      <input name="in" type="float" nodename="N_remap1_float" />
    </sign>
    <multiply name="N_gamma_float" type="float">
      <input name="in1" type="float" nodename="N_pow_float" />
      <input name="in2" type="float" nodename="N_sign_float" />
    </multiply>
    <remap name="N_remap2_float" type="float">
      <input name="in" type="float" nodename="N_gamma_float" />
      <input name="inlow" type="float" value="0.0" />
      <input name="inhigh" type="float" value="1.0" />
      <input name="outlow" type="float" interfacename="outlow" />
      <input name="outhigh" type="float" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_float" type="float">
      <input name="in" type="float" nodename="N_remap2_float" />
      <input name="low" type="float" interfacename="outlow" />
      <input name="high" type="float" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_float" type="float">
      <input name="in1" type="float" nodename="N_clamp_float" />
      <input name="in2" type="float" nodename="N_remap2_float" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="float" nodename="N_switch_float" />
  </nodegraph>
  <nodegraph name="NG_range_color3" nodedef="ND_range_color3">
    <remap name="N_remap1_color3" type="color3">
      <input name="in" type="color3" interfacename="in" />
      <input name="inlow" type="color3" interfacename="inlow" />
      <input name="inhigh" type="color3" interfacename="inhigh" />
      <input name="outlow" type="color3" value="0.0, 0.0, 0.0" />
      <input name="outhigh" type="color3" value="1.0, 1.0, 1.0" />
    </remap>
    <divide name="N_recip_color3" type="color3">
      <input name="in1" type="color3" value="1.0, 1.0, 1.0" />
      <input name="in2" type="color3" interfacename="gamma" />
    </divide>
    <absval name="N_abs_color3" type="color3">
      <input name="in" type="color3" nodename="N_remap1_color3" />
    </absval>
    <power name="N_pow_color3" type="color3">
      <input name="in1" type="color3" nodename="N_abs_color3" />
      <input name="in2" type="color3" nodename="N_recip_color3" />
    </power>
    <sign name="N_sign_color3" type="color3">
      <input name="in" type="color3" nodename="N_remap1_color3" />
    </sign>
    <multiply name="N_gamma_color3" type="color3">
      <input name="in1" type="color3" nodename="N_pow_color3" />
      <input name="in2" type="color3" nodename="N_sign_color3" />
    </multiply>
    <remap name="N_remap2_color3" type="color3">
      <input name="in" type="color3" nodename="N_gamma_color3" />
      <input name="inlow" type="color3" value="0.0, 0.0, 0.0" />
      <input name="inhigh" type="color3" value="1.0, 1.0, 1.0" />
      <input name="outlow" type="color3" interfacename="outlow" />
      <input name="outhigh" type="color3" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_color3" type="color3">
      <input name="in" type="color3" nodename="N_remap2_color3" />
      <input name="low" type="color3" interfacename="outlow" />
      <input name="high" type="color3" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_color3" type="color3">
      <input name="in1" type="color3" nodename="N_clamp_color3" />
      <input name="in2" type="color3" nodename="N_remap2_color3" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="color3" nodename="N_switch_color3" />
  </nodegraph>
  <nodegraph name="NG_range_color4" nodedef="ND_range_color4">
    <remap name="N_remap1_color4" type="color4">
      <input name="in" type="color4" interfacename="in" />
      <input name="inlow" type="color4" interfacename="inlow" />
      <input name="inhigh" type="color4" interfacename="inhigh" />
      <input name="outlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
      <input name="outhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    </remap>
    <divide name="N_recip_color4" type="color4">
      <input name="in1" type="color4" value="1.0, 1.0, 1.0, 1.0" />
      <input name="in2" type="color4" interfacename="gamma" />
    </divide>
    <absval name="N_abs_color4" type="color4">
      <input name="in" type="color4" nodename="N_remap1_color4" />
    </absval>
    <power name="N_pow_color4" type="color4">
      <input name="in1" type="color4" nodename="N_abs_color4" />
      <input name="in2" type="color4" nodename="N_recip_color4" />
    </power>
    <sign name="N_sign_color4" type="color4">
      <input name="in" type="color4" nodename="N_remap1_color4" />
    </sign>
    <multiply name="N_gamma_color4" type="color4">
      <input name="in1" type="color4" nodename="N_pow_color4" />
      <input name="in2" type="color4" nodename="N_sign_color4" />
    </multiply>
    <remap name="N_remap2_color4" type="color4">
      <input name="in" type="color4" nodename="N_gamma_color4" />
      <input name="inlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
      <input name="inhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
      <input name="outlow" type="color4" interfacename="outlow" />
      <input name="outhigh" type="color4" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_color4" type="color4">
      <input name="in" type="color4" nodename="N_remap2_color4" />
      <input name="low" type="color4" interfacename="outlow" />
      <input name="high" type="color4" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_color4" type="color4">
      <input name="in1" type="color4" nodename="N_clamp_color4" />
      <input name="in2" type="color4" nodename="N_remap2_color4" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="color4" nodename="N_switch_color4" />
  </nodegraph>
  <nodegraph name="NG_range_vector2" nodedef="ND_range_vector2">
    <remap name="N_remap1_vector2" type="vector2">
      <input name="in" type="vector2" interfacename="in" />
      <input name="inlow" type="vector2" interfacename="inlow" />
      <input name="inhigh" type="vector2" interfacename="inhigh" />
      <input name="outlow" type="vector2" value="0.0, 0.0" />
      <input name="outhigh" type="vector2" value="1.0, 1.0" />
    </remap>
    <divide name="N_recip_vector2" type="vector2">
      <input name="in1" type="vector2" value="1.0, 1.0" />
      <input name="in2" type="vector2" interfacename="gamma" />
    </divide>
    <absval name="N_abs_vector2" type="vector2">
      <input name="in" type="vector2" nodename="N_remap1_vector2" />
    </absval>
    <power name="N_pow_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_abs_vector2" />
      <input name="in2" type="vector2" nodename="N_recip_vector2" />
    </power>
    <sign name="N_sign_vector2" type="vector2">
      <input name="in" type="vector2" nodename="N_remap1_vector2" />
    </sign>
    <multiply name="N_gamma_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_pow_vector2" />
      <input name="in2" type="vector2" nodename="N_sign_vector2" />
    </multiply>
    <remap name="N_remap2_vector2" type="vector2">
      <input name="in" type="vector2" nodename="N_gamma_vector2" />
      <input name="inlow" type="vector2" value="0.0, 0.0" />
      <input name="inhigh" type="vector2" value="1.0, 1.0" />
      <input name="outlow" type="vector2" interfacename="outlow" />
      <input name="outhigh" type="vector2" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_vector2" type="vector2">
      <input name="in" type="vector2" nodename="N_remap2_vector2" />
      <input name="low" type="vector2" interfacename="outlow" />
      <input name="high" type="vector2" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_vector2" type="vector2">
      <input name="in1" type="vector2" nodename="N_clamp_vector2" />
      <input name="in2" type="vector2" nodename="N_remap2_vector2" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="vector2" nodename="N_switch_vector2" />
  </nodegraph>
  <nodegraph name="NG_range_vector3" nodedef="ND_range_vector3">
    <remap name="N_remap1_vector3" type="vector3">
      <input name="in" type="vector3" interfacename="in" />
      <input name="inlow" type="vector3" interfacename="inlow" />
      <input name="inhigh" type="vector3" interfacename="inhigh" />
      <input name="outlow" type="vector3" value="0.0, 0.0, 0.0" />
      <input name="outhigh" type="vector3" value="1.0, 1.0, 1.0" />
    </remap>
    <divide name="N_recip_vector3" type="vector3">
      <input name="in1" type="vector3" value="1.0, 1.0, 1.0" />
      <input name="in2" type="vector3" interfacename="gamma" />
    </divide>
    <absval name="N_abs_vector3" type="vector3">
      <input name="in" type="vector3" nodename="N_remap1_vector3" />
    </absval>
    <power name="N_pow_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_abs_vector3" />
      <input name="in2" type="vector3" nodename="N_recip_vector3" />
    </power>
    <sign name="N_sign_vector3" type="vector3">
      <input name="in" type="vector3" nodename="N_remap1_vector3" />
    </sign>
    <multiply name="N_gamma_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_pow_vector3" />
      <input name="in2" type="vector3" nodename="N_sign_vector3" />
    </multiply>
    <remap name="N_remap2_vector3" type="vector3">
      <input name="in" type="vector3" nodename="N_gamma_vector3" />
      <input name="inlow" type="vector3" value="0.0, 0.0, 0.0" />
      <input name="inhigh" type="vector3" value="1.0, 1.0, 1.0" />
      <input name="outlow" type="vector3" interfacename="outlow" />
      <input name="outhigh" type="vector3" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_vector3" type="vector3">
      <input name="in" type="vector3" nodename="N_remap2_vector3" />
      <input name="low" type="vector3" interfacename="outlow" />
      <input name="high" type="vector3" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_vector3" type="vector3">
      <input name="in1" type="vector3" nodename="N_clamp_vector3" />
      <input name="in2" type="vector3" nodename="N_remap2_vector3" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="vector3" nodename="N_switch_vector3" />
  </nodegraph>
  <nodegraph name="NG_range_vector4" nodedef="ND_range_vector4">
    <remap name="N_remap1_vector4" type="vector4">
      <input name="in" type="vector4" interfacename="in" />
      <input name="inlow" type="vector4" interfacename="inlow" />
      <input name="inhigh" type="vector4" interfacename="inhigh" />
      <input name="outlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
      <input name="outhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    </remap>
    <divide name="N_recip_vector4" type="vector4">
      <input name="in1" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
      <input name="in2" type="vector4" interfacename="gamma" />
    </divide>
    <absval name="N_abs_vector4" type="vector4">
      <input name="in" type="vector4" nodename="N_remap1_vector4" />
    </absval>
    <power name="N_pow_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_abs_vector4" />
      <input name="in2" type="vector4" nodename="N_recip_vector4" />
    </power>
    <sign name="N_sign_vector4" type="vector4">
      <input name="in" type="vector4" nodename="N_remap1_vector4" />
    </sign>
    <multiply name="N_gamma_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_pow_vector4" />
      <input name="in2" type="vector4" nodename="N_sign_vector4" />
    </multiply>
    <remap name="N_remap2_vector4" type="vector4">
      <input name="in" type="vector4" nodename="N_gamma_vector4" />
      <input name="inlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
      <input name="inhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
      <input name="outlow" type="vector4" interfacename="outlow" />
      <input name="outhigh" type="vector4" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_vector4" type="vector4">
      <input name="in" type="vector4" nodename="N_remap2_vector4" />
      <input name="low" type="vector4" interfacename="outlow" />
      <input name="high" type="vector4" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_vector4" type="vector4">
      <input name="in1" type="vector4" nodename="N_clamp_vector4" />
      <input name="in2" type="vector4" nodename="N_remap2_vector4" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="vector4" nodename="N_switch_vector4" />
  </nodegraph>
  <nodegraph name="NG_range_color3FA" nodedef="ND_range_color3FA">
    <remap name="N_remap1_color3FA" type="color3">
      <input name="in" type="color3" interfacename="in" />
      <input name="inlow" type="float" interfacename="inlow" />
      <input name="inhigh" type="float" interfacename="inhigh" />
      <input name="outlow" type="float" value="0.0" />
      <input name="outhigh" type="float" value="1.0" />
    </remap>
    <divide name="N_recip_color3FA" type="float">
      <input name="in1" type="float" value="1.0" />
      <input name="in2" type="float" interfacename="gamma" />
    </divide>
    <absval name="N_abs_color3FA" type="color3">
      <input name="in" type="color3" nodename="N_remap1_color3FA" />
    </absval>
    <power name="N_pow_color3FA" type="color3">
      <input name="in1" type="color3" nodename="N_abs_color3FA" />
      <input name="in2" type="float" nodename="N_recip_color3FA" />
    </power>
    <sign name="N_sign_color3FA" type="color3">
      <input name="in" type="color3" nodename="N_remap1_color3FA" />
    </sign>
    <multiply name="N_gamma_color3FA" type="color3">
      <input name="in1" type="color3" nodename="N_pow_color3FA" />
      <input name="in2" type="color3" nodename="N_sign_color3FA" />
    </multiply>
    <remap name="N_remap2_color3FA" type="color3">
      <input name="in" type="color3" nodename="N_gamma_color3FA" />
      <input name="inlow" type="float" value="0.0" />
      <input name="inhigh" type="float" value="1.0" />
      <input name="outlow" type="float" interfacename="outlow" />
      <input name="outhigh" type="float" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_color3FA" type="color3">
      <input name="in" type="color3" nodename="N_remap2_color3FA" />
      <input name="low" type="float" interfacename="outlow" />
      <input name="high" type="float" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_color3FA" type="color3">
      <input name="in1" type="color3" nodename="N_clamp_color3FA" />
      <input name="in2" type="color3" nodename="N_remap2_color3FA" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="color3" nodename="N_switch_color3FA" />
  </nodegraph>
  <nodegraph name="NG_range_color4FA" nodedef="ND_range_color4FA">
    <remap name="N_remap1_color4FA" type="color4">
      <input name="in" type="color4" interfacename="in" />
      <input name="inlow" type="float" interfacename="inlow" />
      <input name="inhigh" type="float" interfacename="inhigh" />
      <input name="outlow" type="float" value="0.0" />
      <input name="outhigh" type="float" value="1.0" />
    </remap>
    <divide name="N_recip_color4FA" type="float">
      <input name="in1" type="float" value="1.0" />
      <input name="in2" type="float" interfacename="gamma" />
    </divide>
    <absval name="N_abs_color4FA" type="color4">
      <input name="in" type="color4" nodename="N_remap1_color4FA" />
    </absval>
    <power name="N_pow_color4FA" type="color4">
      <input name="in1" type="color4" nodename="N_abs_color4FA" />
      <input name="in2" type="float" nodename="N_recip_color4FA" />
    </power>
    <sign name="N_sign_color4FA" type="color4">
      <input name="in" type="color4" nodename="N_remap1_color4FA" />
    </sign>
    <multiply name="N_gamma_color4FA" type="color4">
      <input name="in1" type="color4" nodename="N_pow_color4FA" />
      <input name="in2" type="color4" nodename="N_sign_color4FA" />
    </multiply>
    <remap name="N_remap2_color4FA" type="color4">
      <input name="in" type="color4" nodename="N_gamma_color4FA" />
      <input name="inlow" type="float" value="0.0" />
      <input name="inhigh" type="float" value="1.0" />
      <input name="outlow" type="float" interfacename="outlow" />
      <input name="outhigh" type="float" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_color4FA" type="color4">
      <input name="in" type="color4" nodename="N_remap2_color4FA" />
      <input name="low" type="float" interfacename="outlow" />
      <input name="high" type="float" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_color4FA" type="color4">
      <input name="in1" type="color4" nodename="N_clamp_color4FA" />
      <input name="in2" type="color4" nodename="N_remap2_color4FA" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="color4" nodename="N_switch_color4FA" />
  </nodegraph>
  <nodegraph name="NG_range_vector2FA" nodedef="ND_range_vector2FA">
    <remap name="N_remap1_vector2FA" type="vector2">
      <input name="in" type="vector2" interfacename="in" />
      <input name="inlow" type="float" interfacename="inlow" />
      <input name="inhigh" type="float" interfacename="inhigh" />
      <input name="outlow" type="float" value="0.0" />
      <input name="outhigh" type="float" value="1.0" />
    </remap>
    <divide name="N_recip_vector2FA" type="float">
      <input name="in1" type="float" value="1.0" />
      <input name="in2" type="float" interfacename="gamma" />
    </divide>
    <absval name="N_abs_vector2FA" type="vector2">
      <input name="in" type="vector2" nodename="N_remap1_vector2FA" />
    </absval>
    <power name="N_pow_vector2FA" type="vector2">
      <input name="in1" type="vector2" nodename="N_abs_vector2FA" />
      <input name="in2" type="float" nodename="N_recip_vector2FA" />
    </power>
    <sign name="N_sign_vector2FA" type="vector2">
      <input name="in" type="vector2" nodename="N_remap1_vector2FA" />
    </sign>
    <multiply name="N_gamma_vector2FA" type="vector2">
      <input name="in1" type="vector2" nodename="N_pow_vector2FA" />
      <input name="in2" type="vector2" nodename="N_sign_vector2FA" />
    </multiply>
    <remap name="N_remap2_vector2FA" type="vector2">
      <input name="in" type="vector2" nodename="N_gamma_vector2FA" />
      <input name="inlow" type="float" value="0.0" />
      <input name="inhigh" type="float" value="1.0" />
      <input name="outlow" type="float" interfacename="outlow" />
      <input name="outhigh" type="float" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_vector2FA" type="vector2">
      <input name="in" type="vector2" nodename="N_remap2_vector2FA" />
      <input name="low" type="float" interfacename="outlow" />
      <input name="high" type="float" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_vector2FA" type="vector2">
      <input name="in1" type="vector2" nodename="N_clamp_vector2FA" />
      <input name="in2" type="vector2" nodename="N_remap2_vector2FA" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="vector2" nodename="N_switch_vector2FA" />
  </nodegraph>
  <nodegraph name="NG_range_vector3FA" nodedef="ND_range_vector3FA">
    <remap name="N_remap1_vector3FA" type="vector3">
      <input name="in" type="vector3" interfacename="in" />
      <input name="inlow" type="float" interfacename="inlow" />
      <input name="inhigh" type="float" interfacename="inhigh" />
      <input name="outlow" type="float" value="0.0" />
      <input name="outhigh" type="float" value="1.0" />
    </remap>
    <divide name="N_recip_vector3FA" type="float">
      <input name="in1" type="float" value="1.0" />
      <input name="in2" type="float" interfacename="gamma" />
    </divide>
    <absval name="N_abs_vector3FA" type="vector3">
      <input name="in" type="vector3" nodename="N_remap1_vector3FA" />
    </absval>
    <power name="N_pow_vector3FA" type="vector3">
      <input name="in1" type="vector3" nodename="N_abs_vector3FA" />
      <input name="in2" type="float" nodename="N_recip_vector3FA" />
    </power>
    <sign name="N_sign_vector3FA" type="vector3">
      <input name="in" type="vector3" nodename="N_remap1_vector3FA" />
    </sign>
    <multiply name="N_gamma_vector3FA" type="vector3">
      <input name="in1" type="vector3" nodename="N_pow_vector3FA" />
      <input name="in2" type="vector3" nodename="N_sign_vector3FA" />
    </multiply>
    <remap name="N_remap2_vector3FA" type="vector3">
      <input name="in" type="vector3" nodename="N_gamma_vector3FA" />
      <input name="inlow" type="float" value="0.0" />
      <input name="inhigh" type="float" value="1.0" />
      <input name="outlow" type="float" interfacename="outlow" />
      <input name="outhigh" type="float" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_vector3FA" type="vector3">
      <input name="in" type="vector3" nodename="N_remap2_vector3FA" />
      <input name="low" type="float" interfacename="outlow" />
      <input name="high" type="float" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_vector3FA" type="vector3">
      <input name="in1" type="vector3" nodename="N_clamp_vector3FA" />
      <input name="in2" type="vector3" nodename="N_remap2_vector3FA" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="vector3" nodename="N_switch_vector3FA" />
  </nodegraph>
  <nodegraph name="NG_range_vector4FA" nodedef="ND_range_vector4FA">
    <remap name="N_remap1_vector4FA" type="vector4">
      <input name="in" type="vector4" interfacename="in" />
      <input name="inlow" type="float" interfacename="inlow" />
      <input name="inhigh" type="float" interfacename="inhigh" />
      <input name="outlow" type="float" value="0.0" />
      <input name="outhigh" type="float" value="1.0" />
    </remap>
    <divide name="N_recip_vector4FA" type="float">
      <input name="in1" type="float" value="1.0" />
      <input name="in2" type="float" interfacename="gamma" />
    </divide>
    <absval name="N_abs_vector4FA" type="vector4">
      <input name="in" type="vector4" nodename="N_remap1_vector4FA" />
    </absval>
    <power name="N_pow_vector4FA" type="vector4">
      <input name="in1" type="vector4" nodename="N_abs_vector4FA" />
      <input name="in2" type="float" nodename="N_recip_vector4FA" />
    </power>
    <sign name="N_sign_vector4FA" type="vector4">
      <input name="in" type="vector4" nodename="N_remap1_vector4FA" />
    </sign>
    <multiply name="N_gamma_vector4FA" type="vector4">
      <input name="in1" type="vector4" nodename="N_pow_vector4FA" />
      <input name="in2" type="vector4" nodename="N_sign_vector4FA" />
    </multiply>
    <remap name="N_remap2_vector4FA" type="vector4">
      <input name="in" type="vector4" nodename="N_gamma_vector4FA" />
      <input name="inlow" type="float" value="0.0" />
      <input name="inhigh" type="float" value="1.0" />
      <input name="outlow" type="float" interfacename="outlow" />
      <input name="outhigh" type="float" interfacename="outhigh" />
    </remap>
    <clamp name="N_clamp_vector4FA" type="vector4">
      <input name="in" type="vector4" nodename="N_remap2_vector4FA" />
      <input name="low" type="float" interfacename="outlow" />
      <input name="high" type="float" interfacename="outhigh" />
    </clamp>
    <ifequal name="N_switch_vector4FA" type="vector4">
      <input name="in1" type="vector4" nodename="N_clamp_vector4FA" />
      <input name="in2" type="vector4" nodename="N_remap2_vector4FA" />
      <input name="value1" type="boolean" interfacename="doclamp" />
      <input name="value2" type="boolean" value="true" />
    </ifequal>
    <output name="out" type="vector4" nodename="N_switch_vector4FA" />
  </nodegraph>
  <nodegraph name="NG_hsvadjust_color3" nodedef="ND_hsvadjust_color3">
    <rgbtohsv name="N_inhsv_color3" type="color3">
      <input name="in" type="color3" interfacename="in" />
    </rgbtohsv>
    <convert name="N_camount_color3" type="color3">
      <input name="in" type="vector3" interfacename="amount" />
    </convert>
    <multiply name="N_hchans_color3" type="color3">
      <input name="in1" type="color3" nodename="N_camount_color3" />
      <input name="in2" type="color3" value="1.0, 0.0, 0.0" />
    </multiply>
    <multiply name="N_tmp1_color3" type="color3">
      <input name="in1" type="color3" nodename="N_camount_color3" />
      <input name="in2" type="color3" value="0.0, 1.0, 1.0" />
    </multiply>
    <add name="N_svchans_color3" type="color3">
      <input name="in1" type="color3" nodename="N_tmp1_color3" />
      <input name="in2" type="color3" value="1.0, 0.0, 0.0" />
    </add>
    <add name="N_tmp2_color3" type="color3">
      <input name="in1" type="color3" nodename="N_inhsv_color3" />
      <input name="in2" type="color3" nodename="N_hchans_color3" />
    </add>
    <multiply name="N_tmp3_color3" type="color3">
      <input name="in1" type="color3" nodename="N_tmp2_color3" />
      <input name="in2" type="color3" nodename="N_svchans_color3" />
    </multiply>
    <hsvtorgb name="N_torgb_color3" type="color3">
      <input name="in" type="color3" nodename="N_tmp3_color3" />
    </hsvtorgb>
    <output name="out" type="color3" nodename="N_torgb_color3" />
  </nodegraph>
  <nodegraph name="NG_hsvadjust_color4" nodedef="ND_hsvadjust_color4">
    <rgbtohsv name="N_inhsv_color4" type="color4">
      <input name="in" type="color4" interfacename="in" />
    </rgbtohsv>
    <convert name="N_camt_color3" type="color3">
      <input name="in" type="vector3" interfacename="amount" />
    </convert>
    <convert name="N_camount_color4" type="color4">
      <input name="in" type="color3" nodename="N_camt_color3" />
    </convert>
    <multiply name="N_hchans_color4" type="color4">
      <input name="in1" type="color4" nodename="N_camount_color4" />
      <input name="in2" type="color4" value="1.0, 0.0, 0.0, 0.0" />
    </multiply>
    <multiply name="N_tmp1_color4" type="color4">
      <input name="in1" type="color4" nodename="N_camount_color4" />
      <input name="in2" type="color4" value="0.0, 1.0, 1.0, 0.0" />
    </multiply>
    <add name="N_svchans_color4" type="color4">
      <input name="in1" type="color4" nodename="N_tmp1_color4" />
      <input name="in2" type="color4" value="1.0, 0.0, 0.0, 1.0" />
    </add>
    <add name="N_tmp2_color4" type="color4">
      <input name="in1" type="color4" nodename="N_inhsv_color4" />
      <input name="in2" type="color4" nodename="N_hchans_color4" />
    </add>
    <multiply name="N_tmp3_color4" type="color4">
      <input name="in1" type="color4" nodename="N_tmp2_color4" />
      <input name="in2" type="color4" nodename="N_svchans_color4" />
    </multiply>
    <hsvtorgb name="N_torgb_color4" type="color4">
      <input name="in" type="color4" nodename="N_tmp3_color4" />
    </hsvtorgb>
    <output name="out" type="color4" nodename="N_torgb_color4" />
  </nodegraph>
  <nodegraph name="NG_saturate_color3" nodedef="ND_saturate_color3">
    <luminance name="N_gray_color3" type="color3">
      <input name="in" type="color3" interfacename="in" />
      <input name="lumacoeffs" type="color3" interfacename="lumacoeffs" />
    </luminance>
    <mix name="N_mix_color3" type="color3">
      <input name="bg" type="color3" nodename="N_gray_color3" />
      <input name="fg" type="color3" interfacename="in" />
      <input name="mix" type="float" interfacename="amount" />
    </mix>
    <output name="out" type="color3" nodename="N_mix_color3" />
  </nodegraph>
  <nodegraph name="NG_saturate_color4" nodedef="ND_saturate_color4">
    <luminance name="N_gray_color4" type="color4">
      <input name="in" type="color4" interfacename="in" />
      <input name="lumacoeffs" type="color3" interfacename="lumacoeffs" />
    </luminance>
    <mix name="N_mix_color4" type="color4">
      <input name="bg" type="color4" nodename="N_gray_color4" />
      <input name="fg" type="color4" interfacename="in" />
      <input name="mix" type="float" interfacename="amount" />
    </mix>
    <output name="out" type="color4" nodename="N_mix_color4" />
  </nodegraph>
  <nodegraph name="NG_colorcorrect_color3" nodedef="ND_colorcorrect_color3">
    <multiply name="N_exposure" type="color3">
      <input name="in1" type="color3" nodename="N_contrast" />
      <input name="in2" type="float" nodename="N_exposurepwr" />
    </multiply>
    <contrast name="N_contrast" type="color3">
      <input name="in" type="color3" nodename="N_gain" />
      <input name="amount" type="float" interfacename="contrast" />
      <input name="pivot" type="float" interfacename="contrastpivot" />
    </contrast>
    <power name="N_exposurepwr" type="float">
      <input name="in2" type="float" interfacename="exposure" />
      <input name="in1" type="float" value="2" />
    </power>
    <multiply name="N_gain" type="color3">
      <input name="in1" type="color3" nodename="N_liftadd" />
      <input name="in2" type="float" interfacename="gain" />
    </multiply>
    <add name="N_liftadd" type="color3">
      <input name="in1" type="color3" nodename="N_liftmult" />
      <input name="in2" type="float" interfacename="lift" />
    </add>
    <multiply name="N_liftmult" type="color3">
      <input name="in1" type="color3" nodename="N_gamma" />
      <input name="in2" type="float" nodename="N_liftsubtract" />
    </multiply>
    <range name="N_gamma" type="color3">
      <input name="in" type="color3" nodename="N_saturation" />
      <input name="gamma" type="float" interfacename="gamma" />
      <input name="inlow" type="float" value="0" />
      <input name="inhigh" type="float" value="1" />
      <input name="outlow" type="float" value="0" />
      <input name="outhigh" type="float" value="1" />
      <input name="doclamp" type="boolean" value="false" />
    </range>
    <subtract name="N_liftsubtract" type="float">
      <input name="in2" type="float" interfacename="lift" />
      <input name="in1" type="float" value="1" />
    </subtract>
    <saturate name="N_saturation" type="color3">
      <input name="in" type="color3" nodename="N_hsvadjust" />
      <input name="amount" type="float" interfacename="saturation" />
    </saturate>
    <hsvadjust name="N_hsvadjust" type="color3">
      <input name="in" type="color3" interfacename="in" />
      <input name="amount" type="vector3" nodename="N_parm2hue" />
    </hsvadjust>
    <combine3 name="N_parm2hue" type="vector3">
      <input name="in1" type="float" interfacename="hue" />
      <input name="in2" type="float" value="1" />
      <input name="in3" type="float" value="1" />
    </combine3>
    <output name="out" type="color3" nodename="N_exposure" />
  </nodegraph>
  <nodegraph name="NG_colorcorrect_color4" nodedef="ND_colorcorrect_color4">
    <separate4 name="N_split_color4" type="multioutput">
      <input name="in" type="color4" interfacename="in" />
    </separate4>
    <combine3 name="N_combine_color" type="color3">
      <input name="in1" type="float" nodename="N_split_color4" output="outr" />
      <input name="in2" type="float" nodename="N_split_color4" output="outg" />
      <input name="in3" type="float" nodename="N_split_color4" output="outb" />
    </combine3>
    <colorcorrect name="N_colorcorrect" type="color3">
      <input name="in" type="color3" nodename="N_combine_color" />
      <input name="hue" type="float" interfacename="hue" />
      <input name="saturation" type="float" interfacename="saturation" />
      <input name="gamma" type="float" interfacename="gamma" />
      <input name="lift" type="float" interfacename="lift" />
      <input name="gain" type="float" interfacename="gain" />
      <input name="contrast" type="float" interfacename="contrast" />
      <input name="contrastpivot" type="float" interfacename="contrastpivot" />
      <input name="exposure" type="float" interfacename="exposure" />
    </colorcorrect>
    <separate3 name="N_split_color" type="multioutput">
      <input name="in" type="color3" nodename="N_colorcorrect" />
    </separate3>
    <combine4 name="N_combine_with_alpha" type="color4">
      <input name="in1" type="float" nodename="N_split_color" output="outr" />
      <input name="in2" type="float" nodename="N_split_color" output="outg" />
      <input name="in3" type="float" nodename="N_split_color" output="outb" />
      <input name="in4" type="float" nodename="N_split_color4" output="outa" />
    </combine4>
    <output name="out" type="color4" nodename="N_combine_with_alpha" />
  </nodegraph>
  <nodegraph name="NG_convert_color3_surfaceshader" nodedef="ND_convert_color3_surfaceshader">
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" interfacename="in" />
    </surface_unlit>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_color4_surfaceshader" nodedef="ND_convert_color4_surfaceshader">
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="convert" />
      <input name="opacity" type="float" nodename="extract" />
    </surface_unlit>
    <extract name="extract" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="index" type="integer" uniform="true" value="3" />
    </extract>
    <convert name="convert" type="color3">
      <input name="in" type="color4" interfacename="in" />
    </convert>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_float_surfaceshader" nodedef="ND_convert_float_surfaceshader">
    <convert name="float_to_color3" type="color3">
      <input name="in" type="float" interfacename="in" />
    </convert>
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="float_to_color3" />
    </surface_unlit>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_vector2_surfaceshader" nodedef="ND_convert_vector2_surfaceshader">
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="vec3_to_color3" />
    </surface_unlit>
    <convert name="vec2_to_vec3" type="vector3">
      <input name="in" type="vector2" interfacename="in" />
    </convert>
    <convert name="vec3_to_color3" type="color3">
      <input name="in" type="vector3" nodename="vec2_to_vec3" />
    </convert>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_vector3_surfaceshader" nodedef="ND_convert_vector3_surfaceshader">
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="vec3_to_color3" />
    </surface_unlit>
    <convert name="vec3_to_color3" type="color3">
      <input name="in" type="vector3" interfacename="in" />
    </convert>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_vector4_surfaceshader" nodedef="ND_convert_vector4_surfaceshader">
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="color4_to_color3" />
      <input name="opacity" type="float" nodename="color4_to_float" />
    </surface_unlit>
    <convert name="vec4_to_color4" type="color4">
      <input name="in" type="vector4" interfacename="in" />
    </convert>
    <extract name="color4_to_float" type="float">
      <input name="in" type="color4" nodename="vec4_to_color4" />
      <input name="index" type="integer" uniform="true" value="3" />
    </extract>
    <convert name="color4_to_color3" type="color3">
      <input name="in" type="color4" nodename="vec4_to_color4" />
    </convert>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_integer_surfaceshader" nodedef="ND_convert_integer_surfaceshader">
    <convert name="int_to_float" type="float">
      <input name="in" type="integer" interfacename="in" />
    </convert>
    <convert name="float_to_color3" type="color3">
      <input name="in" type="float" nodename="int_to_float" />
    </convert>
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="float_to_color3" />
    </surface_unlit>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_convert_boolean_surfaceshader" nodedef="ND_convert_boolean_surfaceshader">
    <convert name="bool_to_float" type="float">
      <input name="in" type="boolean" interfacename="in" />
    </convert>
    <convert name="float_to_color3" type="color3">
      <input name="in" type="float" nodename="bool_to_float" />
    </convert>
    <surface_unlit name="surface" type="surfaceshader">
      <input name="emission_color" type="color3" nodename="float_to_color3" />
    </surface_unlit>
    <output name="out" type="surfaceshader" nodename="surface" />
  </nodegraph>
  <nodegraph name="NG_extract_vector2" nodedef="ND_extract_vector2">
    <swizzle name="N_x_vector2" type="float">
      <input name="in" type="vector2" interfacename="in" />
      <input name="channels" type="string" value="x" />
    </swizzle>
    <swizzle name="N_y_vector2" type="float">
      <input name="in" type="vector2" interfacename="in" />
      <input name="channels" type="string" value="y" />
    </swizzle>
    <switch name="N_sw_vector2" type="float">
      <input name="in1" type="float" nodename="N_x_vector2" />
      <input name="in2" type="float" nodename="N_y_vector2" />
      <input name="which" type="integer" interfacename="index" />
    </switch>
    <output name="out" type="float" nodename="N_sw_vector2" />
  </nodegraph>
  <nodegraph name="NG_extract_color3" nodedef="ND_extract_color3">
    <swizzle name="N_r_color3" type="float">
      <input name="in" type="color3" interfacename="in" />
      <input name="channels" type="string" value="r" />
    </swizzle>
    <swizzle name="N_g_color3" type="float">
      <input name="in" type="color3" interfacename="in" />
      <input name="channels" type="string" value="g" />
    </swizzle>
    <swizzle name="N_b_color3" type="float">
      <input name="in" type="color3" interfacename="in" />
      <input name="channels" type="string" value="b" />
    </swizzle>
    <switch name="N_sw_color3" type="float">
      <input name="in1" type="float" nodename="N_r_color3" />
      <input name="in2" type="float" nodename="N_g_color3" />
      <input name="in3" type="float" nodename="N_b_color3" />
      <input name="which" type="integer" interfacename="index" />
    </switch>
    <output name="out" type="float" nodename="N_sw_color3" />
  </nodegraph>
  <nodegraph name="NG_extract_vector3" nodedef="ND_extract_vector3">
    <swizzle name="N_x_vector3" type="float">
      <input name="in" type="vector3" interfacename="in" />
      <input name="channels" type="string" value="x" />
    </swizzle>
    <swizzle name="N_y_vector3" type="float">
      <input name="in" type="vector3" interfacename="in" />
      <input name="channels" type="string" value="y" />
    </swizzle>
    <swizzle name="N_z_vector3" type="float">
      <input name="in" type="vector3" interfacename="in" />
      <input name="channels" type="string" value="z" />
    </swizzle>
    <switch name="N_sw_vector3" type="float">
      <input name="in1" type="float" nodename="N_x_vector3" />
      <input name="in2" type="float" nodename="N_y_vector3" />
      <input name="in3" type="float" nodename="N_z_vector3" />
      <input name="which" type="integer" interfacename="index" />
    </switch>
    <output name="out" type="float" nodename="N_sw_vector3" />
  </nodegraph>
  <nodegraph name="NG_extract_color4" nodedef="ND_extract_color4">
    <swizzle name="N_r_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="r" />
    </swizzle>
    <swizzle name="N_g_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="g" />
    </swizzle>
    <swizzle name="N_b_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="b" />
    </swizzle>
    <swizzle name="N_a_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="a" />
    </swizzle>
    <switch name="N_sw_color4" type="float">
      <input name="in1" type="float" nodename="N_r_color4" />
      <input name="in2" type="float" nodename="N_g_color4" />
      <input name="in3" type="float" nodename="N_b_color4" />
      <input name="in4" type="float" nodename="N_a_color4" />
      <input name="which" type="integer" interfacename="index" />
    </switch>
    <output name="out" type="float" nodename="N_sw_color4" />
  </nodegraph>
  <nodegraph name="NG_extract_vector4" nodedef="ND_extract_vector4">
    <swizzle name="N_x_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="x" />
    </swizzle>
    <swizzle name="N_y_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="y" />
    </swizzle>
    <swizzle name="N_z_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="z" />
    </swizzle>
    <swizzle name="N_w_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="w" />
    </swizzle>
    <switch name="N_sw_vector4" type="float">
      <input name="in1" type="float" nodename="N_x_vector4" />
      <input name="in2" type="float" nodename="N_y_vector4" />
      <input name="in3" type="float" nodename="N_z_vector4" />
      <input name="in4" type="float" nodename="N_w_vector4" />
      <input name="which" type="integer" interfacename="index" />
    </switch>
    <output name="out" type="float" nodename="N_sw_vector4" />
  </nodegraph>
  <nodegraph name="NG_separate2_vector2" nodedef="ND_separate2_vector2">
    <swizzle name="N_x_vector2" type="float">
      <input name="in" type="vector2" interfacename="in" />
      <input name="channels" type="string" value="x" />
    </swizzle>
    <swizzle name="N_y_vector2" type="float">
      <input name="in" type="vector2" interfacename="in" />
      <input name="channels" type="string" value="y" />
    </swizzle>
    <output name="outx" type="float" nodename="N_x_vector2" />
    <output name="outy" type="float" nodename="N_y_vector2" />
  </nodegraph>
  <nodegraph name="NG_separate3_color3" nodedef="ND_separate3_color3">
    <swizzle name="N_r_color3" type="float">
      <input name="in" type="color3" interfacename="in" />
      <input name="channels" type="string" value="r" />
    </swizzle>
    <swizzle name="N_g_color3" type="float">
      <input name="in" type="color3" interfacename="in" />
      <input name="channels" type="string" value="g" />
    </swizzle>
    <swizzle name="N_b_color3" type="float">
      <input name="in" type="color3" interfacename="in" />
      <input name="channels" type="string" value="b" />
    </swizzle>
    <output name="outr" type="float" nodename="N_r_color3" />
    <output name="outg" type="float" nodename="N_g_color3" />
    <output name="outb" type="float" nodename="N_b_color3" />
  </nodegraph>
  <nodegraph name="NG_separate3_vector3" nodedef="ND_separate3_vector3">
    <swizzle name="N_x_vector3" type="float">
      <input name="in" type="vector3" interfacename="in" />
      <input name="channels" type="string" value="x" />
    </swizzle>
    <swizzle name="N_y_vector3" type="float">
      <input name="in" type="vector3" interfacename="in" />
      <input name="channels" type="string" value="y" />
    </swizzle>
    <swizzle name="N_z_vector3" type="float">
      <input name="in" type="vector3" interfacename="in" />
      <input name="channels" type="string" value="z" />
    </swizzle>
    <output name="outx" type="float" nodename="N_x_vector3" />
    <output name="outy" type="float" nodename="N_y_vector3" />
    <output name="outz" type="float" nodename="N_z_vector3" />
  </nodegraph>
  <nodegraph name="NG_separate4_color4" nodedef="ND_separate4_color4">
    <swizzle name="N_r_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="r" />
    </swizzle>
    <swizzle name="N_g_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="g" />
    </swizzle>
    <swizzle name="N_b_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="b" />
    </swizzle>
    <swizzle name="N_a_color4" type="float">
      <input name="in" type="color4" interfacename="in" />
      <input name="channels" type="string" value="a" />
    </swizzle>
    <output name="outr" type="float" nodename="N_r_color4" />
    <output name="outg" type="float" nodename="N_g_color4" />
    <output name="outb" type="float" nodename="N_b_color4" />
    <output name="outa" type="float" nodename="N_a_color4" />
  </nodegraph>
  <nodegraph name="NG_separate4_vector4" nodedef="ND_separate4_vector4">
    <swizzle name="N_x_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="x" />
    </swizzle>
    <swizzle name="N_y_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="y" />
    </swizzle>
    <swizzle name="N_z_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="z" />
    </swizzle>
    <swizzle name="N_w_vector4" type="float">
      <input name="in" type="vector4" interfacename="in" />
      <input name="channels" type="string" value="w" />
    </swizzle>
    <output name="outx" type="float" nodename="N_x_vector4" />
    <output name="outy" type="float" nodename="N_y_vector4" />
    <output name="outz" type="float" nodename="N_z_vector4" />
    <output name="outw" type="float" nodename="N_w_vector4" />
  </nodegraph>
  <!-- Functional graph implementation of custom marble: mymarble -->
  <nodegraph name="NG_mymarble" nodedef="ND_mymarble">
    <position name="obj_pos" type="vector3" />
    <dotproduct name="add_xyz" type="float">
      <input name="in1" type="vector3" nodename="obj_pos" />
      <input name="in2" type="vector3" value="1, 1, 1" />
    </dotproduct>
    <multiply name="scale_xyz" type="float">
      <input name="in1" type="float" nodename="add_xyz" />
      <input name="in2" type="float" interfacename="noise_scale_1" />
    </multiply>
    <multiply name="scale_pos" type="vector3">
      <input name="in1" type="vector3" nodename="obj_pos" />
      <input name="in2" type="float" interfacename="noise_scale_2" />
    </multiply>
    <fractal3d name="noise" type="float">
      <input name="octaves" type="integer" interfacename="noise_octaves" />
      <input name="position" type="vector3" nodename="scale_pos" />
    </fractal3d>
    <multiply name="scale_noise" type="float">
      <input name="in1" type="float" nodename="noise" />
      <input name="in2" type="float" value="3.0" />
    </multiply>
    <add name="sum" type="float">
      <input name="in1" type="float" nodename="scale_xyz" />
      <input name="in2" type="float" nodename="scale_noise" />
    </add>
    <sin name="sin" type="float">
      <input name="in" type="float" nodename="sum" />
    </sin>
    <multiply name="scale" type="float">
      <input name="in1" type="float" nodename="sin" />
      <input name="in2" type="float" value="0.5" />
    </multiply>
    <add name="bias" type="float">
      <input name="in1" type="float" nodename="scale" />
      <input name="in2" type="float" value="0.5" />
    </add>
    <power name="power" type="float">
      <input name="in1" type="float" nodename="bias" />
      <input name="in2" type="float" interfacename="noise_power" />
    </power>
    <mix name="color_mix" type="color3">
      <input name="bg" type="color3" interfacename="base_color_1" />
      <input name="fg" type="color3" interfacename="base_color_2" />
      <input name="mix" type="float" nodename="power" />
    </mix>
    <output name="out" type="color3" nodename="color_mix" />
  </nodegraph>
</materialx>
```
</details>


Functional graph added to: libraries\stdlib\stdlib_ng.mtlx
Definition added to: libraries\stdlib\stdlib_defs.mtlx


#### Example 2: Definition with Source Code Implementation 

In this example we take the previous custom `myadd` definition and add it's definition to the `stdlib` definition file and
add all of it's implementations to eh appropriate per target implementation files

In [35]:
def addDefinitionToDocument(definition, impls, defDoc, implDocs, defComment='', implComment=''):
    '''
    Copy a definition and implementations to a new document.
    
    '''
    if definition and impls and defDoc and implDocs:

        # Add definition comment
        if defComment:
            comment = defDoc.addChildOfCategory('comment')
            comment.setDocString(defComment)                 

        # Create a new definition, and copy the content over. Make sure
        # to use the existing name and category.
        newDef = defDoc.addNodeDef(definition.getName(), '', definition.getCategory())
        newDef.copyContentFrom(definition)        

        # Add implementations to appropriate implementation documents
        for impl, implDoc in zip(impls, implDocs):

            if not implDoc:
                continue

            # Add impl comment
            if implComment:
                comment = implDoc.addChildOfCategory('comment')
                comment.setDocString(implComment)                 

            # Create a new graph and copy the contents over. This will result in a functional graph.
            # Use the definiton document if no graph document specified
            newImpl = implDoc.addImplementation(impl.getName())
            newImpl.copyContentFrom(impl)

if inlined_impls:
    implDocs = []
    skipped_targets = []
    for implFilePath, target in zip(implFilePaths, targets):
        implFilePath = mx.FilePath(defaultLibFolder[0]) / implFilePath
        implDoc = mx.createDocument()
        try:
            mx.readFromXmlFile(implDoc, implFilePath, defaultSearchPath)
            implDocs.append(implDoc)
        except mx.ExceptionFileMissing as err:
            implDocs.append(implDoc)
            skipped_targets.append(target)
            print('No target (%s) impl file to append to %s' % (target, implFilePath.asString()))
        
    addDefinitionToDocument(inline_definition, inlined_impls, defDoc, implDocs, 'Custom add definition (mxadd)', 'Custom add implementation (mxadd)')

    # Examine the definition document
    documentContents = mx.writeToXmlString(defDoc, writeOptions)
    text = '<details><summary>Standard Libray Definitions with New Definition</summary>\n\n' + '```xml\n' + documentContents + '```\n' + '</details>\n' 
    display_markdown(text , raw=True) 

    findDef = defDoc.getNodeDef(inline_definition.getName())
    if findDef:
        print('Definition %s added to: %s' % (inline_definition.getName(), defFilePath.asString()))
        defDoc.removeChild(inline_definition.getName())
        findDef = defDoc.getNodeGraph(inline_definition.getName())

    # Examine the implementation documents
    for implDoc, inline_impl, target in zip(implDocs, inlined_impls, targets):
        if target in skipped_targets:
            continue

        documentContents = mx.writeToXmlString(implDoc, writeOptions)
        text = '<details><summary>Implementation for target ' + target + '</summary>\n\n' + '```xml\n' + documentContents + '```\n' + '</details>\n' 
        display_markdown(text , raw=True)  

        implName = inline_impl.getName()
        if implDoc.getImplementation(implName):
            print('- Source implementation added: %s' % implName)
            implDoc.removeChild(implName)

<details><summary>Standard Libray Definitions with New Definition</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <typedef name="boolean" />
  <typedef name="integer" />
  <typedef name="float" />
  <typedef name="color3" semantic="color" />
  <typedef name="color4" semantic="color" />
  <typedef name="vector2" />
  <typedef name="vector3" />
  <typedef name="vector4" />
  <typedef name="matrix33" />
  <typedef name="matrix44" />
  <typedef name="string" />
  <typedef name="filename" />
  <typedef name="geomname" />
  <typedef name="surfaceshader" semantic="shader" context="surface" />
  <typedef name="displacementshader" semantic="shader" context="displacement" />
  <typedef name="volumeshader" semantic="shader" context="volume" />
  <typedef name="lightshader" semantic="shader" context="light" />
  <typedef name="material" semantic="material" />
  <typedef name="none" />
  <typedef name="integerarray" />
  <typedef name="floatarray" />
  <typedef name="color3array" semantic="color" />
  <typedef name="color4array" semantic="color" />
  <typedef name="vector2array" />
  <typedef name="vector3array" />
  <typedef name="vector4array" />
  <typedef name="stringarray" />
  <typedef name="geomnamearray" />
  <unittypedef name="distance" />
  <unitdef name="UD_stdlib_distance" unittype="distance">
    <unit name="nanometer" scale="0.000000001" />
    <unit name="micron" scale="0.000001" />
    <unit name="millimeter" scale="0.001" />
    <unit name="centimeter" scale="0.01" />
    <unit name="inch" scale="0.0254" />
    <unit name="foot" scale="0.3048" />
    <unit name="yard" scale="0.9144" />
    <unit name="meter" scale="1.0" />
    <unit name="kilometer" scale="1000.0" />
    <unit name="mile" scale="1609.344" />
  </unitdef>
  <unittypedef name="angle" />
  <unitdef name="UD_stdlib_angle" unittype="angle">
    <unit name="degree" scale="1.0" />
    <unit name="radian" scale="57.295779513" />
  </unitdef>
  <geompropdef name="Pobject" type="vector3" geomprop="position" space="object" />
  <geompropdef name="Nobject" type="vector3" geomprop="normal" space="object" />
  <geompropdef name="Tobject" type="vector3" geomprop="tangent" space="object" index="0" />
  <geompropdef name="Bobject" type="vector3" geomprop="bitangent" space="object" index="0" />
  <geompropdef name="Pworld" type="vector3" geomprop="position" space="world" />
  <geompropdef name="Nworld" type="vector3" geomprop="normal" space="world" />
  <geompropdef name="Tworld" type="vector3" geomprop="tangent" space="world" index="0" />
  <geompropdef name="Bworld" type="vector3" geomprop="bitangent" space="world" index="0" />
  <geompropdef name="UV0" type="vector2" geomprop="texcoord" index="0" />
  <nodedef name="ND_surfacematerial" node="surfacematerial" nodegroup="material">
    <input name="surfaceshader" type="surfaceshader" value="" />
    <input name="displacementshader" type="displacementshader" value="" />
    <output name="out" type="material" />
  </nodedef>
  <nodedef name="ND_volumematerial" node="volumematerial" nodegroup="material">
    <input name="volumeshader" type="volumeshader" value="" />
    <output name="out" type="material" />
  </nodedef>
  <nodedef name="ND_surface_unlit" node="surface_unlit" nodegroup="shader" doc="Construct a surface shader from emission and transmission values.">
    <input name="emission" type="float" value="1.0" doc="Surface emission amount." />
    <input name="emission_color" type="color3" value="1,1,1" doc="Surface emission color." />
    <input name="transmission" type="float" value="0.0" doc="Surface transmission amount." />
    <input name="transmission_color" type="color3" value="1,1,1" doc="Surface transmission color." />
    <input name="opacity" type="float" value="1.0" doc="Surface cutout opacity." />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_image_float" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="float" value="0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_image_color3" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_color4" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_vector2" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_vector3" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_image_vector4" node="image" nodegroup="texture2d">
    <input name="file" type="filename" value="" uiname="Filename" uniform="true" />
    <input name="layer" type="string" value="" uiname="Layer" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Default Color" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" uiname="Texture Coordinates" />
    <input name="uaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode U" uniform="true" />
    <input name="vaddressmode" type="string" value="periodic" enum="constant,clamp,periodic,mirror" uiname="Address Mode V" uniform="true" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uiname="Filter Type" uniform="true" />
    <input name="framerange" type="string" value="" uiname="Frame Range" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uiname="Frame Offset" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uiname="Frame End Action" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_float" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_color3" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_color4" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_vector2" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_vector3" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tiledimage_vector4" node="tiledimage" nodegroup="texture2d">
    <input name="file" type="filename" value="" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="uvtiling" type="vector2" value="1.0, 1.0" />
    <input name="uvoffset" type="vector2" value="0.0, 0.0" />
    <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_float" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_color3" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_color4" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_vector2" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_vector3" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_triplanarprojection_vector4" node="triplanarprojection" nodegroup="texture3d">
    <input name="filex" type="filename" value="" uniform="true" />
    <input name="filey" type="filename" value="" uniform="true" />
    <input name="filez" type="filename" value="" uniform="true" />
    <input name="layerx" type="string" value="" uniform="true" />
    <input name="layery" type="string" value="" uniform="true" />
    <input name="layerz" type="string" value="" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="normal" type="vector3" defaultgeomprop="Nobject" />
    <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true" />
    <input name="framerange" type="string" value="" uniform="true" />
    <input name="frameoffset" type="integer" value="0" uniform="true" />
    <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_float" node="constant" nodegroup="procedural">
    <input name="value" type="float" value="0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_constant_color3" node="constant" nodegroup="procedural">
    <input name="value" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_color4" node="constant" nodegroup="procedural">
    <input name="value" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_vector2" node="constant" nodegroup="procedural">
    <input name="value" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_vector3" node="constant" nodegroup="procedural">
    <input name="value" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_vector4" node="constant" nodegroup="procedural">
    <input name="value" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_constant_boolean" node="constant" nodegroup="procedural">
    <input name="value" type="boolean" value="false" />
    <output name="out" type="boolean" default="false" />
  </nodedef>
  <nodedef name="ND_constant_integer" node="constant" nodegroup="procedural">
    <input name="value" type="integer" value="0" />
    <output name="out" type="integer" default="0" />
  </nodedef>
  <nodedef name="ND_constant_matrix33" node="constant" nodegroup="procedural">
    <input name="value" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" default="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
  </nodedef>
  <nodedef name="ND_constant_matrix44" node="constant" nodegroup="procedural">
    <input name="value" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" default="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
  </nodedef>
  <nodedef name="ND_constant_string" node="constant" nodegroup="procedural">
    <input name="value" type="string" value="" uniform="true" />
    <output name="out" type="string" default="" />
  </nodedef>
  <nodedef name="ND_constant_filename" node="constant" nodegroup="procedural">
    <input name="value" type="filename" value="" uniform="true" />
    <output name="out" type="filename" default="" />
  </nodedef>
  <nodedef name="ND_ramplr_float" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="float" value="0.0" />
    <input name="valuer" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_color3" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuer" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_color4" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuer" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_vector2" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="vector2" value="0.0, 0.0" />
    <input name="valuer" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_vector3" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuer" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramplr_vector4" node="ramplr" nodegroup="procedural2d">
    <input name="valuel" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuer" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_float" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="float" value="0.0" />
    <input name="valueb" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_color3" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valueb" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_color4" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valueb" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_vector2" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="vector2" value="0.0, 0.0" />
    <input name="valueb" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_vector3" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valueb" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramptb_vector4" node="ramptb" nodegroup="procedural2d">
    <input name="valuet" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valueb" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_float" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="float" value="0.0" />
    <input name="valuetr" type="float" value="0.0" />
    <input name="valuebl" type="float" value="0.0" />
    <input name="valuebr" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_color3" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuetr" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuebl" type="color3" value="0.0, 0.0, 0.0" />
    <input name="valuebr" type="color3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_color4" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuetr" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebl" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebr" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_vector2" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="vector2" value="0.0, 0.0" />
    <input name="valuetr" type="vector2" value="0.0, 0.0" />
    <input name="valuebl" type="vector2" value="0.0, 0.0" />
    <input name="valuebr" type="vector2" value="0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_vector3" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuetr" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuebl" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="valuebr" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_ramp4_vector4" node="ramp4" nodegroup="procedural2d">
    <input name="valuetl" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuetr" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebl" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="valuebr" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_float" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="float" value="0.0" uiname="Left" />
    <input name="valuer" type="float" value="0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_color3" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="color3" value="0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="color3" value="0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_color4" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_vector2" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="vector2" value="0.0, 0.0" uiname="Left" />
    <input name="valuer" type="vector2" value="0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_vector3" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="vector3" value="0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="vector3" value="0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splitlr_vector4" node="splitlr" nodegroup="procedural2d">
    <input name="valuel" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Left" />
    <input name="valuer" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Right" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_float" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="float" value="0.0" uiname="Top" />
    <input name="valueb" type="float" value="0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_splittb_color3" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="color3" value="0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="color3" value="0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_color4" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="color4" value="0.0, 0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_vector2" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="vector2" value="0.0, 0.0" uiname="Top" />
    <input name="valueb" type="vector2" value="0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_vector3" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="vector3" value="0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="vector3" value="0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_splittb_vector4" node="splittb" nodegroup="procedural2d">
    <input name="valuet" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Top" />
    <input name="valueb" type="vector4" value="0.0, 0.0, 0.0, 0.0" uiname="Bottom" />
    <input name="center" type="float" value="0.5" uiname="Center" uimin="0.0" uimax="1.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_float" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color3" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color4" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector2" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector2" value="1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector3" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector4" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color3FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_color4FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector2FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector3FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise2d_vector4FA" node="noise2d" nodegroup="procedural2d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_float" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color3" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color4" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector2" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector2" value="1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector3" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector4" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color3FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_color4FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector2FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector3FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_noise3d_vector4FA" node="noise3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.0" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_float" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color3" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color4" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector2" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector2" value="1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector3" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector4" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color3FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_color4FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector2FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector3FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_fractal3d_vector4FA" node="fractal3d" nodegroup="procedural3d">
    <input name="amplitude" type="float" value="1.0" />
    <input name="octaves" type="integer" value="3" />
    <input name="lacunarity" type="float" value="2.0" />
    <input name="diminish" type="float" value="0.5" />
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_cellnoise2d_float" node="cellnoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_cellnoise3d_float" node="cellnoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise2d_float" node="worleynoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise2d_vector2" node="worleynoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise2d_vector3" node="worleynoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" defaultgeomprop="UV0" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise3d_float" node="worleynoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise3d_vector2" node="worleynoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_worleynoise3d_vector3" node="worleynoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" defaultgeomprop="Pobject" />
    <input name="jitter" type="float" value="1.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_unifiednoise2d_float" node="unifiednoise2d" nodegroup="procedural2d">
    <input name="texcoord" type="vector2" uifolder="Common" defaultgeomprop="UV0" doc="The input 2d space. Default is the first texture coordinates." />
    <input name="freq" type="vector2" uiname="Frequency" uifolder="Common" value="1, 1" doc="Adjusts the noise frequency, with higher values producing smaller noise shapes. Default is (1,1)." />
    <input name="offset" type="vector2" uiname="Offset" uifolder="Common" value="0, 0" doc="Shift the noise in 2d space. Default is (0,0)." />
    <input name="jitter" type="float" uiname="Jitter" uifolder="Common" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjust uniformity of Worley noise; for other noise types jitters the results." />
    <input name="outmin" type="float" uiname="Output Min" uifolder="Post Process" value="0" doc="The lowest values fit to the noise. Default is 0.0." />
    <input name="outmax" type="float" uiname="Output Max" uifolder="Post Process" value="1" doc="The highest values fit to the noise. Default is 1.0." />
    <input name="clampoutput" type="boolean" uiname="Clamp Output" uifolder="Post Process" value="true" doc="Clamp the output to the min and max output values." />
    <input name="octaves" type="integer" uiname="Octaves" uifolder="Fractal" value="3" doc="The number of octaves of Fractal noise to be generated. Default is 3." />
    <input name="lacunarity" type="float" uiname="Lacunarity" uifolder="Fractal" value="2" doc="The exponential scale between successive octaves of Fractal noise. Default is 2.0." />
    <input name="diminish" type="float" uiname="Diminish" uifolder="Fractal" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="The rate at which noise amplitude is diminished for each octave of Fractal noise. Default is 0.5." />
    <input name="type" type="integer" uiname="Noise Type" uifolder="Common" uisoftmin="0" uisoftmax="3" value="0" enum="Perlin,Cell,Worley,Fractal" enumvalues="0,1,2,3" doc="Menu to select the type of noise: Perlin, Cell, Worley, or Fractal. Default is Perlin." />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_unifiednoise3d_float" node="unifiednoise3d" nodegroup="procedural3d">
    <input name="position" type="vector3" uifolder="Common" defaultgeomprop="Pobject" doc="The input 3d space. Default is position in object-space." />
    <input name="freq" type="vector3" uiname="Frequency" uifolder="Common" value="1, 1, 1" doc="Adjusts the noise frequency, with higher values producing smaller noise shapes. Default is (1,1,1)." />
    <input name="offset" type="vector3" uiname="Offset" uifolder="Common" value="0, 0, 0" doc="Shift the noise in 3d space. Default is (0,0,0)." />
    <input name="jitter" type="float" uiname="Jitter" uifolder="Common" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjust uniformity of Worley noise; for other noise types jitters the results." />
    <input name="outmin" type="float" uiname="Output Min" uifolder="Post Process" value="0" doc="The lowest values fit to the noise. Default is 0.0." />
    <input name="outmax" type="float" uiname="Output Max" uifolder="Post Process" value="1" doc="The highest values fit to the noise. Default is 1.0." />
    <input name="clampoutput" type="boolean" uiname="Clamp Output" uifolder="Post Process" value="true" doc="Clamp the output to the min and max output values." />
    <input name="octaves" type="integer" uiname="Octaves" uifolder="Fractal" value="3" doc="The number of octaves of Fractal noise to be generated. Default is 3." />
    <input name="lacunarity" type="float" uiname="Lacunarity" uifolder="Fractal" value="2" doc="The exponential scale between successive octaves of Fractal noise. Default is 2.0." />
    <input name="diminish" type="float" uiname="Diminish" uifolder="Fractal" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="The rate at which noise amplitude is diminished for each octave of Fractal noise. Default is 0.5." />
    <input name="type" type="integer" uiname="Noise Type" uifolder="Common" uisoftmin="0" uisoftmax="3" value="0" enum="Perlin,Cell,Worley,Fractal" enumvalues="0,1,2,3" doc="Menu to select the type of noise: Perlin, Cell, Worley, or Fractal. Default is Perlin." />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_checkerboard_color3" node="checkerboard" nodegroup="procedural2d">
    <input name="color1" type="color3" uiname="Color 1" value="1.0, 1.0, 1.0" doc="The first color used in the checkerboard pattern." />
    <input name="color2" type="color3" uiname="Color 2" value="0.0, 0.0, 0.0" doc="The second color used in the checkerboard pattern." />
    <input name="freq" type="vector2" uiname="Frequency" value="8, 8" doc="The frequency of checkers, with higher values producing smaller squares. Default is (8, 8)." />
    <input name="offset" type="vector2" uiname="Offset" value="0, 0" doc="Shift the pattern in 2d space. Default is (0, 0)." />
    <input name="texcoord" type="vector2" uiname="Texture Coordinates" defaultgeomprop="UV0" doc="The input 2d space. Default is the first texture coordinates." />
    <output name="out" type="color3" />
  </nodedef>
  <nodedef name="ND_position_vector3" node="position" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_normal_vector3" node="normal" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <output name="out" type="vector3" default="0.0, 1.0, 0.0" />
  </nodedef>
  <nodedef name="ND_tangent_vector3" node="tangent" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector3" default="1.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_bitangent_vector3" node="bitangent" nodegroup="geometric">
    <input name="space" type="string" value="object" enum="model,object,world" uniform="true" />
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 1.0" />
  </nodedef>
  <nodedef name="ND_texcoord_vector2" node="texcoord" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_texcoord_vector3" node="texcoord" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geomcolor_float" node="geomcolor" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_geomcolor_color3" node="geomcolor" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geomcolor_color4" node="geomcolor" nodegroup="geometric">
    <input name="index" type="integer" value="0" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_integer" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="integer" value="0" />
    <output name="out" type="integer" default="0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_boolean" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="boolean" value="false" />
    <output name="out" type="boolean" default="false" />
  </nodedef>
  <nodedef name="ND_geompropvalue_string" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="string" value="" uniform="true" />
    <output name="out" type="string" default="" />
  </nodedef>
  <nodedef name="ND_geompropvalue_float" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="float" value="0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_color3" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_color4" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_vector2" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_vector3" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_geompropvalue_vector4" node="geompropvalue" nodegroup="geometric">
    <input name="geomprop" type="string" value="" uniform="true" />
    <input name="default" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_bump_vector3" node="bump" nodegroup="geometric">
    <input name="height" type="float" uiname="Height" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Amount to offset the surface normal." />
    <input name="scale" type="float" uiname="Scale" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Scalar to adjust the height amount." />
    <input name="normal" type="vector3" uiname="Normal" defaultgeomprop="Nworld" doc="Surface normal; defaults to the current world-space normal." />
    <input name="tangent" type="vector3" uiname="Tangent" defaultgeomprop="Tworld" doc="Surface tangent vector, defaults to the current world-space tangent vector." />
    <output name="out" type="vector3" doc="Offset surface normal; connect this to a shader's 'normal' input." />
  </nodedef>
  <nodedef name="ND_ambientocclusion_float" node="ambientocclusion" nodegroup="global">
    <input name="coneangle" type="float" value="90.0" unittype="angle" unit="degree" />
    <input name="maxdistance" type="float" value="1e38" />
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_frame_float" node="frame" nodegroup="application">
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_time_float" node="time" nodegroup="application">
    <input name="fps" type="float" value="24.0" />
    <output name="out" type="float" default="0.041666667" />
  </nodedef>
  <nodedef name="ND_add_float" node="add" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color3" node="add" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color4" node="add" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector2" node="add" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector3" node="add" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector4" node="add" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix33" node="add" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix44" node="add" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color3FA" node="add" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_color4FA" node="add" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector2FA" node="add" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector3FA" node="add" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_vector4FA" node="add" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix33FA" node="add" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_add_matrix44FA" node="add" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_float" node="subtract" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color3" node="subtract" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color4" node="subtract" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector2" node="subtract" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector3" node="subtract" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector4" node="subtract" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix33" node="subtract" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix44" node="subtract" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color3FA" node="subtract" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_color4FA" node="subtract" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector2FA" node="subtract" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector3FA" node="subtract" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_vector4FA" node="subtract" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix33FA" node="subtract" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_subtract_matrix44FA" node="subtract" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_float" node="multiply" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color3" node="multiply" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color4" node="multiply" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector2" node="multiply" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector3" node="multiply" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector4" node="multiply" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_matrix33" node="multiply" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_matrix44" node="multiply" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color3FA" node="multiply" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_color4FA" node="multiply" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector2FA" node="multiply" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector3FA" node="multiply" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_multiply_vector4FA" node="multiply" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_float" node="divide" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color3" node="divide" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color4" node="divide" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector2" node="divide" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector3" node="divide" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector4" node="divide" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_matrix33" node="divide" nodegroup="math">
    <input name="in1" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="in2" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_matrix44" node="divide" nodegroup="math">
    <input name="in1" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="in2" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color3FA" node="divide" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_color4FA" node="divide" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector2FA" node="divide" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector3FA" node="divide" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_divide_vector4FA" node="divide" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_float" node="modulo" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color3" node="modulo" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color4" node="modulo" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector2" node="modulo" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector3" node="modulo" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector4" node="modulo" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color3FA" node="modulo" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_color4FA" node="modulo" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector2FA" node="modulo" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector3FA" node="modulo" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_modulo_vector4FA" node="modulo" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_invert_float" node="invert" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color3" node="invert" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color4" node="invert" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector2" node="invert" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector3" node="invert" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector4" node="invert" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color3FA" node="invert" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_color4FA" node="invert" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector2FA" node="invert" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector3FA" node="invert" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invert_vector4FA" node="invert" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_float" node="absval" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_color3" node="absval" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_color4" node="absval" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_vector2" node="absval" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_vector3" node="absval" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_absval_vector4" node="absval" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_float" node="floor" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_color3" node="floor" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_color4" node="floor" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_vector2" node="floor" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_vector3" node="floor" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_vector4" node="floor" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_floor_integer" node="floor" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="integer" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_float" node="ceil" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_color3" node="ceil" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_color4" node="ceil" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_vector2" node="ceil" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_vector3" node="ceil" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_vector4" node="ceil" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ceil_integer" node="ceil" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="integer" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_power_float" node="power" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color3" node="power" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color4" node="power" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector2" node="power" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector3" node="power" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector4" node="power" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color3FA" node="power" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_color4FA" node="power" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector2FA" node="power" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector3FA" node="power" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_power_vector4FA" node="power" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_float" node="sin" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_float" node="cos" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_float" node="tan" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_float" node="asin" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_float" node="acos" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_float" node="atan2" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_vector2" node="sin" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_vector2" node="cos" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_vector2" node="tan" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_vector2" node="asin" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_vector2" node="acos" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_vector2" node="atan2" nodegroup="math">
    <input name="in1" type="vector2" value="1.0, 1.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_vector3" node="sin" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_vector3" node="cos" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_vector3" node="tan" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_vector3" node="asin" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_vector3" node="acos" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_vector3" node="atan2" nodegroup="math">
    <input name="in1" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sin_vector4" node="sin" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_cos_vector4" node="cos" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_tan_vector4" node="tan" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_asin_vector4" node="asin" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_acos_vector4" node="acos" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_atan2_vector4" node="atan2" nodegroup="math">
    <input name="in1" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_sqrt_float" node="sqrt" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_float" node="ln" nodegroup="math">
    <input name="in" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_float" node="exp" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sqrt_vector2" node="sqrt" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_vector2" node="ln" nodegroup="math">
    <input name="in" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_vector2" node="exp" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sqrt_vector3" node="sqrt" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_vector3" node="ln" nodegroup="math">
    <input name="in" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_vector3" node="exp" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sqrt_vector4" node="sqrt" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_ln_vector4" node="ln" nodegroup="math">
    <input name="in" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_exp_vector4" node="exp" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_float" node="sign" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_color3" node="sign" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_color4" node="sign" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_vector2" node="sign" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_vector3" node="sign" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_sign_vector4" node="sign" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_float" node="clamp" nodegroup="math">
    <input name="in" type="float" value="0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color3" node="clamp" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="color3" value="0.0, 0.0, 0.0" />
    <input name="high" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color4" node="clamp" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector2" node="clamp" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="vector2" value="0.0, 0.0" />
    <input name="high" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector3" node="clamp" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="high" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector4" node="clamp" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color3FA" node="clamp" nodegroup="math">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_color4FA" node="clamp" nodegroup="math">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector2FA" node="clamp" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector3FA" node="clamp" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_clamp_vector4FA" node="clamp" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_min_float" node="min" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color3" node="min" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color4" node="min" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector2" node="min" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector3" node="min" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector4" node="min" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color3FA" node="min" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_color4FA" node="min" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector2FA" node="min" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector3FA" node="min" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_min_vector4FA" node="min" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_float" node="max" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color3" node="max" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color4" node="max" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector2" node="max" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector3" node="max" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector4" node="max" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color3FA" node="max" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_color4FA" node="max" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector2FA" node="max" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector3FA" node="max" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_max_vector4FA" node="max" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_normalize_vector2" node="normalize" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_normalize_vector3" node="normalize" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_normalize_vector4" node="normalize" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_magnitude_vector2" node="magnitude" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_magnitude_vector3" node="magnitude" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_magnitude_vector4" node="magnitude" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_distance_vector2" node="distance" nodegroup="math">
    <input name="in1" type="vector2" uiname="in1" value="0.0, 0.0" />
    <input name="in2" type="vector2" uiname="in2" value="0.0, 0.0" />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_distance_vector3" node="distance" nodegroup="math">
    <input name="in1" type="vector3" uiname="in1" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" uiname="in2" value="0.0, 0.0, 0.0" />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_distance_vector4" node="distance" nodegroup="math">
    <input name="in1" type="vector4" uiname="in1" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" uiname="in2" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="float" />
  </nodedef>
  <nodedef name="ND_dotproduct_vector2" node="dotproduct" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_dotproduct_vector3" node="dotproduct" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_dotproduct_vector4" node="dotproduct" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_crossproduct_vector3" node="crossproduct" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_transformpoint_vector3" node="transformpoint" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="fromspace" type="string" value="" uniform="true" />
    <input name="tospace" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformvector_vector3" node="transformvector" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="fromspace" type="string" value="" uniform="true" />
    <input name="tospace" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformnormal_vector3" node="transformnormal" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 1.0" />
    <input name="fromspace" type="string" value="" uniform="true" />
    <input name="tospace" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector2M3" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="mat" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector3" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mat" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector3M4" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mat" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transformmatrix_vector4" node="transformmatrix" nodegroup="math">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mat" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_normalmap" node="normalmap" nodegroup="math">
    <input name="in" type="vector3" value="0.5, 0.5, 1.0" />
    <input name="space" type="string" value="tangent" enum="tangent, object" uniform="true" />
    <input name="scale" type="float" value="1.0" />
    <input name="normal" type="vector3" defaultgeomprop="Nworld" />
    <input name="tangent" type="vector3" defaultgeomprop="Tworld" />
    <output name="out" type="vector3" defaultinput="normal" />
  </nodedef>
  <nodedef name="ND_transpose_matrix33" node="transpose" nodegroup="math">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_transpose_matrix44" node="transpose" nodegroup="math">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_determinant_matrix33" node="determinant" nodegroup="math">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_determinant_matrix44" node="determinant" nodegroup="math">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="float" default="1.0" />
  </nodedef>
  <nodedef name="ND_invertmatrix_matrix33" node="invertmatrix" nodegroup="math">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <output name="out" type="matrix33" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_invertmatrix_matrix44" node="invertmatrix" nodegroup="math">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <output name="out" type="matrix44" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rotate2d_vector2" node="rotate2d" nodegroup="math">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="float" value="0.0" unittype="angle" unit="degree" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rotate3d_vector3" node="rotate3d" nodegroup="math">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="0.0" unittype="angle" unit="degree" />
    <input name="axis" type="vector3" value="0.0, 1.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_place2d_vector2" node="place2d" nodegroup="math">
    <input name="texcoord" type="vector2" value="0.0, 0.0" />
    <input name="pivot" type="vector2" value="0.0,0.0" />
    <input name="scale" type="vector2" value="1.0,1.0" />
    <input name="rotate" type="float" value="0.0" unittype="angle" unit="degree" />
    <input name="offset" type="vector2" value="0.0,0.0" />
    <input name="operationorder" type="integer" value="0" enum="SRT, TRS" enumvalues="0, 1" />
    <output name="out" type="vector2" defaultinput="texcoord" />
  </nodedef>
  <nodedef name="ND_arrayappend_integer_integerarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="integer" value="0" />
    <input name="in2" type="integer" value="0" />
    <output name="out" type="integerarray" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_integerarray_integerarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="integerarray" value="" />
    <input name="in2" type="integer" value="0" />
    <output name="out" type="integerarray" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_float_floatarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="floatarray" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_floatarray_floatarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="floatarray" value="" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="floatarray" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_color3_color3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_color3array_color3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color3array" value="" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_color4_color4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_color4array_color4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="color4array" value="" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector2_vector2array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector2array_vector2array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector2array" value="" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector3_vector3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector3array_vector3array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector3array" value="" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector4_vector4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4array" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_vector4array_vector4array" node="arrayappend" nodegroup="math">
    <input name="in1" type="vector4array" value="" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4array" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_arrayappend_string_stringarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="string" value="" />
    <input name="in2" type="string" value="" />
    <output name="out" type="stringarray" default="[]" />
  </nodedef>
  <nodedef name="ND_arrayappend_stringarray_stringarray" node="arrayappend" nodegroup="math">
    <input name="in1" type="stringarray" value="" />
    <input name="in2" type="string" value="" />
    <output name="out" type="stringarray" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_remap_float" node="remap" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color3" node="remap" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="color3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color4" node="remap" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector2" node="remap" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="vector2" value="0.0, 0.0" />
    <input name="inhigh" type="vector2" value="1.0, 1.0" />
    <input name="outlow" type="vector2" value="0.0, 0.0" />
    <input name="outhigh" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector3" node="remap" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector4" node="remap" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color3FA" node="remap" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_color4FA" node="remap" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector2FA" node="remap" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector3FA" node="remap" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_remap_vector4FA" node="remap" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_float" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color3" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="color3" value="0.0, 0.0, 0.0" />
    <input name="high" type="color3" value="1.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color4" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector2" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="vector2" value="0.0, 0.0" />
    <input name="high" type="vector2" value="1.0, 1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector3" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="high" type="vector3" value="1.0, 1.0, 1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector4" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="high" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color3FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_color4FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector2FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector3FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_smoothstep_vector4FA" node="smoothstep" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="low" type="float" value="0.0" />
    <input name="high" type="float" value="1.0" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_float" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_color3" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_color4" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_vector2" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_vector3" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_curveadjust_vector4" node="curveadjust" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="knots" type="vector2array" value="" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_luminance_color3" node="luminance" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_luminance_color4" node="luminance" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rgbtohsv_color3" node="rgbtohsv" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_rgbtohsv_color4" node="rgbtohsv" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvtorgb_color3" node="hsvtorgb" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvtorgb_color4" node="hsvtorgb" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_float" node="contrast" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color3" node="contrast" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="color3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="color3" value="0.5, 0.5, 0.5" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color4" node="contrast" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="color4" value="0.5, 0.5, 0.5, 0.5" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector2" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="vector2" value="1.0, 1.0" />
    <input name="pivot" type="vector2" value="0.5, 0.5" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector3" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="pivot" type="vector3" value="0.5, 0.5, 0.5" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector4" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="pivot" type="vector4" value="0.5, 0.5, 0.5, 0.5" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color3FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_color4FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector2FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector3FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_contrast_vector4FA" node="contrast" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="pivot" type="float" value="0.5" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_float" node="range" nodegroup="adjustment">
    <input name="in" type="float" value="0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color3" node="range" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="color3" value="1.0, 1.0, 1.0" />
    <input name="gamma" type="color3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="color3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="color3" value="1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color4" node="range" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="gamma" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="color4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector2" node="range" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="vector2" value="0.0, 0.0" />
    <input name="inhigh" type="vector2" value="1.0, 1.0" />
    <input name="gamma" type="vector2" value="1.0, 1.0" />
    <input name="outlow" type="vector2" value="0.0, 0.0" />
    <input name="outhigh" type="vector2" value="1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector3" node="range" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="gamma" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="outlow" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector3" value="1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector4" node="range" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="gamma" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="outlow" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="outhigh" type="vector4" value="1.0, 1.0, 1.0, 1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color3FA" node="range" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_color4FA" node="range" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector2FA" node="range" nodegroup="adjustment">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector3FA" node="range" nodegroup="adjustment">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_range_vector4FA" node="range" nodegroup="adjustment">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="inlow" type="float" value="0.0" />
    <input name="inhigh" type="float" value="1.0" />
    <input name="gamma" type="float" value="1.0" />
    <input name="outlow" type="float" value="0.0" />
    <input name="outhigh" type="float" value="1.0" />
    <input name="doclamp" type="boolean" value="false" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvadjust_color3" node="hsvadjust" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="0.0, 1.0, 1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_hsvadjust_color4" node="hsvadjust" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="vector3" value="0.0, 1.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_saturate_color3" node="saturate" nodegroup="adjustment">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_saturate_color4" node="saturate" nodegroup="adjustment">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="amount" type="float" value="1.0" />
    <input name="lumacoeffs" type="color3" value="0.2722287, 0.6740818, 0.0536895" enum="acescg, rec709, rec2020, rec2100" enumvalues="0.2722287,0.6740818,0.0536895, 0.2126,0.7152,0.0722, 0.2627,0.6780,0.0593, 0.2627,0.6780,0.0593" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_colorcorrect_color3" node="colorcorrect" nodegroup="adjustment">
    <input name="in" type="color3" uiname="Input Color" value="1, 1, 1" doc="The input color to be adjusted." />
    <input name="hue" type="float" uiname="Hue" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Rotates the color hue, with values wrapping at 0-1 boundaries." />
    <input name="saturation" type="float" uiname="Saturation" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjusts the input color saturation level." />
    <input name="gamma" type="float" uiname="Gamma" uisoftmin="0.0" uisoftmax="3.0" value="1" doc="Applies a gamma correction to the color." />
    <input name="lift" type="float" uiname="Lift" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Raise the dark color values, leaving the white values unchanged." />
    <input name="gain" type="float" uiname="Gain" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Multiplier increases lighter color values, leaving black values unchanged." />
    <input name="contrast" type="float" uiname="Contrast" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Linearly increase or decrease the color contrast." />
    <input name="contrastpivot" type="float" uiname="Contrast Pivot" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="Pivot value around which contrast applies. This value will not change as contrast is adjusted." />
    <input name="exposure" type="float" uiname="Exposure" uisoftmin="-1.0" uisoftmax="1.0" value="0" doc="Multplier which increases or decreases color brightness by 2^value." />
    <output name="out" type="color3" />
  </nodedef>
  <nodedef name="ND_colorcorrect_color4" node="colorcorrect" nodegroup="adjustment">
    <input name="in" type="color4" uiname="Input Color" value="1, 1, 1, 0" doc="The input color to be adjusted." />
    <input name="hue" type="float" uiname="Hue" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Rotates the color hue, with values wrapping at 0-1 boundaries." />
    <input name="saturation" type="float" uiname="Saturation" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Adjusts the input color saturation level." />
    <input name="gamma" type="float" uiname="Gamma" uisoftmin="0.0" uisoftmax="3.0" value="1" doc="Applies a gamma correction to the color." />
    <input name="lift" type="float" uiname="Lift" uisoftmin="0.0" uisoftmax="1.0" value="0" doc="Raise the dark color values, leaving the white values unchanged." />
    <input name="gain" type="float" uiname="Gain" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Multiplier increases lighter color values, leaving black values unchanged." />
    <input name="contrast" type="float" uiname="Contrast" uisoftmin="0.0" uisoftmax="1.0" value="1" doc="Linearly increase or decrease the color contrast." />
    <input name="contrastpivot" type="float" uiname="Contrast Pivot" uisoftmin="0.0" uisoftmax="1.0" value="0.5" doc="Pivot value around which contrast applies. This value will not change as contrast is adjusted." />
    <input name="exposure" type="float" uiname="Exposure" uisoftmin="-1.0" uisoftmax="1.0" value="0" doc="Multplier which increases or decreases color brightness by 2^value." />
    <output name="out" type="color4" />
  </nodedef>
  <nodedef name="ND_premult_color4" node="premult" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_unpremult_color4" node="unpremult" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_plus_float" node="plus" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_plus_color3" node="plus" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_plus_color4" node="plus" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_minus_float" node="minus" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_minus_color3" node="minus" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_minus_color4" node="minus" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_difference_float" node="difference" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_difference_color3" node="difference" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_difference_color4" node="difference" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_burn_float" node="burn" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_burn_color3" node="burn" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_burn_color4" node="burn" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_dodge_float" node="dodge" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_dodge_color3" node="dodge" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_dodge_color4" node="dodge" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_screen_float" node="screen" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_screen_color3" node="screen" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_screen_color4" node="screen" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_overlay_float" node="overlay" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_overlay_color3" node="overlay" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_overlay_color4" node="overlay" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_disjointover_color4" node="disjointover" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_in_color4" node="in" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mask_color4" node="mask" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_matte_color4" node="matte" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_out_color4" node="out" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_over_color4" node="over" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_inside_float" node="inside" nodegroup="compositing">
    <input name="in" type="float" value="0.0" />
    <input name="mask" type="float" value="1.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_inside_color3" node="inside" nodegroup="compositing">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="1.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_inside_color4" node="inside" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="1.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_outside_float" node="outside" nodegroup="compositing">
    <input name="in" type="float" value="0.0" />
    <input name="mask" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_outside_color3" node="outside" nodegroup="compositing">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_outside_color4" node="outside" nodegroup="compositing">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mask" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_mix_float" node="mix" nodegroup="compositing">
    <input name="fg" type="float" value="0.0" />
    <input name="bg" type="float" value="0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="float" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color3" node="mix" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color3_color3" node="mix" nodegroup="compositing">
    <input name="fg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="color3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="color3" value="0.0, 0.0, 0.0" uisoftmin="0,0,0" uisoftmax="1,1,1" />
    <output name="out" type="color3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color4" node="mix" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_color4_color4" node="mix" nodegroup="compositing">
    <input name="fg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="color4" value="0.0, 0.0, 0.0, 0.0" uisoftmin="0,0,0,0" uisoftmax="1,1,1,1" />
    <output name="out" type="color4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector2" node="mix" nodegroup="compositing">
    <input name="fg" type="vector2" value="0.0, 0.0" />
    <input name="bg" type="vector2" value="0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="vector2" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector2_vector2" node="mix" nodegroup="compositing">
    <input name="fg" type="vector2" value="0.0, 0.0" />
    <input name="bg" type="vector2" value="0.0, 0.0" />
    <input name="mix" type="vector2" value="0.0, 0.0" uisoftmin="0,0" uisoftmax="1,1" />
    <output name="out" type="vector2" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector3" node="mix" nodegroup="compositing">
    <input name="fg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="vector3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector3_vector3" node="mix" nodegroup="compositing">
    <input name="fg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="bg" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="mix" type="vector3" value="0.0, 0.0, 0.0" uisoftmin="0,0,0" uisoftmax="1,1,1" />
    <output name="out" type="vector3" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector4" node="mix" nodegroup="compositing">
    <input name="fg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="vector4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_vector4_vector4" node="mix" nodegroup="compositing">
    <input name="fg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="bg" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="mix" type="vector4" value="0.0, 0.0, 0.0, 0.0" uisoftmin="0,0,0,0" uisoftmax="1,1,1,1" />
    <output name="out" type="vector4" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_surfaceshader" node="mix" nodegroup="compositing">
    <input name="fg" type="surfaceshader" value="" />
    <input name="bg" type="surfaceshader" value="" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="surfaceshader" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_displacementshader" node="mix" nodegroup="compositing">
    <input name="fg" type="displacementshader" value="" />
    <input name="bg" type="displacementshader" value="" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="displacementshader" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_mix_volumeshader" node="mix" nodegroup="compositing">
    <input name="fg" type="volumeshader" value="" />
    <input name="bg" type="volumeshader" value="" />
    <input name="mix" type="float" value="0.0" uisoftmin="0.0" uisoftmax="1.0" />
    <output name="out" type="volumeshader" defaultinput="bg" />
  </nodedef>
  <nodedef name="ND_ifgreater_float" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color3" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color4" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector2" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector3" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector4" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_floatI" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color3I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_color4I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector2I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector3I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreater_vector4I" node="ifgreater" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_float" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color3" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color4" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector2" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector3" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector4" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="float" value="1.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_floatI" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color3I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_color4I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector2I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector3I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifgreatereq_vector4I" node="ifgreatereq" nodegroup="conditional">
    <input name="value1" type="integer" value="1" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_float" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color3" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color4" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector2" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector3" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector4" node="ifequal" nodegroup="conditional">
    <input name="value1" type="float" value="0.0" />
    <input name="value2" type="float" value="0.0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_floatI" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color3I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color4I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector2I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector3I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector4I" node="ifequal" nodegroup="conditional">
    <input name="value1" type="integer" value="0" />
    <input name="value2" type="integer" value="0" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_floatB" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color3B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_color4B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector2B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector3B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_ifequal_vector4B" node="ifequal" nodegroup="conditional">
    <input name="value1" type="boolean" value="false" />
    <input name="value2" type="boolean" value="false" />
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_float" node="switch" nodegroup="conditional">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <input name="in5" type="float" value="0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color3" node="switch" nodegroup="conditional">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="color3" value="0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color4" node="switch" nodegroup="conditional">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector2" node="switch" nodegroup="conditional">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <input name="in3" type="vector2" value="0.0, 0.0" />
    <input name="in4" type="vector2" value="0.0, 0.0" />
    <input name="in5" type="vector2" value="0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector3" node="switch" nodegroup="conditional">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector4" node="switch" nodegroup="conditional">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="float" value="0.0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_floatI" node="switch" nodegroup="conditional">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <input name="in5" type="float" value="0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="float" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color3I" node="switch" nodegroup="conditional">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="color3" value="0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_color4I" node="switch" nodegroup="conditional">
    <input name="in1" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="color4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector2I" node="switch" nodegroup="conditional">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <input name="in3" type="vector2" value="0.0, 0.0" />
    <input name="in4" type="vector2" value="0.0, 0.0" />
    <input name="in5" type="vector2" value="0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="vector2" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector3I" node="switch" nodegroup="conditional">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in3" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in4" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in5" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="vector3" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_switch_vector4I" node="switch" nodegroup="conditional">
    <input name="in1" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in2" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in3" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in4" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="in5" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="which" type="integer" value="0" />
    <output name="out" type="vector4" defaultinput="in1" />
  </nodedef>
  <nodedef name="ND_convert_float_color3" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_color4" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_vector2" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_vector3" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_float_vector4" node="convert" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector2_vector3" node="convert" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector3_color3" node="convert" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector3_vector2" node="convert" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector3_vector4" node="convert" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector4_color4" node="convert" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_vector4_vector3" node="convert" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color3_vector3" node="convert" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color4_vector4" node="convert" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color3_color4" node="convert" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_color4_color3" node="convert" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_convert_boolean_float" node="convert" nodegroup="channel">
    <input name="in" type="boolean" value="false" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_convert_integer_float" node="convert" nodegroup="channel">
    <input name="in" type="integer" value="0" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_convert_color3_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert color3 to shader">
    <input name="in" type="color3" value="0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_color4_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert color4  to shader">
    <input name="in" type="color4" value="0, 0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_float_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert float to shader">
    <input name="in" type="float" value="0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_vector2_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert vector2 to shader">
    <input name="in" type="vector2" value="0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_vector3_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert vector2 to shader">
    <input name="in" type="vector3" value="0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_vector4_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert vector4 to shader">
    <input name="in" type="vector4" value="0, 0, 0, 0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_integer_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert integer to shader">
    <input name="in" type="integer" value="0" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_convert_boolean_surfaceshader" node="convert" version="1.0" isdefaultversion="true" nodegroup="shader" doc="Convert boolean to shader">
    <input name="in" type="boolean" value="false" />
    <output name="out" type="surfaceshader" />
  </nodedef>
  <nodedef name="ND_swizzle_float_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_float_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="float" value="0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_float" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="r" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rr" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color3_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_float" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="r" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rr" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrr" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_color4_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="rrrr" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_float" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="x" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector2_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_float" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="x" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_swizzle_vector3_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_float" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="x" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_color3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_color4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_vector2" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xx" uniform="true" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_vector3" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxx" uniform="true" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_swizzle_vector4_vector4" node="swizzle" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="channels" type="string" value="xxxx" uniform="true" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_combine2_vector2" node="combine2" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector2" default="0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine2_color4CF" node="combine2" nodegroup="channel">
    <input name="in1" type="color3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine2_vector4VF" node="combine2" nodegroup="channel">
    <input name="in1" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="in2" type="float" value="0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine2_vector4VV" node="combine2" nodegroup="channel">
    <input name="in1" type="vector2" value="0.0, 0.0" />
    <input name="in2" type="vector2" value="0.0, 0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine3_color3" node="combine3" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <output name="out" type="color3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine3_vector3" node="combine3" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <output name="out" type="vector3" default="0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine4_color4" node="combine4" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <output name="out" type="color4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_combine4_vector4" node="combine4" nodegroup="channel">
    <input name="in1" type="float" value="0.0" />
    <input name="in2" type="float" value="0.0" />
    <input name="in3" type="float" value="0.0" />
    <input name="in4" type="float" value="0.0" />
    <output name="out" type="vector4" default="0.0, 0.0, 0.0, 0.0" />
  </nodedef>
  <nodedef name="ND_extract_color3" node="extract" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="2" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_color4" node="extract" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="3" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_vector2" node="extract" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="1" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_vector3" node="extract" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="2" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_extract_vector4" node="extract" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="index" type="integer" value="0" uimin="0" uimax="3" uniform="true" />
    <output name="out" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate2_vector2" node="separate2" nodegroup="channel">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <output name="outx" type="float" default="0.0" />
    <output name="outy" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate3_color3" node="separate3" nodegroup="channel">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <output name="outr" type="float" default="0.0" />
    <output name="outg" type="float" default="0.0" />
    <output name="outb" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate3_vector3" node="separate3" nodegroup="channel">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <output name="outx" type="float" default="0.0" />
    <output name="outy" type="float" default="0.0" />
    <output name="outz" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate4_color4" node="separate4" nodegroup="channel">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="outr" type="float" default="0.0" />
    <output name="outg" type="float" default="0.0" />
    <output name="outb" type="float" default="0.0" />
    <output name="outa" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_separate4_vector4" node="separate4" nodegroup="channel">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <output name="outx" type="float" default="0.0" />
    <output name="outy" type="float" default="0.0" />
    <output name="outz" type="float" default="0.0" />
    <output name="outw" type="float" default="0.0" />
  </nodedef>
  <nodedef name="ND_blur_float" node="blur" nodegroup="convolution2d">
    <input name="in" type="float" value="0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_color3" node="blur" nodegroup="convolution2d">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_color4" node="blur" nodegroup="convolution2d">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_vector2" node="blur" nodegroup="convolution2d">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_vector3" node="blur" nodegroup="convolution2d">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_blur_vector4" node="blur" nodegroup="convolution2d">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="size" type="float" value="0.0" />
    <input name="filtertype" type="string" value="box" enum="box,gaussian" uniform="true" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_heighttonormal_vector3" node="heighttonormal" nodegroup="convolution2d">
    <input name="in" type="float" value="0.0" />
    <input name="scale" type="float" value="1.0" />
    <output name="out" type="vector3" default="0.5, 0.5, 1.0" />
  </nodedef>
  <nodedef name="ND_dot_float" node="dot" nodegroup="organization">
    <input name="in" type="float" value="0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="float" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_color3" node="dot" nodegroup="organization">
    <input name="in" type="color3" value="0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="color3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_color4" node="dot" nodegroup="organization">
    <input name="in" type="color4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="color4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_vector2" node="dot" nodegroup="organization">
    <input name="in" type="vector2" value="0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="vector2" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_vector3" node="dot" nodegroup="organization">
    <input name="in" type="vector3" value="0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="vector3" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_vector4" node="dot" nodegroup="organization">
    <input name="in" type="vector4" value="0.0, 0.0, 0.0, 0.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="vector4" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_boolean" node="dot" nodegroup="organization">
    <input name="in" type="boolean" value="false" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="boolean" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_integer" node="dot" nodegroup="organization">
    <input name="in" type="integer" value="0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="integer" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_matrix33" node="dot" nodegroup="organization">
    <input name="in" type="matrix33" value="1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="matrix33" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_matrix44" node="dot" nodegroup="organization">
    <input name="in" type="matrix44" value="1.0,0.0,0.0,0.0, 0.0,1.0,0.0,0.0, 0.0,0.0,1.0,0.0, 0.0,0.0,0.0,1.0" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="matrix44" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_string" node="dot" nodegroup="organization">
    <input name="in" type="string" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="string" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_filename" node="dot" nodegroup="organization">
    <input name="in" type="filename" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="filename" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_surfaceshader" node="dot" nodegroup="organization">
    <input name="in" type="surfaceshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="surfaceshader" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_displacementshader" node="dot" nodegroup="organization">
    <input name="in" type="displacementshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="displacementshader" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_volumeshader" node="dot" nodegroup="organization">
    <input name="in" type="volumeshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="volumeshader" defaultinput="in" />
  </nodedef>
  <nodedef name="ND_dot_lightshader" node="dot" nodegroup="organization">
    <input name="in" type="lightshader" value="" />
    <input name="note" type="string" value="" uniform="true" />
    <output name="out" type="lightshader" defaultinput="in" />
  </nodedef>
  <!-- Custom marble definition: : mymarble -->
  <!--Custom add definition (mxadd)-->
  <nodedef name="ND_myadd_color3" node="myadd">
    <input name="in1" type="color3" value="1, 0, 0" />
    <input name="in2" type="color3" value="0, 0, 0" />
    <output name="out" type="color3" defaultinput="in1" />
  </nodedef>
</materialx>
```
</details>


Definition ND_myadd_color3 added to: libraries\stdlib\stdlib_defs.mtlx


<details><summary>Implementation for target genglsl</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <implementation name="IM_surfacematerial_genglsl" nodedef="ND_surfacematerial" target="genglsl" />
  <implementation name="IM_surface_unlit_genglsl" nodedef="ND_surface_unlit" target="genglsl" />
  <implementation name="IM_image_float_genglsl" nodedef="ND_image_float" file="mx_image_float.glsl" function="mx_image_float" target="genglsl">
    <input name="default" type="float" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color3_genglsl" nodedef="ND_image_color3" file="mx_image_color3.glsl" function="mx_image_color3" target="genglsl">
    <input name="default" type="color3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color4_genglsl" nodedef="ND_image_color4" file="mx_image_color4.glsl" function="mx_image_color4" target="genglsl">
    <input name="default" type="color4" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector2_genglsl" nodedef="ND_image_vector2" file="mx_image_vector2.glsl" function="mx_image_vector2" target="genglsl">
    <input name="default" type="vector2" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector3_genglsl" nodedef="ND_image_vector3" file="mx_image_vector3.glsl" function="mx_image_vector3" target="genglsl">
    <input name="default" type="vector3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector4_genglsl" nodedef="ND_image_vector4" file="mx_image_vector4.glsl" function="mx_image_vector4" target="genglsl">
    <input name="default" type="vector4" implname="default_value" />
  </implementation>
  <implementation name="IM_normalmap_genglsl" nodedef="ND_normalmap" file="mx_normalmap.glsl" function="mx_normalmap" target="genglsl" />
  <implementation name="IM_constant_float_genglsl" nodedef="ND_constant_float" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_color3_genglsl" nodedef="ND_constant_color3" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_color4_genglsl" nodedef="ND_constant_color4" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector2_genglsl" nodedef="ND_constant_vector2" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector3_genglsl" nodedef="ND_constant_vector3" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector4_genglsl" nodedef="ND_constant_vector4" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_boolean_genglsl" nodedef="ND_constant_boolean" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_integer_genglsl" nodedef="ND_constant_integer" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_matrix33_genglsl" nodedef="ND_constant_matrix33" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_matrix44_genglsl" nodedef="ND_constant_matrix44" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_string_genglsl" nodedef="ND_constant_string" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_filename_genglsl" nodedef="ND_constant_filename" target="genglsl" sourcecode="{{value}}" />
  <implementation name="IM_ramplr_float_genglsl" nodedef="ND_ramplr_float" file="mx_ramplr_float.glsl" function="mx_ramplr_float" target="genglsl" />
  <implementation name="IM_ramplr_color3_genglsl" nodedef="ND_ramplr_color3" file="mx_ramplr_vector3.glsl" function="mx_ramplr_vector3" target="genglsl" />
  <implementation name="IM_ramplr_color4_genglsl" nodedef="ND_ramplr_color4" file="mx_ramplr_vector4.glsl" function="mx_ramplr_vector4" target="genglsl" />
  <implementation name="IM_ramplr_vector2_genglsl" nodedef="ND_ramplr_vector2" file="mx_ramplr_vector2.glsl" function="mx_ramplr_vector2" target="genglsl" />
  <implementation name="IM_ramplr_vector3_genglsl" nodedef="ND_ramplr_vector3" file="mx_ramplr_vector3.glsl" function="mx_ramplr_vector3" target="genglsl" />
  <implementation name="IM_ramplr_vector4_genglsl" nodedef="ND_ramplr_vector4" file="mx_ramplr_vector4.glsl" function="mx_ramplr_vector4" target="genglsl" />
  <implementation name="IM_ramptb_float_genglsl" nodedef="ND_ramptb_float" file="mx_ramptb_float.glsl" function="mx_ramptb_float" target="genglsl" />
  <implementation name="IM_ramptb_color3_genglsl" nodedef="ND_ramptb_color3" file="mx_ramptb_vector3.glsl" function="mx_ramptb_vector3" target="genglsl" />
  <implementation name="IM_ramptb_color4_genglsl" nodedef="ND_ramptb_color4" file="mx_ramptb_vector4.glsl" function="mx_ramptb_vector4" target="genglsl" />
  <implementation name="IM_ramptb_vector2_genglsl" nodedef="ND_ramptb_vector2" file="mx_ramptb_vector2.glsl" function="mx_ramptb_vector2" target="genglsl" />
  <implementation name="IM_ramptb_vector3_genglsl" nodedef="ND_ramptb_vector3" file="mx_ramptb_vector3.glsl" function="mx_ramptb_vector3" target="genglsl" />
  <implementation name="IM_ramptb_vector4_genglsl" nodedef="ND_ramptb_vector4" file="mx_ramptb_vector4.glsl" function="mx_ramptb_vector4" target="genglsl" />
  <implementation name="IM_splitlr_float_genglsl" nodedef="ND_splitlr_float" file="mx_splitlr_float.glsl" function="mx_splitlr_float" target="genglsl" />
  <implementation name="IM_splitlr_color3_genglsl" nodedef="ND_splitlr_color3" file="mx_splitlr_vector3.glsl" function="mx_splitlr_vector3" target="genglsl" />
  <implementation name="IM_splitlr_color4_genglsl" nodedef="ND_splitlr_color4" file="mx_splitlr_vector4.glsl" function="mx_splitlr_vector4" target="genglsl" />
  <implementation name="IM_splitlr_vector2_genglsl" nodedef="ND_splitlr_vector2" file="mx_splitlr_vector2.glsl" function="mx_splitlr_vector2" target="genglsl" />
  <implementation name="IM_splitlr_vector3_genglsl" nodedef="ND_splitlr_vector3" file="mx_splitlr_vector3.glsl" function="mx_splitlr_vector3" target="genglsl" />
  <implementation name="IM_splitlr_vector4_genglsl" nodedef="ND_splitlr_vector4" file="mx_splitlr_vector4.glsl" function="mx_splitlr_vector4" target="genglsl" />
  <implementation name="IM_splittb_float_genglsl" nodedef="ND_splittb_float" file="mx_splittb_float.glsl" function="mx_splittb_float" target="genglsl" />
  <implementation name="IM_splittb_color3_genglsl" nodedef="ND_splittb_color3" file="mx_splittb_vector3.glsl" function="mx_splittb_vector3" target="genglsl" />
  <implementation name="IM_splittb_color4_genglsl" nodedef="ND_splittb_color4" file="mx_splittb_vector4.glsl" function="mx_splittb_vector4" target="genglsl" />
  <implementation name="IM_splittb_vector2_genglsl" nodedef="ND_splittb_vector2" file="mx_splittb_vector2.glsl" function="mx_splittb_vector2" target="genglsl" />
  <implementation name="IM_splittb_vector3_genglsl" nodedef="ND_splittb_vector3" file="mx_splittb_vector3.glsl" function="mx_splittb_vector3" target="genglsl" />
  <implementation name="IM_splittb_vector4_genglsl" nodedef="ND_splittb_vector4" file="mx_splittb_vector4.glsl" function="mx_splittb_vector4" target="genglsl" />
  <implementation name="IM_noise2d_float_genglsl" nodedef="ND_noise2d_float" file="mx_noise2d_float.glsl" function="mx_noise2d_float" target="genglsl" />
  <implementation name="IM_noise2d_color3_genglsl" nodedef="ND_noise2d_color3" file="mx_noise2d_vector3.glsl" function="mx_noise2d_vector3" target="genglsl" />
  <implementation name="IM_noise2d_color4_genglsl" nodedef="ND_noise2d_color4" file="mx_noise2d_vector4.glsl" function="mx_noise2d_vector4" target="genglsl" />
  <implementation name="IM_noise2d_color3FA_genglsl" nodedef="ND_noise2d_color3FA" file="mx_noise2d_fa_vector3.glsl" function="mx_noise2d_fa_vector3" target="genglsl" />
  <implementation name="IM_noise2d_color4FA_genglsl" nodedef="ND_noise2d_color4FA" file="mx_noise2d_fa_vector4.glsl" function="mx_noise2d_fa_vector4" target="genglsl" />
  <implementation name="IM_noise2d_vector2_genglsl" nodedef="ND_noise2d_vector2" file="mx_noise2d_vector2.glsl" function="mx_noise2d_vector2" target="genglsl" />
  <implementation name="IM_noise2d_vector3_genglsl" nodedef="ND_noise2d_vector3" file="mx_noise2d_vector3.glsl" function="mx_noise2d_vector3" target="genglsl" />
  <implementation name="IM_noise2d_vector4_genglsl" nodedef="ND_noise2d_vector4" file="mx_noise2d_vector4.glsl" function="mx_noise2d_vector4" target="genglsl" />
  <implementation name="IM_noise2d_vector2FA_genglsl" nodedef="ND_noise2d_vector2FA" file="mx_noise2d_fa_vector2.glsl" function="mx_noise2d_fa_vector2" target="genglsl" />
  <implementation name="IM_noise2d_vector3FA_genglsl" nodedef="ND_noise2d_vector3FA" file="mx_noise2d_fa_vector3.glsl" function="mx_noise2d_fa_vector3" target="genglsl" />
  <implementation name="IM_noise2d_vector4FA_genglsl" nodedef="ND_noise2d_vector4FA" file="mx_noise2d_fa_vector4.glsl" function="mx_noise2d_fa_vector4" target="genglsl" />
  <implementation name="IM_noise3d_float_genglsl" nodedef="ND_noise3d_float" file="mx_noise3d_float.glsl" function="mx_noise3d_float" target="genglsl" />
  <implementation name="IM_noise3d_color3_genglsl" nodedef="ND_noise3d_color3" file="mx_noise3d_vector3.glsl" function="mx_noise3d_vector3" target="genglsl" />
  <implementation name="IM_noise3d_color4_genglsl" nodedef="ND_noise3d_color4" file="mx_noise3d_vector4.glsl" function="mx_noise3d_vector4" target="genglsl" />
  <implementation name="IM_noise3d_color3FA_genglsl" nodedef="ND_noise3d_color3FA" file="mx_noise3d_fa_vector3.glsl" function="mx_noise3d_fa_vector3" target="genglsl" />
  <implementation name="IM_noise3d_color4FA_genglsl" nodedef="ND_noise3d_color4FA" file="mx_noise3d_fa_vector4.glsl" function="mx_noise3d_fa_vector4" target="genglsl" />
  <implementation name="IM_noise3d_vector2_genglsl" nodedef="ND_noise3d_vector2" file="mx_noise3d_vector2.glsl" function="mx_noise3d_vector2" target="genglsl" />
  <implementation name="IM_noise3d_vector3_genglsl" nodedef="ND_noise3d_vector3" file="mx_noise3d_vector3.glsl" function="mx_noise3d_vector3" target="genglsl" />
  <implementation name="IM_noise3d_vector4_genglsl" nodedef="ND_noise3d_vector4" file="mx_noise3d_vector4.glsl" function="mx_noise3d_vector4" target="genglsl" />
  <implementation name="IM_noise3d_vector2FA_genglsl" nodedef="ND_noise3d_vector2FA" file="mx_noise3d_fa_vector2.glsl" function="mx_noise3d_fa_vector2" target="genglsl" />
  <implementation name="IM_noise3d_vector3FA_genglsl" nodedef="ND_noise3d_vector3FA" file="mx_noise3d_fa_vector3.glsl" function="mx_noise3d_fa_vector3" target="genglsl" />
  <implementation name="IM_noise3d_vector4FA_genglsl" nodedef="ND_noise3d_vector4FA" file="mx_noise3d_fa_vector4.glsl" function="mx_noise3d_fa_vector4" target="genglsl" />
  <implementation name="IM_fractal3d_float_genglsl" nodedef="ND_fractal3d_float" file="mx_fractal3d_float.glsl" function="mx_fractal3d_float" target="genglsl" />
  <implementation name="IM_fractal3d_color3_genglsl" nodedef="ND_fractal3d_color3" file="mx_fractal3d_vector3.glsl" function="mx_fractal3d_vector3" target="genglsl" />
  <implementation name="IM_fractal3d_color4_genglsl" nodedef="ND_fractal3d_color4" file="mx_fractal3d_vector4.glsl" function="mx_fractal3d_vector4" target="genglsl" />
  <implementation name="IM_fractal3d_color3FA_genglsl" nodedef="ND_fractal3d_color3FA" file="mx_fractal3d_fa_vector3.glsl" function="mx_fractal3d_fa_vector3" target="genglsl" />
  <implementation name="IM_fractal3d_color4FA_genglsl" nodedef="ND_fractal3d_color4FA" file="mx_fractal3d_fa_vector4.glsl" function="mx_fractal3d_fa_vector4" target="genglsl" />
  <implementation name="IM_fractal3d_vector2_genglsl" nodedef="ND_fractal3d_vector2" file="mx_fractal3d_vector2.glsl" function="mx_fractal3d_vector2" target="genglsl" />
  <implementation name="IM_fractal3d_vector3_genglsl" nodedef="ND_fractal3d_vector3" file="mx_fractal3d_vector3.glsl" function="mx_fractal3d_vector3" target="genglsl" />
  <implementation name="IM_fractal3d_vector4_genglsl" nodedef="ND_fractal3d_vector4" file="mx_fractal3d_vector4.glsl" function="mx_fractal3d_vector4" target="genglsl" />
  <implementation name="IM_fractal3d_vector2FA_genglsl" nodedef="ND_fractal3d_vector2FA" file="mx_fractal3d_fa_vector2.glsl" function="mx_fractal3d_fa_vector2" target="genglsl" />
  <implementation name="IM_fractal3d_vector3FA_genglsl" nodedef="ND_fractal3d_vector3FA" file="mx_fractal3d_fa_vector3.glsl" function="mx_fractal3d_fa_vector3" target="genglsl" />
  <implementation name="IM_fractal3d_vector4FA_genglsl" nodedef="ND_fractal3d_vector4FA" file="mx_fractal3d_fa_vector4.glsl" function="mx_fractal3d_fa_vector4" target="genglsl" />
  <implementation name="IM_cellnoise2d_float_genglsl" nodedef="ND_cellnoise2d_float" file="mx_cellnoise2d_float.glsl" function="mx_cellnoise2d_float" target="genglsl" />
  <implementation name="IM_cellnoise3d_float_genglsl" nodedef="ND_cellnoise3d_float" file="mx_cellnoise3d_float.glsl" function="mx_cellnoise3d_float" target="genglsl" />
  <implementation name="IM_worleynoise2d_float_genglsl" nodedef="ND_worleynoise2d_float" file="mx_worleynoise2d_float.glsl" function="mx_worleynoise2d_float" target="genglsl" />
  <implementation name="IM_worleynoise2d_vector2_genglsl" nodedef="ND_worleynoise2d_vector2" file="mx_worleynoise2d_vector2.glsl" function="mx_worleynoise2d_vector2" target="genglsl" />
  <implementation name="IM_worleynoise2d_vector3_genglsl" nodedef="ND_worleynoise2d_vector3" file="mx_worleynoise2d_vector3.glsl" function="mx_worleynoise2d_vector3" target="genglsl" />
  <implementation name="IM_worleynoise3d_float_genglsl" nodedef="ND_worleynoise3d_float" file="mx_worleynoise3d_float.glsl" function="mx_worleynoise3d_float" target="genglsl" />
  <implementation name="IM_worleynoise3d_vector2_genglsl" nodedef="ND_worleynoise3d_vector2" file="mx_worleynoise3d_vector2.glsl" function="mx_worleynoise3d_vector2" target="genglsl" />
  <implementation name="IM_worleynoise3d_vector3_genglsl" nodedef="ND_worleynoise3d_vector3" file="mx_worleynoise3d_vector3.glsl" function="mx_worleynoise3d_vector3" target="genglsl" />
  <implementation name="IM_position_vector3_genglsl" nodedef="ND_position_vector3" target="genglsl" />
  <implementation name="IM_normal_vector3_genglsl" nodedef="ND_normal_vector3" target="genglsl" />
  <implementation name="IM_tangent_vector3_genglsl" nodedef="ND_tangent_vector3" target="genglsl" />
  <implementation name="IM_bitangent_vector3_genglsl" nodedef="ND_bitangent_vector3" target="genglsl" />
  <implementation name="IM_texcoord_vector2_genglsl" nodedef="ND_texcoord_vector2" target="genglsl" />
  <implementation name="IM_texcoord_vector3_genglsl" nodedef="ND_texcoord_vector3" target="genglsl" />
  <implementation name="IM_geomcolor_float_genglsl" nodedef="ND_geomcolor_float" target="genglsl" />
  <implementation name="IM_geomcolor_color3_genglsl" nodedef="ND_geomcolor_color3" target="genglsl" />
  <implementation name="IM_geomcolor_color4_genglsl" nodedef="ND_geomcolor_color4" target="genglsl" />
  <implementation name="IM_geompropvalue_integer_genglsl" nodedef="ND_geompropvalue_integer" function="mx_geompropvalue_int" target="genglsl" />
  <implementation name="IM_geompropvalue_boolean_genglsl" nodedef="ND_geompropvalue_boolean" function="mx_geompropvalue_bool" target="genglsl" />
  <implementation name="IM_geompropvalue_string_genglsl" nodedef="ND_geompropvalue_string" function="mx_geompropvalue_string" target="genglsl" />
  <implementation name="IM_geompropvalue_float_genglsl" nodedef="ND_geompropvalue_float" function="mx_geompropvalue_float" target="genglsl" />
  <implementation name="IM_geompropvalue_color3_genglsl" nodedef="ND_geompropvalue_color3" function="mx_geompropvalue_color" target="genglsl" />
  <implementation name="IM_geompropvalue_color4_genglsl" nodedef="ND_geompropvalue_color4" function="mx_geompropvalue_color4" target="genglsl" />
  <implementation name="IM_geompropvalue_vector2_genglsl" nodedef="ND_geompropvalue_vector2" function="mx_geompropvalue_vector2" target="genglsl" />
  <implementation name="IM_geompropvalue_vector3_genglsl" nodedef="ND_geompropvalue_vector3" function="mx_geompropvalue_vector" target="genglsl" />
  <implementation name="IM_geompropvalue_vector4_genglsl" nodedef="ND_geompropvalue_vector4" function="mx_geompropvalue_vector4" target="genglsl" />
  <implementation name="IM_frame_float_genglsl" nodedef="ND_frame_float" function="mx_frame_float" target="genglsl" />
  <implementation name="IM_time_float_genglsl" nodedef="ND_time_float" function="mx_time_float" target="genglsl" />
  <implementation name="IM_add_float_genglsl" nodedef="ND_add_float" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color3_genglsl" nodedef="ND_add_color3" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color3FA_genglsl" nodedef="ND_add_color3FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color4_genglsl" nodedef="ND_add_color4" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color4FA_genglsl" nodedef="ND_add_color4FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector2_genglsl" nodedef="ND_add_vector2" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector2FA_genglsl" nodedef="ND_add_vector2FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector3_genglsl" nodedef="ND_add_vector3" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector3FA_genglsl" nodedef="ND_add_vector3FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector4_genglsl" nodedef="ND_add_vector4" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector4FA_genglsl" nodedef="ND_add_vector4FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix33_genglsl" nodedef="ND_add_matrix33" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix33FA_genglsl" nodedef="ND_add_matrix33FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix44_genglsl" nodedef="ND_add_matrix44" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix44FA_genglsl" nodedef="ND_add_matrix44FA" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_subtract_float_genglsl" nodedef="ND_subtract_float" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color3_genglsl" nodedef="ND_subtract_color3" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color3FA_genglsl" nodedef="ND_subtract_color3FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color4_genglsl" nodedef="ND_subtract_color4" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color4FA_genglsl" nodedef="ND_subtract_color4FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector2_genglsl" nodedef="ND_subtract_vector2" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector2FA_genglsl" nodedef="ND_subtract_vector2FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector3_genglsl" nodedef="ND_subtract_vector3" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector3FA_genglsl" nodedef="ND_subtract_vector3FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector4_genglsl" nodedef="ND_subtract_vector4" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector4FA_genglsl" nodedef="ND_subtract_vector4FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix33_genglsl" nodedef="ND_subtract_matrix33" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix33FA_genglsl" nodedef="ND_subtract_matrix33FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix44_genglsl" nodedef="ND_subtract_matrix44" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix44FA_genglsl" nodedef="ND_subtract_matrix44FA" target="genglsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_multiply_float_genglsl" nodedef="ND_multiply_float" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color3_genglsl" nodedef="ND_multiply_color3" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color3FA_genglsl" nodedef="ND_multiply_color3FA" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color4_genglsl" nodedef="ND_multiply_color4" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color4FA_genglsl" nodedef="ND_multiply_color4FA" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector2_genglsl" nodedef="ND_multiply_vector2" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector2FA_genglsl" nodedef="ND_multiply_vector2FA" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector3_genglsl" nodedef="ND_multiply_vector3" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector3FA_genglsl" nodedef="ND_multiply_vector3FA" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector4_genglsl" nodedef="ND_multiply_vector4" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector4FA_genglsl" nodedef="ND_multiply_vector4FA" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_matrix33_genglsl" nodedef="ND_multiply_matrix33" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_matrix44_genglsl" nodedef="ND_multiply_matrix44" target="genglsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_divide_float_genglsl" nodedef="ND_divide_float" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color3_genglsl" nodedef="ND_divide_color3" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color3FA_genglsl" nodedef="ND_divide_color3FA" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color4_genglsl" nodedef="ND_divide_color4" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color4FA_genglsl" nodedef="ND_divide_color4FA" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector2_genglsl" nodedef="ND_divide_vector2" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector2FA_genglsl" nodedef="ND_divide_vector2FA" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector3_genglsl" nodedef="ND_divide_vector3" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector3FA_genglsl" nodedef="ND_divide_vector3FA" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector4_genglsl" nodedef="ND_divide_vector4" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector4FA_genglsl" nodedef="ND_divide_vector4FA" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_matrix33_genglsl" nodedef="ND_divide_matrix33" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_matrix44_genglsl" nodedef="ND_divide_matrix44" target="genglsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_modulo_float_genglsl" nodedef="ND_modulo_float" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color3_genglsl" nodedef="ND_modulo_color3" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color3FA_genglsl" nodedef="ND_modulo_color3FA" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color4_genglsl" nodedef="ND_modulo_color4" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color4FA_genglsl" nodedef="ND_modulo_color4FA" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector2_genglsl" nodedef="ND_modulo_vector2" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector2FA_genglsl" nodedef="ND_modulo_vector2FA" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector3_genglsl" nodedef="ND_modulo_vector3" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector3FA_genglsl" nodedef="ND_modulo_vector3FA" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector4_genglsl" nodedef="ND_modulo_vector4" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector4FA_genglsl" nodedef="ND_modulo_vector4FA" target="genglsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_invert_float_genglsl" nodedef="ND_invert_float" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color3_genglsl" nodedef="ND_invert_color3" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color3FA_genglsl" nodedef="ND_invert_color3FA" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color4_genglsl" nodedef="ND_invert_color4" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color4FA_genglsl" nodedef="ND_invert_color4FA" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector2_genglsl" nodedef="ND_invert_vector2" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector2FA_genglsl" nodedef="ND_invert_vector2FA" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector3_genglsl" nodedef="ND_invert_vector3" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector3FA_genglsl" nodedef="ND_invert_vector3FA" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector4_genglsl" nodedef="ND_invert_vector4" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector4FA_genglsl" nodedef="ND_invert_vector4FA" target="genglsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_absval_float_genglsl" nodedef="ND_absval_float" target="genglsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_color3_genglsl" nodedef="ND_absval_color3" target="genglsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_color4_genglsl" nodedef="ND_absval_color4" target="genglsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector2_genglsl" nodedef="ND_absval_vector2" target="genglsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector3_genglsl" nodedef="ND_absval_vector3" target="genglsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector4_genglsl" nodedef="ND_absval_vector4" target="genglsl" sourcecode="abs({{in}})" />
  <implementation name="IM_floor_float_genglsl" nodedef="ND_floor_float" target="genglsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_color3_genglsl" nodedef="ND_floor_color3" target="genglsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_color4_genglsl" nodedef="ND_floor_color4" target="genglsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector2_genglsl" nodedef="ND_floor_vector2" target="genglsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector3_genglsl" nodedef="ND_floor_vector3" target="genglsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector4_genglsl" nodedef="ND_floor_vector4" target="genglsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_integer_genglsl" nodedef="ND_floor_integer" target="genglsl" sourcecode="int(floor({{in}}))" />
  <implementation name="IM_ceil_float_genglsl" nodedef="ND_ceil_float" target="genglsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_color3_genglsl" nodedef="ND_ceil_color3" target="genglsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_color4_genglsl" nodedef="ND_ceil_color4" target="genglsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector2_genglsl" nodedef="ND_ceil_vector2" target="genglsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector3_genglsl" nodedef="ND_ceil_vector3" target="genglsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector4_genglsl" nodedef="ND_ceil_vector4" target="genglsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_integer_genglsl" nodedef="ND_ceil_integer" target="genglsl" sourcecode="int(ceil({{in}}))" />
  <implementation name="IM_power_float_genglsl" nodedef="ND_power_float" target="genglsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color3_genglsl" nodedef="ND_power_color3" target="genglsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color3FA_genglsl" nodedef="ND_power_color3FA" target="genglsl" sourcecode="pow({{in1}}, vec3({{in2}}))" />
  <implementation name="IM_power_color4_genglsl" nodedef="ND_power_color4" target="genglsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color4FA_genglsl" nodedef="ND_power_color4FA" target="genglsl" sourcecode="pow({{in1}}, vec4({{in2}}))" />
  <implementation name="IM_power_vector2_genglsl" nodedef="ND_power_vector2" target="genglsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector2FA_genglsl" nodedef="ND_power_vector2FA" target="genglsl" sourcecode="pow({{in1}}, vec2({{in2}}))" />
  <implementation name="IM_power_vector3_genglsl" nodedef="ND_power_vector3" target="genglsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector3FA_genglsl" nodedef="ND_power_vector3FA" target="genglsl" sourcecode="pow({{in1}}, vec3({{in2}}))" />
  <implementation name="IM_power_vector4_genglsl" nodedef="ND_power_vector4" target="genglsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector4FA_genglsl" nodedef="ND_power_vector4FA" target="genglsl" sourcecode="pow({{in1}}, vec4({{in2}}))" />
  <implementation name="IM_sin_float_genglsl" nodedef="ND_sin_float" target="genglsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_float_genglsl" nodedef="ND_cos_float" target="genglsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_float_genglsl" nodedef="ND_tan_float" target="genglsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_float_genglsl" nodedef="ND_asin_float" target="genglsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_float_genglsl" nodedef="ND_acos_float" target="genglsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_float_genglsl" nodedef="ND_atan2_float" target="genglsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sin_vector2_genglsl" nodedef="ND_sin_vector2" target="genglsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector2_genglsl" nodedef="ND_cos_vector2" target="genglsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector2_genglsl" nodedef="ND_tan_vector2" target="genglsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector2_genglsl" nodedef="ND_asin_vector2" target="genglsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector2_genglsl" nodedef="ND_acos_vector2" target="genglsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector2_genglsl" nodedef="ND_atan2_vector2" target="genglsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sin_vector3_genglsl" nodedef="ND_sin_vector3" target="genglsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector3_genglsl" nodedef="ND_cos_vector3" target="genglsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector3_genglsl" nodedef="ND_tan_vector3" target="genglsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector3_genglsl" nodedef="ND_asin_vector3" target="genglsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector3_genglsl" nodedef="ND_acos_vector3" target="genglsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector3_genglsl" nodedef="ND_atan2_vector3" target="genglsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sin_vector4_genglsl" nodedef="ND_sin_vector4" target="genglsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector4_genglsl" nodedef="ND_cos_vector4" target="genglsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector4_genglsl" nodedef="ND_tan_vector4" target="genglsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector4_genglsl" nodedef="ND_asin_vector4" target="genglsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector4_genglsl" nodedef="ND_acos_vector4" target="genglsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector4_genglsl" nodedef="ND_atan2_vector4" target="genglsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sqrt_float_genglsl" nodedef="ND_sqrt_float" target="genglsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector2_genglsl" nodedef="ND_sqrt_vector2" target="genglsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector3_genglsl" nodedef="ND_sqrt_vector3" target="genglsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector4_genglsl" nodedef="ND_sqrt_vector4" target="genglsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_ln_float_genglsl" nodedef="ND_ln_float" target="genglsl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector2_genglsl" nodedef="ND_ln_vector2" target="genglsl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector3_genglsl" nodedef="ND_ln_vector3" target="genglsl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector4_genglsl" nodedef="ND_ln_vector4" target="genglsl" sourcecode="log({{in}})" />
  <implementation name="IM_exp_float_genglsl" nodedef="ND_exp_float" target="genglsl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector2_genglsl" nodedef="ND_exp_vector2" target="genglsl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector3_genglsl" nodedef="ND_exp_vector3" target="genglsl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector4_genglsl" nodedef="ND_exp_vector4" target="genglsl" sourcecode="exp({{in}})" />
  <implementation name="IM_sign_float_genglsl" nodedef="ND_sign_float" target="genglsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_color3_genglsl" nodedef="ND_sign_color3" target="genglsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_color4_genglsl" nodedef="ND_sign_color4" target="genglsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector2_genglsl" nodedef="ND_sign_vector2" target="genglsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector3_genglsl" nodedef="ND_sign_vector3" target="genglsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector4_genglsl" nodedef="ND_sign_vector4" target="genglsl" sourcecode="sign({{in}})" />
  <implementation name="IM_clamp_float_genglsl" nodedef="ND_clamp_float" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color3_genglsl" nodedef="ND_clamp_color3" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color3FA_genglsl" nodedef="ND_clamp_color3FA" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color4_genglsl" nodedef="ND_clamp_color4" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color4FA_genglsl" nodedef="ND_clamp_color4FA" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector2_genglsl" nodedef="ND_clamp_vector2" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector2FA_genglsl" nodedef="ND_clamp_vector2FA" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector3_genglsl" nodedef="ND_clamp_vector3" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector3FA_genglsl" nodedef="ND_clamp_vector3FA" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector4_genglsl" nodedef="ND_clamp_vector4" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector4FA_genglsl" nodedef="ND_clamp_vector4FA" target="genglsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_min_float_genglsl" nodedef="ND_min_float" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color3_genglsl" nodedef="ND_min_color3" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color3FA_genglsl" nodedef="ND_min_color3FA" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color4_genglsl" nodedef="ND_min_color4" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color4FA_genglsl" nodedef="ND_min_color4FA" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector2_genglsl" nodedef="ND_min_vector2" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector2FA_genglsl" nodedef="ND_min_vector2FA" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector3_genglsl" nodedef="ND_min_vector3" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector3FA_genglsl" nodedef="ND_min_vector3FA" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector4_genglsl" nodedef="ND_min_vector4" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector4FA_genglsl" nodedef="ND_min_vector4FA" target="genglsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_max_float_genglsl" nodedef="ND_max_float" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color3_genglsl" nodedef="ND_max_color3" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color3FA_genglsl" nodedef="ND_max_color3FA" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color4_genglsl" nodedef="ND_max_color4" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color4FA_genglsl" nodedef="ND_max_color4FA" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector2_genglsl" nodedef="ND_max_vector2" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector2FA_genglsl" nodedef="ND_max_vector2FA" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector3_genglsl" nodedef="ND_max_vector3" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector3FA_genglsl" nodedef="ND_max_vector3FA" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector4_genglsl" nodedef="ND_max_vector4" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector4FA_genglsl" nodedef="ND_max_vector4FA" target="genglsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_normalize_vector2_genglsl" nodedef="ND_normalize_vector2" target="genglsl" sourcecode="normalize({{in}})" />
  <implementation name="IM_normalize_vector3_genglsl" nodedef="ND_normalize_vector3" target="genglsl" sourcecode="normalize({{in}})" />
  <implementation name="IM_normalize_vector4_genglsl" nodedef="ND_normalize_vector4" target="genglsl" sourcecode="normalize({{in}})" />
  <implementation name="IM_magnitude_vector2_genglsl" nodedef="ND_magnitude_vector2" target="genglsl" sourcecode="length({{in}})" />
  <implementation name="IM_magnitude_vector3_genglsl" nodedef="ND_magnitude_vector3" target="genglsl" sourcecode="length({{in}})" />
  <implementation name="IM_magnitude_vector4_genglsl" nodedef="ND_magnitude_vector4" target="genglsl" sourcecode="length({{in}})" />
  <implementation name="IM_dotproduct_vector2_genglsl" nodedef="ND_dotproduct_vector2" target="genglsl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_dotproduct_vector3_genglsl" nodedef="ND_dotproduct_vector3" target="genglsl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_dotproduct_vector4_genglsl" nodedef="ND_dotproduct_vector4" target="genglsl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_crossproduct_vector3_genglsl" nodedef="ND_crossproduct_vector3" target="genglsl" sourcecode="cross({{in1}}, {{in2}})" />
  <implementation name="IM_transformpoint_vector3_genglsl" nodedef="ND_transformpoint_vector3" target="genglsl" />
  <implementation name="IM_transformvector_vector3_genglsl" nodedef="ND_transformvector_vector3" target="genglsl" />
  <implementation name="IM_transformnormal_vector3_genglsl" nodedef="ND_transformnormal_vector3" target="genglsl" />
  <implementation name="IM_transformmatrix_vector2M3_genglsl" nodedef="ND_transformmatrix_vector2M3" function="mx_transformmatrix_vector2M3" file="mx_transformmatrix_vector2M3.glsl" target="genglsl" />
  <implementation name="IM_transformmatrix_vector3_genglsl" nodedef="ND_transformmatrix_vector3" target="genglsl" sourcecode="{{mat}} * {{in}}" />
  <implementation name="IM_transformmatrix_vector3M4_genglsl" nodedef="ND_transformmatrix_vector3M4" function="mx_transformmatrix_vector3M4" file="mx_transformmatrix_vector3M4.glsl" target="genglsl" />
  <implementation name="IM_transformmatrix_vector4_genglsl" nodedef="ND_transformmatrix_vector4" target="genglsl" sourcecode="{{mat}} * {{in}}" />
  <implementation name="IM_transpose_matrix33_genglsl" nodedef="ND_transpose_matrix33" target="genglsl" sourcecode="transpose({{in}})" />
  <implementation name="IM_transpose_matrix44_genglsl" nodedef="ND_transpose_matrix44" target="genglsl" sourcecode="transpose({{in}})" />
  <implementation name="IM_determinant_matrix33_genglsl" nodedef="ND_determinant_matrix33" target="genglsl" sourcecode="determinant({{in}})" />
  <implementation name="IM_determinant_matrix44_genglsl" nodedef="ND_determinant_matrix44" target="genglsl" sourcecode="determinant({{in}})" />
  <implementation name="IM_invertmatrix_matrix33_genglsl" nodedef="ND_invertmatrix_matrix33" target="genglsl" sourcecode="inverse({{in}})" />
  <implementation name="IM_invertmatrix_matrix44_genglsl" nodedef="ND_invertmatrix_matrix44" target="genglsl" sourcecode="inverse({{in}})" />
  <implementation name="IM_rotate2d_vector2_genglsl" nodedef="ND_rotate2d_vector2" file="mx_rotate_vector2.glsl" function="mx_rotate_vector2" target="genglsl" />
  <implementation name="IM_rotate3d_vector3_genglsl" nodedef="ND_rotate3d_vector3" file="mx_rotate_vector3.glsl" function="mx_rotate_vector3" target="genglsl" />
  <implementation name="IM_remap_float_genglsl" nodedef="ND_remap_float" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color3_genglsl" nodedef="ND_remap_color3" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color3FA_genglsl" nodedef="ND_remap_color3FA" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color4_genglsl" nodedef="ND_remap_color4" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color4FA_genglsl" nodedef="ND_remap_color4FA" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector2_genglsl" nodedef="ND_remap_vector2" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector2FA_genglsl" nodedef="ND_remap_vector2FA" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector3_genglsl" nodedef="ND_remap_vector3" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector3FA_genglsl" nodedef="ND_remap_vector3FA" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector4_genglsl" nodedef="ND_remap_vector4" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector4FA_genglsl" nodedef="ND_remap_vector4FA" target="genglsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_smoothstep_float_genglsl" nodedef="ND_smoothstep_float" file="mx_smoothstep_float.glsl" function="mx_smoothstep_float" target="genglsl" />
  <implementation name="IM_smoothstep_color3_genglsl" nodedef="ND_smoothstep_color3" file="mx_smoothstep_vec3.glsl" function="mx_smoothstep_vec3" target="genglsl" />
  <implementation name="IM_smoothstep_color3FA_genglsl" nodedef="ND_smoothstep_color3FA" file="mx_smoothstep_vec3FA.glsl" function="mx_smoothstep_vec3FA" target="genglsl" />
  <implementation name="IM_smoothstep_color4_genglsl" nodedef="ND_smoothstep_color4" file="mx_smoothstep_vec4.glsl" function="mx_smoothstep_vec4" target="genglsl" />
  <implementation name="IM_smoothstep_color4FA_genglsl" nodedef="ND_smoothstep_color4FA" file="mx_smoothstep_vec4FA.glsl" function="mx_smoothstep_vec4FA" target="genglsl" />
  <implementation name="IM_smoothstep_vector2_genglsl" nodedef="ND_smoothstep_vector2" file="mx_smoothstep_vec2.glsl" function="mx_smoothstep_vec2" target="genglsl" />
  <implementation name="IM_smoothstep_vector2FA_genglsl" nodedef="ND_smoothstep_vector2FA" file="mx_smoothstep_vec2FA.glsl" function="mx_smoothstep_vec2FA" target="genglsl" />
  <implementation name="IM_smoothstep_vector3_genglsl" nodedef="ND_smoothstep_vector3" file="mx_smoothstep_vec3.glsl" function="mx_smoothstep_vec3" target="genglsl" />
  <implementation name="IM_smoothstep_vector3FA_genglsl" nodedef="ND_smoothstep_vector3FA" file="mx_smoothstep_vec3FA.glsl" function="mx_smoothstep_vec3FA" target="genglsl" />
  <implementation name="IM_smoothstep_vector4_genglsl" nodedef="ND_smoothstep_vector4" file="mx_smoothstep_vec4.glsl" function="mx_smoothstep_vec4" target="genglsl" />
  <implementation name="IM_smoothstep_vector4FA_genglsl" nodedef="ND_smoothstep_vector4FA" file="mx_smoothstep_vec4FA.glsl" function="mx_smoothstep_vec4FA" target="genglsl" />
  <implementation name="IM_luminance_color3_genglsl" nodedef="ND_luminance_color3" file="mx_luminance_color3.glsl" function="mx_luminance_color3" target="genglsl" />
  <implementation name="IM_luminance_color4_genglsl" nodedef="ND_luminance_color4" file="mx_luminance_color4.glsl" function="mx_luminance_color4" target="genglsl" />
  <implementation name="IM_rgbtohsv_color3_genglsl" nodedef="ND_rgbtohsv_color3" file="mx_rgbtohsv_color3.glsl" function="mx_rgbtohsv_color3" target="genglsl" />
  <implementation name="IM_rgbtohsv_color4_genglsl" nodedef="ND_rgbtohsv_color4" file="mx_rgbtohsv_color4.glsl" function="mx_rgbtohsv_color4" target="genglsl" />
  <implementation name="IM_hsvtorgb_color3_genglsl" nodedef="ND_hsvtorgb_color3" file="mx_hsvtorgb_color3.glsl" function="mx_hsvtorgb_color3" target="genglsl" />
  <implementation name="IM_hsvtorgb_color4_genglsl" nodedef="ND_hsvtorgb_color4" file="mx_hsvtorgb_color4.glsl" function="mx_hsvtorgb_color4" target="genglsl" />
  <implementation name="IM_premult_color4_genglsl" nodedef="ND_premult_color4" file="mx_premult_color4.glsl" function="mx_premult_color4" target="genglsl" />
  <implementation name="IM_unpremult_color4_genglsl" nodedef="ND_unpremult_color4" file="mx_unpremult_color4.glsl" function="mx_unpremult_color4" target="genglsl" />
  <implementation name="IM_plus_float_genglsl" nodedef="ND_plus_float" target="genglsl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_plus_color3_genglsl" nodedef="ND_plus_color3" target="genglsl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_plus_color4_genglsl" nodedef="ND_plus_color4" target="genglsl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_float_genglsl" nodedef="ND_minus_float" target="genglsl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_color3_genglsl" nodedef="ND_minus_color3" target="genglsl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_color4_genglsl" nodedef="ND_minus_color4" target="genglsl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_float_genglsl" nodedef="ND_difference_float" target="genglsl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_color3_genglsl" nodedef="ND_difference_color3" target="genglsl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_color4_genglsl" nodedef="ND_difference_color4" target="genglsl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_burn_float_genglsl" nodedef="ND_burn_float" file="mx_burn_float.glsl" function="mx_burn_float" target="genglsl" />
  <implementation name="IM_burn_color3_genglsl" nodedef="ND_burn_color3" file="mx_burn_color3.glsl" function="mx_burn_color3" target="genglsl" />
  <implementation name="IM_burn_color4_genglsl" nodedef="ND_burn_color4" file="mx_burn_color4.glsl" function="mx_burn_color4" target="genglsl" />
  <implementation name="IM_dodge_float_genglsl" nodedef="ND_dodge_float" file="mx_dodge_float.glsl" function="mx_dodge_float" target="genglsl" />
  <implementation name="IM_dodge_color3_genglsl" nodedef="ND_dodge_color3" file="mx_dodge_color3.glsl" function="mx_dodge_color3" target="genglsl" />
  <implementation name="IM_dodge_color4_genglsl" nodedef="ND_dodge_color4" file="mx_dodge_color4.glsl" function="mx_dodge_color4" target="genglsl" />
  <implementation name="IM_screen_float_genglsl" nodedef="ND_screen_float" target="genglsl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_screen_color3_genglsl" nodedef="ND_screen_color3" target="genglsl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_screen_color4_genglsl" nodedef="ND_screen_color4" target="genglsl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_overlay_float_genglsl" nodedef="ND_overlay_float" target="genglsl" sourcecode="({{fg}} < 0.5) ? ({{mix}}*2.0*{{fg}}*{{bg}}) + ((1.0-{{mix}})*{{bg}}) : ({{mix}}*(1.0-(1.0-{{fg}})*(1.0-{{bg}}))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_overlay_color3_genglsl" nodedef="ND_overlay_color3" file="mx_overlay_color3.glsl" function="mx_overlay_color3" target="genglsl" />
  <implementation name="IM_overlay_color4_genglsl" nodedef="ND_overlay_color4" file="mx_overlay_color4.glsl" function="mx_overlay_color4" target="genglsl" />
  <implementation name="IM_disjointover_color4_genglsl" nodedef="ND_disjointover_color4" file="mx_disjointover_color4.glsl" function="mx_disjointover_color4" target="genglsl" />
  <implementation name="IM_in_color4_genglsl" nodedef="ND_in_color4" target="genglsl" sourcecode="({{fg}}*{{bg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_mask_color4_genglsl" nodedef="ND_mask_color4" target="genglsl" sourcecode="({{bg}}*{{fg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_matte_color4_genglsl" nodedef="ND_matte_color4" target="genglsl" sourcecode="vec4( {{fg}}.xyz*{{fg}}.w + {{bg}}.xyz*(1.0-{{fg}}.w), {{fg}}.w + ({{bg}}.w*(1.0-{{fg}}.w)) ) * {{mix}} + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_out_color4_genglsl" nodedef="ND_out_color4" target="genglsl" sourcecode="({{fg}}*(1.0-{{bg}}.a)  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_over_color4_genglsl" nodedef="ND_over_color4" target="genglsl" sourcecode="({{fg}} + ({{bg}}*(1.0-{{fg}}[3]))) * {{mix}} + {{bg}} * (1.0-{{mix}})" />
  <implementation name="IM_inside_float_genglsl" nodedef="ND_inside_float" target="genglsl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_inside_color3_genglsl" nodedef="ND_inside_color3" target="genglsl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_inside_color4_genglsl" nodedef="ND_inside_color4" target="genglsl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_outside_float_genglsl" nodedef="ND_outside_float" target="genglsl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_outside_color3_genglsl" nodedef="ND_outside_color3" target="genglsl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_outside_color4_genglsl" nodedef="ND_outside_color4" target="genglsl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_mix_float_genglsl" nodedef="ND_mix_float" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color3_genglsl" nodedef="ND_mix_color3" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color3_color3_genglsl" nodedef="ND_mix_color3_color3" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color4_genglsl" nodedef="ND_mix_color4" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color4_color4_genglsl" nodedef="ND_mix_color4_color4" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector2_genglsl" nodedef="ND_mix_vector2" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector2_vector2_genglsl" nodedef="ND_mix_vector2_vector2" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector3_genglsl" nodedef="ND_mix_vector3" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector3_vector3_genglsl" nodedef="ND_mix_vector3_vector3" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector4_genglsl" nodedef="ND_mix_vector4" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector4_vector4_genglsl" nodedef="ND_mix_vector4_vector4" target="genglsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_surfaceshader_genglsl" nodedef="ND_mix_surfaceshader" function="mx_mix_surfaceshader" file="mx_mix_surfaceshader.glsl" target="genglsl" />
  <implementation name="IM_ifgreater_float_genglsl" nodedef="ND_ifgreater_float" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color3_genglsl" nodedef="ND_ifgreater_color3" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color4_genglsl" nodedef="ND_ifgreater_color4" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector2_genglsl" nodedef="ND_ifgreater_vector2" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector3_genglsl" nodedef="ND_ifgreater_vector3" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector4_genglsl" nodedef="ND_ifgreater_vector4" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_floatI_genglsl" nodedef="ND_ifgreater_floatI" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color3I_genglsl" nodedef="ND_ifgreater_color3I" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color4I_genglsl" nodedef="ND_ifgreater_color4I" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector2I_genglsl" nodedef="ND_ifgreater_vector2I" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector3I_genglsl" nodedef="ND_ifgreater_vector3I" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector4I_genglsl" nodedef="ND_ifgreater_vector4I" target="genglsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_float_genglsl" nodedef="ND_ifgreatereq_float" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color3_genglsl" nodedef="ND_ifgreatereq_color3" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color4_genglsl" nodedef="ND_ifgreatereq_color4" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector2_genglsl" nodedef="ND_ifgreatereq_vector2" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector3_genglsl" nodedef="ND_ifgreatereq_vector3" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector4_genglsl" nodedef="ND_ifgreatereq_vector4" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_floatI_genglsl" nodedef="ND_ifgreatereq_floatI" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color3I_genglsl" nodedef="ND_ifgreatereq_color3I" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color4I_genglsl" nodedef="ND_ifgreatereq_color4I" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector2I_genglsl" nodedef="ND_ifgreatereq_vector2I" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector3I_genglsl" nodedef="ND_ifgreatereq_vector3I" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector4I_genglsl" nodedef="ND_ifgreatereq_vector4I" target="genglsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_float_genglsl" nodedef="ND_ifequal_float" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color3_genglsl" nodedef="ND_ifequal_color3" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color4_genglsl" nodedef="ND_ifequal_color4" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector2_genglsl" nodedef="ND_ifequal_vector2" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector3_genglsl" nodedef="ND_ifequal_vector3" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector4_genglsl" nodedef="ND_ifequal_vector4" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_floatI_genglsl" nodedef="ND_ifequal_floatI" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color3I_genglsl" nodedef="ND_ifequal_color3I" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color4I_genglsl" nodedef="ND_ifequal_color4I" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector2I_genglsl" nodedef="ND_ifequal_vector2I" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector3I_genglsl" nodedef="ND_ifequal_vector3I" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector4I_genglsl" nodedef="ND_ifequal_vector4I" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_floatB_genglsl" nodedef="ND_ifequal_floatB" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color3B_genglsl" nodedef="ND_ifequal_color3B" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color4B_genglsl" nodedef="ND_ifequal_color4B" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector2B_genglsl" nodedef="ND_ifequal_vector2B" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector3B_genglsl" nodedef="ND_ifequal_vector3B" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector4B_genglsl" nodedef="ND_ifequal_vector4B" target="genglsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_switch_float_genglsl" nodedef="ND_switch_float" target="genglsl" />
  <implementation name="IM_switch_color3_genglsl" nodedef="ND_switch_color3" target="genglsl" />
  <implementation name="IM_switch_color4_genglsl" nodedef="ND_switch_color4" target="genglsl" />
  <implementation name="IM_switch_vector2_genglsl" nodedef="ND_switch_vector2" target="genglsl" />
  <implementation name="IM_switch_vector3_genglsl" nodedef="ND_switch_vector3" target="genglsl" />
  <implementation name="IM_switch_vector4_genglsl" nodedef="ND_switch_vector4" target="genglsl" />
  <implementation name="IM_switch_floatI_genglsl" nodedef="ND_switch_floatI" target="genglsl" />
  <implementation name="IM_switch_color3I_genglsl" nodedef="ND_switch_color3I" target="genglsl" />
  <implementation name="IM_switch_color4I_genglsl" nodedef="ND_switch_color4I" target="genglsl" />
  <implementation name="IM_switch_vector2I_genglsl" nodedef="ND_switch_vector2I" target="genglsl" />
  <implementation name="IM_switch_vector3I_genglsl" nodedef="ND_switch_vector3I" target="genglsl" />
  <implementation name="IM_switch_vector4I_genglsl" nodedef="ND_switch_vector4I" target="genglsl" />
  <implementation name="IM_convert_float_color3_genglsl" nodedef="ND_convert_float_color3" target="genglsl" />
  <implementation name="IM_convert_float_color4_genglsl" nodedef="ND_convert_float_color4" target="genglsl" />
  <implementation name="IM_convert_float_vector2_genglsl" nodedef="ND_convert_float_vector2" target="genglsl" />
  <implementation name="IM_convert_float_vector3_genglsl" nodedef="ND_convert_float_vector3" target="genglsl" />
  <implementation name="IM_convert_float_vector4_genglsl" nodedef="ND_convert_float_vector4" target="genglsl" />
  <implementation name="IM_convert_vector2_vector3_genglsl" nodedef="ND_convert_vector2_vector3" target="genglsl" />
  <implementation name="IM_convert_vector3_vector2_genglsl" nodedef="ND_convert_vector3_vector2" target="genglsl" />
  <implementation name="IM_convert_vector3_color3_genglsl" nodedef="ND_convert_vector3_color3" target="genglsl" />
  <implementation name="IM_convert_vector3_vector4_genglsl" nodedef="ND_convert_vector3_vector4" target="genglsl" />
  <implementation name="IM_convert_vector4_vector3_genglsl" nodedef="ND_convert_vector4_vector3" target="genglsl" />
  <implementation name="IM_convert_vector4_color4_genglsl" nodedef="ND_convert_vector4_color4" target="genglsl" />
  <implementation name="IM_convert_color3_vector3_genglsl" nodedef="ND_convert_color3_vector3" target="genglsl" />
  <implementation name="IM_convert_color4_vector4_genglsl" nodedef="ND_convert_color4_vector4" target="genglsl" />
  <implementation name="IM_convert_color3_color4_genglsl" nodedef="ND_convert_color3_color4" target="genglsl" />
  <implementation name="IM_convert_color4_color3_genglsl" nodedef="ND_convert_color4_color3" target="genglsl" />
  <implementation name="IM_convert_boolean_float_genglsl" nodedef="ND_convert_boolean_float" target="genglsl" />
  <implementation name="IM_convert_integer_float_genglsl" nodedef="ND_convert_integer_float" target="genglsl" />
  <implementation name="IM_swizzle_float_color3_genglsl" nodedef="ND_swizzle_float_color3" target="genglsl" />
  <implementation name="IM_swizzle_float_color4_genglsl" nodedef="ND_swizzle_float_color4" target="genglsl" />
  <implementation name="IM_swizzle_float_vector2_genglsl" nodedef="ND_swizzle_float_vector2" target="genglsl" />
  <implementation name="IM_swizzle_float_vector3_genglsl" nodedef="ND_swizzle_float_vector3" target="genglsl" />
  <implementation name="IM_swizzle_float_vector4_genglsl" nodedef="ND_swizzle_float_vector4" target="genglsl" />
  <implementation name="IM_swizzle_color3_float_genglsl" nodedef="ND_swizzle_color3_float" target="genglsl" />
  <implementation name="IM_swizzle_color3_color3_genglsl" nodedef="ND_swizzle_color3_color3" target="genglsl" />
  <implementation name="IM_swizzle_color3_color4_genglsl" nodedef="ND_swizzle_color3_color4" target="genglsl" />
  <implementation name="IM_swizzle_color3_vector2_genglsl" nodedef="ND_swizzle_color3_vector2" target="genglsl" />
  <implementation name="IM_swizzle_color3_vector3_genglsl" nodedef="ND_swizzle_color3_vector3" target="genglsl" />
  <implementation name="IM_swizzle_color3_vector4_genglsl" nodedef="ND_swizzle_color3_vector4" target="genglsl" />
  <implementation name="IM_swizzle_color4_float_genglsl" nodedef="ND_swizzle_color4_float" target="genglsl" />
  <implementation name="IM_swizzle_color4_color3_genglsl" nodedef="ND_swizzle_color4_color3" target="genglsl" />
  <implementation name="IM_swizzle_color4_color4_genglsl" nodedef="ND_swizzle_color4_color4" target="genglsl" />
  <implementation name="IM_swizzle_color4_vector2_genglsl" nodedef="ND_swizzle_color4_vector2" target="genglsl" />
  <implementation name="IM_swizzle_color4_vector3_genglsl" nodedef="ND_swizzle_color4_vector3" target="genglsl" />
  <implementation name="IM_swizzle_color4_vector4_genglsl" nodedef="ND_swizzle_color4_vector4" target="genglsl" />
  <implementation name="IM_swizzle_vector2_float_genglsl" nodedef="ND_swizzle_vector2_float" target="genglsl" />
  <implementation name="IM_swizzle_vector2_color3_genglsl" nodedef="ND_swizzle_vector2_color3" target="genglsl" />
  <implementation name="IM_swizzle_vector2_color4_genglsl" nodedef="ND_swizzle_vector2_color4" target="genglsl" />
  <implementation name="IM_swizzle_vector2_vector2_genglsl" nodedef="ND_swizzle_vector2_vector2" target="genglsl" />
  <implementation name="IM_swizzle_vector2_vector3_genglsl" nodedef="ND_swizzle_vector2_vector3" target="genglsl" />
  <implementation name="IM_swizzle_vector2_vector4_genglsl" nodedef="ND_swizzle_vector2_vector4" target="genglsl" />
  <implementation name="IM_swizzle_vector3_float_genglsl" nodedef="ND_swizzle_vector3_float" target="genglsl" />
  <implementation name="IM_swizzle_vector3_color3_genglsl" nodedef="ND_swizzle_vector3_color3" target="genglsl" />
  <implementation name="IM_swizzle_vector3_color4_genglsl" nodedef="ND_swizzle_vector3_color4" target="genglsl" />
  <implementation name="IM_swizzle_vector3_vector2_genglsl" nodedef="ND_swizzle_vector3_vector2" target="genglsl" />
  <implementation name="IM_swizzle_vector3_vector3_genglsl" nodedef="ND_swizzle_vector3_vector3" target="genglsl" />
  <implementation name="IM_swizzle_vector3_vector4_genglsl" nodedef="ND_swizzle_vector3_vector4" target="genglsl" />
  <implementation name="IM_swizzle_vector4_float_genglsl" nodedef="ND_swizzle_vector4_float" target="genglsl" />
  <implementation name="IM_swizzle_vector4_color3_genglsl" nodedef="ND_swizzle_vector4_color3" target="genglsl" />
  <implementation name="IM_swizzle_vector4_color4_genglsl" nodedef="ND_swizzle_vector4_color4" target="genglsl" />
  <implementation name="IM_swizzle_vector4_vector2_genglsl" nodedef="ND_swizzle_vector4_vector2" target="genglsl" />
  <implementation name="IM_swizzle_vector4_vector3_genglsl" nodedef="ND_swizzle_vector4_vector3" target="genglsl" />
  <implementation name="IM_swizzle_vector4_vector4_genglsl" nodedef="ND_swizzle_vector4_vector4" target="genglsl" />
  <implementation name="IM_combine2_vector2_genglsl" nodedef="ND_combine2_vector2" target="genglsl" />
  <implementation name="IM_combine2_color4CF_genglsl" nodedef="ND_combine2_color4CF" target="genglsl" />
  <implementation name="IM_combine2_vector4VF_genglsl" nodedef="ND_combine2_vector4VF" target="genglsl" />
  <implementation name="IM_combine2_vector4VV_genglsl" nodedef="ND_combine2_vector4VV" target="genglsl" />
  <implementation name="IM_combine3_color3_genglsl" nodedef="ND_combine3_color3" target="genglsl" />
  <implementation name="IM_combine3_vector3_genglsl" nodedef="ND_combine3_vector3" target="genglsl" />
  <implementation name="IM_combine4_color4_genglsl" nodedef="ND_combine4_color4" target="genglsl" />
  <implementation name="IM_combine4_vector4_genglsl" nodedef="ND_combine4_vector4" target="genglsl" />
  <implementation name="IM_blur_float_genglsl" nodedef="ND_blur_float" target="genglsl" />
  <implementation name="IM_blur_color3_genglsl" nodedef="ND_blur_color3" target="genglsl" />
  <implementation name="IM_blur_color4_genglsl" nodedef="ND_blur_color4" target="genglsl" />
  <implementation name="IM_blur_vector2_genglsl" nodedef="ND_blur_vector2" target="genglsl" />
  <implementation name="IM_blur_vector3_genglsl" nodedef="ND_blur_vector3" target="genglsl" />
  <implementation name="IM_blur_vector4_genglsl" nodedef="ND_blur_vector4" target="genglsl" />
  <implementation name="IM_heighttonormal_vector3_genglsl" nodedef="ND_heighttonormal_vector3" target="genglsl" />
  <implementation name="IM_dot_float_genglsl" nodedef="ND_dot_float" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_color3_genglsl" nodedef="ND_dot_color3" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_color4_genglsl" nodedef="ND_dot_color4" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector2_genglsl" nodedef="ND_dot_vector2" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector3_genglsl" nodedef="ND_dot_vector3" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector4_genglsl" nodedef="ND_dot_vector4" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_integer_genglsl" nodedef="ND_dot_integer" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_boolean_genglsl" nodedef="ND_dot_boolean" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_matrix33_genglsl" nodedef="ND_dot_matrix33" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_matrix44_genglsl" nodedef="ND_dot_matrix44" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_string_genglsl" nodedef="ND_dot_string" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_filename_genglsl" nodedef="ND_dot_filename" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_surfaceshader_genglsl" nodedef="ND_dot_surfaceshader" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_displacementshader_genglsl" nodedef="ND_dot_displacementshader" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_volumeshader_genglsl" nodedef="ND_dot_volumeshader" target="genglsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_lightshader_genglsl" nodedef="ND_dot_lightshader" target="genglsl" sourcecode="{{in}}" />
  <!--Custom add implementation (mxadd)-->
  <implementation name="IM_myadd_color3_genglsl" nodedef="ND_myadd_color3" target="genglsl" sourcecode="{{in1}} + {{in2}}" />
</materialx>
```
</details>


- Source implementation added: IM_myadd_color3_genglsl


<details><summary>Implementation for target genmdl</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <implementation name="IM_surfacematerial_genmdl" nodedef="ND_surfacematerial" target="genmdl" />
  <implementation name="IM_surface_unlit_genmdl" nodedef="ND_surface_unlit" sourcecode="mx::stdlib::mx_surface_unlit({{emission}}, {{emission_color}}, {{transmission}}, {{transmission_color}}, {{opacity}})" target="genmdl" />
  <implementation name="IM_image_float_genmdl" nodedef="ND_image_float" sourcecode="mx::stdlib::mx_image_float({{file}}, {{layer}}, {{default}}, {{texcoord}}, {{uaddressmode}}, {{vaddressmode}}, {{filtertype}}, {{framerange}}, {{frameoffset}}, {{frameendaction}})" target="genmdl">
    <input name="default" type="float" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color3_genmdl" nodedef="ND_image_color3" sourcecode="mx::stdlib::mx_image_color3({{file}}, {{layer}}, {{default}}, {{texcoord}}, {{uaddressmode}}, {{vaddressmode}}, {{filtertype}}, {{framerange}}, {{frameoffset}}, {{frameendaction}})" target="genmdl">
    <input name="default" type="color3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color4_genmdl" nodedef="ND_image_color4" sourcecode="mx::stdlib::mx_image_color4({{file}}, {{layer}}, {{default}}, {{texcoord}}, {{uaddressmode}}, {{vaddressmode}}, {{filtertype}}, {{framerange}}, {{frameoffset}}, {{frameendaction}})" target="genmdl">
    <input name="default" type="color4" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector2_genmdl" nodedef="ND_image_vector2" sourcecode="mx::stdlib::mx_image_vector2({{file}}, {{layer}}, {{default}}, {{texcoord}}, {{uaddressmode}}, {{vaddressmode}}, {{filtertype}}, {{framerange}}, {{frameoffset}}, {{frameendaction}})" target="genmdl">
    <input name="default" type="vector2" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector3_genmdl" nodedef="ND_image_vector3" sourcecode="mx::stdlib::mx_image_vector3({{file}}, {{layer}}, {{default}}, {{texcoord}}, {{uaddressmode}}, {{vaddressmode}}, {{filtertype}}, {{framerange}}, {{frameoffset}}, {{frameendaction}})" target="genmdl">
    <input name="default" type="vector3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector4_genmdl" nodedef="ND_image_vector4" sourcecode="mx::stdlib::mx_image_vector4({{file}}, {{layer}}, {{default}}, {{texcoord}}, {{uaddressmode}}, {{vaddressmode}}, {{filtertype}}, {{framerange}}, {{frameoffset}}, {{frameendaction}})" target="genmdl">
    <input name="default" type="vector4" implname="default_value" />
  </implementation>
  <implementation name="IM_normalmap_genmdl" nodedef="ND_normalmap" sourcecode="mx::stdlib::mx_normalmap(mxp_in:{{in}}, mxp_space:{{space}}, mxp_scale:{{scale}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}})" target="genmdl" />
  <implementation name="IM_constant_float_genmdl" nodedef="ND_constant_float" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_color3_genmdl" nodedef="ND_constant_color3" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_color4_genmdl" nodedef="ND_constant_color4" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_vector2_genmdl" nodedef="ND_constant_vector2" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_vector3_genmdl" nodedef="ND_constant_vector3" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_vector4_genmdl" nodedef="ND_constant_vector4" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_boolean_genmdl" nodedef="ND_constant_boolean" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_integer_genmdl" nodedef="ND_constant_integer" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_matrix33_genmdl" nodedef="ND_constant_matrix33" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_matrix44_genmdl" nodedef="ND_constant_matrix44" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_string_genmdl" nodedef="ND_constant_string" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_constant_filename_genmdl" nodedef="ND_constant_filename" sourcecode="{{value}}" target="genmdl" />
  <implementation name="IM_ramplr_float_genmdl" nodedef="ND_ramplr_float" sourcecode="mx::stdlib::mx_ramplr_float(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramplr_color3_genmdl" nodedef="ND_ramplr_color3" sourcecode="mx::stdlib::mx_ramplr_color3(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramplr_color4_genmdl" nodedef="ND_ramplr_color4" sourcecode="mx::stdlib::mx_ramplr_color4(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramplr_vector2_genmdl" nodedef="ND_ramplr_vector2" sourcecode="mx::stdlib::mx_ramplr_vector2(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramplr_vector3_genmdl" nodedef="ND_ramplr_vector3" sourcecode="mx::stdlib::mx_ramplr_vector3(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramplr_vector4_genmdl" nodedef="ND_ramplr_vector4" sourcecode="mx::stdlib::mx_ramplr_vector4(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramptb_float_genmdl" nodedef="ND_ramptb_float" sourcecode="mx::stdlib::mx_ramptb_float(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramptb_color3_genmdl" nodedef="ND_ramptb_color3" sourcecode="mx::stdlib::mx_ramptb_color3(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramptb_color4_genmdl" nodedef="ND_ramptb_color4" sourcecode="mx::stdlib::mx_ramptb_color4(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramptb_vector2_genmdl" nodedef="ND_ramptb_vector2" sourcecode="mx::stdlib::mx_ramptb_vector2(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramptb_vector3_genmdl" nodedef="ND_ramptb_vector3" sourcecode="mx::stdlib::mx_ramptb_vector3(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_ramptb_vector4_genmdl" nodedef="ND_ramptb_vector4" sourcecode="mx::stdlib::mx_ramptb_vector4(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splitlr_float_genmdl" nodedef="ND_splitlr_float" sourcecode="mx::stdlib::mx_splitlr_float(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splitlr_color3_genmdl" nodedef="ND_splitlr_color3" sourcecode="mx::stdlib::mx_splitlr_color3(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splitlr_color4_genmdl" nodedef="ND_splitlr_color4" sourcecode="mx::stdlib::mx_splitlr_color4(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splitlr_vector2_genmdl" nodedef="ND_splitlr_vector2" sourcecode="mx::stdlib::mx_splitlr_vector2(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splitlr_vector3_genmdl" nodedef="ND_splitlr_vector3" sourcecode="mx::stdlib::mx_splitlr_vector3(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splitlr_vector4_genmdl" nodedef="ND_splitlr_vector4" sourcecode="mx::stdlib::mx_splitlr_vector4(mxp_valuel:{{valuel}}, mxp_valuer:{{valuer}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splittb_float_genmdl" nodedef="ND_splittb_float" sourcecode="mx::stdlib::mx_splittb_float(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splittb_color3_genmdl" nodedef="ND_splittb_color3" sourcecode="mx::stdlib::mx_splittb_color3(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splittb_color4_genmdl" nodedef="ND_splittb_color4" sourcecode="mx::stdlib::mx_splittb_color4(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splittb_vector2_genmdl" nodedef="ND_splittb_vector2" sourcecode="mx::stdlib::mx_splittb_vector2(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splittb_vector3_genmdl" nodedef="ND_splittb_vector3" sourcecode="mx::stdlib::mx_splittb_vector3(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_splittb_vector4_genmdl" nodedef="ND_splittb_vector4" sourcecode="mx::stdlib::mx_splittb_vector4(mxp_valuet:{{valuet}}, mxp_valueb:{{valueb}}, mxp_center:{{center}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_float_genmdl" nodedef="ND_noise2d_float" sourcecode="mx::stdlib::mx_noise2d_float(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_color3_genmdl" nodedef="ND_noise2d_color3" sourcecode="mx::stdlib::mx_noise2d_color3(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_color4_genmdl" nodedef="ND_noise2d_color4" sourcecode="mx::stdlib::mx_noise2d_color4(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_vector2_genmdl" nodedef="ND_noise2d_vector2" sourcecode="mx::stdlib::mx_noise2d_float2(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_vector3_genmdl" nodedef="ND_noise2d_vector3" sourcecode="mx::stdlib::mx_noise2d_float3(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_vector4_genmdl" nodedef="ND_noise2d_vector4" sourcecode="mx::stdlib::mx_noise2d_float4(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_color3FA_genmdl" nodedef="ND_noise2d_color3FA" sourcecode="mx::stdlib::mx_noise2d_color3FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_color4FA_genmdl" nodedef="ND_noise2d_color4FA" sourcecode="mx::stdlib::mx_noise2d_color4FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_vector2FA_genmdl" nodedef="ND_noise2d_vector2FA" sourcecode="mx::stdlib::mx_noise2d_float2FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_vector3FA_genmdl" nodedef="ND_noise2d_vector3FA" sourcecode="mx::stdlib::mx_noise2d_float3FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise2d_vector4FA_genmdl" nodedef="ND_noise2d_vector4FA" sourcecode="mx::stdlib::mx_noise2d_float4FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_noise3d_float_genmdl" nodedef="ND_noise3d_float" sourcecode="mx::stdlib::mx_noise3d_float(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_color3_genmdl" nodedef="ND_noise3d_color3" sourcecode="mx::stdlib::mx_noise3d_color3(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_color4_genmdl" nodedef="ND_noise3d_color4" sourcecode="mx::stdlib::mx_noise3d_color4(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_vector2_genmdl" nodedef="ND_noise3d_vector2" sourcecode="mx::stdlib::mx_noise3d_float2(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_vector3_genmdl" nodedef="ND_noise3d_vector3" sourcecode="mx::stdlib::mx_noise3d_float3(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_vector4_genmdl" nodedef="ND_noise3d_vector4" sourcecode="mx::stdlib::mx_noise3d_float4(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_color3FA_genmdl" nodedef="ND_noise3d_color3FA" sourcecode="mx::stdlib::mx_noise3d_color3FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_color4FA_genmdl" nodedef="ND_noise3d_color4FA" sourcecode="mx::stdlib::mx_noise3d_color4FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_vector2FA_genmdl" nodedef="ND_noise3d_vector2FA" sourcecode="mx::stdlib::mx_noise3d_float2FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_vector3FA_genmdl" nodedef="ND_noise3d_vector3FA" sourcecode="mx::stdlib::mx_noise3d_float3FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_noise3d_vector4FA_genmdl" nodedef="ND_noise3d_vector4FA" sourcecode="mx::stdlib::mx_noise3d_float4FA(mxp_amplitude:{{amplitude}}, mxp_pivot:{{pivot}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_float_genmdl" nodedef="ND_fractal3d_float" sourcecode="mx::stdlib::mx_fractal3d_float(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_color3_genmdl" nodedef="ND_fractal3d_color3" sourcecode="mx::stdlib::mx_fractal3d_color3(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_color4_genmdl" nodedef="ND_fractal3d_color4" sourcecode="mx::stdlib::mx_fractal3d_color4(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_vector2_genmdl" nodedef="ND_fractal3d_vector2" sourcecode="mx::stdlib::mx_fractal3d_float2(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_vector3_genmdl" nodedef="ND_fractal3d_vector3" sourcecode="mx::stdlib::mx_fractal3d_float3(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_vector4_genmdl" nodedef="ND_fractal3d_vector4" sourcecode="mx::stdlib::mx_fractal3d_float4(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_color3FA_genmdl" nodedef="ND_fractal3d_color3FA" sourcecode="mx::stdlib::mx_fractal3d_color3FA(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_color4FA_genmdl" nodedef="ND_fractal3d_color4FA" sourcecode="mx::stdlib::mx_fractal3d_color4FA(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_vector2FA_genmdl" nodedef="ND_fractal3d_vector2FA" sourcecode="mx::stdlib::mx_fractal3d_float2FA(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_vector3FA_genmdl" nodedef="ND_fractal3d_vector3FA" sourcecode="mx::stdlib::mx_fractal3d_float3FA(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_fractal3d_vector4FA_genmdl" nodedef="ND_fractal3d_vector4FA" sourcecode="mx::stdlib::mx_fractal3d_float4FA(mxp_amplitude:{{amplitude}}, mxp_octaves:{{octaves}}, mxp_lacunarity:{{lacunarity}}, mxp_diminish:{{diminish}}, mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_cellnoise2d_float_genmdl" nodedef="ND_cellnoise2d_float" sourcecode="mx::stdlib::mx_cellnoise2d_float(mxp_texcoord:{{texcoord}})" target="genmdl" />
  <implementation name="IM_cellnoise3d_float_genmdl" nodedef="ND_cellnoise3d_float" sourcecode="mx::stdlib::mx_cellnoise3d_float(mxp_position:{{position}})" target="genmdl" />
  <implementation name="IM_worleynoise2d_float_genmdl" nodedef="ND_worleynoise2d_float" sourcecode="mx::stdlib::mx_worleynoise2d_float(mxp_texcoord:{{texcoord}}, mxp_jitter:{{jitter}})" target="genmdl" />
  <implementation name="IM_worleynoise2d_vector2_genmdl" nodedef="ND_worleynoise2d_vector2" sourcecode="mx::stdlib::mx_worleynoise2d_float2(mxp_texcoord:{{texcoord}}, mxp_jitter:{{jitter}})" target="genmdl" />
  <implementation name="IM_worleynoise2d_vector3_genmdl" nodedef="ND_worleynoise2d_vector3" sourcecode="mx::stdlib::mx_worleynoise2d_float3(mxp_texcoord:{{texcoord}}, mxp_jitter:{{jitter}})" target="genmdl" />
  <implementation name="IM_worleynoise3d_float_genmdl" nodedef="ND_worleynoise3d_float" sourcecode="mx::stdlib::mx_worleynoise3d_float(mxp_position:{{position}}, mxp_jitter:{{jitter}})" target="genmdl" />
  <implementation name="IM_worleynoise3d_vector2_genmdl" nodedef="ND_worleynoise3d_vector2" sourcecode="mx::stdlib::mx_worleynoise3d_float2(mxp_position:{{position}}, mxp_jitter:{{jitter}})" target="genmdl" />
  <implementation name="IM_worleynoise3d_vector3_genmdl" nodedef="ND_worleynoise3d_vector3" sourcecode="mx::stdlib::mx_worleynoise3d_float3(mxp_position:{{position}}, mxp_jitter:{{jitter}})" target="genmdl" />
  <implementation name="IM_ambientocclusion_float_genmdl" nodedef="ND_ambientocclusion_float" sourcecode="mx::stdlib::mx_ambientocclusion_float(mxp_coneangle:{{coneangle}}, mxp_maxdistance:{{maxdistance}})" target="genmdl" />
  <implementation name="IM_position_vector3_genmdl" nodedef="ND_position_vector3" sourcecode="mx::stdlib::mx_position_vector3(mxp_space:{{space}})" target="genmdl" />
  <implementation name="IM_normal_vector3_genmdl" nodedef="ND_normal_vector3" sourcecode="mx::stdlib::mx_normal_vector3(mxp_space:{{space}})" target="genmdl" />
  <implementation name="IM_tangent_vector3_genmdl" nodedef="ND_tangent_vector3" sourcecode="mx::stdlib::mx_tangent_vector3(mxp_space:{{space}}, mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_bitangent_vector3_genmdl" nodedef="ND_bitangent_vector3" sourcecode="mx::stdlib::mx_bitangent_vector3(mxp_space:{{space}}, mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_texcoord_vector2_genmdl" nodedef="ND_texcoord_vector2" sourcecode="mx::stdlib::mx_texcoord_vector2(mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_texcoord_vector3_genmdl" nodedef="ND_texcoord_vector3" sourcecode="mx::stdlib::mx_texcoord_vector3(mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_geomcolor_float_genmdl" nodedef="ND_geomcolor_float" sourcecode="mx::stdlib::mx_geomcolor_float(mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_geomcolor_color3_genmdl" nodedef="ND_geomcolor_color3" sourcecode="mx::stdlib::mx_geomcolor_color3(mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_geomcolor_color4_genmdl" nodedef="ND_geomcolor_color4" sourcecode="mx::stdlib::mx_geomcolor_color4(mxp_index:{{index}})" target="genmdl" />
  <implementation name="IM_geompropvalue_integer_genmdl" nodedef="ND_geompropvalue_integer" sourcecode="mx::stdlib::mx_geompropvalue_integer(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_boolean_genmdl" nodedef="ND_geompropvalue_boolean" sourcecode="mx::stdlib::mx_geompropvalue_boolean(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_string_genmdl" nodedef="ND_geompropvalue_string" sourcecode="mx::stdlib::mx_geompropvalue_string(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_float_genmdl" nodedef="ND_geompropvalue_float" sourcecode="mx::stdlib::mx_geompropvalue_float(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_color3_genmdl" nodedef="ND_geompropvalue_color3" sourcecode="mx::stdlib::mx_geompropvalue_color3(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_color4_genmdl" nodedef="ND_geompropvalue_color4" sourcecode="mx::stdlib::mx_geompropvalue_color4(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_vector2_genmdl" nodedef="ND_geompropvalue_vector2" sourcecode="mx::stdlib::mx_geompropvalue_vector2(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_vector3_genmdl" nodedef="ND_geompropvalue_vector3" sourcecode="mx::stdlib::mx_geompropvalue_vector3(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_geompropvalue_vector4_genmdl" nodedef="ND_geompropvalue_vector4" sourcecode="mx::stdlib::mx_geompropvalue_vector4(mxp_geomprop:{{geomprop}}, mxp_default:{{default}})" target="genmdl" />
  <implementation name="IM_frame_float_genmdl" nodedef="ND_frame_float" sourcecode="mx::stdlib::mx_frame_float()" target="genmdl" />
  <implementation name="IM_time_float_genmdl" nodedef="ND_time_float" sourcecode="mx::stdlib::mx_time_float()" target="genmdl" />
  <implementation name="IM_add_float_genmdl" nodedef="ND_add_float" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_color3_genmdl" nodedef="ND_add_color3" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_color3FA_genmdl" nodedef="ND_add_color3FA" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_color4_genmdl" nodedef="ND_add_color4" sourcecode="mx_add({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_add_color4FA_genmdl" nodedef="ND_add_color4FA" sourcecode="mx_add({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_add_vector2_genmdl" nodedef="ND_add_vector2" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_vector2FA_genmdl" nodedef="ND_add_vector2FA" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_vector3_genmdl" nodedef="ND_add_vector3" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_vector3FA_genmdl" nodedef="ND_add_vector3FA" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_vector4_genmdl" nodedef="ND_add_vector4" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_vector4FA_genmdl" nodedef="ND_add_vector4FA" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_matrix33_genmdl" nodedef="ND_add_matrix33" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_matrix33FA_genmdl" nodedef="ND_add_matrix33FA" sourcecode="{{in1}} + float3x3({{in2}})" target="genmdl" />
  <implementation name="IM_add_matrix44_genmdl" nodedef="ND_add_matrix44" sourcecode="{{in1}} + {{in2}}" target="genmdl" />
  <implementation name="IM_add_matrix44FA_genmdl" nodedef="ND_add_matrix44FA" sourcecode="{{in1}} + float4x4({{in2}})" target="genmdl" />
  <implementation name="IM_subtract_float_genmdl" nodedef="ND_subtract_float" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_color3_genmdl" nodedef="ND_subtract_color3" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_color3FA_genmdl" nodedef="ND_subtract_color3FA" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_color4_genmdl" nodedef="ND_subtract_color4" sourcecode="mx_subtract({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_subtract_color4FA_genmdl" nodedef="ND_subtract_color4FA" sourcecode="mx_subtract({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_subtract_vector2_genmdl" nodedef="ND_subtract_vector2" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_vector2FA_genmdl" nodedef="ND_subtract_vector2FA" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_vector3_genmdl" nodedef="ND_subtract_vector3" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_vector3FA_genmdl" nodedef="ND_subtract_vector3FA" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_vector4_genmdl" nodedef="ND_subtract_vector4" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_vector4FA_genmdl" nodedef="ND_subtract_vector4FA" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_matrix33_genmdl" nodedef="ND_subtract_matrix33" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_matrix33FA_genmdl" nodedef="ND_subtract_matrix33FA" sourcecode="{{in1}} - float3x3({{in2}})" target="genmdl" />
  <implementation name="IM_subtract_matrix44_genmdl" nodedef="ND_subtract_matrix44" sourcecode="{{in1}} - {{in2}}" target="genmdl" />
  <implementation name="IM_subtract_matrix44FA_genmdl" nodedef="ND_subtract_matrix44FA" sourcecode="{{in1}} - float4x4({{in2}})" target="genmdl" />
  <implementation name="IM_multiply_float_genmdl" nodedef="ND_multiply_float" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_color3_genmdl" nodedef="ND_multiply_color3" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_color3FA_genmdl" nodedef="ND_multiply_color3FA" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_color4_genmdl" nodedef="ND_multiply_color4" sourcecode="mx_multiply({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_multiply_color4FA_genmdl" nodedef="ND_multiply_color4FA" sourcecode="mx_multiply({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_multiply_vector2_genmdl" nodedef="ND_multiply_vector2" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_vector2FA_genmdl" nodedef="ND_multiply_vector2FA" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_vector3_genmdl" nodedef="ND_multiply_vector3" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_vector3FA_genmdl" nodedef="ND_multiply_vector3FA" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_vector4_genmdl" nodedef="ND_multiply_vector4" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_vector4FA_genmdl" nodedef="ND_multiply_vector4FA" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_matrix33_genmdl" nodedef="ND_multiply_matrix33" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_multiply_matrix44_genmdl" nodedef="ND_multiply_matrix44" sourcecode="{{in1}} * {{in2}}" target="genmdl" />
  <implementation name="IM_divide_float_genmdl" nodedef="ND_divide_float" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_color3_genmdl" nodedef="ND_divide_color3" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_color3FA_genmdl" nodedef="ND_divide_color3FA" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_color4_genmdl" nodedef="ND_divide_color4" sourcecode="mx_divide({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_divide_color4FA_genmdl" nodedef="ND_divide_color4FA" sourcecode="mx_divide({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_divide_vector2_genmdl" nodedef="ND_divide_vector2" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_vector2FA_genmdl" nodedef="ND_divide_vector2FA" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_vector3_genmdl" nodedef="ND_divide_vector3" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_vector3FA_genmdl" nodedef="ND_divide_vector3FA" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_vector4_genmdl" nodedef="ND_divide_vector4" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_vector4FA_genmdl" nodedef="ND_divide_vector4FA" sourcecode="{{in1}} / {{in2}}" target="genmdl" />
  <implementation name="IM_divide_matrix33_genmdl" nodedef="ND_divide_matrix33" sourcecode="{{in1}}" target="genmdl" />
  <implementation name="IM_divide_matrix44_genmdl" nodedef="ND_divide_matrix44" sourcecode="{{in1}}" target="genmdl" />
  <implementation name="IM_modulo_float_genmdl" nodedef="ND_modulo_float" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_color3_genmdl" nodedef="ND_modulo_color3" sourcecode="mx::stdlib::mx_modulo_color3({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_color3FA_genmdl" nodedef="ND_modulo_color3FA" sourcecode="mx::stdlib::mx_modulo_color3FA({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_color4_genmdl" nodedef="ND_modulo_color4" sourcecode="mx::stdlib::mx_modulo_color4({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_color4FA_genmdl" nodedef="ND_modulo_color4FA" sourcecode="mx::stdlib::mx_modulo_color4FA({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_vector2_genmdl" nodedef="ND_modulo_vector2" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_vector2FA_genmdl" nodedef="ND_modulo_vector2FA" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_vector3_genmdl" nodedef="ND_modulo_vector3" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_vector3FA_genmdl" nodedef="ND_modulo_vector3FA" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_vector4_genmdl" nodedef="ND_modulo_vector4" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_modulo_vector4FA_genmdl" nodedef="ND_modulo_vector4FA" sourcecode="mx_mod({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_invert_float_genmdl" nodedef="ND_invert_float" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_color3_genmdl" nodedef="ND_invert_color3" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_color3FA_genmdl" nodedef="ND_invert_color3FA" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_color4_genmdl" nodedef="ND_invert_color4" sourcecode="mx::stdlib::mx_invert_color4({{in}}, {{amount}})" target="genmdl" />
  <implementation name="IM_invert_color4FA_genmdl" nodedef="ND_invert_color4FA" sourcecode="mx::stdlib::mx_invert_color4FA({{in}}, {{amount}})" target="genmdl" />
  <implementation name="IM_invert_vector2_genmdl" nodedef="ND_invert_vector2" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_vector2FA_genmdl" nodedef="ND_invert_vector2FA" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_vector3_genmdl" nodedef="ND_invert_vector3" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_vector3FA_genmdl" nodedef="ND_invert_vector3FA" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_vector4_genmdl" nodedef="ND_invert_vector4" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_invert_vector4FA_genmdl" nodedef="ND_invert_vector4FA" sourcecode="{{amount}} - {{in}}" target="genmdl" />
  <implementation name="IM_absval_float_genmdl" nodedef="ND_absval_float" sourcecode="math::abs({{in}})" target="genmdl" />
  <implementation name="IM_absval_color3_genmdl" nodedef="ND_absval_color3" sourcecode="math::abs({{in}})" target="genmdl" />
  <implementation name="IM_absval_color4_genmdl" nodedef="ND_absval_color4" sourcecode="mx::stdlib::mx_absval_color4({{in}})" target="genmdl" />
  <implementation name="IM_absval_vector2_genmdl" nodedef="ND_absval_vector2" sourcecode="math::abs({{in}})" target="genmdl" />
  <implementation name="IM_absval_vector3_genmdl" nodedef="ND_absval_vector3" sourcecode="math::abs({{in}})" target="genmdl" />
  <implementation name="IM_absval_vector4_genmdl" nodedef="ND_absval_vector4" sourcecode="math::abs({{in}})" target="genmdl" />
  <implementation name="IM_floor_float_genmdl" nodedef="ND_floor_float" sourcecode="math::floor({{in}})" target="genmdl" />
  <implementation name="IM_floor_color3_genmdl" nodedef="ND_floor_color3" sourcecode="mx::stdlib::mx_floor_color3({{in}})" target="genmdl" />
  <implementation name="IM_floor_color4_genmdl" nodedef="ND_floor_color4" sourcecode="mx::stdlib::mx_floor_color4({{in}})" target="genmdl" />
  <implementation name="IM_floor_vector2_genmdl" nodedef="ND_floor_vector2" sourcecode="math::floor({{in}})" target="genmdl" />
  <implementation name="IM_floor_vector3_genmdl" nodedef="ND_floor_vector3" sourcecode="math::floor({{in}})" target="genmdl" />
  <implementation name="IM_floor_vector4_genmdl" nodedef="ND_floor_vector4" sourcecode="math::floor({{in}})" target="genmdl" />
  <implementation name="IM_floor_integer_genmdl" nodedef="ND_floor_integer" sourcecode="int(math::floor({{in}}))" target="genmdl" />
  <implementation name="IM_ceil_float_genmdl" nodedef="ND_ceil_float" sourcecode="math::ceil({{in}})" target="genmdl" />
  <implementation name="IM_ceil_color3_genmdl" nodedef="ND_ceil_color3" sourcecode="mx::stdlib::mx_ceil_color3({{in}})" target="genmdl" />
  <implementation name="IM_ceil_color4_genmdl" nodedef="ND_ceil_color4" sourcecode="mx::stdlib::mx_ceil_color4({{in}})" target="genmdl" />
  <implementation name="IM_ceil_vector2_genmdl" nodedef="ND_ceil_vector2" sourcecode="math::ceil({{in}})" target="genmdl" />
  <implementation name="IM_ceil_vector3_genmdl" nodedef="ND_ceil_vector3" sourcecode="math::ceil({{in}})" target="genmdl" />
  <implementation name="IM_ceil_vector4_genmdl" nodedef="ND_ceil_vector4" sourcecode="math::ceil({{in}})" target="genmdl" />
  <implementation name="IM_ceil_integer_genmdl" nodedef="ND_ceil_integer" sourcecode="int(math::ceil({{in}}))" target="genmdl" />
  <implementation name="IM_power_float_genmdl" nodedef="ND_power_float" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_color3_genmdl" nodedef="ND_power_color3" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_color3FA_genmdl" nodedef="ND_power_color3FA" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_color4_genmdl" nodedef="ND_power_color4" sourcecode="mx::stdlib::mx_power_color4({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_color4FA_genmdl" nodedef="ND_power_color4FA" sourcecode="mx::stdlib::mx_power_color4FA({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_vector2_genmdl" nodedef="ND_power_vector2" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_vector2FA_genmdl" nodedef="ND_power_vector2FA" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_vector3_genmdl" nodedef="ND_power_vector3" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_vector3FA_genmdl" nodedef="ND_power_vector3FA" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_vector4_genmdl" nodedef="ND_power_vector4" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_power_vector4FA_genmdl" nodedef="ND_power_vector4FA" sourcecode="math::pow({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_sin_float_genmdl" nodedef="ND_sin_float" sourcecode="math::sin({{in}})" target="genmdl" />
  <implementation name="IM_cos_float_genmdl" nodedef="ND_cos_float" sourcecode="math::cos({{in}})" target="genmdl" />
  <implementation name="IM_tan_float_genmdl" nodedef="ND_tan_float" sourcecode="math::tan({{in}})" target="genmdl" />
  <implementation name="IM_asin_float_genmdl" nodedef="ND_asin_float" sourcecode="math::asin({{in}})" target="genmdl" />
  <implementation name="IM_acos_float_genmdl" nodedef="ND_acos_float" sourcecode="math::acos({{in}})" target="genmdl" />
  <implementation name="IM_atan2_float_genmdl" nodedef="ND_atan2_float" sourcecode="math::atan2({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_sin_vector2_genmdl" nodedef="ND_sin_vector2" sourcecode="math::sin({{in}})" target="genmdl" />
  <implementation name="IM_cos_vector2_genmdl" nodedef="ND_cos_vector2" sourcecode="math::cos({{in}})" target="genmdl" />
  <implementation name="IM_tan_vector2_genmdl" nodedef="ND_tan_vector2" sourcecode="math::tan({{in}})" target="genmdl" />
  <implementation name="IM_asin_vector2_genmdl" nodedef="ND_asin_vector2" sourcecode="math::asin({{in}})" target="genmdl" />
  <implementation name="IM_acos_vector2_genmdl" nodedef="ND_acos_vector2" sourcecode="math::acos({{in}})" target="genmdl" />
  <implementation name="IM_atan2_vector2_genmdl" nodedef="ND_atan2_vector2" sourcecode="math::atan2({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_sin_vector3_genmdl" nodedef="ND_sin_vector3" sourcecode="math::sin({{in}})" target="genmdl" />
  <implementation name="IM_cos_vector3_genmdl" nodedef="ND_cos_vector3" sourcecode="math::cos({{in}})" target="genmdl" />
  <implementation name="IM_tan_vector3_genmdl" nodedef="ND_tan_vector3" sourcecode="math::tan({{in}})" target="genmdl" />
  <implementation name="IM_asin_vector3_genmdl" nodedef="ND_asin_vector3" sourcecode="math::asin({{in}})" target="genmdl" />
  <implementation name="IM_acos_vector3_genmdl" nodedef="ND_acos_vector3" sourcecode="math::acos({{in}})" target="genmdl" />
  <implementation name="IM_atan2_vector3_genmdl" nodedef="ND_atan2_vector3" sourcecode="math::atan2({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_sin_vector4_genmdl" nodedef="ND_sin_vector4" sourcecode="math::sin({{in}})" target="genmdl" />
  <implementation name="IM_cos_vector4_genmdl" nodedef="ND_cos_vector4" sourcecode="math::cos({{in}})" target="genmdl" />
  <implementation name="IM_tan_vector4_genmdl" nodedef="ND_tan_vector4" sourcecode="math::tan({{in}})" target="genmdl" />
  <implementation name="IM_asin_vector4_genmdl" nodedef="ND_asin_vector4" sourcecode="math::asin({{in}})" target="genmdl" />
  <implementation name="IM_acos_vector4_genmdl" nodedef="ND_acos_vector4" sourcecode="math::acos({{in}})" target="genmdl" />
  <implementation name="IM_atan2_vector4_genmdl" nodedef="ND_atan2_vector4" sourcecode="math::atan2({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_sqrt_float_genmdl" nodedef="ND_sqrt_float" sourcecode="math::sqrt({{in}})" target="genmdl" />
  <implementation name="IM_sqrt_vector2_genmdl" nodedef="ND_sqrt_vector2" sourcecode="math::sqrt({{in}})" target="genmdl" />
  <implementation name="IM_sqrt_vector3_genmdl" nodedef="ND_sqrt_vector3" sourcecode="math::sqrt({{in}})" target="genmdl" />
  <implementation name="IM_sqrt_vector4_genmdl" nodedef="ND_sqrt_vector4" sourcecode="math::sqrt({{in}})" target="genmdl" />
  <implementation name="IM_ln_float_genmdl" nodedef="ND_ln_float" sourcecode="math::log({{in}})" target="genmdl" />
  <implementation name="IM_ln_vector2_genmdl" nodedef="ND_ln_vector2" sourcecode="math::log({{in}})" target="genmdl" />
  <implementation name="IM_ln_vector3_genmdl" nodedef="ND_ln_vector3" sourcecode="math::log({{in}})" target="genmdl" />
  <implementation name="IM_ln_vector4_genmdl" nodedef="ND_ln_vector4" sourcecode="math::log({{in}})" target="genmdl" />
  <implementation name="IM_exp_float_genmdl" nodedef="ND_exp_float" sourcecode="math::exp({{in}})" target="genmdl" />
  <implementation name="IM_exp_vector2_genmdl" nodedef="ND_exp_vector2" sourcecode="math::exp({{in}})" target="genmdl" />
  <implementation name="IM_exp_vector3_genmdl" nodedef="ND_exp_vector3" sourcecode="math::exp({{in}})" target="genmdl" />
  <implementation name="IM_exp_vector4_genmdl" nodedef="ND_exp_vector4" sourcecode="math::exp({{in}})" target="genmdl" />
  <implementation name="IM_sign_float_genmdl" nodedef="ND_sign_float" sourcecode="math::sign({{in}})" target="genmdl" />
  <implementation name="IM_sign_color3_genmdl" nodedef="ND_sign_color3" sourcecode="mx::stdlib::mx_sign_color3({{in}})" target="genmdl" />
  <implementation name="IM_sign_color4_genmdl" nodedef="ND_sign_color4" sourcecode="mx::stdlib::mx_sign_color4({{in}})" target="genmdl" />
  <implementation name="IM_sign_vector2_genmdl" nodedef="ND_sign_vector2" sourcecode="math::sign({{in}})" target="genmdl" />
  <implementation name="IM_sign_vector3_genmdl" nodedef="ND_sign_vector3" sourcecode="math::sign({{in}})" target="genmdl" />
  <implementation name="IM_sign_vector4_genmdl" nodedef="ND_sign_vector4" sourcecode="math::sign({{in}})" target="genmdl" />
  <implementation name="IM_clamp_float_genmdl" nodedef="ND_clamp_float" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_color3_genmdl" nodedef="ND_clamp_color3" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_color3FA_genmdl" nodedef="ND_clamp_color3FA" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_color4_genmdl" nodedef="ND_clamp_color4" sourcecode="mx::stdlib::mx_clamp_color4({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_color4FA_genmdl" nodedef="ND_clamp_color4FA" sourcecode="mx::stdlib::mx_clamp_color4FA({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_vector2_genmdl" nodedef="ND_clamp_vector2" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_vector2FA_genmdl" nodedef="ND_clamp_vector2FA" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_vector3_genmdl" nodedef="ND_clamp_vector3" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_vector3FA_genmdl" nodedef="ND_clamp_vector3FA" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_vector4_genmdl" nodedef="ND_clamp_vector4" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_clamp_vector4FA_genmdl" nodedef="ND_clamp_vector4FA" sourcecode="math::clamp({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_min_float_genmdl" nodedef="ND_min_float" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_color3_genmdl" nodedef="ND_min_color3" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_color3FA_genmdl" nodedef="ND_min_color3FA" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_color4_genmdl" nodedef="ND_min_color4" sourcecode="mx::stdlib::mx_min_color4({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_color4FA_genmdl" nodedef="ND_min_color4FA" sourcecode="mx::stdlib::mx_min_color4({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_vector2_genmdl" nodedef="ND_min_vector2" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_vector2FA_genmdl" nodedef="ND_min_vector2FA" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_vector3_genmdl" nodedef="ND_min_vector3" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_vector3FA_genmdl" nodedef="ND_min_vector3FA" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_vector4_genmdl" nodedef="ND_min_vector4" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_min_vector4FA_genmdl" nodedef="ND_min_vector4FA" sourcecode="math::min({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_float_genmdl" nodedef="ND_max_float" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_color3_genmdl" nodedef="ND_max_color3" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_color3FA_genmdl" nodedef="ND_max_color3FA" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_color4_genmdl" nodedef="ND_max_color4" sourcecode="mx::stdlib::mx_max_color4({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_color4FA_genmdl" nodedef="ND_max_color4FA" sourcecode="mx::stdlib::mx_max_color4({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_vector2_genmdl" nodedef="ND_max_vector2" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_vector2FA_genmdl" nodedef="ND_max_vector2FA" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_vector3_genmdl" nodedef="ND_max_vector3" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_vector3FA_genmdl" nodedef="ND_max_vector3FA" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_vector4_genmdl" nodedef="ND_max_vector4" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_max_vector4FA_genmdl" nodedef="ND_max_vector4FA" sourcecode="math::max({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_normalize_vector2_genmdl" nodedef="ND_normalize_vector2" sourcecode="math::normalize({{in}})" target="genmdl" />
  <implementation name="IM_normalize_vector3_genmdl" nodedef="ND_normalize_vector3" sourcecode="math::normalize({{in}})" target="genmdl" />
  <implementation name="IM_normalize_vector4_genmdl" nodedef="ND_normalize_vector4" sourcecode="math::normalize({{in}})" target="genmdl" />
  <implementation name="IM_magnitude_vector2_genmdl" nodedef="ND_magnitude_vector2" sourcecode="math::length({{in}})" target="genmdl" />
  <implementation name="IM_magnitude_vector3_genmdl" nodedef="ND_magnitude_vector3" sourcecode="math::length({{in}})" target="genmdl" />
  <implementation name="IM_magnitude_vector4_genmdl" nodedef="ND_magnitude_vector4" sourcecode="math::length({{in}})" target="genmdl" />
  <implementation name="IM_dotproduct_vector2_genmdl" nodedef="ND_dotproduct_vector2" sourcecode="math::dot({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_dotproduct_vector3_genmdl" nodedef="ND_dotproduct_vector3" sourcecode="math::dot({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_dotproduct_vector4_genmdl" nodedef="ND_dotproduct_vector4" sourcecode="math::dot({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_crossproduct_vector3_genmdl" nodedef="ND_crossproduct_vector3" sourcecode="math::cross({{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_transformpoint_vector3_genmdl" nodedef="ND_transformpoint_vector3" sourcecode="mx::stdlib::mx_transformpoint_vector3({{in}}, {{fromspace}}, {{tospace}})" target="genmdl" />
  <implementation name="IM_transformvector_vector3_genmdl" nodedef="ND_transformvector_vector3" sourcecode="mx::stdlib::mx_transformvector_vector3({{in}}, {{fromspace}}, {{tospace}})" target="genmdl" />
  <implementation name="IM_transformnormal_vector3_genmdl" nodedef="ND_transformnormal_vector3" sourcecode="mx::stdlib::mx_transformnormal_vector3({{in}}, {{fromspace}}, {{tospace}})" target="genmdl" />
  <implementation name="IM_transformmatrix_vector2M3_genmdl" nodedef="ND_transformmatrix_vector2M3" sourcecode="mx::stdlib::mx_transformmatrix_vector2M3({{in}}, {{mat}})" target="genmdl" />
  <implementation name="IM_transformmatrix_vector3_genmdl" nodedef="ND_transformmatrix_vector3" sourcecode="mx::stdlib::mx_transformmatrix_vector3({{in}}, {{mat}})" target="genmdl" />
  <implementation name="IM_transformmatrix_vector3M4_genmdl" nodedef="ND_transformmatrix_vector3M4" sourcecode="mx::stdlib::mx_transformmatrix_vector3M4({{in}}, {{mat}})" target="genmdl" />
  <implementation name="IM_transformmatrix_vector4_genmdl" nodedef="ND_transformmatrix_vector4" sourcecode="mx::stdlib::mx_transformmatrix_vector4({{in}}, {{mat}})" target="genmdl" />
  <implementation name="IM_transpose_matrix33_genmdl" nodedef="ND_transpose_matrix33" sourcecode="math::transpose({{in}})" target="genmdl" />
  <implementation name="IM_transpose_matrix44_genmdl" nodedef="ND_transpose_matrix44" sourcecode="math::transpose({{in}})" target="genmdl" />
  <implementation name="IM_determinant_matrix33_genmdl" nodedef="ND_determinant_matrix33" sourcecode="mx::stdlib::mx_determinant_matrix33({{in}})" target="genmdl" />
  <implementation name="IM_determinant_matrix44_genmdl" nodedef="ND_determinant_matrix44" sourcecode="mx::stdlib::mx_determinant_matrix44({{in}})" target="genmdl" />
  <implementation name="IM_invertmatrix_matrix33_genmdl" nodedef="ND_invertmatrix_matrix33" sourcecode="mx::stdlib::mx_invertmatrix_matrix33({{in}})" target="genmdl" />
  <implementation name="IM_invertmatrix_matrix44_genmdl" nodedef="ND_invertmatrix_matrix44" sourcecode="mx::stdlib::mx_invertmatrix_matrix44({{in}})" target="genmdl" />
  <implementation name="IM_rotate2d_vector2_genmdl" nodedef="ND_rotate2d_vector2" sourcecode="mx::stdlib::mx_rotate2d_vector2(mxp_in:{{in}}, mxp_amount:{{amount}})" target="genmdl" />
  <implementation name="IM_rotate3d_vector3_genmdl" nodedef="ND_rotate3d_vector3" sourcecode="mx::stdlib::mx_rotate3d_vector3(mxp_in:{{in}}, mxp_amount:{{amount}}, mxp_axis:{{axis}})" target="genmdl" />
  <implementation name="IM_remap_float_genmdl" nodedef="ND_remap_float" sourcecode="mx::stdlib::mx_remap_float({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_color3_genmdl" nodedef="ND_remap_color3" sourcecode="mx::stdlib::mx_remap_color3({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_color4_genmdl" nodedef="ND_remap_color4" sourcecode="mx::stdlib::mx_remap_color4({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_vector2_genmdl" nodedef="ND_remap_vector2" sourcecode="mx::stdlib::mx_remap_vector2({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_vector3_genmdl" nodedef="ND_remap_vector3" sourcecode="mx::stdlib::mx_remap_vector3({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_vector4_genmdl" nodedef="ND_remap_vector4" sourcecode="mx::stdlib::mx_remap_vector4({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_color3FA_genmdl" nodedef="ND_remap_color3FA" sourcecode="mx::stdlib::mx_remap_color3FA({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_color4FA_genmdl" nodedef="ND_remap_color4FA" sourcecode="mx::stdlib::mx_remap_color4FA({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_vector2FA_genmdl" nodedef="ND_remap_vector2FA" sourcecode="mx::stdlib::mx_remap_vector2FA({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_vector3FA_genmdl" nodedef="ND_remap_vector3FA" sourcecode="mx::stdlib::mx_remap_vector3FA({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_remap_vector4FA_genmdl" nodedef="ND_remap_vector4FA" sourcecode="mx::stdlib::mx_remap_vector4FA({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}})" target="genmdl" />
  <implementation name="IM_smoothstep_float_genmdl" nodedef="ND_smoothstep_float" sourcecode="mx::stdlib::mx_smoothstep_float({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_color3_genmdl" nodedef="ND_smoothstep_color3" sourcecode="mx::stdlib::mx_smoothstep_color3({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_color4_genmdl" nodedef="ND_smoothstep_color4" sourcecode="mx::stdlib::mx_smoothstep_color4({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_vector2_genmdl" nodedef="ND_smoothstep_vector2" sourcecode="mx::stdlib::mx_smoothstep_vector2({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_vector3_genmdl" nodedef="ND_smoothstep_vector3" sourcecode="mx::stdlib::mx_smoothstep_vector3({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_vector4_genmdl" nodedef="ND_smoothstep_vector4" sourcecode="mx::stdlib::mx_smoothstep_vector4({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_color3FA_genmdl" nodedef="ND_smoothstep_color3FA" sourcecode="mx::stdlib::mx_smoothstep_color3FA({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_color4FA_genmdl" nodedef="ND_smoothstep_color4FA" sourcecode="mx::stdlib::mx_smoothstep_color4FA({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_vector2FA_genmdl" nodedef="ND_smoothstep_vector2FA" sourcecode="mx::stdlib::mx_smoothstep_vector2FA({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_vector3FA_genmdl" nodedef="ND_smoothstep_vector3FA" sourcecode="mx::stdlib::mx_smoothstep_vector3FA({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_smoothstep_vector4FA_genmdl" nodedef="ND_smoothstep_vector4FA" sourcecode="mx::stdlib::mx_smoothstep_vector4FA({{in}}, {{low}}, {{high}})" target="genmdl" />
  <implementation name="IM_luminance_color3_genmdl" nodedef="ND_luminance_color3" sourcecode="mx::stdlib::mx_luminance_color3({{in}})" target="genmdl" />
  <implementation name="IM_luminance_color4_genmdl" nodedef="ND_luminance_color4" sourcecode="mx::stdlib::mx_luminance_color4({{in}})" target="genmdl" />
  <implementation name="IM_rgbtohsv_color3_genmdl" nodedef="ND_rgbtohsv_color3" sourcecode="mx::stdlib::mx_rgbtohsv_color3({{in}})" target="genmdl" />
  <implementation name="IM_rgbtohsv_color4_genmdl" nodedef="ND_rgbtohsv_color4" sourcecode="mx::stdlib::mx_rgbtohsv_color4({{in}})" target="genmdl" />
  <implementation name="IM_hsvtorgb_color3_genmdl" nodedef="ND_hsvtorgb_color3" sourcecode="mx::stdlib::mx_hsvtorgb_color3({{in}})" target="genmdl" />
  <implementation name="IM_hsvtorgb_color4_genmdl" nodedef="ND_hsvtorgb_color4" sourcecode="mx::stdlib::mx_hsvtorgb_color4({{in}})" target="genmdl" />
  <implementation name="IM_premult_color4_genmdl" nodedef="ND_premult_color4" sourcecode="mx::stdlib::mx_premult_color4({{in}})" target="genmdl" />
  <implementation name="IM_unpremult_color4_genmdl" nodedef="ND_unpremult_color4" sourcecode="mx::stdlib::mx_unpremult_color4({{in}})" target="genmdl" />
  <implementation name="IM_plus_float_genmdl" nodedef="ND_plus_float" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_plus_color3_genmdl" nodedef="ND_plus_color3" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_plus_color4_genmdl" nodedef="ND_plus_color4" sourcecode="mx::stdlib::mx_plus_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_minus_float_genmdl" nodedef="ND_minus_float" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_minus_color3_genmdl" nodedef="ND_minus_color3" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_minus_color4_genmdl" nodedef="ND_minus_color4" sourcecode="mx::stdlib::mx_minus_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_difference_float_genmdl" nodedef="ND_difference_float" sourcecode="({{mix}}*math::abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_difference_color3_genmdl" nodedef="ND_difference_color3" sourcecode="({{mix}}*math::abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_difference_color4_genmdl" nodedef="ND_difference_color4" sourcecode="mx::stdlib::mx_difference_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_burn_float_genmdl" nodedef="ND_burn_float" sourcecode="mx::stdlib::mx_burn_float({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_burn_color3_genmdl" nodedef="ND_burn_color3" sourcecode="mx::stdlib::mx_burn_color3({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_burn_color4_genmdl" nodedef="ND_burn_color4" sourcecode="mx::stdlib::mx_burn_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_dodge_float_genmdl" nodedef="ND_dodge_float" sourcecode="mx::stdlib::mx_dodge_float({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_dodge_color3_genmdl" nodedef="ND_dodge_color3" sourcecode="mx::stdlib::mx_dodge_color3({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_dodge_color4_genmdl" nodedef="ND_dodge_color4" sourcecode="mx::stdlib::mx_dodge_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_screen_float_genmdl" nodedef="ND_screen_float" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_screen_color3_genmdl" nodedef="ND_screen_color3" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" target="genmdl" />
  <implementation name="IM_screen_color4_genmdl" nodedef="ND_screen_color4" sourcecode="mx::stdlib::mx_screen_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_overlay_float_genmdl" nodedef="ND_overlay_float" sourcecode="mx::stdlib::mx_overlay_float({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_overlay_color3_genmdl" nodedef="ND_overlay_color3" sourcecode="mx::stdlib::mx_overlay_color3({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_overlay_color4_genmdl" nodedef="ND_overlay_color4" sourcecode="mx::stdlib::mx_overlay_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_disjointover_color4_genmdl" nodedef="ND_disjointover_color4" sourcecode="mx::stdlib::mx_disjointover_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_in_color4_genmdl" nodedef="ND_in_color4" sourcecode="mx::stdlib::mx_in_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mask_color4_genmdl" nodedef="ND_mask_color4" sourcecode="mx::stdlib::mx_mask_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_matte_color4_genmdl" nodedef="ND_matte_color4" sourcecode="mx::stdlib::mx_matte_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_out_color4_genmdl" nodedef="ND_out_color4" sourcecode="mx::stdlib::mx_out_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_over_color4_genmdl" nodedef="ND_over_color4" sourcecode="mx::stdlib::mx_over_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_inside_float_genmdl" nodedef="ND_inside_float" sourcecode="{{in}} * {{mask}}" target="genmdl" />
  <implementation name="IM_inside_color3_genmdl" nodedef="ND_inside_color3" sourcecode="{{in}} * {{mask}}" target="genmdl" />
  <implementation name="IM_inside_color4_genmdl" nodedef="ND_inside_color4" sourcecode="mx_multiply({{in}}, {{mask}})" target="genmdl" />
  <implementation name="IM_outside_float_genmdl" nodedef="ND_outside_float" sourcecode="{{in}} * (1.0 - {{mask}})" target="genmdl" />
  <implementation name="IM_outside_color3_genmdl" nodedef="ND_outside_color3" sourcecode="{{in}} * (1.0 - {{mask}})" target="genmdl" />
  <implementation name="IM_outside_color4_genmdl" nodedef="ND_outside_color4" sourcecode="mx_multiply({{in}}, 1.0 - {{mask}})" target="genmdl" />
  <implementation name="IM_mix_float_genmdl" nodedef="ND_mix_float" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_color3_genmdl" nodedef="ND_mix_color3" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_color3_color3_genmdl" nodedef="ND_mix_color3_color3" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_color4_genmdl" nodedef="ND_mix_color4" sourcecode="mx::stdlib::mx_mix_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_color4_color4_genmdl" nodedef="ND_mix_color4_color4" sourcecode="mx::stdlib::mx_mix_color4_color4({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_vector2_genmdl" nodedef="ND_mix_vector2" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_vector2_vector2_genmdl" nodedef="ND_mix_vector2_vector2" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_vector3_genmdl" nodedef="ND_mix_vector3" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_vector3_vector3_genmdl" nodedef="ND_mix_vector3_vector3" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_vector4_genmdl" nodedef="ND_mix_vector4" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_vector4_vector4_genmdl" nodedef="ND_mix_vector4_vector4" sourcecode="math::lerp({{bg}}, {{fg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_mix_surfaceshader_genmdl" nodedef="ND_mix_surfaceshader" sourcecode="mx::stdlib::mx_mix_surfaceshader({{fg}}, {{bg}}, {{mix}})" target="genmdl" />
  <implementation name="IM_ifgreater_float_genmdl" nodedef="ND_ifgreater_float" sourcecode="mx::stdlib::mx_ifgreater_float({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_color3_genmdl" nodedef="ND_ifgreater_color3" sourcecode="mx::stdlib::mx_ifgreater_color3({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_color4_genmdl" nodedef="ND_ifgreater_color4" sourcecode="mx::stdlib::mx_ifgreater_color4({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_vector2_genmdl" nodedef="ND_ifgreater_vector2" sourcecode="mx::stdlib::mx_ifgreater_vector2({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_vector3_genmdl" nodedef="ND_ifgreater_vector3" sourcecode="mx::stdlib::mx_ifgreater_vector3({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_vector4_genmdl" nodedef="ND_ifgreater_vector4" sourcecode="mx::stdlib::mx_ifgreater_vector4({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_floatI_genmdl" nodedef="ND_ifgreater_floatI" sourcecode="mx::stdlib::mx_ifgreater_floatI({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_color3I_genmdl" nodedef="ND_ifgreater_color3I" sourcecode="mx::stdlib::mx_ifgreater_color3I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_color4I_genmdl" nodedef="ND_ifgreater_color4I" sourcecode="mx::stdlib::mx_ifgreater_color4I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_vector2I_genmdl" nodedef="ND_ifgreater_vector2I" sourcecode="mx::stdlib::mx_ifgreater_vector2I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_vector3I_genmdl" nodedef="ND_ifgreater_vector3I" sourcecode="mx::stdlib::mx_ifgreater_vector3I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreater_vector4I_genmdl" nodedef="ND_ifgreater_vector4I" sourcecode="mx::stdlib::mx_ifgreater_vector4I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_float_genmdl" nodedef="ND_ifgreatereq_float" sourcecode="mx::stdlib::mx_ifgreatereq_float({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_color3_genmdl" nodedef="ND_ifgreatereq_color3" sourcecode="mx::stdlib::mx_ifgreatereq_color3({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_color4_genmdl" nodedef="ND_ifgreatereq_color4" sourcecode="mx::stdlib::mx_ifgreatereq_color4({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_vector2_genmdl" nodedef="ND_ifgreatereq_vector2" sourcecode="mx::stdlib::mx_ifgreatereq_vector2({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_vector3_genmdl" nodedef="ND_ifgreatereq_vector3" sourcecode="mx::stdlib::mx_ifgreatereq_vector3({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_vector4_genmdl" nodedef="ND_ifgreatereq_vector4" sourcecode="mx::stdlib::mx_ifgreatereq_vector4({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_floatI_genmdl" nodedef="ND_ifgreatereq_floatI" sourcecode="mx::stdlib::mx_ifgreatereq_floatI({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_color3I_genmdl" nodedef="ND_ifgreatereq_color3I" sourcecode="mx::stdlib::mx_ifgreatereq_color3I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_color4I_genmdl" nodedef="ND_ifgreatereq_color4I" sourcecode="mx::stdlib::mx_ifgreatereq_color4I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_vector2I_genmdl" nodedef="ND_ifgreatereq_vector2I" sourcecode="mx::stdlib::mx_ifgreatereq_vector2I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_vector3I_genmdl" nodedef="ND_ifgreatereq_vector3I" sourcecode="mx::stdlib::mx_ifgreatereq_vector3I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifgreatereq_vector4I_genmdl" nodedef="ND_ifgreatereq_vector4I" sourcecode="mx::stdlib::mx_ifgreatereq_vector4I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_float_genmdl" nodedef="ND_ifequal_float" sourcecode="mx::stdlib::mx_ifequal_float({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_color3_genmdl" nodedef="ND_ifequal_color3" sourcecode="mx::stdlib::mx_ifequal_color3({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_color4_genmdl" nodedef="ND_ifequal_color4" sourcecode="mx::stdlib::mx_ifequal_color4({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector2_genmdl" nodedef="ND_ifequal_vector2" sourcecode="mx::stdlib::mx_ifequal_vector2({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector3_genmdl" nodedef="ND_ifequal_vector3" sourcecode="mx::stdlib::mx_ifequal_vector3({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector4_genmdl" nodedef="ND_ifequal_vector4" sourcecode="mx::stdlib::mx_ifequal_vector4({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_floatI_genmdl" nodedef="ND_ifequal_floatI" sourcecode="mx::stdlib::mx_ifequal_floatI({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_color3I_genmdl" nodedef="ND_ifequal_color3I" sourcecode="mx::stdlib::mx_ifequal_color3I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_color4I_genmdl" nodedef="ND_ifequal_color4I" sourcecode="mx::stdlib::mx_ifequal_color4I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector2I_genmdl" nodedef="ND_ifequal_vector2I" sourcecode="mx::stdlib::mx_ifequal_vector2I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector3I_genmdl" nodedef="ND_ifequal_vector3I" sourcecode="mx::stdlib::mx_ifequal_vector3I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector4I_genmdl" nodedef="ND_ifequal_vector4I" sourcecode="mx::stdlib::mx_ifequal_vector4I({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_floatB_genmdl" nodedef="ND_ifequal_floatB" sourcecode="mx::stdlib::mx_ifequal_floatB({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_color3B_genmdl" nodedef="ND_ifequal_color3B" sourcecode="mx::stdlib::mx_ifequal_color3B({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_color4B_genmdl" nodedef="ND_ifequal_color4B" sourcecode="mx::stdlib::mx_ifequal_color4B({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector2B_genmdl" nodedef="ND_ifequal_vector2B" sourcecode="mx::stdlib::mx_ifequal_vector2B({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector3B_genmdl" nodedef="ND_ifequal_vector3B" sourcecode="mx::stdlib::mx_ifequal_vector3B({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_ifequal_vector4B_genmdl" nodedef="ND_ifequal_vector4B" sourcecode="mx::stdlib::mx_ifequal_vector4B({{value1}}, {{value2}}, {{in1}}, {{in2}})" target="genmdl" />
  <implementation name="IM_switch_float_genmdl" nodedef="ND_switch_float" sourcecode="mx::stdlib::mx_switch_float({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_color3_genmdl" nodedef="ND_switch_color3" sourcecode="mx::stdlib::mx_switch_color3({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_color4_genmdl" nodedef="ND_switch_color4" sourcecode="mx::stdlib::mx_switch_color4({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_vector2_genmdl" nodedef="ND_switch_vector2" sourcecode="mx::stdlib::mx_switch_vector2({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_vector3_genmdl" nodedef="ND_switch_vector3" sourcecode="mx::stdlib::mx_switch_vector3({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_vector4_genmdl" nodedef="ND_switch_vector4" sourcecode="mx::stdlib::mx_switch_vector4({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_floatI_genmdl" nodedef="ND_switch_floatI" sourcecode="mx::stdlib::mx_switch_floatI({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_color3I_genmdl" nodedef="ND_switch_color3I" sourcecode="mx::stdlib::mx_switch_color3I({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_color4I_genmdl" nodedef="ND_switch_color4I" sourcecode="mx::stdlib::mx_switch_color4I({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_vector2I_genmdl" nodedef="ND_switch_vector2I" sourcecode="mx::stdlib::mx_switch_vector2I({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_vector3I_genmdl" nodedef="ND_switch_vector3I" sourcecode="mx::stdlib::mx_switch_vector3I({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_switch_vector4I_genmdl" nodedef="ND_switch_vector4I" sourcecode="mx::stdlib::mx_switch_vector4I({{in1}}, {{in2}}, {{in3}}, {{in4}}, {{in5}}, {{which}})" target="genmdl" />
  <implementation name="IM_convert_float_color3_genmdl" nodedef="ND_convert_float_color3" target="genmdl" />
  <implementation name="IM_convert_float_color4_genmdl" nodedef="ND_convert_float_color4" target="genmdl" />
  <implementation name="IM_convert_float_vector2_genmdl" nodedef="ND_convert_float_vector2" target="genmdl" />
  <implementation name="IM_convert_float_vector3_genmdl" nodedef="ND_convert_float_vector3" target="genmdl" />
  <implementation name="IM_convert_float_vector4_genmdl" nodedef="ND_convert_float_vector4" target="genmdl" />
  <implementation name="IM_convert_vector2_vector3_genmdl" nodedef="ND_convert_vector2_vector3" target="genmdl" />
  <implementation name="IM_convert_vector3_color3_genmdl" nodedef="ND_convert_vector3_color3" target="genmdl" />
  <implementation name="IM_convert_vector3_vector2_genmdl" nodedef="ND_convert_vector3_vector2" target="genmdl" />
  <implementation name="IM_convert_vector3_vector4_genmdl" nodedef="ND_convert_vector3_vector4" target="genmdl" />
  <implementation name="IM_convert_vector4_color4_genmdl" nodedef="ND_convert_vector4_color4" target="genmdl" />
  <implementation name="IM_convert_vector4_vector3_genmdl" nodedef="ND_convert_vector4_vector3" target="genmdl" />
  <implementation name="IM_convert_color3_vector3_genmdl" nodedef="ND_convert_color3_vector3" target="genmdl" />
  <implementation name="IM_convert_color4_vector4_genmdl" nodedef="ND_convert_color4_vector4" target="genmdl" />
  <implementation name="IM_convert_color3_color4_genmdl" nodedef="ND_convert_color3_color4" target="genmdl" />
  <implementation name="IM_convert_color4_color3_genmdl" nodedef="ND_convert_color4_color3" target="genmdl" />
  <implementation name="IM_convert_boolean_float_genmdl" nodedef="ND_convert_boolean_float" target="genmdl" />
  <implementation name="IM_convert_integer_float_genmdl" nodedef="ND_convert_integer_float" target="genmdl" />
  <implementation name="IM_swizzle_float_color3_genmdl" nodedef="ND_swizzle_float_color3" target="genmdl" />
  <implementation name="IM_swizzle_float_color4_genmdl" nodedef="ND_swizzle_float_color4" target="genmdl" />
  <implementation name="IM_swizzle_float_vector2_genmdl" nodedef="ND_swizzle_float_vector2" target="genmdl" />
  <implementation name="IM_swizzle_float_vector3_genmdl" nodedef="ND_swizzle_float_vector3" target="genmdl" />
  <implementation name="IM_swizzle_float_vector4_genmdl" nodedef="ND_swizzle_float_vector4" target="genmdl" />
  <implementation name="IM_swizzle_color3_float_genmdl" nodedef="ND_swizzle_color3_float" target="genmdl" />
  <implementation name="IM_swizzle_color3_color3_genmdl" nodedef="ND_swizzle_color3_color3" target="genmdl" />
  <implementation name="IM_swizzle_color3_color4_genmdl" nodedef="ND_swizzle_color3_color4" target="genmdl" />
  <implementation name="IM_swizzle_color3_vector2_genmdl" nodedef="ND_swizzle_color3_vector2" target="genmdl" />
  <implementation name="IM_swizzle_color3_vector3_genmdl" nodedef="ND_swizzle_color3_vector3" target="genmdl" />
  <implementation name="IM_swizzle_color3_vector4_genmdl" nodedef="ND_swizzle_color3_vector4" target="genmdl" />
  <implementation name="IM_swizzle_color4_float_genmdl" nodedef="ND_swizzle_color4_float" target="genmdl" />
  <implementation name="IM_swizzle_color4_color3_genmdl" nodedef="ND_swizzle_color4_color3" target="genmdl" />
  <implementation name="IM_swizzle_color4_color4_genmdl" nodedef="ND_swizzle_color4_color4" target="genmdl" />
  <implementation name="IM_swizzle_color4_vector2_genmdl" nodedef="ND_swizzle_color4_vector2" target="genmdl" />
  <implementation name="IM_swizzle_color4_vector3_genmdl" nodedef="ND_swizzle_color4_vector3" target="genmdl" />
  <implementation name="IM_swizzle_color4_vector4_genmdl" nodedef="ND_swizzle_color4_vector4" target="genmdl" />
  <implementation name="IM_swizzle_vector2_float_genmdl" nodedef="ND_swizzle_vector2_float" target="genmdl" />
  <implementation name="IM_swizzle_vector2_color3_genmdl" nodedef="ND_swizzle_vector2_color3" target="genmdl" />
  <implementation name="IM_swizzle_vector2_color4_genmdl" nodedef="ND_swizzle_vector2_color4" target="genmdl" />
  <implementation name="IM_swizzle_vector2_vector2_genmdl" nodedef="ND_swizzle_vector2_vector2" target="genmdl" />
  <implementation name="IM_swizzle_vector2_vector3_genmdl" nodedef="ND_swizzle_vector2_vector3" target="genmdl" />
  <implementation name="IM_swizzle_vector2_vector4_genmdl" nodedef="ND_swizzle_vector2_vector4" target="genmdl" />
  <implementation name="IM_swizzle_vector3_float_genmdl" nodedef="ND_swizzle_vector3_float" target="genmdl" />
  <implementation name="IM_swizzle_vector3_color3_genmdl" nodedef="ND_swizzle_vector3_color3" target="genmdl" />
  <implementation name="IM_swizzle_vector3_color4_genmdl" nodedef="ND_swizzle_vector3_color4" target="genmdl" />
  <implementation name="IM_swizzle_vector3_vector2_genmdl" nodedef="ND_swizzle_vector3_vector2" target="genmdl" />
  <implementation name="IM_swizzle_vector3_vector3_genmdl" nodedef="ND_swizzle_vector3_vector3" target="genmdl" />
  <implementation name="IM_swizzle_vector3_vector4_genmdl" nodedef="ND_swizzle_vector3_vector4" target="genmdl" />
  <implementation name="IM_swizzle_vector4_float_genmdl" nodedef="ND_swizzle_vector4_float" target="genmdl" />
  <implementation name="IM_swizzle_vector4_color3_genmdl" nodedef="ND_swizzle_vector4_color3" target="genmdl" />
  <implementation name="IM_swizzle_vector4_color4_genmdl" nodedef="ND_swizzle_vector4_color4" target="genmdl" />
  <implementation name="IM_swizzle_vector4_vector2_genmdl" nodedef="ND_swizzle_vector4_vector2" target="genmdl" />
  <implementation name="IM_swizzle_vector4_vector3_genmdl" nodedef="ND_swizzle_vector4_vector3" target="genmdl" />
  <implementation name="IM_swizzle_vector4_vector4_genmdl" nodedef="ND_swizzle_vector4_vector4" target="genmdl" />
  <implementation name="IM_combine2_vector2_genmdl" nodedef="ND_combine2_vector2" target="genmdl" />
  <implementation name="IM_combine2_color4CF_genmdl" nodedef="ND_combine2_color4CF" target="genmdl" />
  <implementation name="IM_combine2_vector4VF_genmdl" nodedef="ND_combine2_vector4VF" target="genmdl" />
  <implementation name="IM_combine2_vector4VV_genmdl" nodedef="ND_combine2_vector4VV" target="genmdl" />
  <implementation name="IM_combine3_color3_genmdl" nodedef="ND_combine3_color3" target="genmdl" />
  <implementation name="IM_combine3_vector3_genmdl" nodedef="ND_combine3_vector3" target="genmdl" />
  <implementation name="IM_combine4_color4_genmdl" nodedef="ND_combine4_color4" target="genmdl" />
  <implementation name="IM_combine4_vector4_genmdl" nodedef="ND_combine4_vector4" target="genmdl" />
  <implementation name="IM_blur_float_genmdl" nodedef="ND_blur_float" target="genmdl" />
  <implementation name="IM_blur_color3_genmdl" nodedef="ND_blur_color3" target="genmdl" />
  <implementation name="IM_blur_color4_genmdl" nodedef="ND_blur_color4" target="genmdl" />
  <implementation name="IM_blur_vector2_genmdl" nodedef="ND_blur_vector2" target="genmdl" />
  <implementation name="IM_blur_vector3_genmdl" nodedef="ND_blur_vector3" target="genmdl" />
  <implementation name="IM_blur_vector4_genmdl" nodedef="ND_blur_vector4" target="genmdl" />
  <implementation name="IM_heighttonormal_vector3_genmdl" nodedef="ND_heighttonormal_vector3" target="genmdl" />
  <implementation name="IM_dot_float_genmdl" nodedef="ND_dot_float" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_color3_genmdl" nodedef="ND_dot_color3" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_color4_genmdl" nodedef="ND_dot_color4" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_vector2_genmdl" nodedef="ND_dot_vector2" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_vector3_genmdl" nodedef="ND_dot_vector3" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_vector4_genmdl" nodedef="ND_dot_vector4" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_boolean_genmdl" nodedef="ND_dot_boolean" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_integer_genmdl" nodedef="ND_dot_integer" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_matrix33_genmdl" nodedef="ND_dot_matrix33" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_matrix44_genmdl" nodedef="ND_dot_matrix44" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_string_genmdl" nodedef="ND_dot_string" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_filename_genmdl" nodedef="ND_dot_filename" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_surfaceshader_genmdl" nodedef="ND_dot_surfaceshader" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_displacementshader_genmdl" nodedef="ND_dot_displacementshader" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_volumeshader_genmdl" nodedef="ND_dot_volumeshader" sourcecode="{{in}}" target="genmdl" />
  <implementation name="IM_dot_lightshader_genmdl" nodedef="ND_dot_lightshader" sourcecode="{{in}}" target="genmdl" />
  <!--Custom add implementation (mxadd)-->
  <implementation name="IM_myadd_color3_genmdl" nodedef="ND_myadd_color3" target="genmdl" sourcecode="{{in1}} + {{in2}}" />
</materialx>
```
</details>


- Source implementation added: IM_myadd_color3_genmdl


<details><summary>Implementation for target genmsl</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <implementation name="IM_surfacematerial_genmsl" nodedef="ND_surfacematerial" target="genmsl" />
  <implementation name="IM_surface_unlit_genmsl" nodedef="ND_surface_unlit" target="genmsl" />
  <implementation name="IM_image_float_genmsl" nodedef="ND_image_float" file="../genglsl/mx_image_float.glsl" function="mx_image_float" target="genmsl">
    <input name="default" type="float" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color3_genmsl" nodedef="ND_image_color3" file="../genglsl/mx_image_color3.glsl" function="mx_image_color3" target="genmsl">
    <input name="default" type="color3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color4_genmsl" nodedef="ND_image_color4" file="../genglsl/mx_image_color4.glsl" function="mx_image_color4" target="genmsl">
    <input name="default" type="color4" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector2_genmsl" nodedef="ND_image_vector2" file="../genglsl/mx_image_vector2.glsl" function="mx_image_vector2" target="genmsl">
    <input name="default" type="vector2" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector3_genmsl" nodedef="ND_image_vector3" file="../genglsl/mx_image_vector3.glsl" function="mx_image_vector3" target="genmsl">
    <input name="default" type="vector3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector4_genmsl" nodedef="ND_image_vector4" file="../genglsl/mx_image_vector4.glsl" function="mx_image_vector4" target="genmsl">
    <input name="default" type="vector4" implname="default_value" />
  </implementation>
  <implementation name="IM_normalmap_genmsl" nodedef="ND_normalmap" file="mx_normalmap.metal" function="mx_normalmap" target="genmsl" />
  <implementation name="IM_constant_float_genmsl" nodedef="ND_constant_float" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_color3_genmsl" nodedef="ND_constant_color3" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_color4_genmsl" nodedef="ND_constant_color4" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector2_genmsl" nodedef="ND_constant_vector2" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector3_genmsl" nodedef="ND_constant_vector3" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector4_genmsl" nodedef="ND_constant_vector4" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_boolean_genmsl" nodedef="ND_constant_boolean" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_integer_genmsl" nodedef="ND_constant_integer" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_matrix33_genmsl" nodedef="ND_constant_matrix33" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_matrix44_genmsl" nodedef="ND_constant_matrix44" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_string_genmsl" nodedef="ND_constant_string" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_constant_filename_genmsl" nodedef="ND_constant_filename" target="genmsl" sourcecode="{{value}}" />
  <implementation name="IM_ramplr_float_genmsl" nodedef="ND_ramplr_float" file="../genglsl/mx_ramplr_float.glsl" function="mx_ramplr_float" target="genmsl" />
  <implementation name="IM_ramplr_color3_genmsl" nodedef="ND_ramplr_color3" file="../genglsl/mx_ramplr_vector3.glsl" function="mx_ramplr_vector3" target="genmsl" />
  <implementation name="IM_ramplr_color4_genmsl" nodedef="ND_ramplr_color4" file="../genglsl/mx_ramplr_vector4.glsl" function="mx_ramplr_vector4" target="genmsl" />
  <implementation name="IM_ramplr_vector2_genmsl" nodedef="ND_ramplr_vector2" file="../genglsl/mx_ramplr_vector2.glsl" function="mx_ramplr_vector2" target="genmsl" />
  <implementation name="IM_ramplr_vector3_genmsl" nodedef="ND_ramplr_vector3" file="../genglsl/mx_ramplr_vector3.glsl" function="mx_ramplr_vector3" target="genmsl" />
  <implementation name="IM_ramplr_vector4_genmsl" nodedef="ND_ramplr_vector4" file="../genglsl/mx_ramplr_vector4.glsl" function="mx_ramplr_vector4" target="genmsl" />
  <implementation name="IM_ramptb_float_genmsl" nodedef="ND_ramptb_float" file="../genglsl/mx_ramptb_float.glsl" function="mx_ramptb_float" target="genmsl" />
  <implementation name="IM_ramptb_color3_genmsl" nodedef="ND_ramptb_color3" file="../genglsl/mx_ramptb_vector3.glsl" function="mx_ramptb_vector3" target="genmsl" />
  <implementation name="IM_ramptb_color4_genmsl" nodedef="ND_ramptb_color4" file="../genglsl/mx_ramptb_vector4.glsl" function="mx_ramptb_vector4" target="genmsl" />
  <implementation name="IM_ramptb_vector2_genmsl" nodedef="ND_ramptb_vector2" file="../genglsl/mx_ramptb_vector2.glsl" function="mx_ramptb_vector2" target="genmsl" />
  <implementation name="IM_ramptb_vector3_genmsl" nodedef="ND_ramptb_vector3" file="../genglsl/mx_ramptb_vector3.glsl" function="mx_ramptb_vector3" target="genmsl" />
  <implementation name="IM_ramptb_vector4_genmsl" nodedef="ND_ramptb_vector4" file="../genglsl/mx_ramptb_vector4.glsl" function="mx_ramptb_vector4" target="genmsl" />
  <implementation name="IM_splitlr_float_genmsl" nodedef="ND_splitlr_float" file="../genglsl/mx_splitlr_float.glsl" function="mx_splitlr_float" target="genmsl" />
  <implementation name="IM_splitlr_color3_genmsl" nodedef="ND_splitlr_color3" file="../genglsl/mx_splitlr_vector3.glsl" function="mx_splitlr_vector3" target="genmsl" />
  <implementation name="IM_splitlr_color4_genmsl" nodedef="ND_splitlr_color4" file="../genglsl/mx_splitlr_vector4.glsl" function="mx_splitlr_vector4" target="genmsl" />
  <implementation name="IM_splitlr_vector2_genmsl" nodedef="ND_splitlr_vector2" file="../genglsl/mx_splitlr_vector2.glsl" function="mx_splitlr_vector2" target="genmsl" />
  <implementation name="IM_splitlr_vector3_genmsl" nodedef="ND_splitlr_vector3" file="../genglsl/mx_splitlr_vector3.glsl" function="mx_splitlr_vector3" target="genmsl" />
  <implementation name="IM_splitlr_vector4_genmsl" nodedef="ND_splitlr_vector4" file="../genglsl/mx_splitlr_vector4.glsl" function="mx_splitlr_vector4" target="genmsl" />
  <implementation name="IM_splittb_float_genmsl" nodedef="ND_splittb_float" file="../genglsl/mx_splittb_float.glsl" function="mx_splittb_float" target="genmsl" />
  <implementation name="IM_splittb_color3_genmsl" nodedef="ND_splittb_color3" file="../genglsl/mx_splittb_vector3.glsl" function="mx_splittb_vector3" target="genmsl" />
  <implementation name="IM_splittb_color4_genmsl" nodedef="ND_splittb_color4" file="../genglsl/mx_splittb_vector4.glsl" function="mx_splittb_vector4" target="genmsl" />
  <implementation name="IM_splittb_vector2_genmsl" nodedef="ND_splittb_vector2" file="../genglsl/mx_splittb_vector2.glsl" function="mx_splittb_vector2" target="genmsl" />
  <implementation name="IM_splittb_vector3_genmsl" nodedef="ND_splittb_vector3" file="../genglsl/mx_splittb_vector3.glsl" function="mx_splittb_vector3" target="genmsl" />
  <implementation name="IM_splittb_vector4_genmsl" nodedef="ND_splittb_vector4" file="../genglsl/mx_splittb_vector4.glsl" function="mx_splittb_vector4" target="genmsl" />
  <implementation name="IM_noise2d_float_genmsl" nodedef="ND_noise2d_float" file="../genglsl/mx_noise2d_float.glsl" function="mx_noise2d_float" target="genmsl" />
  <implementation name="IM_noise2d_color3_genmsl" nodedef="ND_noise2d_color3" file="../genglsl/mx_noise2d_vector3.glsl" function="mx_noise2d_vector3" target="genmsl" />
  <implementation name="IM_noise2d_color4_genmsl" nodedef="ND_noise2d_color4" file="../genglsl/mx_noise2d_vector4.glsl" function="mx_noise2d_vector4" target="genmsl" />
  <implementation name="IM_noise2d_color3FA_genmsl" nodedef="ND_noise2d_color3FA" file="../genglsl/mx_noise2d_fa_vector3.glsl" function="mx_noise2d_fa_vector3" target="genmsl" />
  <implementation name="IM_noise2d_color4FA_genmsl" nodedef="ND_noise2d_color4FA" file="../genglsl/mx_noise2d_fa_vector4.glsl" function="mx_noise2d_fa_vector4" target="genmsl" />
  <implementation name="IM_noise2d_vector2_genmsl" nodedef="ND_noise2d_vector2" file="../genglsl/mx_noise2d_vector2.glsl" function="mx_noise2d_vector2" target="genmsl" />
  <implementation name="IM_noise2d_vector3_genmsl" nodedef="ND_noise2d_vector3" file="../genglsl/mx_noise2d_vector3.glsl" function="mx_noise2d_vector3" target="genmsl" />
  <implementation name="IM_noise2d_vector4_genmsl" nodedef="ND_noise2d_vector4" file="../genglsl/mx_noise2d_vector4.glsl" function="mx_noise2d_vector4" target="genmsl" />
  <implementation name="IM_noise2d_vector2FA_genmsl" nodedef="ND_noise2d_vector2FA" file="../genglsl/mx_noise2d_fa_vector2.glsl" function="mx_noise2d_fa_vector2" target="genmsl" />
  <implementation name="IM_noise2d_vector3FA_genmsl" nodedef="ND_noise2d_vector3FA" file="../genglsl/mx_noise2d_fa_vector3.glsl" function="mx_noise2d_fa_vector3" target="genmsl" />
  <implementation name="IM_noise2d_vector4FA_genmsl" nodedef="ND_noise2d_vector4FA" file="../genglsl/mx_noise2d_fa_vector4.glsl" function="mx_noise2d_fa_vector4" target="genmsl" />
  <implementation name="IM_noise3d_float_genmsl" nodedef="ND_noise3d_float" file="../genglsl/mx_noise3d_float.glsl" function="mx_noise3d_float" target="genmsl" />
  <implementation name="IM_noise3d_color3_genmsl" nodedef="ND_noise3d_color3" file="../genglsl/mx_noise3d_vector3.glsl" function="mx_noise3d_vector3" target="genmsl" />
  <implementation name="IM_noise3d_color4_genmsl" nodedef="ND_noise3d_color4" file="../genglsl/mx_noise3d_vector4.glsl" function="mx_noise3d_vector4" target="genmsl" />
  <implementation name="IM_noise3d_color3FA_genmsl" nodedef="ND_noise3d_color3FA" file="../genglsl/mx_noise3d_fa_vector3.glsl" function="mx_noise3d_fa_vector3" target="genmsl" />
  <implementation name="IM_noise3d_color4FA_genmsl" nodedef="ND_noise3d_color4FA" file="../genglsl/mx_noise3d_fa_vector4.glsl" function="mx_noise3d_fa_vector4" target="genmsl" />
  <implementation name="IM_noise3d_vector2_genmsl" nodedef="ND_noise3d_vector2" file="../genglsl/mx_noise3d_vector2.glsl" function="mx_noise3d_vector2" target="genmsl" />
  <implementation name="IM_noise3d_vector3_genmsl" nodedef="ND_noise3d_vector3" file="../genglsl/mx_noise3d_vector3.glsl" function="mx_noise3d_vector3" target="genmsl" />
  <implementation name="IM_noise3d_vector4_genmsl" nodedef="ND_noise3d_vector4" file="../genglsl/mx_noise3d_vector4.glsl" function="mx_noise3d_vector4" target="genmsl" />
  <implementation name="IM_noise3d_vector2FA_genmsl" nodedef="ND_noise3d_vector2FA" file="../genglsl/mx_noise3d_fa_vector2.glsl" function="mx_noise3d_fa_vector2" target="genmsl" />
  <implementation name="IM_noise3d_vector3FA_genmsl" nodedef="ND_noise3d_vector3FA" file="../genglsl/mx_noise3d_fa_vector3.glsl" function="mx_noise3d_fa_vector3" target="genmsl" />
  <implementation name="IM_noise3d_vector4FA_genmsl" nodedef="ND_noise3d_vector4FA" file="../genglsl/mx_noise3d_fa_vector4.glsl" function="mx_noise3d_fa_vector4" target="genmsl" />
  <implementation name="IM_fractal3d_float_genmsl" nodedef="ND_fractal3d_float" file="../genglsl/mx_fractal3d_float.glsl" function="mx_fractal3d_float" target="genmsl" />
  <implementation name="IM_fractal3d_color3_genmsl" nodedef="ND_fractal3d_color3" file="../genglsl/mx_fractal3d_vector3.glsl" function="mx_fractal3d_vector3" target="genmsl" />
  <implementation name="IM_fractal3d_color4_genmsl" nodedef="ND_fractal3d_color4" file="../genglsl/mx_fractal3d_vector4.glsl" function="mx_fractal3d_vector4" target="genmsl" />
  <implementation name="IM_fractal3d_color3FA_genmsl" nodedef="ND_fractal3d_color3FA" file="../genglsl/mx_fractal3d_fa_vector3.glsl" function="mx_fractal3d_fa_vector3" target="genmsl" />
  <implementation name="IM_fractal3d_color4FA_genmsl" nodedef="ND_fractal3d_color4FA" file="../genglsl/mx_fractal3d_fa_vector4.glsl" function="mx_fractal3d_fa_vector4" target="genmsl" />
  <implementation name="IM_fractal3d_vector2_genmsl" nodedef="ND_fractal3d_vector2" file="../genglsl/mx_fractal3d_vector2.glsl" function="mx_fractal3d_vector2" target="genmsl" />
  <implementation name="IM_fractal3d_vector3_genmsl" nodedef="ND_fractal3d_vector3" file="../genglsl/mx_fractal3d_vector3.glsl" function="mx_fractal3d_vector3" target="genmsl" />
  <implementation name="IM_fractal3d_vector4_genmsl" nodedef="ND_fractal3d_vector4" file="../genglsl/mx_fractal3d_vector4.glsl" function="mx_fractal3d_vector4" target="genmsl" />
  <implementation name="IM_fractal3d_vector2FA_genmsl" nodedef="ND_fractal3d_vector2FA" file="../genglsl/mx_fractal3d_fa_vector2.glsl" function="mx_fractal3d_fa_vector2" target="genmsl" />
  <implementation name="IM_fractal3d_vector3FA_genmsl" nodedef="ND_fractal3d_vector3FA" file="../genglsl/mx_fractal3d_fa_vector3.glsl" function="mx_fractal3d_fa_vector3" target="genmsl" />
  <implementation name="IM_fractal3d_vector4FA_genmsl" nodedef="ND_fractal3d_vector4FA" file="../genglsl/mx_fractal3d_fa_vector4.glsl" function="mx_fractal3d_fa_vector4" target="genmsl" />
  <implementation name="IM_cellnoise2d_float_genmsl" nodedef="ND_cellnoise2d_float" file="../genglsl/mx_cellnoise2d_float.glsl" function="mx_cellnoise2d_float" target="genmsl" />
  <implementation name="IM_cellnoise3d_float_genmsl" nodedef="ND_cellnoise3d_float" file="../genglsl/mx_cellnoise3d_float.glsl" function="mx_cellnoise3d_float" target="genmsl" />
  <implementation name="IM_worleynoise2d_float_genmsl" nodedef="ND_worleynoise2d_float" file="../genglsl/mx_worleynoise2d_float.glsl" function="mx_worleynoise2d_float" target="genmsl" />
  <implementation name="IM_worleynoise2d_vector2_genmsl" nodedef="ND_worleynoise2d_vector2" file="../genglsl/mx_worleynoise2d_vector2.glsl" function="mx_worleynoise2d_vector2" target="genmsl" />
  <implementation name="IM_worleynoise2d_vector3_genmsl" nodedef="ND_worleynoise2d_vector3" file="../genglsl/mx_worleynoise2d_vector3.glsl" function="mx_worleynoise2d_vector3" target="genmsl" />
  <implementation name="IM_worleynoise3d_float_genmsl" nodedef="ND_worleynoise3d_float" file="../genglsl/mx_worleynoise3d_float.glsl" function="mx_worleynoise3d_float" target="genmsl" />
  <implementation name="IM_worleynoise3d_vector2_genmsl" nodedef="ND_worleynoise3d_vector2" file="../genglsl/mx_worleynoise3d_vector2.glsl" function="mx_worleynoise3d_vector2" target="genmsl" />
  <implementation name="IM_worleynoise3d_vector3_genmsl" nodedef="ND_worleynoise3d_vector3" file="../genglsl/mx_worleynoise3d_vector3.glsl" function="mx_worleynoise3d_vector3" target="genmsl" />
  <implementation name="IM_position_vector3_genmsl" nodedef="ND_position_vector3" target="genmsl" />
  <implementation name="IM_normal_vector3_genmsl" nodedef="ND_normal_vector3" target="genmsl" />
  <implementation name="IM_tangent_vector3_genmsl" nodedef="ND_tangent_vector3" target="genmsl" />
  <implementation name="IM_bitangent_vector3_genmsl" nodedef="ND_bitangent_vector3" target="genmsl" />
  <implementation name="IM_texcoord_vector2_genmsl" nodedef="ND_texcoord_vector2" target="genmsl" />
  <implementation name="IM_texcoord_vector3_genmsl" nodedef="ND_texcoord_vector3" target="genmsl" />
  <implementation name="IM_geomcolor_float_genmsl" nodedef="ND_geomcolor_float" target="genmsl" />
  <implementation name="IM_geomcolor_color3_genmsl" nodedef="ND_geomcolor_color3" target="genmsl" />
  <implementation name="IM_geomcolor_color4_genmsl" nodedef="ND_geomcolor_color4" target="genmsl" />
  <implementation name="IM_geompropvalue_integer_genmsl" nodedef="ND_geompropvalue_integer" function="mx_geompropvalue_int" target="genmsl" />
  <implementation name="IM_geompropvalue_boolean_genmsl" nodedef="ND_geompropvalue_boolean" function="mx_geompropvalue_bool" target="genmsl" />
  <implementation name="IM_geompropvalue_string_genmsl" nodedef="ND_geompropvalue_string" function="mx_geompropvalue_string" target="genmsl" />
  <implementation name="IM_geompropvalue_float_genmsl" nodedef="ND_geompropvalue_float" function="mx_geompropvalue_float" target="genmsl" />
  <implementation name="IM_geompropvalue_color3_genmsl" nodedef="ND_geompropvalue_color3" function="mx_geompropvalue_color" target="genmsl" />
  <implementation name="IM_geompropvalue_color4_genmsl" nodedef="ND_geompropvalue_color4" function="mx_geompropvalue_color4" target="genmsl" />
  <implementation name="IM_geompropvalue_vector2_genmsl" nodedef="ND_geompropvalue_vector2" function="mx_geompropvalue_vector2" target="genmsl" />
  <implementation name="IM_geompropvalue_vector3_genmsl" nodedef="ND_geompropvalue_vector3" function="mx_geompropvalue_vector" target="genmsl" />
  <implementation name="IM_geompropvalue_vector4_genmsl" nodedef="ND_geompropvalue_vector4" function="mx_geompropvalue_vector4" target="genmsl" />
  <implementation name="IM_frame_float_genmsl" nodedef="ND_frame_float" function="mx_frame_float" target="genmsl" />
  <implementation name="IM_time_float_genmsl" nodedef="ND_time_float" function="mx_time_float" target="genmsl" />
  <implementation name="IM_add_float_genmsl" nodedef="ND_add_float" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color3_genmsl" nodedef="ND_add_color3" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color3FA_genmsl" nodedef="ND_add_color3FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color4_genmsl" nodedef="ND_add_color4" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color4FA_genmsl" nodedef="ND_add_color4FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector2_genmsl" nodedef="ND_add_vector2" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector2FA_genmsl" nodedef="ND_add_vector2FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector3_genmsl" nodedef="ND_add_vector3" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector3FA_genmsl" nodedef="ND_add_vector3FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector4_genmsl" nodedef="ND_add_vector4" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector4FA_genmsl" nodedef="ND_add_vector4FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix33_genmsl" nodedef="ND_add_matrix33" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix33FA_genmsl" nodedef="ND_add_matrix33FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix44_genmsl" nodedef="ND_add_matrix44" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix44FA_genmsl" nodedef="ND_add_matrix44FA" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_subtract_float_genmsl" nodedef="ND_subtract_float" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color3_genmsl" nodedef="ND_subtract_color3" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color3FA_genmsl" nodedef="ND_subtract_color3FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color4_genmsl" nodedef="ND_subtract_color4" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color4FA_genmsl" nodedef="ND_subtract_color4FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector2_genmsl" nodedef="ND_subtract_vector2" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector2FA_genmsl" nodedef="ND_subtract_vector2FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector3_genmsl" nodedef="ND_subtract_vector3" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector3FA_genmsl" nodedef="ND_subtract_vector3FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector4_genmsl" nodedef="ND_subtract_vector4" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector4FA_genmsl" nodedef="ND_subtract_vector4FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix33_genmsl" nodedef="ND_subtract_matrix33" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix33FA_genmsl" nodedef="ND_subtract_matrix33FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix44_genmsl" nodedef="ND_subtract_matrix44" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix44FA_genmsl" nodedef="ND_subtract_matrix44FA" target="genmsl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_multiply_float_genmsl" nodedef="ND_multiply_float" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color3_genmsl" nodedef="ND_multiply_color3" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color3FA_genmsl" nodedef="ND_multiply_color3FA" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color4_genmsl" nodedef="ND_multiply_color4" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color4FA_genmsl" nodedef="ND_multiply_color4FA" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector2_genmsl" nodedef="ND_multiply_vector2" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector2FA_genmsl" nodedef="ND_multiply_vector2FA" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector3_genmsl" nodedef="ND_multiply_vector3" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector3FA_genmsl" nodedef="ND_multiply_vector3FA" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector4_genmsl" nodedef="ND_multiply_vector4" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector4FA_genmsl" nodedef="ND_multiply_vector4FA" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_matrix33_genmsl" nodedef="ND_multiply_matrix33" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_matrix44_genmsl" nodedef="ND_multiply_matrix44" target="genmsl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_divide_float_genmsl" nodedef="ND_divide_float" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color3_genmsl" nodedef="ND_divide_color3" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color3FA_genmsl" nodedef="ND_divide_color3FA" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color4_genmsl" nodedef="ND_divide_color4" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color4FA_genmsl" nodedef="ND_divide_color4FA" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector2_genmsl" nodedef="ND_divide_vector2" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector2FA_genmsl" nodedef="ND_divide_vector2FA" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector3_genmsl" nodedef="ND_divide_vector3" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector3FA_genmsl" nodedef="ND_divide_vector3FA" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector4_genmsl" nodedef="ND_divide_vector4" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector4FA_genmsl" nodedef="ND_divide_vector4FA" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_matrix33_genmsl" nodedef="ND_divide_matrix33" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_matrix44_genmsl" nodedef="ND_divide_matrix44" target="genmsl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_modulo_float_genmsl" nodedef="ND_modulo_float" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color3_genmsl" nodedef="ND_modulo_color3" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color3FA_genmsl" nodedef="ND_modulo_color3FA" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color4_genmsl" nodedef="ND_modulo_color4" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color4FA_genmsl" nodedef="ND_modulo_color4FA" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector2_genmsl" nodedef="ND_modulo_vector2" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector2FA_genmsl" nodedef="ND_modulo_vector2FA" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector3_genmsl" nodedef="ND_modulo_vector3" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector3FA_genmsl" nodedef="ND_modulo_vector3FA" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector4_genmsl" nodedef="ND_modulo_vector4" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector4FA_genmsl" nodedef="ND_modulo_vector4FA" target="genmsl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_invert_float_genmsl" nodedef="ND_invert_float" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color3_genmsl" nodedef="ND_invert_color3" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color3FA_genmsl" nodedef="ND_invert_color3FA" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color4_genmsl" nodedef="ND_invert_color4" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color4FA_genmsl" nodedef="ND_invert_color4FA" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector2_genmsl" nodedef="ND_invert_vector2" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector2FA_genmsl" nodedef="ND_invert_vector2FA" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector3_genmsl" nodedef="ND_invert_vector3" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector3FA_genmsl" nodedef="ND_invert_vector3FA" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector4_genmsl" nodedef="ND_invert_vector4" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector4FA_genmsl" nodedef="ND_invert_vector4FA" target="genmsl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_absval_float_genmsl" nodedef="ND_absval_float" target="genmsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_color3_genmsl" nodedef="ND_absval_color3" target="genmsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_color4_genmsl" nodedef="ND_absval_color4" target="genmsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector2_genmsl" nodedef="ND_absval_vector2" target="genmsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector3_genmsl" nodedef="ND_absval_vector3" target="genmsl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector4_genmsl" nodedef="ND_absval_vector4" target="genmsl" sourcecode="abs({{in}})" />
  <implementation name="IM_floor_float_genmsl" nodedef="ND_floor_float" target="genmsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_color3_genmsl" nodedef="ND_floor_color3" target="genmsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_color4_genmsl" nodedef="ND_floor_color4" target="genmsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector2_genmsl" nodedef="ND_floor_vector2" target="genmsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector3_genmsl" nodedef="ND_floor_vector3" target="genmsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector4_genmsl" nodedef="ND_floor_vector4" target="genmsl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_integer_genmsl" nodedef="ND_floor_integer" target="genmsl" sourcecode="int(floor({{in}}))" />
  <implementation name="IM_ceil_float_genmsl" nodedef="ND_ceil_float" target="genmsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_color3_genmsl" nodedef="ND_ceil_color3" target="genmsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_color4_genmsl" nodedef="ND_ceil_color4" target="genmsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector2_genmsl" nodedef="ND_ceil_vector2" target="genmsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector3_genmsl" nodedef="ND_ceil_vector3" target="genmsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector4_genmsl" nodedef="ND_ceil_vector4" target="genmsl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_integer_genmsl" nodedef="ND_ceil_integer" target="genmsl" sourcecode="int(ceil({{in}}))" />
  <implementation name="IM_power_float_genmsl" nodedef="ND_power_float" target="genmsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color3_genmsl" nodedef="ND_power_color3" target="genmsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color3FA_genmsl" nodedef="ND_power_color3FA" target="genmsl" sourcecode="pow({{in1}}, vec3({{in2}}))" />
  <implementation name="IM_power_color4_genmsl" nodedef="ND_power_color4" target="genmsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color4FA_genmsl" nodedef="ND_power_color4FA" target="genmsl" sourcecode="pow({{in1}}, vec4({{in2}}))" />
  <implementation name="IM_power_vector2_genmsl" nodedef="ND_power_vector2" target="genmsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector2FA_genmsl" nodedef="ND_power_vector2FA" target="genmsl" sourcecode="pow({{in1}}, vec2({{in2}}))" />
  <implementation name="IM_power_vector3_genmsl" nodedef="ND_power_vector3" target="genmsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector3FA_genmsl" nodedef="ND_power_vector3FA" target="genmsl" sourcecode="pow({{in1}}, vec3({{in2}}))" />
  <implementation name="IM_power_vector4_genmsl" nodedef="ND_power_vector4" target="genmsl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector4FA_genmsl" nodedef="ND_power_vector4FA" target="genmsl" sourcecode="pow({{in1}}, vec4({{in2}}))" />
  <implementation name="IM_sin_float_genmsl" nodedef="ND_sin_float" target="genmsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_float_genmsl" nodedef="ND_cos_float" target="genmsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_float_genmsl" nodedef="ND_tan_float" target="genmsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_float_genmsl" nodedef="ND_asin_float" target="genmsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_float_genmsl" nodedef="ND_acos_float" target="genmsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_float_genmsl" nodedef="ND_atan2_float" target="genmsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sin_vector2_genmsl" nodedef="ND_sin_vector2" target="genmsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector2_genmsl" nodedef="ND_cos_vector2" target="genmsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector2_genmsl" nodedef="ND_tan_vector2" target="genmsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector2_genmsl" nodedef="ND_asin_vector2" target="genmsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector2_genmsl" nodedef="ND_acos_vector2" target="genmsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector2_genmsl" nodedef="ND_atan2_vector2" target="genmsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sin_vector3_genmsl" nodedef="ND_sin_vector3" target="genmsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector3_genmsl" nodedef="ND_cos_vector3" target="genmsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector3_genmsl" nodedef="ND_tan_vector3" target="genmsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector3_genmsl" nodedef="ND_asin_vector3" target="genmsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector3_genmsl" nodedef="ND_acos_vector3" target="genmsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector3_genmsl" nodedef="ND_atan2_vector3" target="genmsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sin_vector4_genmsl" nodedef="ND_sin_vector4" target="genmsl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector4_genmsl" nodedef="ND_cos_vector4" target="genmsl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector4_genmsl" nodedef="ND_tan_vector4" target="genmsl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector4_genmsl" nodedef="ND_asin_vector4" target="genmsl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector4_genmsl" nodedef="ND_acos_vector4" target="genmsl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector4_genmsl" nodedef="ND_atan2_vector4" target="genmsl" sourcecode="atan({{in1}}, {{in2}})" />
  <implementation name="IM_sqrt_float_genmsl" nodedef="ND_sqrt_float" target="genmsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector2_genmsl" nodedef="ND_sqrt_vector2" target="genmsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector3_genmsl" nodedef="ND_sqrt_vector3" target="genmsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector4_genmsl" nodedef="ND_sqrt_vector4" target="genmsl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_ln_float_genmsl" nodedef="ND_ln_float" target="genmsl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector2_genmsl" nodedef="ND_ln_vector2" target="genmsl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector3_genmsl" nodedef="ND_ln_vector3" target="genmsl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector4_genmsl" nodedef="ND_ln_vector4" target="genmsl" sourcecode="log({{in}})" />
  <implementation name="IM_exp_float_genmsl" nodedef="ND_exp_float" target="genmsl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector2_genmsl" nodedef="ND_exp_vector2" target="genmsl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector3_genmsl" nodedef="ND_exp_vector3" target="genmsl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector4_genmsl" nodedef="ND_exp_vector4" target="genmsl" sourcecode="exp({{in}})" />
  <implementation name="IM_sign_float_genmsl" nodedef="ND_sign_float" target="genmsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_color3_genmsl" nodedef="ND_sign_color3" target="genmsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_color4_genmsl" nodedef="ND_sign_color4" target="genmsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector2_genmsl" nodedef="ND_sign_vector2" target="genmsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector3_genmsl" nodedef="ND_sign_vector3" target="genmsl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector4_genmsl" nodedef="ND_sign_vector4" target="genmsl" sourcecode="sign({{in}})" />
  <implementation name="IM_clamp_float_genmsl" nodedef="ND_clamp_float" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color3_genmsl" nodedef="ND_clamp_color3" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color3FA_genmsl" nodedef="ND_clamp_color3FA" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color4_genmsl" nodedef="ND_clamp_color4" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color4FA_genmsl" nodedef="ND_clamp_color4FA" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector2_genmsl" nodedef="ND_clamp_vector2" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector2FA_genmsl" nodedef="ND_clamp_vector2FA" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector3_genmsl" nodedef="ND_clamp_vector3" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector3FA_genmsl" nodedef="ND_clamp_vector3FA" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector4_genmsl" nodedef="ND_clamp_vector4" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector4FA_genmsl" nodedef="ND_clamp_vector4FA" target="genmsl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_min_float_genmsl" nodedef="ND_min_float" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color3_genmsl" nodedef="ND_min_color3" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color3FA_genmsl" nodedef="ND_min_color3FA" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color4_genmsl" nodedef="ND_min_color4" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color4FA_genmsl" nodedef="ND_min_color4FA" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector2_genmsl" nodedef="ND_min_vector2" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector2FA_genmsl" nodedef="ND_min_vector2FA" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector3_genmsl" nodedef="ND_min_vector3" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector3FA_genmsl" nodedef="ND_min_vector3FA" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector4_genmsl" nodedef="ND_min_vector4" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector4FA_genmsl" nodedef="ND_min_vector4FA" target="genmsl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_max_float_genmsl" nodedef="ND_max_float" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color3_genmsl" nodedef="ND_max_color3" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color3FA_genmsl" nodedef="ND_max_color3FA" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color4_genmsl" nodedef="ND_max_color4" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color4FA_genmsl" nodedef="ND_max_color4FA" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector2_genmsl" nodedef="ND_max_vector2" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector2FA_genmsl" nodedef="ND_max_vector2FA" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector3_genmsl" nodedef="ND_max_vector3" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector3FA_genmsl" nodedef="ND_max_vector3FA" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector4_genmsl" nodedef="ND_max_vector4" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector4FA_genmsl" nodedef="ND_max_vector4FA" target="genmsl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_normalize_vector2_genmsl" nodedef="ND_normalize_vector2" target="genmsl" sourcecode="normalize({{in}})" />
  <implementation name="IM_normalize_vector3_genmsl" nodedef="ND_normalize_vector3" target="genmsl" sourcecode="normalize({{in}})" />
  <implementation name="IM_normalize_vector4_genmsl" nodedef="ND_normalize_vector4" target="genmsl" sourcecode="normalize({{in}})" />
  <implementation name="IM_magnitude_vector2_genmsl" nodedef="ND_magnitude_vector2" target="genmsl" sourcecode="length({{in}})" />
  <implementation name="IM_magnitude_vector3_genmsl" nodedef="ND_magnitude_vector3" target="genmsl" sourcecode="length({{in}})" />
  <implementation name="IM_magnitude_vector4_genmsl" nodedef="ND_magnitude_vector4" target="genmsl" sourcecode="length({{in}})" />
  <implementation name="IM_dotproduct_vector2_genmsl" nodedef="ND_dotproduct_vector2" target="genmsl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_dotproduct_vector3_genmsl" nodedef="ND_dotproduct_vector3" target="genmsl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_dotproduct_vector4_genmsl" nodedef="ND_dotproduct_vector4" target="genmsl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_crossproduct_vector3_genmsl" nodedef="ND_crossproduct_vector3" target="genmsl" sourcecode="cross({{in1}}, {{in2}})" />
  <implementation name="IM_transformpoint_vector3_genmsl" nodedef="ND_transformpoint_vector3" target="genmsl" />
  <implementation name="IM_transformvector_vector3_genmsl" nodedef="ND_transformvector_vector3" target="genmsl" />
  <implementation name="IM_transformnormal_vector3_genmsl" nodedef="ND_transformnormal_vector3" target="genmsl" />
  <implementation name="IM_transformmatrix_vector2M3_genmsl" nodedef="ND_transformmatrix_vector2M3" function="mx_transformmatrix_vector2M3" file="../genglsl/mx_transformmatrix_vector2M3.glsl" target="genmsl" />
  <implementation name="IM_transformmatrix_vector3_genmsl" nodedef="ND_transformmatrix_vector3" target="genmsl" sourcecode="{{mat}} * {{in}}" />
  <implementation name="IM_transformmatrix_vector3M4_genmsl" nodedef="ND_transformmatrix_vector3M4" function="mx_transformmatrix_vector3M4" file="../genglsl/mx_transformmatrix_vector3M4.glsl" target="genmsl" />
  <implementation name="IM_transformmatrix_vector4_genmsl" nodedef="ND_transformmatrix_vector4" target="genmsl" sourcecode="{{mat}} * {{in}}" />
  <implementation name="IM_transpose_matrix33_genmsl" nodedef="ND_transpose_matrix33" target="genmsl" sourcecode="transpose({{in}})" />
  <implementation name="IM_transpose_matrix44_genmsl" nodedef="ND_transpose_matrix44" target="genmsl" sourcecode="transpose({{in}})" />
  <implementation name="IM_determinant_matrix33_genmsl" nodedef="ND_determinant_matrix33" target="genmsl" sourcecode="determinant({{in}})" />
  <implementation name="IM_determinant_matrix44_genmsl" nodedef="ND_determinant_matrix44" target="genmsl" sourcecode="determinant({{in}})" />
  <implementation name="IM_invertmatrix_matrix33_genmsl" nodedef="ND_invertmatrix_matrix33" target="genmsl" sourcecode="inverse({{in}})" />
  <implementation name="IM_invertmatrix_matrix44_genmsl" nodedef="ND_invertmatrix_matrix44" target="genmsl" sourcecode="inverse({{in}})" />
  <implementation name="IM_rotate2d_vector2_genmsl" nodedef="ND_rotate2d_vector2" file="../genglsl/mx_rotate_vector2.glsl" function="mx_rotate_vector2" target="genmsl" />
  <implementation name="IM_rotate3d_vector3_genmsl" nodedef="ND_rotate3d_vector3" file="../genglsl/mx_rotate_vector3.glsl" function="mx_rotate_vector3" target="genmsl" />
  <implementation name="IM_remap_float_genmsl" nodedef="ND_remap_float" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color3_genmsl" nodedef="ND_remap_color3" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color3FA_genmsl" nodedef="ND_remap_color3FA" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color4_genmsl" nodedef="ND_remap_color4" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_color4FA_genmsl" nodedef="ND_remap_color4FA" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector2_genmsl" nodedef="ND_remap_vector2" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector2FA_genmsl" nodedef="ND_remap_vector2FA" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector3_genmsl" nodedef="ND_remap_vector3" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector3FA_genmsl" nodedef="ND_remap_vector3FA" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector4_genmsl" nodedef="ND_remap_vector4" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_remap_vector4FA_genmsl" nodedef="ND_remap_vector4FA" target="genmsl" sourcecode="{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})" />
  <implementation name="IM_smoothstep_float_genmsl" nodedef="ND_smoothstep_float" file="mx_smoothstep_float.metal" function="mx_smoothstep_float" target="genmsl" />
  <implementation name="IM_smoothstep_color3_genmsl" nodedef="ND_smoothstep_color3" file="mx_smoothstep_vec3.metal" function="mx_smoothstep_vec3" target="genmsl" />
  <implementation name="IM_smoothstep_color3FA_genmsl" nodedef="ND_smoothstep_color3FA" file="mx_smoothstep_vec3FA.metal" function="mx_smoothstep_vec3FA" target="genmsl" />
  <implementation name="IM_smoothstep_color4_genmsl" nodedef="ND_smoothstep_color4" file="mx_smoothstep_vec4.metal" function="mx_smoothstep_vec4" target="genmsl" />
  <implementation name="IM_smoothstep_color4FA_genmsl" nodedef="ND_smoothstep_color4FA" file="mx_smoothstep_vec4FA.metal" function="mx_smoothstep_vec4FA" target="genmsl" />
  <implementation name="IM_smoothstep_vector2_genmsl" nodedef="ND_smoothstep_vector2" file="mx_smoothstep_vec2.metal" function="mx_smoothstep_vec2" target="genmsl" />
  <implementation name="IM_smoothstep_vector2FA_genmsl" nodedef="ND_smoothstep_vector2FA" file="mx_smoothstep_vec2FA.metal" function="mx_smoothstep_vec2FA" target="genmsl" />
  <implementation name="IM_smoothstep_vector3_genmsl" nodedef="ND_smoothstep_vector3" file="mx_smoothstep_vec3.metal" function="mx_smoothstep_vec3" target="genmsl" />
  <implementation name="IM_smoothstep_vector3FA_genmsl" nodedef="ND_smoothstep_vector3FA" file="mx_smoothstep_vec3FA.metal" function="mx_smoothstep_vec3FA" target="genmsl" />
  <implementation name="IM_smoothstep_vector4_genmsl" nodedef="ND_smoothstep_vector4" file="mx_smoothstep_vec4.metal" function="mx_smoothstep_vec4" target="genmsl" />
  <implementation name="IM_smoothstep_vector4FA_genmsl" nodedef="ND_smoothstep_vector4FA" file="mx_smoothstep_vec4FA.metal" function="mx_smoothstep_vec4FA" target="genmsl" />
  <implementation name="IM_luminance_color3_genmsl" nodedef="ND_luminance_color3" file="../genglsl/mx_luminance_color3.glsl" function="mx_luminance_color3" target="genmsl" />
  <implementation name="IM_luminance_color4_genmsl" nodedef="ND_luminance_color4" file="../genglsl/mx_luminance_color4.glsl" function="mx_luminance_color4" target="genmsl" />
  <implementation name="IM_rgbtohsv_color3_genmsl" nodedef="ND_rgbtohsv_color3" file="../genglsl/mx_rgbtohsv_color3.glsl" function="mx_rgbtohsv_color3" target="genmsl" />
  <implementation name="IM_rgbtohsv_color4_genmsl" nodedef="ND_rgbtohsv_color4" file="../genglsl/mx_rgbtohsv_color4.glsl" function="mx_rgbtohsv_color4" target="genmsl" />
  <implementation name="IM_hsvtorgb_color3_genmsl" nodedef="ND_hsvtorgb_color3" file="../genglsl/mx_hsvtorgb_color3.glsl" function="mx_hsvtorgb_color3" target="genmsl" />
  <implementation name="IM_hsvtorgb_color4_genmsl" nodedef="ND_hsvtorgb_color4" file="../genglsl/mx_hsvtorgb_color4.glsl" function="mx_hsvtorgb_color4" target="genmsl" />
  <implementation name="IM_premult_color4_genmsl" nodedef="ND_premult_color4" file="../genglsl/mx_premult_color4.glsl" function="mx_premult_color4" target="genmsl" />
  <implementation name="IM_unpremult_color4_genmsl" nodedef="ND_unpremult_color4" file="../genglsl/mx_unpremult_color4.glsl" function="mx_unpremult_color4" target="genmsl" />
  <implementation name="IM_plus_float_genmsl" nodedef="ND_plus_float" target="genmsl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_plus_color3_genmsl" nodedef="ND_plus_color3" target="genmsl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_plus_color4_genmsl" nodedef="ND_plus_color4" target="genmsl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_float_genmsl" nodedef="ND_minus_float" target="genmsl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_color3_genmsl" nodedef="ND_minus_color3" target="genmsl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_color4_genmsl" nodedef="ND_minus_color4" target="genmsl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_float_genmsl" nodedef="ND_difference_float" target="genmsl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_color3_genmsl" nodedef="ND_difference_color3" target="genmsl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_color4_genmsl" nodedef="ND_difference_color4" target="genmsl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_burn_float_genmsl" nodedef="ND_burn_float" file="mx_burn_float.metal" function="mx_burn_float" target="genmsl" />
  <implementation name="IM_burn_color3_genmsl" nodedef="ND_burn_color3" file="mx_burn_color3.metal" function="mx_burn_color3" target="genmsl" />
  <implementation name="IM_burn_color4_genmsl" nodedef="ND_burn_color4" file="mx_burn_color4.metal" function="mx_burn_color4" target="genmsl" />
  <implementation name="IM_dodge_float_genmsl" nodedef="ND_dodge_float" file="mx_dodge_float.metal" function="mx_dodge_float" target="genmsl" />
  <implementation name="IM_dodge_color3_genmsl" nodedef="ND_dodge_color3" file="mx_dodge_color3.metal" function="mx_dodge_color3" target="genmsl" />
  <implementation name="IM_dodge_color4_genmsl" nodedef="ND_dodge_color4" file="mx_dodge_color4.metal" function="mx_dodge_color4" target="genmsl" />
  <implementation name="IM_screen_float_genmsl" nodedef="ND_screen_float" target="genmsl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_screen_color3_genmsl" nodedef="ND_screen_color3" target="genmsl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_screen_color4_genmsl" nodedef="ND_screen_color4" target="genmsl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_overlay_float_genmsl" nodedef="ND_overlay_float" target="genmsl" sourcecode="({{fg}} < 0.5) ? ({{mix}}*2.0*{{fg}}*{{bg}}) + ((1.0-{{mix}})*{{bg}}) : ({{mix}}*(1.0-(1.0-{{fg}})*(1.0-{{bg}}))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_overlay_color3_genmsl" nodedef="ND_overlay_color3" file="../genglsl/mx_overlay_color3.glsl" function="mx_overlay_color3" target="genmsl" />
  <implementation name="IM_overlay_color4_genmsl" nodedef="ND_overlay_color4" file="../genglsl/mx_overlay_color4.glsl" function="mx_overlay_color4" target="genmsl" />
  <implementation name="IM_disjointover_color4_genmsl" nodedef="ND_disjointover_color4" file="../genglsl/mx_disjointover_color4.glsl" function="mx_disjointover_color4" target="genmsl" />
  <implementation name="IM_in_color4_genmsl" nodedef="ND_in_color4" target="genmsl" sourcecode="({{fg}}*{{bg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_mask_color4_genmsl" nodedef="ND_mask_color4" target="genmsl" sourcecode="({{bg}}*{{fg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_matte_color4_genmsl" nodedef="ND_matte_color4" target="genmsl" sourcecode="vec4( {{fg}}.xyz*{{fg}}.w + {{bg}}.xyz*(1.0-{{fg}}.w), {{fg}}.w + ({{bg}}.w*(1.0-{{fg}}.w)) ) * {{mix}} + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_out_color4_genmsl" nodedef="ND_out_color4" target="genmsl" sourcecode="({{fg}}*(1.0-{{bg}}.a)  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_over_color4_genmsl" nodedef="ND_over_color4" target="genmsl" sourcecode="({{fg}} + ({{bg}}*(1.0-{{fg}}[3]))) * {{mix}} + {{bg}} * (1.0-{{mix}})" />
  <implementation name="IM_inside_float_genmsl" nodedef="ND_inside_float" target="genmsl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_inside_color3_genmsl" nodedef="ND_inside_color3" target="genmsl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_inside_color4_genmsl" nodedef="ND_inside_color4" target="genmsl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_outside_float_genmsl" nodedef="ND_outside_float" target="genmsl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_outside_color3_genmsl" nodedef="ND_outside_color3" target="genmsl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_outside_color4_genmsl" nodedef="ND_outside_color4" target="genmsl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_mix_float_genmsl" nodedef="ND_mix_float" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color3_genmsl" nodedef="ND_mix_color3" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color3_color3_genmsl" nodedef="ND_mix_color3_color3" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color4_genmsl" nodedef="ND_mix_color4" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color4_color4_genmsl" nodedef="ND_mix_color4_color4" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector2_genmsl" nodedef="ND_mix_vector2" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector2_vector2_genmsl" nodedef="ND_mix_vector2_vector2" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector3_genmsl" nodedef="ND_mix_vector3" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector3_vector3_genmsl" nodedef="ND_mix_vector3_vector3" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector4_genmsl" nodedef="ND_mix_vector4" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector4_vector4_genmsl" nodedef="ND_mix_vector4_vector4" target="genmsl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_surfaceshader_genmsl" nodedef="ND_mix_surfaceshader" function="mx_mix_surfaceshader" file="../genglsl/mx_mix_surfaceshader.glsl" target="genmsl" />
  <implementation name="IM_ifgreater_float_genmsl" nodedef="ND_ifgreater_float" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color3_genmsl" nodedef="ND_ifgreater_color3" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color4_genmsl" nodedef="ND_ifgreater_color4" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector2_genmsl" nodedef="ND_ifgreater_vector2" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector3_genmsl" nodedef="ND_ifgreater_vector3" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector4_genmsl" nodedef="ND_ifgreater_vector4" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_floatI_genmsl" nodedef="ND_ifgreater_floatI" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color3I_genmsl" nodedef="ND_ifgreater_color3I" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_color4I_genmsl" nodedef="ND_ifgreater_color4I" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector2I_genmsl" nodedef="ND_ifgreater_vector2I" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector3I_genmsl" nodedef="ND_ifgreater_vector3I" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreater_vector4I_genmsl" nodedef="ND_ifgreater_vector4I" target="genmsl" sourcecode="({{value1}} > {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_float_genmsl" nodedef="ND_ifgreatereq_float" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color3_genmsl" nodedef="ND_ifgreatereq_color3" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color4_genmsl" nodedef="ND_ifgreatereq_color4" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector2_genmsl" nodedef="ND_ifgreatereq_vector2" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector3_genmsl" nodedef="ND_ifgreatereq_vector3" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector4_genmsl" nodedef="ND_ifgreatereq_vector4" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_floatI_genmsl" nodedef="ND_ifgreatereq_floatI" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color3I_genmsl" nodedef="ND_ifgreatereq_color3I" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_color4I_genmsl" nodedef="ND_ifgreatereq_color4I" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector2I_genmsl" nodedef="ND_ifgreatereq_vector2I" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector3I_genmsl" nodedef="ND_ifgreatereq_vector3I" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifgreatereq_vector4I_genmsl" nodedef="ND_ifgreatereq_vector4I" target="genmsl" sourcecode="({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_float_genmsl" nodedef="ND_ifequal_float" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color3_genmsl" nodedef="ND_ifequal_color3" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color4_genmsl" nodedef="ND_ifequal_color4" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector2_genmsl" nodedef="ND_ifequal_vector2" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector3_genmsl" nodedef="ND_ifequal_vector3" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector4_genmsl" nodedef="ND_ifequal_vector4" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_floatI_genmsl" nodedef="ND_ifequal_floatI" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color3I_genmsl" nodedef="ND_ifequal_color3I" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color4I_genmsl" nodedef="ND_ifequal_color4I" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector2I_genmsl" nodedef="ND_ifequal_vector2I" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector3I_genmsl" nodedef="ND_ifequal_vector3I" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector4I_genmsl" nodedef="ND_ifequal_vector4I" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_floatB_genmsl" nodedef="ND_ifequal_floatB" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color3B_genmsl" nodedef="ND_ifequal_color3B" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_color4B_genmsl" nodedef="ND_ifequal_color4B" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector2B_genmsl" nodedef="ND_ifequal_vector2B" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector3B_genmsl" nodedef="ND_ifequal_vector3B" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_ifequal_vector4B_genmsl" nodedef="ND_ifequal_vector4B" target="genmsl" sourcecode="({{value1}} == {{value2}}) ? {{in1}} : {{in2}}" />
  <implementation name="IM_switch_float_genmsl" nodedef="ND_switch_float" target="genmsl" />
  <implementation name="IM_switch_color3_genmsl" nodedef="ND_switch_color3" target="genmsl" />
  <implementation name="IM_switch_color4_genmsl" nodedef="ND_switch_color4" target="genmsl" />
  <implementation name="IM_switch_vector2_genmsl" nodedef="ND_switch_vector2" target="genmsl" />
  <implementation name="IM_switch_vector3_genmsl" nodedef="ND_switch_vector3" target="genmsl" />
  <implementation name="IM_switch_vector4_genmsl" nodedef="ND_switch_vector4" target="genmsl" />
  <implementation name="IM_switch_floatI_genmsl" nodedef="ND_switch_floatI" target="genmsl" />
  <implementation name="IM_switch_color3I_genmsl" nodedef="ND_switch_color3I" target="genmsl" />
  <implementation name="IM_switch_color4I_genmsl" nodedef="ND_switch_color4I" target="genmsl" />
  <implementation name="IM_switch_vector2I_genmsl" nodedef="ND_switch_vector2I" target="genmsl" />
  <implementation name="IM_switch_vector3I_genmsl" nodedef="ND_switch_vector3I" target="genmsl" />
  <implementation name="IM_switch_vector4I_genmsl" nodedef="ND_switch_vector4I" target="genmsl" />
  <implementation name="IM_convert_float_color3_genmsl" nodedef="ND_convert_float_color3" target="genmsl" />
  <implementation name="IM_convert_float_color4_genmsl" nodedef="ND_convert_float_color4" target="genmsl" />
  <implementation name="IM_convert_float_vector2_genmsl" nodedef="ND_convert_float_vector2" target="genmsl" />
  <implementation name="IM_convert_float_vector3_genmsl" nodedef="ND_convert_float_vector3" target="genmsl" />
  <implementation name="IM_convert_float_vector4_genmsl" nodedef="ND_convert_float_vector4" target="genmsl" />
  <implementation name="IM_convert_vector2_vector3_genmsl" nodedef="ND_convert_vector2_vector3" target="genmsl" />
  <implementation name="IM_convert_vector3_vector2_genmsl" nodedef="ND_convert_vector3_vector2" target="genmsl" />
  <implementation name="IM_convert_vector3_color3_genmsl" nodedef="ND_convert_vector3_color3" target="genmsl" />
  <implementation name="IM_convert_vector3_vector4_genmsl" nodedef="ND_convert_vector3_vector4" target="genmsl" />
  <implementation name="IM_convert_vector4_vector3_genmsl" nodedef="ND_convert_vector4_vector3" target="genmsl" />
  <implementation name="IM_convert_vector4_color4_genmsl" nodedef="ND_convert_vector4_color4" target="genmsl" />
  <implementation name="IM_convert_color3_vector3_genmsl" nodedef="ND_convert_color3_vector3" target="genmsl" />
  <implementation name="IM_convert_color4_vector4_genmsl" nodedef="ND_convert_color4_vector4" target="genmsl" />
  <implementation name="IM_convert_color3_color4_genmsl" nodedef="ND_convert_color3_color4" target="genmsl" />
  <implementation name="IM_convert_color4_color3_genmsl" nodedef="ND_convert_color4_color3" target="genmsl" />
  <implementation name="IM_convert_boolean_float_genmsl" nodedef="ND_convert_boolean_float" target="genmsl" />
  <implementation name="IM_convert_integer_float_genmsl" nodedef="ND_convert_integer_float" target="genmsl" />
  <implementation name="IM_swizzle_float_color3_genmsl" nodedef="ND_swizzle_float_color3" target="genmsl" />
  <implementation name="IM_swizzle_float_color4_genmsl" nodedef="ND_swizzle_float_color4" target="genmsl" />
  <implementation name="IM_swizzle_float_vector2_genmsl" nodedef="ND_swizzle_float_vector2" target="genmsl" />
  <implementation name="IM_swizzle_float_vector3_genmsl" nodedef="ND_swizzle_float_vector3" target="genmsl" />
  <implementation name="IM_swizzle_float_vector4_genmsl" nodedef="ND_swizzle_float_vector4" target="genmsl" />
  <implementation name="IM_swizzle_color3_float_genmsl" nodedef="ND_swizzle_color3_float" target="genmsl" />
  <implementation name="IM_swizzle_color3_color3_genmsl" nodedef="ND_swizzle_color3_color3" target="genmsl" />
  <implementation name="IM_swizzle_color3_color4_genmsl" nodedef="ND_swizzle_color3_color4" target="genmsl" />
  <implementation name="IM_swizzle_color3_vector2_genmsl" nodedef="ND_swizzle_color3_vector2" target="genmsl" />
  <implementation name="IM_swizzle_color3_vector3_genmsl" nodedef="ND_swizzle_color3_vector3" target="genmsl" />
  <implementation name="IM_swizzle_color3_vector4_genmsl" nodedef="ND_swizzle_color3_vector4" target="genmsl" />
  <implementation name="IM_swizzle_color4_float_genmsl" nodedef="ND_swizzle_color4_float" target="genmsl" />
  <implementation name="IM_swizzle_color4_color3_genmsl" nodedef="ND_swizzle_color4_color3" target="genmsl" />
  <implementation name="IM_swizzle_color4_color4_genmsl" nodedef="ND_swizzle_color4_color4" target="genmsl" />
  <implementation name="IM_swizzle_color4_vector2_genmsl" nodedef="ND_swizzle_color4_vector2" target="genmsl" />
  <implementation name="IM_swizzle_color4_vector3_genmsl" nodedef="ND_swizzle_color4_vector3" target="genmsl" />
  <implementation name="IM_swizzle_color4_vector4_genmsl" nodedef="ND_swizzle_color4_vector4" target="genmsl" />
  <implementation name="IM_swizzle_vector2_float_genmsl" nodedef="ND_swizzle_vector2_float" target="genmsl" />
  <implementation name="IM_swizzle_vector2_color3_genmsl" nodedef="ND_swizzle_vector2_color3" target="genmsl" />
  <implementation name="IM_swizzle_vector2_color4_genmsl" nodedef="ND_swizzle_vector2_color4" target="genmsl" />
  <implementation name="IM_swizzle_vector2_vector2_genmsl" nodedef="ND_swizzle_vector2_vector2" target="genmsl" />
  <implementation name="IM_swizzle_vector2_vector3_genmsl" nodedef="ND_swizzle_vector2_vector3" target="genmsl" />
  <implementation name="IM_swizzle_vector2_vector4_genmsl" nodedef="ND_swizzle_vector2_vector4" target="genmsl" />
  <implementation name="IM_swizzle_vector3_float_genmsl" nodedef="ND_swizzle_vector3_float" target="genmsl" />
  <implementation name="IM_swizzle_vector3_color3_genmsl" nodedef="ND_swizzle_vector3_color3" target="genmsl" />
  <implementation name="IM_swizzle_vector3_color4_genmsl" nodedef="ND_swizzle_vector3_color4" target="genmsl" />
  <implementation name="IM_swizzle_vector3_vector2_genmsl" nodedef="ND_swizzle_vector3_vector2" target="genmsl" />
  <implementation name="IM_swizzle_vector3_vector3_genmsl" nodedef="ND_swizzle_vector3_vector3" target="genmsl" />
  <implementation name="IM_swizzle_vector3_vector4_genmsl" nodedef="ND_swizzle_vector3_vector4" target="genmsl" />
  <implementation name="IM_swizzle_vector4_float_genmsl" nodedef="ND_swizzle_vector4_float" target="genmsl" />
  <implementation name="IM_swizzle_vector4_color3_genmsl" nodedef="ND_swizzle_vector4_color3" target="genmsl" />
  <implementation name="IM_swizzle_vector4_color4_genmsl" nodedef="ND_swizzle_vector4_color4" target="genmsl" />
  <implementation name="IM_swizzle_vector4_vector2_genmsl" nodedef="ND_swizzle_vector4_vector2" target="genmsl" />
  <implementation name="IM_swizzle_vector4_vector3_genmsl" nodedef="ND_swizzle_vector4_vector3" target="genmsl" />
  <implementation name="IM_swizzle_vector4_vector4_genmsl" nodedef="ND_swizzle_vector4_vector4" target="genmsl" />
  <implementation name="IM_combine2_vector2_genmsl" nodedef="ND_combine2_vector2" target="genmsl" />
  <implementation name="IM_combine2_color4CF_genmsl" nodedef="ND_combine2_color4CF" target="genmsl" />
  <implementation name="IM_combine2_vector4VF_genmsl" nodedef="ND_combine2_vector4VF" target="genmsl" />
  <implementation name="IM_combine2_vector4VV_genmsl" nodedef="ND_combine2_vector4VV" target="genmsl" />
  <implementation name="IM_combine3_color3_genmsl" nodedef="ND_combine3_color3" target="genmsl" />
  <implementation name="IM_combine3_vector3_genmsl" nodedef="ND_combine3_vector3" target="genmsl" />
  <implementation name="IM_combine4_color4_genmsl" nodedef="ND_combine4_color4" target="genmsl" />
  <implementation name="IM_combine4_vector4_genmsl" nodedef="ND_combine4_vector4" target="genmsl" />
  <implementation name="IM_blur_float_genmsl" nodedef="ND_blur_float" target="genmsl" />
  <implementation name="IM_blur_color3_genmsl" nodedef="ND_blur_color3" target="genmsl" />
  <implementation name="IM_blur_color4_genmsl" nodedef="ND_blur_color4" target="genmsl" />
  <implementation name="IM_blur_vector2_genmsl" nodedef="ND_blur_vector2" target="genmsl" />
  <implementation name="IM_blur_vector3_genmsl" nodedef="ND_blur_vector3" target="genmsl" />
  <implementation name="IM_blur_vector4_genmsl" nodedef="ND_blur_vector4" target="genmsl" />
  <implementation name="IM_heighttonormal_vector3_genmsl" nodedef="ND_heighttonormal_vector3" target="genmsl" />
  <implementation name="IM_dot_float_genmsl" nodedef="ND_dot_float" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_color3_genmsl" nodedef="ND_dot_color3" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_color4_genmsl" nodedef="ND_dot_color4" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector2_genmsl" nodedef="ND_dot_vector2" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector3_genmsl" nodedef="ND_dot_vector3" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector4_genmsl" nodedef="ND_dot_vector4" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_integer_genmsl" nodedef="ND_dot_integer" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_boolean_genmsl" nodedef="ND_dot_boolean" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_matrix33_genmsl" nodedef="ND_dot_matrix33" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_matrix44_genmsl" nodedef="ND_dot_matrix44" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_string_genmsl" nodedef="ND_dot_string" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_filename_genmsl" nodedef="ND_dot_filename" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_surfaceshader_genmsl" nodedef="ND_dot_surfaceshader" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_displacementshader_genmsl" nodedef="ND_dot_displacementshader" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_volumeshader_genmsl" nodedef="ND_dot_volumeshader" target="genmsl" sourcecode="{{in}}" />
  <implementation name="IM_dot_lightshader_genmsl" nodedef="ND_dot_lightshader" target="genmsl" sourcecode="{{in}}" />
  <!--Custom add implementation (mxadd)-->
  <implementation name="IM_myadd_color3_genmsl" nodedef="ND_myadd_color3" target="genmsl" sourcecode="{{in1}} + {{in2}}" />
</materialx>
```
</details>


- Source implementation added: IM_myadd_color3_genmsl


<details><summary>Implementation for target genosl</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <implementation name="IM_surfacematerial_genosl" nodedef="ND_surfacematerial" target="genosl" />
  <implementation name="IM_surface_unlit_genosl" nodedef="ND_surface_unlit" file="mx_surface_unlit.osl" function="mx_surface_unlit" target="genosl" />
  <implementation name="IM_image_float_genosl" nodedef="ND_image_float" file="mx_image_float.osl" function="mx_image_float" target="genosl">
    <input name="default" type="float" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color3_genosl" nodedef="ND_image_color3" file="mx_image_color3.osl" function="mx_image_color3" target="genosl">
    <input name="default" type="color3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_color4_genosl" nodedef="ND_image_color4" file="mx_image_color4.osl" function="mx_image_color4" target="genosl">
    <input name="default" type="color4" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector2_genosl" nodedef="ND_image_vector2" file="mx_image_vector2.osl" function="mx_image_vector2" target="genosl">
    <input name="default" type="vector2" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector3_genosl" nodedef="ND_image_vector3" file="mx_image_vector3.osl" function="mx_image_vector3" target="genosl">
    <input name="default" type="vector3" implname="default_value" />
  </implementation>
  <implementation name="IM_image_vector4_genosl" nodedef="ND_image_vector4" file="mx_image_vector4.osl" function="mx_image_vector4" target="genosl">
    <input name="default" type="vector4" implname="default_value" />
  </implementation>
  <implementation name="IM_normalmap_genosl" nodedef="ND_normalmap" file="mx_normalmap.osl" function="mx_normalmap" target="genosl" />
  <implementation name="IM_constant_float_genosl" nodedef="ND_constant_float" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_color3_genosl" nodedef="ND_constant_color3" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_color4_genosl" nodedef="ND_constant_color4" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector2_genosl" nodedef="ND_constant_vector2" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector3_genosl" nodedef="ND_constant_vector3" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_vector4_genosl" nodedef="ND_constant_vector4" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_boolean_genosl" nodedef="ND_constant_boolean" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_integer_genosl" nodedef="ND_constant_integer" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_matrix33_genosl" nodedef="ND_constant_matrix33" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_matrix44_genosl" nodedef="ND_constant_matrix44" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_string_genosl" nodedef="ND_constant_string" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_constant_filename_genosl" nodedef="ND_constant_filename" target="genosl" sourcecode="{{value}}" />
  <implementation name="IM_ramplr_float_genosl" nodedef="ND_ramplr_float" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))" />
  <implementation name="IM_ramplr_color3_genosl" nodedef="ND_ramplr_color3" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))" />
  <implementation name="IM_ramplr_color4_genosl" nodedef="ND_ramplr_color4" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))" />
  <implementation name="IM_ramplr_vector2_genosl" nodedef="ND_ramplr_vector2" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))" />
  <implementation name="IM_ramplr_vector3_genosl" nodedef="ND_ramplr_vector3" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))" />
  <implementation name="IM_ramplr_vector4_genosl" nodedef="ND_ramplr_vector4" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))" />
  <implementation name="IM_ramptb_float_genosl" nodedef="ND_ramptb_float" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))" />
  <implementation name="IM_ramptb_color3_genosl" nodedef="ND_ramptb_color3" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))" />
  <implementation name="IM_ramptb_color4_genosl" nodedef="ND_ramptb_color4" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))" />
  <implementation name="IM_ramptb_vector2_genosl" nodedef="ND_ramptb_vector2" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))" />
  <implementation name="IM_ramptb_vector3_genosl" nodedef="ND_ramptb_vector3" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))" />
  <implementation name="IM_ramptb_vector4_genosl" nodedef="ND_ramptb_vector4" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))" />
  <implementation name="IM_splitlr_float_genosl" nodedef="ND_splitlr_float" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))" />
  <implementation name="IM_splitlr_color3_genosl" nodedef="ND_splitlr_color3" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))" />
  <implementation name="IM_splitlr_color4_genosl" nodedef="ND_splitlr_color4" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))" />
  <implementation name="IM_splitlr_vector2_genosl" nodedef="ND_splitlr_vector2" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))" />
  <implementation name="IM_splitlr_vector3_genosl" nodedef="ND_splitlr_vector3" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))" />
  <implementation name="IM_splitlr_vector4_genosl" nodedef="ND_splitlr_vector4" target="genosl" sourcecode="mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))" />
  <implementation name="IM_splittb_float_genosl" nodedef="ND_splittb_float" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))" />
  <implementation name="IM_splittb_color3_genosl" nodedef="ND_splittb_color3" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))" />
  <implementation name="IM_splittb_color4_genosl" nodedef="ND_splittb_color4" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))" />
  <implementation name="IM_splittb_vector2_genosl" nodedef="ND_splittb_vector2" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))" />
  <implementation name="IM_splittb_vector3_genosl" nodedef="ND_splittb_vector3" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))" />
  <implementation name="IM_splittb_vector4_genosl" nodedef="ND_splittb_vector4" target="genosl" sourcecode="mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))" />
  <implementation name="IM_noise2d_float_genosl" nodedef="ND_noise2d_float" file="mx_noise2d_float.osl" function="mx_noise2d_float" target="genosl" />
  <implementation name="IM_noise2d_color3_genosl" nodedef="ND_noise2d_color3" file="mx_noise2d_color3.osl" function="mx_noise2d_color3" target="genosl" />
  <implementation name="IM_noise2d_color4_genosl" nodedef="ND_noise2d_color4" file="mx_noise2d_color4.osl" function="mx_noise2d_color4" target="genosl" />
  <implementation name="IM_noise2d_color3FA_genosl" nodedef="ND_noise2d_color3FA" file="mx_noise2d_fa_color3.osl" function="mx_noise2d_fa_color3" target="genosl" />
  <implementation name="IM_noise2d_color4FA_genosl" nodedef="ND_noise2d_color4FA" file="mx_noise2d_fa_color4.osl" function="mx_noise2d_fa_color4" target="genosl" />
  <implementation name="IM_noise2d_vector2_genosl" nodedef="ND_noise2d_vector2" file="mx_noise2d_vector2.osl" function="mx_noise2d_vector2" target="genosl" />
  <implementation name="IM_noise2d_vector3_genosl" nodedef="ND_noise2d_vector3" file="mx_noise2d_vector3.osl" function="mx_noise2d_vector3" target="genosl" />
  <implementation name="IM_noise2d_vector4_genosl" nodedef="ND_noise2d_vector4" file="mx_noise2d_vector4.osl" function="mx_noise2d_vector4" target="genosl" />
  <implementation name="IM_noise2d_vector2FA_genosl" nodedef="ND_noise2d_vector2FA" file="mx_noise2d_fa_vector2.osl" function="mx_noise2d_fa_vector2" target="genosl" />
  <implementation name="IM_noise2d_vector3FA_genosl" nodedef="ND_noise2d_vector3FA" file="mx_noise2d_fa_vector3.osl" function="mx_noise2d_fa_vector3" target="genosl" />
  <implementation name="IM_noise2d_vector4FA_genosl" nodedef="ND_noise2d_vector4FA" file="mx_noise2d_fa_vector4.osl" function="mx_noise2d_fa_vector4" target="genosl" />
  <implementation name="IM_noise3d_float_genosl" nodedef="ND_noise3d_float" file="mx_noise3d_float.osl" function="mx_noise3d_float" target="genosl" />
  <implementation name="IM_noise3d_color3_genosl" nodedef="ND_noise3d_color3" file="mx_noise3d_color3.osl" function="mx_noise3d_color3" target="genosl" />
  <implementation name="IM_noise3d_color4_genosl" nodedef="ND_noise3d_color4" file="mx_noise3d_color4.osl" function="mx_noise3d_color4" target="genosl" />
  <implementation name="IM_noise3d_color3FA_genosl" nodedef="ND_noise3d_color3FA" file="mx_noise3d_fa_color3.osl" function="mx_noise3d_fa_color3" target="genosl" />
  <implementation name="IM_noise3d_color4FA_genosl" nodedef="ND_noise3d_color4FA" file="mx_noise3d_fa_color4.osl" function="mx_noise3d_fa_color4" target="genosl" />
  <implementation name="IM_noise3d_vector2_genosl" nodedef="ND_noise3d_vector2" file="mx_noise3d_vector2.osl" function="mx_noise3d_vector2" target="genosl" />
  <implementation name="IM_noise3d_vector3_genosl" nodedef="ND_noise3d_vector3" file="mx_noise3d_vector3.osl" function="mx_noise3d_vector3" target="genosl" />
  <implementation name="IM_noise3d_vector4_genosl" nodedef="ND_noise3d_vector4" file="mx_noise3d_vector4.osl" function="mx_noise3d_vector4" target="genosl" />
  <implementation name="IM_noise3d_vector2FA_genosl" nodedef="ND_noise3d_vector2FA" file="mx_noise3d_fa_vector2.osl" function="mx_noise3d_fa_vector2" target="genosl" />
  <implementation name="IM_noise3d_vector3FA_genosl" nodedef="ND_noise3d_vector3FA" file="mx_noise3d_fa_vector3.osl" function="mx_noise3d_fa_vector3" target="genosl" />
  <implementation name="IM_noise3d_vector4FA_genosl" nodedef="ND_noise3d_vector4FA" file="mx_noise3d_fa_vector4.osl" function="mx_noise3d_fa_vector4" target="genosl" />
  <implementation name="IM_fractal3d_float_genosl" nodedef="ND_fractal3d_float" file="mx_fractal3d_float.osl" function="mx_fractal3d_float" target="genosl" />
  <implementation name="IM_fractal3d_color3_genosl" nodedef="ND_fractal3d_color3" file="mx_fractal3d_color3.osl" function="mx_fractal3d_color3" target="genosl" />
  <implementation name="IM_fractal3d_color4_genosl" nodedef="ND_fractal3d_color4" file="mx_fractal3d_color4.osl" function="mx_fractal3d_color4" target="genosl" />
  <implementation name="IM_fractal3d_color3FA_genosl" nodedef="ND_fractal3d_color3FA" file="mx_fractal3d_fa_color3.osl" function="mx_fractal3d_fa_color3" target="genosl" />
  <implementation name="IM_fractal3d_color4FA_genosl" nodedef="ND_fractal3d_color4FA" file="mx_fractal3d_fa_color4.osl" function="mx_fractal3d_fa_color4" target="genosl" />
  <implementation name="IM_fractal3d_vector2_genosl" nodedef="ND_fractal3d_vector2" file="mx_fractal3d_vector2.osl" function="mx_fractal3d_vector2" target="genosl" />
  <implementation name="IM_fractal3d_vector3_genosl" nodedef="ND_fractal3d_vector3" file="mx_fractal3d_vector3.osl" function="mx_fractal3d_vector3" target="genosl" />
  <implementation name="IM_fractal3d_vector4_genosl" nodedef="ND_fractal3d_vector4" file="mx_fractal3d_vector4.osl" function="mx_fractal3d_vector4" target="genosl" />
  <implementation name="IM_fractal3d_vector2FA_genosl" nodedef="ND_fractal3d_vector2FA" file="mx_fractal3d_fa_vector2.osl" function="mx_fractal3d_fa_vector2" target="genosl" />
  <implementation name="IM_fractal3d_vector3FA_genosl" nodedef="ND_fractal3d_vector3FA" file="mx_fractal3d_fa_vector3.osl" function="mx_fractal3d_fa_vector3" target="genosl" />
  <implementation name="IM_fractal3d_vector4FA_genosl" nodedef="ND_fractal3d_vector4FA" file="mx_fractal3d_fa_vector4.osl" function="mx_fractal3d_fa_vector4" target="genosl" />
  <implementation name="IM_cellnoise2d_float_genosl" nodedef="ND_cellnoise2d_float" file="mx_cellnoise2d_float.osl" function="mx_cellnoise2d_float" target="genosl" />
  <implementation name="IM_cellnoise3d_float_genosl" nodedef="ND_cellnoise3d_float" file="mx_cellnoise3d_float.osl" function="mx_cellnoise3d_float" target="genosl" />
  <implementation name="IM_worleynoise2d_float_genosl" nodedef="ND_worleynoise2d_float" file="mx_worleynoise2d_float.osl" function="mx_worleynoise2d_float" target="genosl" />
  <implementation name="IM_worleynoise2d_vector2_genosl" nodedef="ND_worleynoise2d_vector2" file="mx_worleynoise2d_vector2.osl" function="mx_worleynoise2d_vector2" target="genosl" />
  <implementation name="IM_worleynoise2d_vector3_genosl" nodedef="ND_worleynoise2d_vector3" file="mx_worleynoise2d_vector3.osl" function="mx_worleynoise2d_vector3" target="genosl" />
  <implementation name="IM_worleynoise3d_float_genosl" nodedef="ND_worleynoise3d_float" file="mx_worleynoise3d_float.osl" function="mx_worleynoise3d_float" target="genosl" />
  <implementation name="IM_worleynoise3d_vector2_genosl" nodedef="ND_worleynoise3d_vector2" file="mx_worleynoise3d_vector2.osl" function="mx_worleynoise3d_vector2" target="genosl" />
  <implementation name="IM_worleynoise3d_vector3_genosl" nodedef="ND_worleynoise3d_vector3" file="mx_worleynoise3d_vector3.osl" function="mx_worleynoise3d_vector3" target="genosl" />
  <implementation name="IM_ambientocclusion_float_genosl" nodedef="ND_ambientocclusion_float" file="mx_ambientocclusion_float.osl" function="mx_ambientocclusion_float" target="genosl" />
  <implementation name="IM_position_vector3_genosl" nodedef="ND_position_vector3" target="genosl" sourcecode="transform({{space}}, P)" />
  <implementation name="IM_normal_vector3_genosl" nodedef="ND_normal_vector3" target="genosl" sourcecode="transform({{space}}, N)" />
  <implementation name="IM_tangent_vector3_genosl" nodedef="ND_tangent_vector3" target="genosl" sourcecode="transform({{space}}, normalize(dPdu))" />
  <implementation name="IM_bitangent_vector3_genosl" nodedef="ND_bitangent_vector3" target="genosl" sourcecode="transform({{space}}, normalize(dPdv))" />
  <implementation name="IM_texcoord_vector2_genosl" nodedef="ND_texcoord_vector2" target="genosl" sourcecode="vector2(u,v)" />
  <implementation name="IM_texcoord_vector3_genosl" nodedef="ND_texcoord_vector3" target="genosl" sourcecode="vector(u,v,0)" />
  <implementation name="IM_geomcolor_float_genosl" nodedef="ND_geomcolor_float" file="mx_geomcolor_float.osl" function="mx_geomcolor_float" target="genosl" />
  <implementation name="IM_geomcolor_color3_genosl" nodedef="ND_geomcolor_color3" file="mx_geomcolor_color3.osl" function="mx_geomcolor_color3" target="genosl" />
  <implementation name="IM_geomcolor_color4_genosl" nodedef="ND_geomcolor_color4" file="mx_geomcolor_color4.osl" function="mx_geomcolor_color4" target="genosl" />
  <implementation name="IM_geompropvalue_integer_genosl" nodedef="ND_geompropvalue_integer" file="mx_geompropvalue_integer.osl" function="mx_geompropvalue_integer" target="genosl" />
  <implementation name="IM_geompropvalue_boolean_genosl" nodedef="ND_geompropvalue_boolean" file="mx_geompropvalue_boolean.osl" function="mx_geompropvalue_boolean" target="genosl" />
  <implementation name="IM_geompropvalue_string_genosl" nodedef="ND_geompropvalue_string" file="mx_geompropvalue_string.osl" function="mx_geompropvalue_string" target="genosl" />
  <implementation name="IM_geompropvalue_float_genosl" nodedef="ND_geompropvalue_float" file="mx_geompropvalue_float.osl" function="mx_geompropvalue_float" target="genosl" />
  <implementation name="IM_geompropvalue_color3_genosl" nodedef="ND_geompropvalue_color3" file="mx_geompropvalue_color3.osl" function="mx_geompropvalue_color" target="genosl" />
  <implementation name="IM_geompropvalue_color4_genosl" nodedef="ND_geompropvalue_color4" file="mx_geompropvalue_color4.osl" function="mx_geompropvalue_color4" target="genosl" />
  <implementation name="IM_geompropvalue_vector2_genosl" nodedef="ND_geompropvalue_vector2" file="mx_geompropvalue_vector2.osl" function="mx_geompropvalue_vector2" target="genosl" />
  <implementation name="IM_geompropvalue_vector3_genosl" nodedef="ND_geompropvalue_vector3" file="mx_geompropvalue_vector3.osl" function="mx_geompropvalue_vector" target="genosl" />
  <implementation name="IM_geompropvalue_vector4_genosl" nodedef="ND_geompropvalue_vector4" file="mx_geompropvalue_vector4.osl" function="mx_geompropvalue_vector4" target="genosl" />
  <implementation name="IM_frame_float_genosl" nodedef="ND_frame_float" file="mx_frame_float.osl" function="mx_frame_float" target="genosl" />
  <implementation name="IM_time_float_genosl" nodedef="ND_time_float" file="mx_time_float.osl" function="mx_time_float" target="genosl" />
  <implementation name="IM_add_float_genosl" nodedef="ND_add_float" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color3_genosl" nodedef="ND_add_color3" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color3FA_genosl" nodedef="ND_add_color3FA" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color4_genosl" nodedef="ND_add_color4" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_color4FA_genosl" nodedef="ND_add_color4FA" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector2_genosl" nodedef="ND_add_vector2" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector2FA_genosl" nodedef="ND_add_vector2FA" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector3_genosl" nodedef="ND_add_vector3" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector3FA_genosl" nodedef="ND_add_vector3FA" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector4_genosl" nodedef="ND_add_vector4" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_vector4FA_genosl" nodedef="ND_add_vector4FA" target="genosl" sourcecode="{{in1}} + {{in2}}" />
  <implementation name="IM_add_matrix33_genosl" nodedef="ND_add_matrix33" sourcecode="mx_add({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_add_matrix33FA_genosl" nodedef="ND_add_matrix33FA" sourcecode="mx_add({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_add_matrix44_genosl" nodedef="ND_add_matrix44" sourcecode="mx_add({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_add_matrix44FA_genosl" nodedef="ND_add_matrix44FA" sourcecode="mx_add({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_subtract_float_genosl" nodedef="ND_subtract_float" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color3_genosl" nodedef="ND_subtract_color3" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color3FA_genosl" nodedef="ND_subtract_color3FA" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color4_genosl" nodedef="ND_subtract_color4" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_color4FA_genosl" nodedef="ND_subtract_color4FA" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector2_genosl" nodedef="ND_subtract_vector2" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector2FA_genosl" nodedef="ND_subtract_vector2FA" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector3_genosl" nodedef="ND_subtract_vector3" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector3FA_genosl" nodedef="ND_subtract_vector3FA" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector4_genosl" nodedef="ND_subtract_vector4" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_vector4FA_genosl" nodedef="ND_subtract_vector4FA" target="genosl" sourcecode="{{in1}} - {{in2}}" />
  <implementation name="IM_subtract_matrix33_genosl" nodedef="ND_subtract_matrix33" sourcecode="mx_subtract({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_subtract_matrix33FA_genosl" nodedef="ND_subtract_matrix33FA" sourcecode="mx_subtract({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_subtract_matrix44_genosl" nodedef="ND_subtract_matrix44" sourcecode="mx_subtract({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_subtract_matrix44FA_genosl" nodedef="ND_subtract_matrix44FA" sourcecode="mx_subtract({{in1}}, {{in2}})" target="genosl" />
  <implementation name="IM_multiply_float_genosl" nodedef="ND_multiply_float" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color3_genosl" nodedef="ND_multiply_color3" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color3FA_genosl" nodedef="ND_multiply_color3FA" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color4_genosl" nodedef="ND_multiply_color4" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_color4FA_genosl" nodedef="ND_multiply_color4FA" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector2_genosl" nodedef="ND_multiply_vector2" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector2FA_genosl" nodedef="ND_multiply_vector2FA" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector3_genosl" nodedef="ND_multiply_vector3" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector3FA_genosl" nodedef="ND_multiply_vector3FA" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector4_genosl" nodedef="ND_multiply_vector4" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_vector4FA_genosl" nodedef="ND_multiply_vector4FA" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_matrix33_genosl" nodedef="ND_multiply_matrix33" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_multiply_matrix44_genosl" nodedef="ND_multiply_matrix44" target="genosl" sourcecode="{{in1}} * {{in2}}" />
  <implementation name="IM_divide_float_genosl" nodedef="ND_divide_float" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color3_genosl" nodedef="ND_divide_color3" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color3FA_genosl" nodedef="ND_divide_color3FA" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color4_genosl" nodedef="ND_divide_color4" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_color4FA_genosl" nodedef="ND_divide_color4FA" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector2_genosl" nodedef="ND_divide_vector2" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector2FA_genosl" nodedef="ND_divide_vector2FA" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector3_genosl" nodedef="ND_divide_vector3" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector3FA_genosl" nodedef="ND_divide_vector3FA" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector4_genosl" nodedef="ND_divide_vector4" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_vector4FA_genosl" nodedef="ND_divide_vector4FA" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_matrix33_genosl" nodedef="ND_divide_matrix33" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_divide_matrix44_genosl" nodedef="ND_divide_matrix44" target="genosl" sourcecode="{{in1}} / {{in2}}" />
  <implementation name="IM_modulo_float_genosl" nodedef="ND_modulo_float" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color3_genosl" nodedef="ND_modulo_color3" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color3FA_genosl" nodedef="ND_modulo_color3FA" target="genosl" sourcecode="mod({{in1}}, color({{in2}},{{in2}},{{in2}}))" />
  <implementation name="IM_modulo_color4_genosl" nodedef="ND_modulo_color4" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_color4FA_genosl" nodedef="ND_modulo_color4FA" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector2_genosl" nodedef="ND_modulo_vector2" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector2FA_genosl" nodedef="ND_modulo_vector2FA" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector3_genosl" nodedef="ND_modulo_vector3" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector3FA_genosl" nodedef="ND_modulo_vector3FA" target="genosl" sourcecode="mod({{in1}}, vector({{in2}},{{in2}},{{in2}}))" />
  <implementation name="IM_modulo_vector4_genosl" nodedef="ND_modulo_vector4" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_modulo_vector4FA_genosl" nodedef="ND_modulo_vector4FA" target="genosl" sourcecode="mod({{in1}}, {{in2}})" />
  <implementation name="IM_invert_float_genosl" nodedef="ND_invert_float" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color3_genosl" nodedef="ND_invert_color3" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color3FA_genosl" nodedef="ND_invert_color3FA" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color4_genosl" nodedef="ND_invert_color4" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_color4FA_genosl" nodedef="ND_invert_color4FA" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector2_genosl" nodedef="ND_invert_vector2" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector2FA_genosl" nodedef="ND_invert_vector2FA" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector3_genosl" nodedef="ND_invert_vector3" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector3FA_genosl" nodedef="ND_invert_vector3FA" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector4_genosl" nodedef="ND_invert_vector4" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_invert_vector4FA_genosl" nodedef="ND_invert_vector4FA" target="genosl" sourcecode="{{amount}} - {{in}}" />
  <implementation name="IM_absval_float_genosl" nodedef="ND_absval_float" target="genosl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_color3_genosl" nodedef="ND_absval_color3" target="genosl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_color4_genosl" nodedef="ND_absval_color4" target="genosl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector2_genosl" nodedef="ND_absval_vector2" target="genosl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector3_genosl" nodedef="ND_absval_vector3" target="genosl" sourcecode="abs({{in}})" />
  <implementation name="IM_absval_vector4_genosl" nodedef="ND_absval_vector4" target="genosl" sourcecode="abs({{in}})" />
  <implementation name="IM_floor_float_genosl" nodedef="ND_floor_float" target="genosl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_color3_genosl" nodedef="ND_floor_color3" target="genosl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_color4_genosl" nodedef="ND_floor_color4" target="genosl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector2_genosl" nodedef="ND_floor_vector2" target="genosl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector3_genosl" nodedef="ND_floor_vector3" target="genosl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_vector4_genosl" nodedef="ND_floor_vector4" target="genosl" sourcecode="floor({{in}})" />
  <implementation name="IM_floor_integer_genosl" nodedef="ND_floor_integer" target="genosl" sourcecode="int(floor({{in}}))" />
  <implementation name="IM_ceil_float_genosl" nodedef="ND_ceil_float" target="genosl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_color3_genosl" nodedef="ND_ceil_color3" target="genosl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_color4_genosl" nodedef="ND_ceil_color4" target="genosl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector2_genosl" nodedef="ND_ceil_vector2" target="genosl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector3_genosl" nodedef="ND_ceil_vector3" target="genosl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_vector4_genosl" nodedef="ND_ceil_vector4" target="genosl" sourcecode="ceil({{in}})" />
  <implementation name="IM_ceil_integer_genosl" nodedef="ND_ceil_integer" target="genosl" sourcecode="int(ceil({{in}}))" />
  <implementation name="IM_power_float_genosl" nodedef="ND_power_float" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color3_genosl" nodedef="ND_power_color3" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color3FA_genosl" nodedef="ND_power_color3FA" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color4_genosl" nodedef="ND_power_color4" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_color4FA_genosl" nodedef="ND_power_color4FA" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector2_genosl" nodedef="ND_power_vector2" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector2FA_genosl" nodedef="ND_power_vector2FA" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector3_genosl" nodedef="ND_power_vector3" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector3FA_genosl" nodedef="ND_power_vector3FA" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector4_genosl" nodedef="ND_power_vector4" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_power_vector4FA_genosl" nodedef="ND_power_vector4FA" target="genosl" sourcecode="pow({{in1}}, {{in2}})" />
  <implementation name="IM_sin_float_genosl" nodedef="ND_sin_float" target="genosl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_float_genosl" nodedef="ND_cos_float" target="genosl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_float_genosl" nodedef="ND_tan_float" target="genosl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_float_genosl" nodedef="ND_asin_float" target="genosl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_float_genosl" nodedef="ND_acos_float" target="genosl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_float_genosl" nodedef="ND_atan2_float" target="genosl" sourcecode="atan2({{in1}},{{in2}})" />
  <implementation name="IM_sin_vector2_genosl" nodedef="ND_sin_vector2" target="genosl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector2_genosl" nodedef="ND_cos_vector2" target="genosl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector2_genosl" nodedef="ND_tan_vector2" target="genosl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector2_genosl" nodedef="ND_asin_vector2" target="genosl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector2_genosl" nodedef="ND_acos_vector2" target="genosl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector2_genosl" nodedef="ND_atan2_vector2" target="genosl" sourcecode="atan2({{in1}},{{in2}})" />
  <implementation name="IM_sin_vector3_genosl" nodedef="ND_sin_vector3" target="genosl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector3_genosl" nodedef="ND_cos_vector3" target="genosl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector3_genosl" nodedef="ND_tan_vector3" target="genosl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector3_genosl" nodedef="ND_asin_vector3" target="genosl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector3_genosl" nodedef="ND_acos_vector3" target="genosl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector3_genosl" nodedef="ND_atan2_vector3" target="genosl" sourcecode="atan2({{in1}},{{in2}})" />
  <implementation name="IM_sin_vector4_genosl" nodedef="ND_sin_vector4" target="genosl" sourcecode="sin({{in}})" />
  <implementation name="IM_cos_vector4_genosl" nodedef="ND_cos_vector4" target="genosl" sourcecode="cos({{in}})" />
  <implementation name="IM_tan_vector4_genosl" nodedef="ND_tan_vector4" target="genosl" sourcecode="tan({{in}})" />
  <implementation name="IM_asin_vector4_genosl" nodedef="ND_asin_vector4" target="genosl" sourcecode="asin({{in}})" />
  <implementation name="IM_acos_vector4_genosl" nodedef="ND_acos_vector4" target="genosl" sourcecode="acos({{in}})" />
  <implementation name="IM_atan2_vector4_genosl" nodedef="ND_atan2_vector4" target="genosl" sourcecode="atan2({{in1}},{{in2}})" />
  <implementation name="IM_sqrt_float_genosl" nodedef="ND_sqrt_float" target="genosl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector2_genosl" nodedef="ND_sqrt_vector2" target="genosl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector3_genosl" nodedef="ND_sqrt_vector3" target="genosl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_sqrt_vector4_genosl" nodedef="ND_sqrt_vector4" target="genosl" sourcecode="sqrt({{in}})" />
  <implementation name="IM_ln_float_genosl" nodedef="ND_ln_float" target="genosl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector2_genosl" nodedef="ND_ln_vector2" target="genosl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector3_genosl" nodedef="ND_ln_vector3" target="genosl" sourcecode="log({{in}})" />
  <implementation name="IM_ln_vector4_genosl" nodedef="ND_ln_vector4" target="genosl" sourcecode="log({{in}})" />
  <implementation name="IM_exp_float_genosl" nodedef="ND_exp_float" target="genosl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector2_genosl" nodedef="ND_exp_vector2" target="genosl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector3_genosl" nodedef="ND_exp_vector3" target="genosl" sourcecode="exp({{in}})" />
  <implementation name="IM_exp_vector4_genosl" nodedef="ND_exp_vector4" target="genosl" sourcecode="exp({{in}})" />
  <implementation name="IM_sign_float_genosl" nodedef="ND_sign_float" target="genosl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_color3_genosl" nodedef="ND_sign_color3" target="genosl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_color4_genosl" nodedef="ND_sign_color4" target="genosl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector2_genosl" nodedef="ND_sign_vector2" target="genosl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector3_genosl" nodedef="ND_sign_vector3" target="genosl" sourcecode="sign({{in}})" />
  <implementation name="IM_sign_vector4_genosl" nodedef="ND_sign_vector4" target="genosl" sourcecode="sign({{in}})" />
  <implementation name="IM_clamp_float_genosl" nodedef="ND_clamp_float" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color3_genosl" nodedef="ND_clamp_color3" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color3FA_genosl" nodedef="ND_clamp_color3FA" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color4_genosl" nodedef="ND_clamp_color4" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_color4FA_genosl" nodedef="ND_clamp_color4FA" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector2_genosl" nodedef="ND_clamp_vector2" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector2FA_genosl" nodedef="ND_clamp_vector2FA" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector3_genosl" nodedef="ND_clamp_vector3" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector3FA_genosl" nodedef="ND_clamp_vector3FA" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector4_genosl" nodedef="ND_clamp_vector4" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_clamp_vector4FA_genosl" nodedef="ND_clamp_vector4FA" target="genosl" sourcecode="clamp({{in}}, {{low}}, {{high}})" />
  <implementation name="IM_min_float_genosl" nodedef="ND_min_float" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color3_genosl" nodedef="ND_min_color3" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color3FA_genosl" nodedef="ND_min_color3FA" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color4_genosl" nodedef="ND_min_color4" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_color4FA_genosl" nodedef="ND_min_color4FA" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector2_genosl" nodedef="ND_min_vector2" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector2FA_genosl" nodedef="ND_min_vector2FA" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector3_genosl" nodedef="ND_min_vector3" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector3FA_genosl" nodedef="ND_min_vector3FA" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector4_genosl" nodedef="ND_min_vector4" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_min_vector4FA_genosl" nodedef="ND_min_vector4FA" target="genosl" sourcecode="min({{in1}}, {{in2}})" />
  <implementation name="IM_max_float_genosl" nodedef="ND_max_float" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color3_genosl" nodedef="ND_max_color3" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color3FA_genosl" nodedef="ND_max_color3FA" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color4_genosl" nodedef="ND_max_color4" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_color4FA_genosl" nodedef="ND_max_color4FA" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector2_genosl" nodedef="ND_max_vector2" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector2FA_genosl" nodedef="ND_max_vector2FA" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector3_genosl" nodedef="ND_max_vector3" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector3FA_genosl" nodedef="ND_max_vector3FA" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector4_genosl" nodedef="ND_max_vector4" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_max_vector4FA_genosl" nodedef="ND_max_vector4FA" target="genosl" sourcecode="max({{in1}}, {{in2}})" />
  <implementation name="IM_normalize_vector2_genosl" nodedef="ND_normalize_vector2" target="genosl" sourcecode="normalize({{in}})" />
  <implementation name="IM_normalize_vector3_genosl" nodedef="ND_normalize_vector3" target="genosl" sourcecode="normalize({{in}})" />
  <implementation name="IM_normalize_vector4_genosl" nodedef="ND_normalize_vector4" target="genosl" sourcecode="normalize({{in}})" />
  <implementation name="IM_magnitude_vector2_genosl" nodedef="ND_magnitude_vector2" target="genosl" sourcecode="length({{in}})" />
  <implementation name="IM_magnitude_vector3_genosl" nodedef="ND_magnitude_vector3" target="genosl" sourcecode="length({{in}})" />
  <implementation name="IM_magnitude_vector4_genosl" nodedef="ND_magnitude_vector4" target="genosl" sourcecode="length({{in}})" />
  <implementation name="IM_dotproduct_vector2_genosl" nodedef="ND_dotproduct_vector2" target="genosl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_dotproduct_vector3_genosl" nodedef="ND_dotproduct_vector3" target="genosl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_dotproduct_vector4_genosl" nodedef="ND_dotproduct_vector4" target="genosl" sourcecode="dot({{in1}}, {{in2}})" />
  <implementation name="IM_crossproduct_vector3_genosl" nodedef="ND_crossproduct_vector3" target="genosl" sourcecode="cross({{in1}}, {{in2}})" />
  <implementation name="IM_transformpoint_vector3_genosl" nodedef="ND_transformpoint_vector3" target="genosl" sourcecode="transform({{fromspace}}, {{tospace}}, point({{in}}))" />
  <implementation name="IM_transformvector_vector3_genosl" nodedef="ND_transformvector_vector3" target="genosl" sourcecode="transform({{fromspace}}, {{tospace}}, {{in}})" />
  <implementation name="IM_transformnormal_vector3_genosl" nodedef="ND_transformnormal_vector3" target="genosl" sourcecode="transform({{fromspace}}, {{tospace}}, normal({{in}}))" />
  <implementation name="IM_transformmatrix_vector2M3_genosl" nodedef="ND_transformmatrix_vector2M3" function="mx_transformmatrix_vector2M3" file="mx_transformmatrix_vector2M3.osl" target="genosl" />
  <implementation name="IM_transformmatrix_vector3_genosl" nodedef="ND_transformmatrix_vector3" target="genosl" sourcecode="transform({{mat}}, {{in}})" />
  <implementation name="IM_transformmatrix_vector3M4_genosl" nodedef="ND_transformmatrix_vector3M4" target="genosl" sourcecode="transform({{mat}}, {{in}})" />
  <implementation name="IM_transformmatrix_vector4_genosl" nodedef="ND_transformmatrix_vector4" target="genosl" sourcecode="transform({{mat}}, {{in}})" />
  <implementation name="IM_transpose_matrix33_genosl" nodedef="ND_transpose_matrix33" target="genosl" sourcecode="transpose({{in}})" />
  <implementation name="IM_transpose_matrix44_genosl" nodedef="ND_transpose_matrix44" target="genosl" sourcecode="transpose({{in}})" />
  <implementation name="IM_determinant_matrix33_genosl" nodedef="ND_determinant_matrix33" target="genosl" sourcecode="determinant({{in}})" />
  <implementation name="IM_determinant_matrix44_genosl" nodedef="ND_determinant_matrix44" target="genosl" sourcecode="determinant({{in}})" />
  <implementation name="IM_invertmatrix_matrix33_genosl" nodedef="ND_invertmatrix_matrix33" target="genosl" sourcecode="1 / {{in}}" />
  <implementation name="IM_invertmatrix_matrix44_genosl" nodedef="ND_invertmatrix_matrix44" target="genosl" sourcecode="1 / {{in}}" />
  <implementation name="IM_rotate2d_vector2_genosl" nodedef="ND_rotate2d_vector2" file="mx_rotate_vector2.osl" function="mx_rotate_vector2" target="genosl" />
  <implementation name="IM_rotate3d_vector3_genosl" nodedef="ND_rotate3d_vector3" file="mx_rotate_vector3.osl" function="mx_rotate_vector3" target="genosl" />
  <implementation name="IM_remap_float_genosl" nodedef="ND_remap_float" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_color3_genosl" nodedef="ND_remap_color3" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_color3FA_genosl" nodedef="ND_remap_color3FA" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_color4_genosl" nodedef="ND_remap_color4" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_color4FA_genosl" nodedef="ND_remap_color4FA" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_vector2_genosl" nodedef="ND_remap_vector2" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_vector2FA_genosl" nodedef="ND_remap_vector2FA" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_vector3_genosl" nodedef="ND_remap_vector3" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_vector3FA_genosl" nodedef="ND_remap_vector3FA" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_vector4_genosl" nodedef="ND_remap_vector4" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_remap_vector4FA_genosl" nodedef="ND_remap_vector4FA" target="genosl" sourcecode="mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)" />
  <implementation name="IM_smoothstep_float_genosl" nodedef="ND_smoothstep_float" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_color3_genosl" nodedef="ND_smoothstep_color3" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_color3FA_genosl" nodedef="ND_smoothstep_color3FA" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_color4_genosl" nodedef="ND_smoothstep_color4" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_color4FA_genosl" nodedef="ND_smoothstep_color4FA" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_vector2_genosl" nodedef="ND_smoothstep_vector2" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_vector2FA_genosl" nodedef="ND_smoothstep_vector2FA" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_vector3_genosl" nodedef="ND_smoothstep_vector3" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_vector3FA_genosl" nodedef="ND_smoothstep_vector3FA" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_vector4_genosl" nodedef="ND_smoothstep_vector4" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_smoothstep_vector4FA_genosl" nodedef="ND_smoothstep_vector4FA" target="genosl" sourcecode="smoothstep({{low}}, {{high}}, {{in}})" />
  <implementation name="IM_luminance_color3_genosl" nodedef="ND_luminance_color3" file="mx_luminance_color3.osl" function="mx_luminance_color3" target="genosl" />
  <implementation name="IM_luminance_color4_genosl" nodedef="ND_luminance_color4" file="mx_luminance_color4.osl" function="mx_luminance_color4" target="genosl" />
  <implementation name="IM_rgbtohsv_color3_genosl" nodedef="ND_rgbtohsv_color3" file="mx_rgbtohsv_color3.osl" function="mx_rgbtohsv_color3" target="genosl" />
  <implementation name="IM_rgbtohsv_color4_genosl" nodedef="ND_rgbtohsv_color4" file="mx_rgbtohsv_color4.osl" function="mx_rgbtohsv_color4" target="genosl" />
  <implementation name="IM_hsvtorgb_color3_genosl" nodedef="ND_hsvtorgb_color3" file="mx_hsvtorgb_color3.osl" function="mx_hsvtorgb_color3" target="genosl" />
  <implementation name="IM_hsvtorgb_color4_genosl" nodedef="ND_hsvtorgb_color4" file="mx_hsvtorgb_color4.osl" function="mx_hsvtorgb_color4" target="genosl" />
  <implementation name="IM_premult_color4_genosl" nodedef="ND_premult_color4" file="mx_premult_color4.osl" function="mx_premult_color4" target="genosl" />
  <implementation name="IM_unpremult_color4_genosl" nodedef="ND_unpremult_color4" file="mx_unpremult_color4.osl" function="mx_unpremult_color4" target="genosl" />
  <implementation name="IM_plus_float_genosl" nodedef="ND_plus_float" target="genosl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_plus_color3_genosl" nodedef="ND_plus_color3" target="genosl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_plus_color4_genosl" nodedef="ND_plus_color4" target="genosl" sourcecode="({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_float_genosl" nodedef="ND_minus_float" target="genosl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_color3_genosl" nodedef="ND_minus_color3" target="genosl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_minus_color4_genosl" nodedef="ND_minus_color4" target="genosl" sourcecode="({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_float_genosl" nodedef="ND_difference_float" target="genosl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_color3_genosl" nodedef="ND_difference_color3" target="genosl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_difference_color4_genosl" nodedef="ND_difference_color4" target="genosl" sourcecode="({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_burn_float_genosl" nodedef="ND_burn_float" file="mx_burn_float.osl" function="mx_burn_float" target="genosl" />
  <implementation name="IM_burn_color3_genosl" nodedef="ND_burn_color3" file="mx_burn_color3.osl" function="mx_burn_color3" target="genosl" />
  <implementation name="IM_burn_color4_genosl" nodedef="ND_burn_color4" file="mx_burn_color4.osl" function="mx_burn_color4" target="genosl" />
  <implementation name="IM_dodge_float_genosl" nodedef="ND_dodge_float" file="mx_dodge_float.osl" function="mx_dodge_float" target="genosl" />
  <implementation name="IM_dodge_color3_genosl" nodedef="ND_dodge_color3" file="mx_dodge_color3.osl" function="mx_dodge_color3" target="genosl" />
  <implementation name="IM_dodge_color4_genosl" nodedef="ND_dodge_color4" file="mx_dodge_color4.osl" function="mx_dodge_color4" target="genosl" />
  <implementation name="IM_screen_float_genosl" nodedef="ND_screen_float" target="genosl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_screen_color3_genosl" nodedef="ND_screen_color3" target="genosl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_screen_color4_genosl" nodedef="ND_screen_color4" target="genosl" sourcecode="({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_overlay_float_genosl" nodedef="ND_overlay_float" target="genosl" sourcecode="({{fg}} < 0.5) ? ({{mix}}*2.0*{{fg}}*{{bg}}) + ((1.0-{{mix}})*{{bg}}) : ({{mix}}*(1.0-(1.0-{{fg}})*(1.0-{{bg}}))) + ((1.0-{{mix}})*{{bg}})" />
  <implementation name="IM_overlay_color3_genosl" nodedef="ND_overlay_color3" file="mx_overlay_color3.osl" function="mx_overlay_color3" target="genosl" />
  <implementation name="IM_overlay_color4_genosl" nodedef="ND_overlay_color4" file="mx_overlay_color4.osl" function="mx_overlay_color4" target="genosl" />
  <implementation name="IM_disjointover_color4_genosl" nodedef="ND_disjointover_color4" file="mx_disjointover_color4.osl" function="mx_disjointover_color4" target="genosl" />
  <implementation name="IM_in_color4_genosl" nodedef="ND_in_color4" target="genosl" sourcecode="({{fg}}*{{bg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}))" />
  <implementation name="IM_mask_color4_genosl" nodedef="ND_mask_color4" target="genosl" sourcecode="({{bg}}*{{fg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));" />
  <implementation name="IM_matte_color4_genosl" nodedef="ND_matte_color4" target="genosl" sourcecode="color4({{fg}}.rgb*{{fg}}.a + {{bg}}.rgb*(1.0-{{fg}}.a), {{fg}}.a + ({{bg}}.a*(1.0-{{fg}}.a)) ) * {{mix}} + ({{bg}} * (1.0-{{mix}}))" />
  <implementation name="IM_out_color4_genosl" nodedef="ND_out_color4" target="genosl" sourcecode="({{fg}}*(1.0-{{bg}}.a)  * {{mix}}) + ({{bg}} * (1.0-{{mix}}))" />
  <implementation name="IM_over_color4_genosl" nodedef="ND_over_color4" target="genosl" sourcecode="({{fg}} + ({{bg}}*(1.0-{{fg}}.a))) * {{mix}} + {{bg}} * (1.0-{{mix}})" />
  <implementation name="IM_inside_float_genosl" nodedef="ND_inside_float" target="genosl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_inside_color3_genosl" nodedef="ND_inside_color3" target="genosl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_inside_color4_genosl" nodedef="ND_inside_color4" target="genosl" sourcecode="{{in}} * {{mask}}" />
  <implementation name="IM_outside_float_genosl" nodedef="ND_outside_float" target="genosl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_outside_color3_genosl" nodedef="ND_outside_color3" target="genosl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_outside_color4_genosl" nodedef="ND_outside_color4" target="genosl" sourcecode="{{in}} * (1.0 - {{mask}})" />
  <implementation name="IM_mix_float_genosl" nodedef="ND_mix_float" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color3_genosl" nodedef="ND_mix_color3" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color3_color3_genosl" nodedef="ND_mix_color3_color3" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color4_genosl" nodedef="ND_mix_color4" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_color4_color4_genosl" nodedef="ND_mix_color4_color4" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector2_genosl" nodedef="ND_mix_vector2" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector2_vector2_genosl" nodedef="ND_mix_vector2_vector2" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector3_genosl" nodedef="ND_mix_vector3" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector3_vector3_genosl" nodedef="ND_mix_vector3_vector3" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector4_genosl" nodedef="ND_mix_vector4" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_vector4_vector4_genosl" nodedef="ND_mix_vector4_vector4" target="genosl" sourcecode="mix({{bg}}, {{fg}}, {{mix}})" />
  <implementation name="IM_mix_surfaceshader_genosl" nodedef="ND_mix_surfaceshader" file="mx_mix_surfaceshader.osl" function="mx_mix_surfaceshader" target="genosl" />
  <implementation name="IM_ifgreater_float_genosl" nodedef="ND_ifgreater_float" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_color3_genosl" nodedef="ND_ifgreater_color3" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_color4_genosl" nodedef="ND_ifgreater_color4" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_vector2_genosl" nodedef="ND_ifgreater_vector2" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_vector3_genosl" nodedef="ND_ifgreater_vector3" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_vector4_genosl" nodedef="ND_ifgreater_vector4" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_floatI_genosl" nodedef="ND_ifgreater_floatI" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_color3I_genosl" nodedef="ND_ifgreater_color3I" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_color4I_genosl" nodedef="ND_ifgreater_color4I" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_vector2I_genosl" nodedef="ND_ifgreater_vector2I" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_vector3I_genosl" nodedef="ND_ifgreater_vector3I" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreater_vector4I_genosl" nodedef="ND_ifgreater_vector4I" target="genosl" sourcecode="mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_float_genosl" nodedef="ND_ifgreatereq_float" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_color3_genosl" nodedef="ND_ifgreatereq_color3" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_color4_genosl" nodedef="ND_ifgreatereq_color4" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_vector2_genosl" nodedef="ND_ifgreatereq_vector2" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_vector3_genosl" nodedef="ND_ifgreatereq_vector3" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_vector4_genosl" nodedef="ND_ifgreatereq_vector4" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_floatI_genosl" nodedef="ND_ifgreatereq_floatI" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_color3I_genosl" nodedef="ND_ifgreatereq_color3I" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_color4I_genosl" nodedef="ND_ifgreatereq_color4I" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_vector2I_genosl" nodedef="ND_ifgreatereq_vector2I" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_vector3I_genosl" nodedef="ND_ifgreatereq_vector3I" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifgreatereq_vector4I_genosl" nodedef="ND_ifgreatereq_vector4I" target="genosl" sourcecode="mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_float_genosl" nodedef="ND_ifequal_float" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_color3_genosl" nodedef="ND_ifequal_color3" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_color4_genosl" nodedef="ND_ifequal_color4" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector2_genosl" nodedef="ND_ifequal_vector2" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector3_genosl" nodedef="ND_ifequal_vector3" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector4_genosl" nodedef="ND_ifequal_vector4" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_floatI_genosl" nodedef="ND_ifequal_floatI" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_color3I_genosl" nodedef="ND_ifequal_color3I" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_color4I_genosl" nodedef="ND_ifequal_color4I" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector2I_genosl" nodedef="ND_ifequal_vector2I" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector3I_genosl" nodedef="ND_ifequal_vector3I" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector4I_genosl" nodedef="ND_ifequal_vector4I" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_floatB_genosl" nodedef="ND_ifequal_floatB" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_color3B_genosl" nodedef="ND_ifequal_color3B" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_color4B_genosl" nodedef="ND_ifequal_color4B" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector2B_genosl" nodedef="ND_ifequal_vector2B" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector3B_genosl" nodedef="ND_ifequal_vector3B" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_ifequal_vector4B_genosl" nodedef="ND_ifequal_vector4B" target="genosl" sourcecode="mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})" />
  <implementation name="IM_switch_float_genosl" nodedef="ND_switch_float" target="genosl" />
  <implementation name="IM_switch_color3_genosl" nodedef="ND_switch_color3" target="genosl" />
  <implementation name="IM_switch_color4_genosl" nodedef="ND_switch_color4" target="genosl" />
  <implementation name="IM_switch_vector2_genosl" nodedef="ND_switch_vector2" target="genosl" />
  <implementation name="IM_switch_vector3_genosl" nodedef="ND_switch_vector3" target="genosl" />
  <implementation name="IM_switch_vector4_genosl" nodedef="ND_switch_vector4" target="genosl" />
  <implementation name="IM_switch_floatI_genosl" nodedef="ND_switch_floatI" target="genosl" />
  <implementation name="IM_switch_color3I_genosl" nodedef="ND_switch_color3I" target="genosl" />
  <implementation name="IM_switch_color4I_genosl" nodedef="ND_switch_color4I" target="genosl" />
  <implementation name="IM_switch_vector2I_genosl" nodedef="ND_switch_vector2I" target="genosl" />
  <implementation name="IM_switch_vector3I_genosl" nodedef="ND_switch_vector3I" target="genosl" />
  <implementation name="IM_switch_vector4I_genosl" nodedef="ND_switch_vector4I" target="genosl" />
  <implementation name="IM_convert_float_color3_genosl" nodedef="ND_convert_float_color3" target="genosl" />
  <implementation name="IM_convert_float_color4_genosl" nodedef="ND_convert_float_color4" target="genosl" />
  <implementation name="IM_convert_float_vector2_genosl" nodedef="ND_convert_float_vector2" target="genosl" />
  <implementation name="IM_convert_float_vector3_genosl" nodedef="ND_convert_float_vector3" target="genosl" />
  <implementation name="IM_convert_float_vector4_genosl" nodedef="ND_convert_float_vector4" target="genosl" />
  <implementation name="IM_convert_vector2_vector3_genosl" nodedef="ND_convert_vector2_vector3" target="genosl" />
  <implementation name="IM_convert_vector3_vector2_genosl" nodedef="ND_convert_vector3_vector2" target="genosl" />
  <implementation name="IM_convert_vector3_color3_genosl" nodedef="ND_convert_vector3_color3" target="genosl" />
  <implementation name="IM_convert_vector3_vector4_genosl" nodedef="ND_convert_vector3_vector4" target="genosl" />
  <implementation name="IM_convert_vector4_vector3_genosl" nodedef="ND_convert_vector4_vector3" target="genosl" />
  <implementation name="IM_convert_vector4_color4_genosl" nodedef="ND_convert_vector4_color4" target="genosl" />
  <implementation name="IM_convert_color3_vector3_genosl" nodedef="ND_convert_color3_vector3" target="genosl" />
  <implementation name="IM_convert_color4_vector4_genosl" nodedef="ND_convert_color4_vector4" target="genosl" />
  <implementation name="IM_convert_color3_color4_genosl" nodedef="ND_convert_color3_color4" target="genosl" />
  <implementation name="IM_convert_color4_color3_genosl" nodedef="ND_convert_color4_color3" target="genosl" />
  <implementation name="IM_convert_boolean_float_genosl" nodedef="ND_convert_boolean_float" target="genosl" />
  <implementation name="IM_convert_integer_float_genosl" nodedef="ND_convert_integer_float" target="genosl" />
  <implementation name="IM_swizzle_float_color3_genosl" nodedef="ND_swizzle_float_color3" target="genosl" />
  <implementation name="IM_swizzle_float_color4_genosl" nodedef="ND_swizzle_float_color4" target="genosl" />
  <implementation name="IM_swizzle_float_vector2_genosl" nodedef="ND_swizzle_float_vector2" target="genosl" />
  <implementation name="IM_swizzle_float_vector3_genosl" nodedef="ND_swizzle_float_vector3" target="genosl" />
  <implementation name="IM_swizzle_float_vector4_genosl" nodedef="ND_swizzle_float_vector4" target="genosl" />
  <implementation name="IM_swizzle_color3_float_genosl" nodedef="ND_swizzle_color3_float" target="genosl" />
  <implementation name="IM_swizzle_color3_color3_genosl" nodedef="ND_swizzle_color3_color3" target="genosl" />
  <implementation name="IM_swizzle_color3_color4_genosl" nodedef="ND_swizzle_color3_color4" target="genosl" />
  <implementation name="IM_swizzle_color3_vector2_genosl" nodedef="ND_swizzle_color3_vector2" target="genosl" />
  <implementation name="IM_swizzle_color3_vector3_genosl" nodedef="ND_swizzle_color3_vector3" target="genosl" />
  <implementation name="IM_swizzle_color3_vector4_genosl" nodedef="ND_swizzle_color3_vector4" target="genosl" />
  <implementation name="IM_swizzle_color4_float_genosl" nodedef="ND_swizzle_color4_float" target="genosl" />
  <implementation name="IM_swizzle_color4_color3_genosl" nodedef="ND_swizzle_color4_color3" target="genosl" />
  <implementation name="IM_swizzle_color4_color4_genosl" nodedef="ND_swizzle_color4_color4" target="genosl" />
  <implementation name="IM_swizzle_color4_vector2_genosl" nodedef="ND_swizzle_color4_vector2" target="genosl" />
  <implementation name="IM_swizzle_color4_vector3_genosl" nodedef="ND_swizzle_color4_vector3" target="genosl" />
  <implementation name="IM_swizzle_color4_vector4_genosl" nodedef="ND_swizzle_color4_vector4" target="genosl" />
  <implementation name="IM_swizzle_vector2_float_genosl" nodedef="ND_swizzle_vector2_float" target="genosl" />
  <implementation name="IM_swizzle_vector2_color3_genosl" nodedef="ND_swizzle_vector2_color3" target="genosl" />
  <implementation name="IM_swizzle_vector2_color4_genosl" nodedef="ND_swizzle_vector2_color4" target="genosl" />
  <implementation name="IM_swizzle_vector2_vector2_genosl" nodedef="ND_swizzle_vector2_vector2" target="genosl" />
  <implementation name="IM_swizzle_vector2_vector3_genosl" nodedef="ND_swizzle_vector2_vector3" target="genosl" />
  <implementation name="IM_swizzle_vector2_vector4_genosl" nodedef="ND_swizzle_vector2_vector4" target="genosl" />
  <implementation name="IM_swizzle_vector3_float_genosl" nodedef="ND_swizzle_vector3_float" target="genosl" />
  <implementation name="IM_swizzle_vector3_color3_genosl" nodedef="ND_swizzle_vector3_color3" target="genosl" />
  <implementation name="IM_swizzle_vector3_color4_genosl" nodedef="ND_swizzle_vector3_color4" target="genosl" />
  <implementation name="IM_swizzle_vector3_vector2_genosl" nodedef="ND_swizzle_vector3_vector2" target="genosl" />
  <implementation name="IM_swizzle_vector3_vector3_genosl" nodedef="ND_swizzle_vector3_vector3" target="genosl" />
  <implementation name="IM_swizzle_vector3_vector4_genosl" nodedef="ND_swizzle_vector3_vector4" target="genosl" />
  <implementation name="IM_swizzle_vector4_float_genosl" nodedef="ND_swizzle_vector4_float" target="genosl" />
  <implementation name="IM_swizzle_vector4_color3_genosl" nodedef="ND_swizzle_vector4_color3" target="genosl" />
  <implementation name="IM_swizzle_vector4_color4_genosl" nodedef="ND_swizzle_vector4_color4" target="genosl" />
  <implementation name="IM_swizzle_vector4_vector2_genosl" nodedef="ND_swizzle_vector4_vector2" target="genosl" />
  <implementation name="IM_swizzle_vector4_vector3_genosl" nodedef="ND_swizzle_vector4_vector3" target="genosl" />
  <implementation name="IM_swizzle_vector4_vector4_genosl" nodedef="ND_swizzle_vector4_vector4" target="genosl" />
  <implementation name="IM_combine2_vector2_genosl" nodedef="ND_combine2_vector2" target="genosl" />
  <implementation name="IM_combine2_color4CF_genosl" nodedef="ND_combine2_color4CF" target="genosl" />
  <implementation name="IM_combine2_vector4VF_genosl" nodedef="ND_combine2_vector4VF" target="genosl" />
  <implementation name="IM_combine2_vector4VV_genosl" nodedef="ND_combine2_vector4VV" target="genosl" />
  <implementation name="IM_combine3_color3_genosl" nodedef="ND_combine3_color3" target="genosl" />
  <implementation name="IM_combine3_vector3_genosl" nodedef="ND_combine3_vector3" target="genosl" />
  <implementation name="IM_combine4_color4_genosl" nodedef="ND_combine4_color4" target="genosl" />
  <implementation name="IM_combine4_vector4_genosl" nodedef="ND_combine4_vector4" target="genosl" />
  <implementation name="IM_blur_float_genosl" nodedef="ND_blur_float" target="genosl" />
  <implementation name="IM_blur_color3_genosl" nodedef="ND_blur_color3" target="genosl" />
  <implementation name="IM_blur_color4_genosl" nodedef="ND_blur_color4" target="genosl" />
  <implementation name="IM_blur_vector2_genosl" nodedef="ND_blur_vector2" target="genosl" />
  <implementation name="IM_blur_vector3_genosl" nodedef="ND_blur_vector3" target="genosl" />
  <implementation name="IM_blur_vector4_genosl" nodedef="ND_blur_vector4" target="genosl" />
  <implementation name="IM_heighttonormal_vector3_genosl" nodedef="ND_heighttonormal_vector3" file="mx_heighttonormal_vector3.osl" function="mx_heighttonormal_vector3" target="genosl" />
  <implementation name="IM_dot_float_genosl" nodedef="ND_dot_float" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_color3_genosl" nodedef="ND_dot_color3" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_color4_genosl" nodedef="ND_dot_color4" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector2_genosl" nodedef="ND_dot_vector2" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector3_genosl" nodedef="ND_dot_vector3" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_vector4_genosl" nodedef="ND_dot_vector4" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_boolean_genosl" nodedef="ND_dot_boolean" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_integer_genosl" nodedef="ND_dot_integer" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_matrix33_genosl" nodedef="ND_dot_matrix33" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_matrix44_genosl" nodedef="ND_dot_matrix44" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_string_genosl" nodedef="ND_dot_string" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_filename_genosl" nodedef="ND_dot_filename" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_surfaceshader_genosl" nodedef="ND_dot_surfaceshader" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_displacementshader_genosl" nodedef="ND_dot_displacementshader" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_volumeshader_genosl" nodedef="ND_dot_volumeshader" target="genosl" sourcecode="{{in}}" />
  <implementation name="IM_dot_lightshader_genosl" nodedef="ND_dot_lightshader" target="genosl" sourcecode="{{in}}" />
  <!--Custom add implementation (mxadd)-->
  <implementation name="IM_myadd_color3_genosl" nodedef="ND_myadd_color3" target="genosl" sourcecode="{{in1}} + {{in2}}" />
</materialx>
```
</details>


- Source implementation added: IM_myadd_color3_genosl
